In [ ]:
import pandas as pd
import numpy as np
import time #mesurer la durée de nos codes
import math
import requests #permet de formuler des requetes HTTP en Python : sera utile pour les query Wikidata et l'API OpenstreetMap
import re 
import csv
%pip install mkwikidata
import mkwikidata

# Departements 

In [ ]:
# on définit ici la liste des départements, à la fois sous forme de dictionnaire et de liste qui nous seront utiles dans la suite
# le dictionnaire a été récupéré à l'adresse https://gist.github.com/mlorant/b4d7bb6f96c47776c8082cf7af44ad95
# L'outre-mer n'est pas prise en compte
DEPARTMENTS = {
    '01': 'Ain', 
    '02': 'Aisne', 
    '03': 'Allier', 
    '04': 'Alpes-de-Haute-Provence', 
    '05': 'Hautes-Alpes',
    '06': 'Alpes-Maritimes', 
    '07': 'Ardèche', 
    '08': 'Ardennes', 
    '09': 'Ariège', 
    '10': 'Aube', 
    '11': 'Aude',
    '12': 'Aveyron', 
    '13': 'Bouches-du-Rhône', 
    '14': 'Calvados', 
    '15': 'Cantal', 
    '16': 'Charente',
    '17': 'Charente-Maritime', 
    '18': 'Cher', 
    '19': 'Corrèze', 
    '2A': 'Corse-du-Sud', 
    '2B': 'Haute-Corse',
    '21': 'Côte-d\'Or', 
    '22': 'Côtes-d\'Armor', 
    '23': 'Creuse', 
    '24': 'Dordogne', 
    '25': 'Doubs', 
    '26': 'Drôme',
    '27': 'Eure', 
    '28': 'Eure-et-Loir', 
    '29': 'Finistère', 
    '30': 'Gard', 
    '31': 'Haute-Garonne', 
    '32': 'Gers',
    '33': 'Gironde', 
    '34': 'Hérault', 
    '35': 'Ille-et-Vilaine', 
    '36': 'Indre', 
    '37': 'Indre-et-Loire',
    '38': 'Isère', 
    '39': 'Jura', 
    '40': 'Landes', 
    '41': 'Loir-et-Cher', 
    '42': 'Loire', 
    '43': 'Haute-Loire',
    '44': 'Loire-Atlantique', 
    '45': 'Loiret', 
    '46': 'Lot', 
    '47': 'Lot-et-Garonne', 
    '48': 'Lozère',
    '49': 'Maine-et-Loire', 
    '50': 'Manche', 
    '51': 'Marne', 
    '52': 'Haute-Marne', 
    '53': 'Mayenne',
    '54': 'Meurthe-et-Moselle', 
    '55': 'Meuse', 
    '56': 'Morbihan', 
    '57': 'Moselle', 
    '58': 'Nièvre', 
    '59': 'Nord',
    '60': 'Oise', 
    '61': 'Orne', 
    '62': 'Pas-de-Calais', 
    '63': 'Puy-de-Dôme', 
    '64': 'Pyrénées-Atlantiques',
    '65': 'Hautes-Pyrénées', 
    '66': 'Pyrénées-Orientales', 
    '67': 'Bas-Rhin', 
    '68': 'Haut-Rhin', 
    '69': 'Rhône',
    '70': 'Haute-Saône', 
    '71': 'Saône-et-Loire', 
    '72': 'Sarthe', 
    '73': 'Savoie', 
    '74': 'Haute-Savoie',
    '75': 'Paris', 
    '76': 'Seine-Maritime', 
    '77': 'Seine-et-Marne', 
    '78': 'Yvelines', 
    '79': 'Deux-Sèvres',
    '80': 'Somme', 
    '81': 'Tarn', 
    '82': 'Tarn-et-Garonne', 
    '83': 'Var', 
    '84': 'Vaucluse', 
    '85': 'Vendée',
    '86': 'Vienne', 
    '87': 'Haute-Vienne', 
    '88': 'Vosges', 
    '89': 'Yonne', 
    '90': 'Territoire de Belfort',
    '91': 'Essonne', 
    '92': 'Hauts-de-Seine', 
    '93': 'Seine-Saint-Denis', 
    '94': 'Val-de-Marne', 
    '95': 'Val-d\'Oise',
}
liste_dep = list(DEPARTMENTS.values())
print(liste_dep)

['Ain', 'Aisne', 'Allier', 'Alpes-de-Haute-Provence', 'Hautes-Alpes', 'Alpes-Maritimes', 'Ardèche', 'Ardennes', 'Ariège', 'Aube', 'Aude', 'Aveyron', 'Bouches-du-Rhône', 'Calvados', 'Cantal', 'Charente', 'Charente-Maritime', 'Cher', 'Corrèze', 'Corse-du-Sud', 'Haute-Corse', "Côte-d'Or", "Côtes-d'Armor", 'Creuse', 'Dordogne', 'Doubs', 'Drôme', 'Eure', 'Eure-et-Loir', 'Finistère', 'Gard', 'Haute-Garonne', 'Gers', 'Gironde', 'Hérault', 'Ille-et-Vilaine', 'Indre', 'Indre-et-Loire', 'Isère', 'Jura', 'Landes', 'Loir-et-Cher', 'Loire', 'Haute-Loire', 'Loire-Atlantique', 'Loiret', 'Lot', 'Lot-et-Garonne', 'Lozère', 'Maine-et-Loire', 'Manche', 'Marne', 'Haute-Marne', 'Mayenne', 'Meurthe-et-Moselle', 'Meuse', 'Morbihan', 'Moselle', 'Nièvre', 'Nord', 'Oise', 'Orne', 'Pas-de-Calais', 'Puy-de-Dôme', 'Pyrénées-Atlantiques', 'Hautes-Pyrénées', 'Pyrénées-Orientales', 'Bas-Rhin', 'Haut-Rhin', 'Rhône', 'Haute-Saône', 'Saône-et-Loire', 'Sarthe', 'Savoie', 'Haute-Savoie', 'Paris', 'Seine-Maritime', 'Seine-

# Queries Wikidata

Le but de cette section est d'obtenir au final un dictionnaire avec pour clés  les communes françaises et pour valeurs la population de la commune

In [ ]:
#this function gives the wikidata code of a departement i.e. the value that we write in wd:Q
def get_wikidata_code(departement_name):
  #double { to avoid conflict with format
  query = f"""
  SELECT ?item
  WHERE {{
    ?item rdfs:label "{departement_name}"@fr.
    ?item wdt:P31 wd:Q6465.  # elements qui sont des départements
  }}
  LIMIT 1
  """
  query_result = mkwikidata.run_query(query, params={ })
  address = query_result['results']['bindings'][0]['item']['value']
  
  #we keep the part after the Q in the URI of the departement
  i = 0
  for elem in address:
      if elem == 'Q':
          #print(i)
          iQ = i
      i += 1
  code = address[iQ+1:]

  return code

In [ ]:
#query that gives the list of cities in a departement with their population 
def get_list_cities(departement_code):
  #double { to avoid conflict with format
  query = f"""
  SELECT ?city ?cityLabel ?population
  WHERE {{
    ?city wdt:P31 wd:Q484170 . #est une commune française
    ?city wdt:P1082 ?population. #on récupère sa population
    ?city wdt:P131 wd:Q{departement_code}. #commune du département untel
    FILTER(?population > 2500) #on se limite aux communes de population > 2500 : justifié car autour de chaque petit village on a forcément à quelques kilomètres une commune de plus de 2500 habitants
    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "fr". }}
  }}
  """

  query_result = mkwikidata.run_query(query, params={ })
  print(query_result)

  list_cities_departement = []
  for i in range(len(query_result['results']['bindings'])):
    list_cities_departement.append((query_result['results']['bindings'][i]['cityLabel']['value'], query_result['results']['bindings'][i]['population']['value']))
  return list_cities_departement

In [ ]:
#ATTENTION cette cellule prend du temps à s'executer : pour faciliter la réplication de ce notebook nous fournissons dans la cellule suivante le dictionnaire dico obtenu
dico = {}

for i in range(len(liste_dep)): 

  departement_code = get_wikidata_code(liste_dep[i])
  time.sleep(5) #l'API limitant le nombre de requetes par seconde, on est obligé de mettre des pauses pour ne pas avoir d'erreur
  list_cities = get_list_cities(departement_code)
  time.sleep(5)
  dico[liste_dep[i]] = list_cities

In [ ]:
dico = {'Ain': [('Beynost', '4754'), ('Saint-Maurice-de-Beynost', '3962'), ('Villars-les-Dombes', '4978'), ('Bourg-en-Bresse', '41111'), ('Châtillon-en-Michaille', '3923'), ('Belley', '9182'), ('Bellegarde-sur-Valserine', '11450'), ('Oyonnax', '22456'), ('Ambérieu-en-Bugey', '14134'), ('Ambronay', '2800'), ('Nantua', '3454'), ('Gex', '13121'), ('Bâgé-la-Ville', '3188'), ('Châtillon-sur-Chalaronne', '4931'), ('Arbent', '3349'), ('Attignat', '3251'), ('Chalamont', '2589'), ('Ceyzériat', '3194'), ('Bellignat', '3652'), ('Béligneux', '3376'), ('La Boisse', '3283'), ('Ferney-Voltaire', '10026'), ('Lagnieu', '7161'), ('Cessy', '4808'), ('Montluel', '6886'), ('Balan', '2563'), ('Dagneux', '4777'), ('Divonne-les-Bains', '9951'), ('Vonnas', '3181'), ('Hauteville-Lompnes', '3652'), ('Meximieux', '7923'), ('Loyettes', '3225'), ('Massieux', '2670'), ('Villieu-Loyes-Mollon', '3717'), ('Péronnas', '6520'), ('Culoz', '3004'), ('Jassans-Riottier', '6346'), ('Montrevel-en-Bresse', '2576'), ('Trévoux', '6920'), ("Pont-d'Ain", '2924'), ('Ornex', '4483'), ('Montréal-la-Cluse', '3456'), ('Saint-Maurice-de-Gourdans', '2645'), ('Prévessin-Moëns', '8711'), ('Péron', '2717'), ('Saint-Genis-Pouilly', '13943'), ('Miribel', '10122'), ('Neyron', '2569'), ('Polliat', '2547'), ('Thoiry', '6021'), ('Saint-Étienne-du-Bois', '2542'), ('Saint-Didier-sur-Chalaronne', '2938'), ('Viriat', '6595'), ('Saint-Denis-lès-Bourg', '5943'), ('Feillens', '3365'), ('Montmerle-sur-Saône', '3772'), ('Reyrieux', '4984'), ('Saint-André-de-Corcy', '3369'), ('Replonges', '3791'), ('Val-Revermont', '2538'), ('Bâgé-Dommartin', '4065'), ('Valserhône', '16378'), ("Plateau d'Hauteville", '4844'), ('Culoz-Béon', '3426')], 'Aisne': [('Laon', '24304'), ('Soissons', '28712'), ('Saint-Quentin', '53570'), ('Château-Thierry', '15254'), ('Vervins', '2587'), ('Villers-Cotterêts', '10424'), ('Chauny', '11653'), ('Hirson', '8710'), ('Guise', '4645'), ('Bohain-en-Vermandois', '5686'), ('Fère-en-Tardenois', '3001'), ('La Fère', '2838'), ('Tergnier', '13588'), ('Crouy', '2923'), ('Athies-sous-Laon', '2606'), ('Saint-Michel', '3314'), ('Charly-sur-Marne', '2585'), ('Beautor', '2714'), ('Fresnoy-le-Grand', '2958'), ('Le Nouvion-en-Thiérache', '2557'), ('Essômes-sur-Marne', '2745'), ('Villeneuve-Saint-Germain', '2549'), ('Gauchy', '5235'), ('Belleu', '3832'), ('Quessy', '2950'), ('Anizy-le-Grand', '2574'), ('Villeneuve-sur-Aisne', '2751')], 'Allier': [('Saint-Yorre', '2570'), ('Vichy', '24980'), ('Moulins', '19246'), ('Montluçon', '34361'), ('Désertines', '4459'), ('Dompierre-sur-Besbre', '3032'), ("Bourbon-l'Archambault", '2572'), ('Bellerive-sur-Allier', '8741'), ('Prémilhat', '2507'), ('Domérat', '8729'), ('Lapalisse', '3144'), ('Gannat', '5830'), ('Huriel', '2645'), ('Avermes', '3953'), ('Abrest', '2904'), ('Saint-Pourçain-sur-Sioule', '5221'), ('Cusset', '12474'), ('Yzeure', '12838'), ('Saint-Germain-des-Fossés', '3654'), ('Néris-les-Bains', '2570'), ('Commentry', '6132'), ('Varennes-sur-Allier', '3618'), ('Creuzier-le-Vieux', '3297')], 'Alpes-de-Haute-Provence': [('Sainte-Tulle', '3442'), ('Volx', '3206'), ('Pierrevert', '3905'), ('Forcalquier', '5121'), ('Digne-les-Bains', '16482'), ('Barcelonnette', '2564'), ('Manosque', '22528'), ('Sisteron', '7595'), ('Château-Arnoux-Saint-Auban', '5089'), ('Peyruis', '2813'), ('Gréoux-les-Bains', '2639'), ('Valensole', '3173'), ('Les Mées', '3751'), ('Oraison', '5867'), ('Villeneuve', '4323')], 'Hautes-Alpes': [('Laragne-Montéglin', '3543'), ('Gap', '40631'), ('Briançon', '11339'), ('Embrun', '6335'), ('Chorges', '3061'), ('Veynes', '3216'), ('La Bâtie-Neuve', '2562')], 'Alpes-Maritimes': [('Saint-André-de-la-Roche', '5695'), ('Saint-Vallier-de-Thiey', '3654'), ('Colomars', '3482'), ('Saint-Cézaire-sur-Siagne', '3903'), ('La Colle-sur-Loup', '7898'), ('Saint-Martin-du-Var', '3095'), ('Le Tignet', '3066'), ("L'Escarène", '2554'), ('Tourrette-Levens', '5013'), ('Contes', '7437'), ('Drap', '5117'), ('Nice', '343477'), ('Cannes', '74545'), ('Antibes', '73438'), ('Menton', '30525'), ('Vence', '18940'), ('Cagnes-sur-Mer', '52178'), ('Villefranche-sur-Mer', '5033'), ('Le Cannet', '41887'), ('Saint-Paul-de-Vence', '3252'), ('Beaulieu-sur-Mer', '3755'), ('Saint-Laurent-du-Var', '29169'), ('Roquebrune-Cap-Martin', '12839'), ('Mougins', '19982'), ('Vallauris', '27364'), ('Mandelieu-la-Napoule', '21998'), ('Biot', '9668'), ('La Turbie', '2981'), ('Levens', '4761'), ('Beausoleil', '13358'), ("Cap-d'Ail", '4523'), ('Pégomas', '7996'), ('Villeneuve-Loubet', '16285'), ('Mouans-Sartoux', '9887'), ('Châteauneuf-Grasse', '3664'), ('Roquefort-les-Pins', '7183'), ('Valbonne', '13486'), ('La Trinité', '10115'), ('La Gaude', '7010'), ('Tourrettes-sur-Loup', '4041'), ('Sospel', '3805'), ('Saint-Jeannet', '4246'), ('Le Rouret', '4072'), ('Carros', '12550'), ('Auribeau-sur-Siagne', '3200'), ('Peymeinade', '8192'), ('Gattières', '4173'), ('La Roquette-sur-Siagne', '5385'), ('Le Bar-sur-Loup', '2921')], 'Ardèche': [('Saint-Étienne-de-Fontbellon', '2833'), ('Le Teil', '8832'), ('Le Pouzin', '2879'), ('Chomérac', '3015'), ('Bourg-Saint-Andéol', '7187'), ('Saint-Péray', '7580'), ('Saint-Jean-de-Muzols', '2508'), ('Davézieux', '3168'), ('Le Cheylard', '2877'), ('Guilherand-Granges', '10756'), ('Viviers', '3689'), ('Vals-les-Bains', '3530'), ('Charmes-sur-Rhône', '2979'), ('Les Vans', '2655'), ('Villeneuve-de-Berg', '3001'), ('La Voulte-sur-Rhône', '4848'), ('Vernosc-lès-Annonay', '2614'), ('Roiffieux', '2746'), ('Cruas', '3063'), ('Privas', '8465'), ('Aubenas', '12479'), ('Tournon-sur-Rhône', '10622'), ('Annonay', '16297')], 'Ardennes': [('Carignan', '2879'), ('Villers-Semeuse', '3568'), ('Vivier-au-Court', '2918'), ('Vrigne-aux-Bois', '3184'), ('Charleville-Mézières', '46436'), ('Sedan', '16092'), ('Rethel', '7467'), ('Vouziers', '3919'), ('Givet', '6476'), ('Fumay', '3280'), ('Nouzonville', '5689'), ('Bogny-sur-Meuse', '5018'), ('Revin', '5836'), ('Charleville', '24668'), ('Mézières', '11799'), ('Vrigne aux Bois', '3572')], 'Ariège': [('La Tour-du-Crieu', '3218'), ('Varilhes', '3495'), ('Foix', '9493'), ('Pamiers', '15659'), ('Saint-Girons', '6418'), ('Lavelanet', '6031'), ('Mirepoix', '3102'), ('Saint-Jean-du-Falga', '2859'), ('Mazères', '3873'), ('Tarascon-sur-Ariège', '3024'), ('Saverdun', '4828')], 'Aube': [('Les Noës-près-Troyes', '3280'), ('Sainte-Savine', '10508'), ('Bréviandes', '2851'), ('Saint-Julien-les-Villas', '6765'), ('Bar-sur-Aube', '4787'), ('Arcis-sur-Aube', '2803'), ('Bar-sur-Seine', '2961'), ('La Chapelle-Saint-Luc', '11741'), ('Nogent-sur-Seine', '5966'), ('Brienne-le-Château', '2772'), ('La Rivière-de-Corps', '3591'), ('Saint-André-les-Vergers', '12784'), ('Romilly-sur-Seine', '14623'), ('Saint-Parres-aux-Tertres', '3167'), ('Rosières-près-Troyes', '4521'), ('Saint-Lyé', '2885'), ('Pont-Sainte-Marie', '5154'), ('Villenauxe-la-Grande', '2726'), ('Aix-Villemaur-Pâlis', '3546')], 'Aude': [('Vinassan', '2658'), ('Bram', '3208'), ("Salles-d'Aude", '3254'), ('Quillan', '3212'), ('Coursan', '5825'), ('Sigean', '5533'), ('Leucate', '4612'), ("Sallèles-d'Aude", '3024'), ('Port-la-Nouvelle', '5795'), ('Fleury', '3758'), ('Conques-sur-Orbiel', '2581'), ('Gruissan', '5087'), ('Villemoustaussou', '4469'), ('Pennautier', '2634'), ('Narbonne', '55516'), ('Carcassonne', '46825'), ('Lézignan-Corbières', '11317'), ('Ouveillan', '2564'), ('Trèbes', '5502'), ("Cuxac-d'Aude", '4033'), ('Limoux', '10012'), ('Castelnaudary', '12187'), ('Quillan', '3212')], 'Aveyron': [('Olemps', '3439'), ('Sébazac-Concourès', '3256'), ('Viviez-les-Albres', '2580'), ('Aubin', '3741'), ('Baraqueville', '3150'), ('Druelle', '2512'), ('Rodez', '24475'), ('Millau', '21979'), ('Villefranche-de-Rouergue', '11602'), ('Luc-la-Primaube', '6005'), ('Espalion', '4601'), ('Decazeville', '5323'), ('Onet-le-Château', '11665'), ('Bozouls', '2923'), ('Capdenac-Gare', '4419'), ('Saint-Affrique', '8023'), ("Sévérac d'Aveyron", '4069'), ('Druelle Balsac', '3205')], 'Bouches-du-Rhône': [('Saint-Savournin', '3443'), ("La Roque-d'Anthéron", '5426'), ('Roquefort-la-Bédoule', '6015'), ('Fuveau', '10161'), ('Meyreuil', '5821'), ('Port-de-Bouc', '16651'), ('Grans', '5196'), ('Rognac', '12086'), ('Meyrargues', '3855'), ('Gréasque', '4271'), ('Rognonas', '4058'), ('Le Puy-Sainte-Réparade', '5882'), ('Velaux', '8582'), ('Coudoux', '3702'), ('Septèmes-les-Vallons', '11196'), ("Plan-d'Orgon", '3515'), ('Saint-Martin-de-Crau', '13413'), ('Mouriès', '3410'), ('Ventabren', '5407'), ('Marseille', '870321'), ('Aix-en-Provence', '145133'), ('Arles', '50454'), ('Bouc-Bel-Air', '14838'), ('Aubagne', '47535'), ('Istres', '43626'), ('Vitrolles', '33333'), ('Saint-Rémy-de-Provence', '9769'), ('La Ciotat', '35993'), ('Salon-de-Provence', '45386'), ('Tarascon', '15811'), ('Miramas', '26428'), ('Martigues', '48574'), ('Carry-le-Rouet', '5690'), ('Cassis', '6905'), ('Les Pennes-Mirabeau', '21623'), ('Marignane', '32384'), ('Gémenos', '6635'), ('Port-Saint-Louis-du-Rhône', '8424'), ('Orgon', '2821'), ('Graveson', '4856'), ('Allauch', '21372'), ('Éguilles', '8030'), ("Berre-l'Étang", '13660'), ('Gardanne', '21521'), ('Barbentane', '4192'), ('Auriol', '12922'), ('Alleins', '2626'), ('Cabannes', '4439'), ('Châteauneuf-les-Martigues', '17606'), ('La Bouilladisse', '6270'), ('Cabriès', '9918'), ('Charleval', '2669'), ('Eyguières', '6844'), ('La Penne-sur-Huveaune', '6519'), ('Lançon-Provence', '8941'), ('Fos-sur-Mer', '15687'), ('Gignac-la-Nerthe', '9887'), ('Eyragues', '4496'), ('Saint-Andiol', '3346'), ('Pélissanne', '10487'), ('Châteaurenard', '16063'), ('Ceyreste', '4729'), ('Lambesc', '9909'), ('Cuges-les-Pins', '5244'), ('Noves', '5847'), ('La Destrousse', '3768'), ('Sausset-les-Pins', '7581'), ('Ensuès-la-Redonne', '5783'), ('Maillane', '2667'), ('Fontvieille', '3544'), ('Plan-de-Cuques', '11649'), ('Peynier', '3540'), ('Carnoux-en-Provence', '6498'), ('Sénas', '6894'), ('Saint-Victoret', '6631'), ('Trets', '10303'), ('Mollégès', '2656'), ('Le Rove', '5145'), ('Peyrolles-en-Provence', '5134'), ('Peypin', '5598'), ('Mallemort', '6134'), ('Rousset', '4918'), ('Saint-Chamas', '8591'), ('Saint-Mitre-les-Remparts', '5820'), ('La Fare-les-Oliviers', '8744'), ('Rognes', '4637'), ('Mimet', '4213'), ('Jouques', '4406'), ('Roquevaire', '8713'), ('Simiane-Collongue', '5808'), ('Saint-Cannat', '5638'), ('Venelles', '8400')], 'Calvados': [('Bayeux', '12682'), ('Lisieux', '20038'), ('Falaise', '7968'), ('Cabourg', '3558'), ('Deauville', '3565'), ('Vire', '10762'), ("Pont-l'Évêque", '4627'), ('Honfleur', '6733'), ('Aunay-sur-Odon', '3209'), ('Hérouville-Saint-Clair', '22555'), ('Saint-Pierre-sur-Dives', '3373'), ('Condé-sur-Noireau', '4551'), ('Trouville-sur-Mer', '4603'), ('Ouistreham', '9344'), ('Troarn', '3459'), ('Courseulles-sur-Mer', '4196'), ('Biéville-Beuville', '3561'), ('Blainville-sur-Orne', '5855'), ('Villers-sur-Mer', '2543'), ('Bretteville-sur-Odon', '4005'), ('Cormelles-le-Royal', '4957'), ('Mézidon-Canon', '4787'), ('Lion-sur-Mer', '2536'), ('Mondeville', '9973'), ('La Rivière-Saint-Sauveur', '2547'), ('Louvigny', '2683'), ('Touques', '3731'), ('Fontaine-Étoupefour', '2617'), ('Fleury-sur-Orne', '4938'), ('Colleville-Montgomery', '2555'), ('Argences', '3795'), ('Douvres-la-Délivrande', '4973'), ('Carpiquet', '2886'), ('Villers-Bocage', '3160'), ('Giberville', '4916'), ('Dives-sur-Mer', '5276'), ('Le Molay-Littry', '3016'), ('Isigny-sur-Mer', '3610'), ('Saint-Contest', '2525'), ('Colombelles', '6976'), ('Saint-Martin-de-Fontenay', '2523'), ('Moult', '2654'), ('Démouville', '3117'), ("Bretteville-l'Orgueilleuse", '3149'), ('Verson', '3636'), ('Hermanville-sur-Mer', '3110'), ('Luc-sur-Mer', '3213'), ('Ifs', '11696'), ('Valdallière', '5755'), ('Souleuvre en Bocage', '8950'), ('Condé-en-Normandie', '6452'), ('Valorbiquet', '2507'), ('Thury-Harcourt-le-Hom', '3629'), ("Livarot-Pays-d'Auge", '6220'), ('Vire Normandie', '16590'), ('Saline', '5428'), ('Thue et Mue', '6120'), ('Moult-Chicheboville', '3273'), ('Saint-Pierre-en-Auge', '7329'), ("Mézidon Vallée d'Auge", '9586'), ("Les Monts d'Aunay", '4595'), ('Noues de Sienne', '4357'), ('Isigny-sur-Mer', '2508'), ("Pont-l'Évêque", '3188'), ("Pont-l'Évêque", '3466'), ("Pont-l'Évêque", '3709'), ("Pont-l'Évêque", '3767'), ("Pont-l'Évêque", '3843'), ("Pont-l'Évêque", '4133'), ("Pont-l'Évêque", '4158'), ("Pont-l'Évêque", '4432'), ("Pont-l'Évêque", '4488'), ("Pont-l'Évêque", '4498'), ("Pont-l'Évêque", '4534'), ("Pont-l'Évêque", '4579'), ('Troarn', '3574'), ('Troarn', '3645'), ('Troarn', '5609')], 'Cantal': [('Arpajon-sur-Cère', '6279'), ('Mauriac', '3498'), ('Ytrac', '4335'), ('Aurillac', '25593'), ('Saint-Flour', '6458')], 'Charente': [('Brie', '4233'), ('Fléac', '3818'), ("L'Isle-d'Espagnac", '5626'), ('Saint-Yrieix-sur-Charente', '7381'), ('Cognac', '18670'), ('Angoulême', '41603'), ('Confolens', '2604'), ('Jarnac', '4434'), ('Barbezieux-Saint-Hilaire', '4714'), ('Champniers', '5149'), ('La Couronne', '7764'), ('La Rochefoucauld', '2911'), ('Ruffec', '3372'), ('Chasseneuil-sur-Bonnieure', '3083'), ('Châteaubernard', '3746'), ('Ruelle-sur-Touvre', '7305'), ('Magnac-sur-Touvre', '3223'), ('Châteauneuf-sur-Charente', '3537'), ('Soyaux', '9761'), ('Roullet-Saint-Estèphe', '4277'), ('Saint-Michel', '3218'), ('Gond-Pontouvre', '6119'), ('Montmoreau', '2564'), ('Confolens', '2712'), ('Rouillac', '2964'), ('La Rochefoucauld-en-Angoumois', '3977'), ('Terres-de-Haute-Charente', '3900'), ('Barbezieux', '4851')], 'Charente-Maritime': [('Marennes', '5637'), ('Saujon', '7180'), ('Saint-Georges-des-Coteaux', '2741'), ('Marans', '4505'), ('La Jarrie', '3336'), ("Saint-Pierre-d'Oléron", '6634'), ('Pons', '4203'), ('Fouras', '3934'), ('Montendre', '3226'), ('Soubise', '3047'), ('La Flotte', '2785'), ('Surgères', '6786'), ('Saint-Georges-de-Didonne', '5342'), ("Dolus-d'Oléron", '3144'), ('Sainte-Marie-de-Ré', '3363'), ('Meschers-sur-Gironde', '3123'), ('Vaux-sur-Mer', '3926'), ('Nieul-sur-Mer', '5859'), ('Bourcefranc-le-Chapus', '3506'), ("L'Houmeau", '2865'), ('Saint-Xandre', '5091'), ('Marsilly', '3130'), ('Saint-Jean-de-Liversay', '2975'), ('Dompierre-sur-Mer', '5638'), ('Sainte-Soulle', '4846'), ('Angoulins', '4050'), ('Châtelaillon-Plage', '5900'), ('Aytré', '9247'), ('Arvert', '3566'), ('Royan', '18419'), ('La Rochelle', '77205'), ('Périgny', '8684'), ('Saintes', '25287'), ('Rochefort', '23584'), ('Jonzac', '3519'), ('Échillais', '3547'), ("Saint-Jean-d'Angély", '6796'), ('Tonnay-Charente', '8097'), ('Médis', '2992'), ("Saint-Georges-d'Oléron", '3742'), ("Aigrefeuille-d'Aunis", '4143'), ("Le Château-d'Oléron", '4270'), ('Saint-Agnant', '2700'), ('La Jarne', '2541'), ('La Tremblade', '4322'), ('Lagord', '7098'), ('Gémozac', '2906'), ('Étaules', '2618'), ('Chaniers', '3587'), ('Breuillet', '3010'), ('Puilboreau', '6556'), ('Saint-Sulpice-de-Royan', '3309'), ('Saint-Palais-sur-Mer', '3907'), ('Marennes-Hiers-Brouage', '6234')], 'Cher': [('Saint-Germain-du-Puy', '5041'), ('La Chapelle-Saint-Ursin', '3663'), ('Saint-Doulchard', '9607'), ("La Guerche-sur-l'Aubois", '3177'), ('Saint-Florent-sur-Cher', '6442'), ('Méreau', '2606'), ('Trouy', '3953'), ('Bourges', '64541'), ('Vierzon', '25464'), ('Saint-Amand-Montrond', '9488'), ('Aubigny-sur-Nère', '5502'), ('Mehun-sur-Yèvre', '6555'), ('Avord', '2740'), ('Sancoins', '2984'), ('Dun-sur-Auron', '3655')], 'Corrèze': [('Malemort', '7984'), ('Argentat-sur-Dordogne', '2896'), ('Brive-la-Gaillarde', '46330'), ('Ussel', '9358'), ('Tulle', '14812'), ('Donzenac', '2678'), ('Ussac', '4215'), ('Malemort-sur-Corrèze', '7476'), ('Uzerche', '2794'), ('Bort-les-Orgues', '2648'), ('Égletons', '4295'), ('Argentat', '2817'), ('Cosnac', '2974'), ('Objat', '3624'), ('Saint-Pantaléon-de-Larche', '4751'), ('Allassac', '3968')], 'Corse-du-Sud': [('Grosseto-Prugna', '3402'), ('Alata', '3456'), ('Bastelicaccia', '4124'), ('Ajaccio', '71361'), ('Bonifacio', '3192'), ('Sartène', '3248'), ('Porto-Vecchio', '11132'), ('Afa', '3229'), ('Propriano', '3699'), ('Sarrola-Carcopino', '3289'), ('Zonza', '2759')], 'Haute-Corse': [('Bastia', '48503'), ('Calvi', '5774'), ('Corte', '7485'), ('Penta-di-Casinca', '3370'), ('Ville-di-Pietrabugno', '3328'), ("L'Île-Rousse", '3163'), ('Biguglia', '7810'), ('Borgo', '8832'), ('Furiani', '5608'), ('Ghisonaccia', '4219'), ('Ventiseri', '2512'), ('Vescovato', '2851'), ('Lucciana', '6007'), ('San-Martino-di-Lota', '2913'), ('Prunelli-di-Fiumorbo', '3667')], "Côte-d'Or": [('Auxonne', '7614'), ('Chenôve', '14025'), ('Longvic', '8545'), ('Marsannay-la-Côte', '5311'), ('Saint-Apollinaire', '7546'), ('Quetigny', '9099'), ('Chevigny-Saint-Sauveur', '11123'), ('Arc-sur-Tille', '2629'), ('Talant', '11713'), ('Plombières-lès-Dijon', '2542'), ('Fontaine-lès-Dijon', '8800'), ('Genlis', '5231'), ('Nuits-Saint-Georges', '5362'), ('Montbard', '4848'), ('Dijon', '158002'), ('Beaune', '20551'), ('Gevrey-Chambertin', '3048'), ('Is-sur-Tille', '4405'), ('Venarey-les-Laumes', '2846'), ('Semur-en-Auxois', '4120'), ('Châtillon-sur-Seine', '5347'), ('Neuilly-Crimolois', '3036')], "Côtes-d'Armor": [('Pordic', '6359'), ('Quévert', '3976'), ('Plouër-sur-Rance', '3551'), ('Ploubazlanec', '3039'), ('Plestin-les-Grèves', '3620'), ('Plancoët', '2981'), ('Ploubalay', '3249'), ('Yffiniac', '4990'), ('Plémet', '3195'), ('Erquy', '3922'), ('Saint-Brieuc', '43605'), ('Guingamp', '7069'), ('Dinan', '14407'), ('Lannion', '20210'), ('Léhon', '3110'), ('Quintin', '2867'), ('Plaintel', '4441'), ('Trégastel', '2535'), ('Pléneuf-Val-André', '4083'), ('Binic', '3826'), ('Broons', '2902'), ('Plérin', '14309'), ('Plœuc-sur-Lié', '3328'), ('Bégard', '4809'), ('Merdrignac', '2937'), ('Plouha', '4541'), ('Plouézec', '3140'), ('Hillion', '4183'), ('Langueux', '7801'), ('Ploufragan', '11383'), ('Taden', '2524'), ('Trélivan', '2862'), ('Ploumagoar', '5409'), ('Grâces', '2545'), ('Plélo', '3242'), ('Saint-Quay-Portrieux', '3121'), ('Perros-Guirec', '7089'), ('Lamballe', '12579'), ('Rostrenen', '3056'), ('Étables-sur-Mer', '3180'), ('Paimpol', '7179'), ('Loudéac', '9605'), ('Pleslin-Trigavou', '3775'), ('Pleudihen-sur-Rance', '2968'), ('Saint-Cast-le-Guildo', '3308'), ('Pleumeur-Bodou', '3857'), ('Lanvallay', '4195'), ('Plédran', '6839'), ('Louannec', '3077'), ('Ploubezre', '3687'), ('Trégueux', '8459'), ('Quessoy', '3856'), ('Pabu', '2748'), ('Plouagat', '2886'), ('Trébeurden', '3641'), ('Le Mené', '6396'), ("Plœuc-L'Hermitage", '4103'), ('Plémet', '3697'), ('Jugon-les-Lacs - Commune nouvelle', '2520'), ('Binic-Étables-sur-Mer', '6934'), ('Beaussais-sur-Mer', '3757'), ('Pordic', '7300'), ('La Roche-Jaudy', '2713'), ('Châtelaudren-Plouagat', '3916'), ('Lamballe-Armor', '16688')], 'Creuse': [('Guéret', '12734'), ('La Souterraine', '4982'), ('Aubusson', '3248')], 'Dordogne': [('La Roche-Chalais', '3022'), ('Prigonrieux', '4143'), ('Nontron', '3055'), ('Coulounieix-Chamiers', '7387'), ('Eymet', '2564'), ('Saint-Astier', '5352'), ('Chancelade', '4378'), ('Boulazac', '7170'), ('Neuvic', '3611'), ('Champcevinel', '2965'), ('Sarlat-la-Canéda', '8816'), ('Périgueux', '29896'), ('Montpon-Ménestérol', '5704'), ('Bergerac', '26693'), ('Notre-Dame-de-Sanilhac', '3185'), ("Marsac-sur-l'Isle", '3142'), ('Ribérac', '3844'), ('Le Bugue', '2588'), ('Mussidan', '2762'), ('Montignac-Lascaux', '2753'), ('Terrasson-Lavilledieu', '6266'), ('Thiviers', '2871'), ('Lamonzie-Saint-Martin', '2609'), ('Lalinde', '2770'), ('La Force', '2653'), ('Trélissac', '7006'), ('Bassillac et Auberoche', '4410'), ('Sanilhac', '4604'), ('Boulazac Isle Manoire', '10737'), ('Brantôme en Périgord', '3670')], 'Doubs': [('Charquemont', '2741'), ('Bethoncourt', '5441'), ('Hérimoncourt', '3662'), ('Maîche', '4266'), ('Saint-Vit', '4922'), ('Valdahon', '5748'), ('Villers-le-Lac', '5188'), ('Pont-de-Roide-Vermondans', '4124'), ('Seloncourt', '5726'), ('Grand-Charmont', '5786'), ('Vieux-Charmont', '2793'), ('Voujeaucourt', '3148'), ('Besançon', '118258'), ('Valentigney', '11272'), ('Montbéliard', '25806'), ('Morteau', '6853'), ('Pontarlier', '17542'), ('École-Valentin', '2633'), ('Sochaux', '3829'), ('Étupes', '3724'), ('Doubs', '3198'), ('Bavans', '3616'), ('Ornans', '4308'), ('Thise', '3020'), ('Miserey-Salines', '2555'), ('Audincourt', '13341'), ('Baume-les-Dames', '4989'), ('Exincourt', '3237'), ("L'Isle-sur-le-Doubs", '2865'), ('Saône', '3231'), ('Les Fins', '3202'), ('Mandeure', '4795'), ('Les Auxons', '2515'), ('Ornans', '4423')], 'Drôme': [('Châteauneuf-sur-Isère', '3945'), ('Livron-sur-Drôme', '9202'), ('Peyrins', '2629'), ('Saint-Jean-en-Royans', '2809'), ('Châteauneuf-du-Rhône', '2724'), ('Montmeyran', '2942'), ('Saint-Marcel-lès-Valence', '6289'), ('Malissard', '3207'), ('Donzère', '5921'), ('Chabeuil', '6814'), ('Étoile-sur-Rhône', '5520'), ('Anneyron', '4063'), ('Aouste-sur-Sye', '2580'), ('Beaumont-lès-Valence', '3984'), ('Loriol-sur-Drôme', '6616'), ('Portes-lès-Valence', '10680'), ('Montélier', '4271'), ("Saint-Donat-sur-l'Herbasse", '4248'), ("Pont-de-l'Isère", '3573'), ('Mours-Saint-Eusèbe', '3229'), ('La Roche-de-Glun', '3395'), ('Valence', '64749'), ('Die', '4718'), ('Nyons', '6772'), ("Saint-Rambert-d'Albon", '6758'), ('Saint-Vallier', '3960'), ('Montélimar', '39818'), ('Romans-sur-Isère', '33098'), ('Bourg-de-Péage', '9957'), ('Saint-Paul-Trois-Châteaux', '8736'), ('Bourg-lès-Valence', '19792'), ("Tain-l'Hermitage", '6106'), ('Pierrelatte', '13510'), ('Dieulefit', '3239'), ('Alixan', '2546'), ('Crest', '8680'), ('Chatuzange-le-Goubet', '5934'), ('Mercurol-Veaunes', '2732')], 'Eure': [('Bernay', '9848'), ('Évreux', '46349'), ("Pont-de-l'Arche", '4146'), ('Les Andelys', '8070'), ('Vernon', '23727'), ('Gisors', '11696'), ('Verneuil-sur-Avre', '6602'), ('Pacy-sur-Eure', '4564'), ('Conches-en-Ouche', '4995'), ('Beaumont-le-Roger', '2794'), ('Pont-Audemer', '8442'), ('Le Vaudreuil', '3681'), ('Bourgtheroulde-Infreville', '3171'), ('Ivry-la-Bataille', '2665'), ('Breteuil', '3108'), ('Louviers', '18518'), ('Brionne', '4199'), ('Romilly-sur-Andelle', '3288'), ('Beuzeville', '4666'), ('Bourg-Achard', '3942'), ('Saint-Sébastien-de-Morsent', '5594'), ('Guichainville', '2954'), ('Gaillon', '6860'), ('Saint-Marcel', '4425'), ('Gravigny', '3994'), ('Étrépagny', '3730'), ('Le Neubourg', '4241'), ('Val-de-Reuil', '12910'), ('Le Bosc-Roger-en-Roumois', '3287'), ('Aubevoye', '4936'), ('Pîtres', '2648'), ("Saint-André-de-l'Eure", '4031'), ('Gasny', '3058'), ('Ézy-sur-Eure', '3702'), ('Mesnil-en-Ouche', '4554'), ("Le Thuit de l'Oison", '3700'), ('Mesnils-sur-Iton', '6126'), ('Vexin-sur-Epte', '5952'), ('Grand Bourgtheroulde', '3957'), ("Le Val d'Hazey", '5350'), ('Bosroumois', '3649'), ('La Chapelle-Longueville', '3338'), ("Verneuil d'Avre et d'Iton", '7614'), ('Breteuil', '4331'), ('Pacy-sur-Eure', '5056'), ('Pont-Audemer', '9982'), ('Mesnils-sur-Iton', '4511')], 'Eure-et-Loir': [('Chartres', '38534'), ('Dreux', '30646'), ('Châteaudun', '13096'), ('Nogent-le-Rotrou', '9491'), ('Épernon', '5634'), ('Illiers-Combray', '3266'), ('Bonneval', '4846'), ('Voves', '3265'), ('Cloyes-sur-le-Loir', '2690'), ('Châteauneuf-en-Thymerais', '2596'), ('Nogent-le-Roi', '3981'), ('La Loupe', '3329'), ('Anet', '2697'), ('Saint-Georges-sur-Eure', '2817'), ('Auneau', '4767'), ('Vernouillet', '12472'), ('Maintenon', '4457'), ('Toury', '2595'), ('Champhol', '3715'), ('Mainvilliers', '11127'), ('Brou', '3311'), ('Le Coudray', '4078'), ('Saint-Rémy-sur-Avre', '4041'), ('Saint-Lubin-des-Joncherets', '4019'), ('Luisant', '6689'), ('Hanches', '2682'), ('Lèves', '5686'), ('Gallardon', '3693'), ('Senonches', '3032'), ('Pierres', '2756'), ('Lucé', '15403'), ('Courville-sur-Eure', '2831'), ('Les Villages Vovéens', '3914'), ('Auneau-Bleury-Saint-Symphorien', '6090'), ('Cloyes-les-Trois-Rivières', '5618'), ("Vald'Yerre", '3653'), ('Janville-en-Beauce', '2571')], 'Finistère': [('Riec-sur-Bélon', '4180'), ('Ergué-Gabéric', '8434'), ('Pluguffan', '4179'), ('Kernével', '3193'), ('Quimper', '63283'), ('Ploudalmézeau', '6312'), ('Brest', '139926'), ('Concarneau', '19816'), ('Plougasnou', '2737'), ('Penmarch', '5149'), ('Morlaix', '14845'), ('Châteaulin', '5164'), ('Saint-Yvi', '3249'), ('Carhaix-Plouguer', '7155'), ('Plouarzel', '3854'), ('Plouvorn', '2862'), ('Moëlan-sur-Mer', '6742'), ('Plouvien', '3832'), ('Pleuven', '3188'), ('Plouigneau', '4905'), ('Crozon', '7386'), ('Plabennec', '8515'), ('Plourin-lès-Morlaix', '4517'), ('Cléder', '3683'), ('Le Folgoët', '3215'), ('Plougonvelin', '4183'), ('Plomelin', '4191'), ("Pont-l'Abbé", '8369'), ('Elliant', '3320'), ('Bourg-Blanc', '3553'), ('Saint-Thégonnec Loc-Eguiner', '3044'), ('Milizac-Guipronvel', '4550'), ('Audierne', '3692'), ('Plouigneau', '5086'), ('Quimperlé', '12220'), ('Pont-Aven', '2821'), ('Saint-Pol-de-Léon', '6709'), ('Douarnenez', '14000'), ('Fouesnant', '9864'), ('Briec', '5675'), ('Roscoff', '3489'), ('Lesneven', '7322'), ('Saint-Renan', '8163'), ('Scaër', '5295'), ('Bannalec', '5668'), ('Guipavas', '15050'), ('Landerneau', '15918'), ('Le Conquet', '2750'), ('Lannilis', '5672'), ('Taulé', '2898'), ('Rosporden', '7614'), ('Gouesnou', '6215'), ('Guilers', '8061'), ('Landivisiau', '9151'), ('Bénodet', '3616'), ('Saint-Thégonnec', '2715'), ('Combrit', '4187'), ('Guilvinec', '2681'), ('Châteauneuf-du-Faou', '3650'), ('Trégunc', '7064'), ('Gouesnach', '2786'), ('La Forêt-Fouesnant', '3438'), ('Plougastel-Daoulas', '13161'), ('Loctudy', '4013'), ('Locmaria-Plouzané', '5092'), ('Plouescat', '3497'), ('Clohars-Carnoët', '4528'), ('Carantec', '3189'), ('Bohars', '3530'), ('Plogonnec', '3178'), ('Plouédern', '2944'), ('Plozévet', '2948'), ('Plouzané', '13496'), ('Guiclan', '2502'), ('Saint-Martin-des-Champs', '4621'), ('Le Relecq-Kerhuon', '11717'), ('Rédené', '2917'), ('Saint-Évarzec', '3503'), ('Plomeur', '3828'), ('Milizac', '3719'), ('Pleyber-Christ', '3184'), ('Landéda', '3576'), ('Plouguerneau', '6640'), ('Névez', '2686'), ('Pleyben', '3640'), ('Plonéour-Lanvern', '6223'), ('Loperhet', '3910'), ('Melgven', '3408'), ('Plouhinec', '3951'), ('Pont-de-Buis-lès-Quimerch', '3639'), ('Plouénan', '2535'), ('Mellac', '3242'), ('Ploudaniel', '3737'), ('Plobannalec-Lesconil', '3568'), ('Plougonven', '3454')], 'Gard': [('Bouillargues', '6272'), ('Saint-Laurent-des-Arbres', '3000'), ('Générac', '4071'), ('Bagard', '2556'), ('Sommières', '5050'), ('Saint-Christol-lez-Alès', '7055'), ('Aubais', '2895'), ('Saint-Julien-les-Rosiers', '3401'), ('Saint-Hippolyte-du-Fort', '3804'), ('Caveirac', '4308'), ("Saint-Laurent-d'Aigouze", '3536'), ('Saint-Hilaire-de-Brethmas', '4503'), ('Milhaud', '5721'), ('Saint-Martin-de-Valgalgues', '4497'), ('Vauvert', '11492'), ('Boisset-et-Gaujac', '2552'), ('Rousson', '4251'), ('Villeneuve-lès-Avignon', '12216'), ('Vergèze', '5308'), ('Caissargues', '4150'), ('Bellegarde', '7353'), ('Aramon', '4219'), ('Bessèges', '2835'), ('Poulx', '4131'), ('Rochefort-du-Gard', '7629'), ('Salindres', '3571'), ('Bagnols-sur-Cèze', '18091'), ('Beauvoisin', '5104'), ('Roquemaure', '5545'), ('Le Grau-du-Roi', '8419'), ('Saint-Ambroix', '3351'), ('Redessan', '4097'), ('Jonquières-Saint-Vincent', '3850'), ('Les Salles-du-Gardon', '2600'), ('Saint-Quentin-la-Poterie', '3064'), ('Calvisson', '5898'), ('Nîmes', '148561'), ('Aigues-Vives', '3338'), ('Gallargues-le-Montueux', '3632'), ('Garons', '4940'), ('Alès', '41837'), ('Aigues-Mortes', '8560'), ('Le Vigan', '3772'), ('Beaucaire', '15906'), ('Uzès', '8387'), ('Quissac', '3299'), ('La Grand-Combe', '4998'), ('Anduze', '3327'), ("Laudun-l'Ardoise", '6331'), ('Saint-Gilles', '13931'), ('Pont-Saint-Esprit', '10369'), ('Aimargues', '5717'), ('Uchaud', '4424'), ('Manduel', '6943'), ('Saint-Privat-des-Vieux', '5275'), ('Marguerittes', '8656'), ('Meynes', '2503'), ('Fourques', '2796'), ('Saint-Geniès-de-Malgoirès', '3052'), ('Bernis', '3421'), ('Rodilhan', '2851'), ('Les Angles', '8476'), ('Clarensac', '4209'), ('Montfrin', '3163'), ('Pujaut', '3921')], 'Haute-Garonne': [('Rieux-Volvestre', '2611'), ('Villefranche-de-Lauragais', '4806'), ('Grenade', '8844'), ('Montastruc-la-Conseillère', '3576'), ('Pibrac', '8578'), ('Baziège', '3395'), ('Fronton', '6271'), ('Beauzelle', '6973'), ('Eaunes', '6349'), ('Merville', '6054'), ('Verfeil', '3658'), ('Frouzins', '9292'), ('Villeneuve-Tolosane', '10050'), ('Montberon', '3088'), ('Labastidette', '2938'), ('Fonsorbes', '12170'), ('Seysses', '9481'), ('Noé', '2912'), ('Miremont', '2623'), ('Roques', '4757'), ('Toulouse', '498003'), ('Fenouillet', '5307'), ('Muret', '24797'), ('Saint-Gaudens', '11773'), ('Auzeville-Tolosane', '4088'), ('Blagnac', '25525'), ('Colomiers', '39968'), ('Tournefeuille', '28117'), ('Auterive', '9923'), ('Revel', '9637'), ('Montréjeau', '2751'), ('Fontenilles', '5893'), ('Launaguet', '9050'), ('Nailloux', '3928'), ('Plaisance-du-Touch', '19402'), ('Castelmaurou', '4359'), ('Cugnaux', '19344'), ('Aussonne', '7209'), ('Ayguesvives', '2679'), ('Balma', '16625'), ('Rieumes', '3578'), ('Cazères', '4799'), ('Caraman', '2544'), ('Carbonne', '5791'), ('Léguevin', '9359'), ('Montesquieu-Volvestre', '3111'), ('Cintegabelle', '2914'), ('Castanet-Tolosan', '14213'), ('Montgiscard', '2530'), ('Saint-Lys', '9521'), ('Roquettes', '4212'), ('Pins-Justaret', '4318'), ('Bruguières', '5947'), ('Cornebarrieu', '7372'), ('Buzet-sur-Tarn', '2870'), ('Aucamville', '8968'), ('Quint-Fonsegrives', '5896'), ("Castelnau-d'Estrétefonds", '6579'), ('Lavernose-Lacasse', '3255'), ('Bessières', '4127'), ('Portet-sur-Garonne', '9812'), ('Villemur-sur-Tarn', '6034'), ("L'Union", '11798'), ('La Salvetat-Saint-Gilles', '8474'), ('Saint-Jean', '11017'), ('Saint-Geniès-Bellevue', '2522'), ('Montrabé', '4111'), ('Pinsaguel', '2840'), ('Mondonville', '5166'), ('Escalquens', '6967'), ('Lherm', '3777'), ('Lagardelle-sur-Lèze', '3169'), ('Saint-Orens-de-Gameville', '12696'), ('Saint-Jory', '6383'), ('Vernet', '3059'), ('Seilh', '3284'), ('Saint-Alban', '6217'), ('Lapeyrouse-Fossat', '2886'), ('Longages', '3160'), ('Gratentour', '4387'), ('Pechbonnieu', '4544'), ('Labarthe-sur-Lèze', '6271'), ('Drémil-Lafage', '2649'), ('Ramonville-Saint-Agne', '14600'), ('Venerque', '2702'), ('Bouloc', '4723'), ('Bérat', '3034'), ('Labège', '4008'), ('Lespinasse', '2828'), ('Castelginest', '10682'), ('Fonbeauzard', '3011'), ('Brax', '2862'), ('Gagnac-sur-Garonne', '3130')], 'Gers': [('Vic-Fezensac', '3546'), ('Eauze', '3975'), ("L'Isle-Jourdain", '9072'), ('Auch', '22173'), ('Condom', '6493'), ('Gimont', '3016'), ('Pavie', '2501'), ('Lectoure', '3672'), ('Fleurance', '5924'), ('Mirande', '3450')], 'Gironde': [('Saint-Ciers-sur-Gironde', '3021'), ('Saint-Savin', '3250'), ('Floirac', '18181'), ('Lège-Cap-Ferret', '8352'), ('Saint-Loubès', '9909'), ('Saint-Laurent-Médoc', '4860'), ('Blaye', '4838'), ('Mérignac', '72197'), ('Langon', '7357'), ('Pessac', '65245'), ('Bazas', '4801'), ('Andernos-les-Bains', '12242'), ('Lesparre-Médoc', '5808'), ('Audenge', '8680'), ('Blanquefort', '15419'), ('Cubzac-les-Ponts', '2501'), ('La Teste-de-Buch', '26168'), ('Gradignan', '25694'), ('Lormont', '23181'), ("Villenave-d'Ornon", '36754'), ('Saint-Médard-en-Jalles', '31808'), ('Ambès', '3053'), ('Ambarès-et-Lagrave', '16636'), ('Bassens', '7472'), ('Belin-Béliet', '5732'), ('Coutras', '8602'), ('La Brède', '4420'), ('Castelnau-de-Médoc', '4804'), ('Bouliac', '3706'), ('La Réole', '4355'), ('Castillon-la-Bataille', '3194'), ('Arès', '6381'), ('Pauillac', '4997'), ('Saint-Quentin-de-Baron', '2559'), ('Sainte-Hélène', '2838'), ('Le Porge', '3294'), ('Saint-Denis-de-Pile', '5688'), ('Sainte-Eulalie', '4832'), ('Pineuilh', '4438'), ('Cézac', '2654'), ('Le Haillan', '11539'), ('Izon', '6167'), ('Bordeaux', '259809'), ('Libourne', '24257'), ('Camblanes-et-Meynac', '2984'), ('Le Teich', '8500'), ('Ludon-Médoc', '5131'), ('Martignas-sur-Jalle', '7624'), ('Avensan', '3018'), ('Sallebœuf', '2630'), ('Le Barp', '5605'), ('Arcachon', '11630'), ('Bègles', '30642'), ('Le Bouscat', '23890'), ('Talence', '43820'), ('Créon', '4832'), ('Marcheprime', '4992'), ('Podensac', '3160'), ('Biganos', '10990'), ('Cadaujac', '6464'), ('Sainte-Foy-la-Grande', '2627'), ('Cestas', '17053'), ('Bruges', '19403'), ('Martillac', '3279'), ('Cadillac-sur-Garonne', '2836'), ('Lanton', '7285'), ('Carignan-de-Bordeaux', '4141'), ('Listrac-Médoc', '2788'), ('Tresses', '4895'), ('Portets', '2757'), ('Yvrac', '2838'), ('Hourtin', '3813'), ('Carbon-Blanc', '8254'), ('Canéjan', '6061'), ("Saint-Médard-d'Eyrans", '3098'), ('Fargues-Saint-Hilaire', '3203'), ('Pompignac', '3170'), ('Saucats', '3255'), ('Eysines', '24488'), ('Toulenne', '2752'), ('Salles', '7411'), ('Léognan', '10571'), ('Soulac-sur-Mer', '2825'), ('Galgon', '3022'), ('Latresne', '3523'), ('Saint-Caprais-de-Bordeaux', '3399'), ('Saint-Sulpice-et-Cameyrac', '4762'), ('Mios', '10781'), ('Saint-Selve', '3423'), ('Lacanau', '5070'), ('Sadirac', '4499'), ('Gujan-Mestras', '21887'), ('Artigues-près-Bordeaux', '8656'), ('Saint-André-de-Cubzac', '12372'), ('Parempuyre', '9136'), ('Arsac', '3747'), ('Le Taillan-Médoc', '10141'), ('Macau', '4410'), ("Saint-Seurin-sur-l'Isle", '3141'), ('Laruscade', '2844'), ('La Lande-de-Fronsac', '2540'), ("Saint-Jean-d'Illac", '8980'), ('Saint-Aubin-de-Médoc', '7495'), ('Vayres', '4123'), ('Le Pian-Médoc', '6890'), ('Montussan', '3364'), ('Cenon', '25496'), ('Val de Virvée', '3621'), ('Margaux-Cantenac', '2876')], 'Hérault': [('Montpellier', '299096'), ('Sussargues', '2802'), ('Béziers', '78308'), ('Saint-Gély-du-Fesc', '10197'), ('Paulhan', '4016'), ('Saint-Brès', '3209'), ('Gigean', '6507'), ('Saint-Mathieu-de-Tréviers', '4883'), ('Saint-Aunès', '3786'), ('Castries', '6294'), ('Montferrier-sur-Lez', '3960'), ('Sète', '43858'), ('Agde', '29600'), ('Saint-Jean-de-Védas', '11086'), ('Lavérune', '3328'), ('Lattes', '17453'), ('Caux', '2538'), ('Cournonterral', '6423'), ('Saint-Just', '3278'), ('Prades-le-Lez', '5908'), ('Jacou', '6808'), ('Lodève', '7477'), ('Montarnaud', '4052'), ("Saint-Georges-d'Orques", '5552'), ('Lunel-Viel', '4367'), ('Frontignan', '23028'), ("Clermont-l'Hérault", '9029'), ('Murviel-lès-Béziers', '3105'), ('Maraussan', '4628'), ('Vendres', '2723'), ('Juvignac', '11344'), ('Bessan', '5162'), ('Nissan-lez-Enserune', '3996'), ('Puisserguier', '3037'), ('Capestang', '3281'), ('Bédarieux', '5743'), ('Lunel', '26385'), ('Aniane', '2918'), ('Mèze', '12307'), ('Thézan-lès-Béziers', '3029'), ('La Grande-Motte', '8696'), ('Castelnau-le-Lez', '22534'), ('Saint-Drézéry', '2735'), ('Fabrègues', '7293'), ('Saint-André-de-Sangonis', '6198'), ('Pézenas', '8002'), ('Montblanc', '2865'), ('Balaruc-le-Vieux', '2648'), ('Vic-la-Gardiole', '3373'), ('Villeneuve-lès-Maguelone', '10344'), ('Marseillan', '7734'), ('Saint-Thibéry', '2807'), ('Ganges', '4098'), ('Saint-Clément-de-Rivière', '5016'), ('Sérignan', '7504'), ('Valras-Plage', '4206'), ('Mauguio', '16705'), ('Portiragnes', '3104'), ('Vias', '5717'), ('Palavas-les-Flots', '5844'), ('Balaruc-les-Bains', '6991'), ('Teyran', '4595'), ('Clapiers', '5411'), ('Mireval', '3310'), ('Canet', '3506'), ('Cers', '2560'), ('Pignan', '7959'), ('Montagnac', '4324'), ('Gignac', '6480'), ('Cazouls-lès-Béziers', '5060'), ('Colombiers', '2684'), ('Boujan-sur-Libron', '3367'), ('Servian', '5233'), ('Baillargues', '7654'), ('Cournonsec', '3454'), ('Saint-Martin-de-Londres', '2766'), ('Pérols', '9016'), ('Le Crès', '9310'), ('Villeveyrac', '3859'), ('Lespignan', '3299'), ('Lansargues', '3099'), ('Vendargues', '6423'), ('Mudaison', '2753'), ('Lignan-sur-Orb', '3218'), ('Montbazin', '2934'), ('Magalas', '3371'), ('Poussan', '5993'), ('Montady', '3960'), ('Florensac', '5057'), ('Villeneuve-lès-Béziers', '4102'), ('Vailhauquès', '2584'), ('Grabels', '8798'), ('Sauvian', '5485'), ('Marsillargues', '6407')], 'Ille-et-Vilaine': [('Rennes', '220488'), ('Fougères', '20595'), ('Dinard', '10181'), ('Montauban-de-Bretagne', '6086'), ('Saint-Malo', '46803'), ('Saint-Aubin-du-Cormier', '4006'), ('Plerguer', '2796'), ('Piré-sur-Seiche', '2666'), ('Cancale', '5226'), ('Redon', '9287'), ('Bréal-sous-Montfort', '6404'), ('Breteil', '3655'), ('Bédée', '4417'), ('Étrelles', '2585'), ('Bains-sur-Oust', '3503'), ('Saint-Coulomb', '2820'), ('Crevin', '2862'), ('Bruz', '18905'), ('Chantepie', '10278'), ('Pont-Péan', '4413'), ('Saint-Domineuc', '2562'), ('Gévezé', '5521'), ('Saint-Grégoire', '9890'), ('Nouvoitou', '3281'), ("L'Hermitage", '4629'), ('Saint-Pierre-de-Plesguen', '2960'), ('Janzé', '8470'), ('Saint-Gilles', '5178'), ('Saint-Jouan-des-Guérets', '2683'), ('Orgères', '5152'), ('La Fresnais', '2571'), ('Miniac-Morvan', '4148'), ('La Chapelle-des-Fougeretz', '4682'), ('Louvigné-du-Désert', '3340'), ('Bourg-des-Comptes', '3295'), ('Chavagne', '4268'), ('Guichen', '8674'), ('Vezin-le-Coquet', '6114'), ('Pacé', '11860'), ('Corps-Nuds', '3434'), ('Bourgbarré', '4515'), ('Pléchâtel', '2780'), ('Saint-Méloir-des-Ondes', '4327'), ('Bain-de-Bretagne', '7314'), ('Cesson-Sévigné', '17082'), ('Montgermont', '3515'), ('Montfort-sur-Meu', '6726'), ('Vitré', '18487'), ('Retiers', '4438'), ('Dol-de-Bretagne', '5738'), ('Combourg', '5963'), ('Châteaugiron', '8028'), ('Saint-Méen-le-Grand', '4599'), ('Châteaubourg', '7336'), ('Tinténiac', '3739'), ('La Guerche-de-Bretagne', '4249'), ('Iffendic', '4553'), ('Saint-Jacques-de-la-Lande', '13948'), ('Pipriac', '3813'), ('Domloup', '3680'), ('Sens-de-Bretagne', '2566'), ('Noyal-sur-Vilaine', '6133'), ('Chartres-de-Bretagne', '8157'), ('Pleumeleuc', '3474'), ('Pleurtuit', '6982'), ('Romillé', '4032'), ('Betton', '12265'), ('Melesse', '6958'), ('Argentré-du-Plessis', '4502'), ('Vern-sur-Seiche', '8169'), ('Saint-Erblon', '3311'), ('Plélan-le-Grand', '4008'), ('Mordelles', '7338'), ("Saint-Aubin-d'Aubigné", '3916'), ('Liffré', '7949'), ('Maure-de-Bretagne', '3507'), ('Saint-Brice-en-Coglès', '3068'), ('Acigné', '6834'), ('Noyal-Châtillon-sur-Seiche', '7003'), ('Laillé', '5116'), ('La Mézière', '4971'), ('La Bouëxière', '4526'), ('Servon-sur-Vilaine', '3804'), ('Thorigné-Fouillard', '8575'), ("Val-d'Izé", '2580'), ('Martigné-Ferchaud', '2583'), ('Guipry', '3830'), ('Guignen', '3986'), ('Lécousse', '3260'), ('Goven', '4321'), ('Le Rheu', '8932'), ('Guipry-Messac', '7107'), ('Maen Roch', '4958'), ("Val d'Anast", '3954'), ('Châteaugiron', '10283'), ('Val-Couesnon', '4137'), ('Rives-du-Couesnon', '2886'), ("Mesnil-Roc'h", '4336'), ('Piré-Chancé', '3031'), ('Montauban-de-Bretagne', '5019'), ('Montauban-de-Bretagne', '5095')], 'Indre': [('Levroux', '2931'), ('Levroux', '2947'), ('Saint-Maur', '3680'), ('Châteauroux', '43122'), ('Issoudun', '11477'), ('Le Blanc', '6250'), ('La Châtre', '4054'), ('Argenton-sur-Creuse', '4880'), ('Buzançais', '4516'), ('Déols', '7609'), ('Villedieu-sur-Indre', '2687'), ('Levroux', '2727'), ('Ardentes', '3850'), ('Le Poinçonnet', '5820'), ('Chabris', '2763'), ('Levroux', '2923'), ('Saint-Maur', '3685')], 'Indre-et-Loire': [('Langeais', '4455'), ('Tours', '137087'), ('Chinon', '8104'), ('Amboise', '12533'), ('Loches', '6199'), ('Azay-le-Rideau', '3501'), ('Bléré', '5257'), ('Joué-lès-Tours', '38444'), ('Descartes', '3366'), ('Langeais', '4667'), ('Sainte-Maure-de-Touraine', '4150'), ('Château-Renault', '4917'), ('Bourgueil', '3894'), ('Vouvray', '3254'), ('Fondettes', '10301'), ('Ballan-Miré', '8102'), ('Montlouis-sur-Loire', '10868'), ('Chambray-lès-Tours', '11880'), ('Saint-Cyr-sur-Loire', '16419'), ('La Riche', '10317'), ('Cinq-Mars-la-Pile', '3729'), ('Rochecorbon', '3145'), ('Luynes', '5101'), ('Montbazon', '4542'), ("Notre-Dame-d'Oé", '4234'), ('Chanceaux-sur-Choisille', '3525'), ('Savonnières', '3234'), ('Nazelles-Négron', '3508'), ('Saint-Avertin', '14995'), ('Vernou-sur-Brenne', '2713'), ('Sorigny', '2752'), ('Azay-sur-Cher', '3099'), ('Esvres', '6109'), ('Athée-sur-Cher', '2716'), ('Artannes-sur-Indre', '2649'), ('Beaumont-en-Véron', '2689'), ('Saint-Pierre-des-Corps', '15967'), ('La Ville-aux-Dames', '5585'), ('La Membrolle-sur-Choisille', '3300'), ('Veigné', '6423'), ('Véretz', '4570'), ('Saint-Branchs', '2625'), ('Monts', '7876'), ('Saint-Martin-le-Beau', '3181'), ('Monnaie', '4643')], 'Isère': [("Villette-d'Anthon", '5113'), ('La Mure', '4920'), ('Meylan', '17786'), ('Roussillon', '8457'), ('Vinay', '4352'), ('Saint-Quentin-Fallavier', '6020'), ('Vizille', '7246'), ('Valencin', '2879'), ('Tignieu-Jameyzieu', '7555'), ('Saint-Georges-de-Commiers', '2537'), ('Saint-Martin-le-Vinoux', '5880'), ('Le Pont-de-Claix', '10716'), ('Sassenage', '11062'), ('Villard-de-Lans', '4234'), ('Voreppe', '9275'), ('Vaulx-Milieu', '2537'), ('Vif', '8547'), ('La Terrasse', '2510'), ('Claix', '7924'), ('Fontaine', '23211'), ('Heyrieux', '4712'), ('Chanas', '2726'), ('Beaurepaire', '4977'), ('La Côte-Saint-André', '4829'), ('Grenoble', '158198'), ('Vienne', '29993'), ('Bourgoin-Jallieu', '28834'), ('Échirolles', '36932'), ('La Tour-du-Pin', '8167'), ('Voiron', '20372'), ("Saint-Martin-d'Hères", '37935'), ("Le Bourg-d'Oisans", '3286'), ('Saint-Laurent-du-Pont', '4510'), ('Charvieu-Chavagneux', '10102'), ('Crémieu', '3375'), ('Allevard', '4023'), ('Tullins', '7816'), ('Pontcharra', '7348'), ('Pont-de-Chéruy', '5980'), ('Saint-Marcellin', '7766'), ('Crolles', '8293'), ('Le Touvet', '3176'), ('Seyssins', '7827'), ('Fontanil-Cornillon', '3172'), ('Les Avenières', '5643'), ('Gières', '7134'), ('Le Grand-Lemps', '3079'), ('Saint-Nazaire-les-Eymes', '2960'), ('Saint-Étienne-de-Saint-Geoirs', '3206'), ('Vaulnaveys-le-Haut', '3965'), ('Salaise-sur-Sanne', '4488'), ('Saint-Étienne-de-Crossey', '2595'), ('Saint-Jean-de-Moirans', '3564'), ('Saint-Siméon-de-Bressieux', '2921'), ('Saint-Clair-de-la-Tour', '3456'), ('Saint-André-le-Gaz', '2760'), ('Saint-Clair-du-Rhône', '3705'), ("Saint-Georges-d'Espéranche", '3439'), ("Saint-Maurice-l'Exil", '6387'), ('Saint-Just-Chaleyssin', '2656'), ('Pont-Évêque', '5313'), ('Le Péage-de-Roussillon', '6516'), ('Montbonnot-Saint-Martin', '5477'), ('Nivolas-Vermelle', '2648'), ('Le Cheylas', '2528'), ('Montalieu-Vercieu', '3467'), ('Coublevie', '5216'), ('Estrablin', '3535'), ('Diémoz', '2799'), ('Champ-sur-Drac', '3172'), ('Saint-Égrève', '15944'), ('Brié-et-Angonnes', '2545'), ('Le Pont-de-Beauvoisin', '3570'), ('Saint-Ismier', '7064'), ('Chapareillan', '2986'), ('Seyssinet-Pariset', '11891'), ('Dolomieu', '3192'), ('La Buisse', '3235'), ('Le Versoud', '4933'), ("L'Isle-d'Abeau", '16584'), ('Saint-Savin', '4202'), ('Saint-Romain-de-Jalionas', '3346'), ('Chasse-sur-Rhône', '6308'), ('Lans-en-Vercors', '2676'), ('Saint-Chef', '3689'), ('Froges', '3392'), ('Villard-Bonnot', '7249'), ('La Verpillière', '7293'), ('Saint-Jean-de-Bournay', '4590'), ('Rives', '6668'), ('Eybens', '9980'), ('Cessieu', '3113'), ('Chavanoz', '4869'), ('Bernin', '3025'), ('Aoste', '2882'), ('Apprieu', '3380'), ('Domène', '6677'), ('Villefontaine', '18835'), ('La Tronche', '6602'), ('Morestel', '4526'), ("Saint-Pierre-d'Allevard", '2887'), ("Saint-Martin-d'Uriage", '5512'), ('Corenc', '4075'), ('Ruy-Montceau', '4717'), ('Moirans', '7499'), ('Renage', '3467'), ('Jarrie', '3779'), ('Varces-Allières-et-Risset', '8301'), ('Crêts en Belledonne', '3372'), ('Autrans-Méaudre en Vercors', '3039'), ('Les Avenières Veyrins-Thuellin', '7804'), ('Les Abrets en Dauphiné', '6389'), ('Villages du Lac de Paladru', '2528')], 'Jura': [('Morez', '4813'), ('Damparis', '2629'), ('Lons-le-Saunier', '17189'), ('Dole', '23711'), ('Saint-Claude', '8985'), ('Arbois', '3295'), ('Tavaux', '3957'), ('Les Rousses', '3648'), ('Salins-les-Bains', '2551'), ('Champagnole', '8014'), ('Montmorot', '3082'), ('Poligny', '4003'), ('Hauts de Bienne', '5275')], 'Landes': [('Morcenx', '4415'), ('Saint-Paul-lès-Dax', '13381'), ('Rion-des-Landes', '3023'), ('Morcenx-la-Nouvelle', '4987'), ('Parentis-en-Born', '6684'), ('Saint-Sever', '4890'), ('Pouillon', '3071'), ('Saint-Geours-de-Maremne', '2752'), ("Pontonx-sur-l'Adour", '2868'), ('Ondres', '5520'), ('Labenne', '6887'), ('Tartas', '3225'), ('Mont-de-Marsan', '29807'), ('Dax', '20843'), ('Capbreton', '9037'), ('Bénesse-Maremne', '3683'), ('Rion-des-Landes', '2542'), ('Tarnos', '12634'), ('Biscarrosse', '13947'), ("Aire-sur-l'Adour", '6189'), ('Seignosse', '3876'), ('Hagetmau', '4651'), ('Narrosse', '3230'), ('Saint-Martin-de-Seignanx', '5776'), ('Soorts-Hossegor', '3480'), ('Soustons', '8235'), ('Sanguinet', '4431'), ('Mimizan', '7255'), ('Labouheyre', '2736'), ('Peyrehorade', '3648'), ('Saint-Vincent-de-Paul', '3307'), ('Saint-Vincent-de-Tyrosse', '7734'), ('Tosse', '3219'), ('Saint-Pierre-du-Mont', '9689')], 'Loir-et-Cher': [('Veuzain-sur-Loire', '3452'), ('Le Controis-en-Sologne', '6723'), ('Blois', '45898'), ('Vendôme', '15856'), ('Romorantin-Lanthenay', '17924'), ('Salbris', '4953'), ('Chailles', '2674'), ('Mont-près-Chambord', '3311'), ('Montrichard', '3145'), ('Contres', '3631'), ('Villefranche-sur-Cher', '2657'), ('Mer', '6171'), ('Onzain', '3275'), ('Montoire-sur-le-Loir', '3721'), ('Selles-sur-Cher', '4573'), ('Saint-Aignan', '2852'), ('Saint-Laurent-Nouan', '4337'), ('Noyers-sur-Cher', '2721'), ('Saint-Gervais-la-Forêt', '3174'), ('Saint-Georges-sur-Cher', '2731'), ('Lamotte-Beuvron', '4667'), ('Cellettes', '2652'), ('Vineuil', '8091'), ('Saint-Ouen', '3087'), ('Cour-Cheverny', '2802'), ('La Chaussée-Saint-Victor', '4524'), ('Beauce la Romaine', '3488'), ('Montrichard Val de Cher', '3719')], 'Loire': [('Sury-le-Comtal', '6653'), ('Le Coteau', '6880'), ('Genilac', '3894'), ('Boën-sur-Lignon', '3120'), ('La Talaudière', '6977'), ('Le Chambon-Feugerolles', '11948'), ('Saint-Just-sur-Loire', '6856'), ('Unieux', '8418'), ('Villerest', '5030'), ('Saint-Just-Saint-Rambert', '15229'), ('Saint-Romain-le-Puy', '4090'), ('Saint-Genest-Malifaux', '2918'), ('Saint-Héand', '3637'), ("L'Horme", '4746'), ('Riorges', '10848'), ('Roche-la-Molière', '9884'), ('Panissières', '2892'), ('Pouilly-sous-Charlieu', '2532'), ('Montrond-les-Bains', '5342'), ('Saint-Genest-Lerpt', '6123'), ('Roanne', '33809'), ('Saint-Étienne', '173821'), ('Montbrison', '15915'), ('Saint-Chamond', '34841'), ('Firminy', '16901'), ('Commelle-Vernay', '3005'), ('Lorette', '4716'), ('Feurs', '8302'), ('La Ricamarie', '7872'), ('Mably', '7653'), ('Bourg-Argental', '2937'), ('Balbigny', '2840'), ('Saint-Paul-en-Jarez', '4817'), ('Renaison', '3198'), ('Rive-de-Gier', '14978'), ('Veauche', '8973'), ('Saint-Galmier', '5826'), ('Charlieu', '3695'), ('Savigneux', '3430'), ('Pélussin', '3738'), ('La Fouillouse', '4666'), ("L'Étrat", '2643'), ('Fraisses', '3733'), ('Chavanay', '2897'), ('Chazelles-sur-Lyon', '5423'), ('Andrézieux-Bouthéon', '9963'), ('Bonson', '4093'), ('Villars', '7942'), ('Saint-Priest-en-Jarez', '6106'), ('La Grand-Croix', '5172'), ('Saint-Jean-Bonnefonds', '6615'), ('Sorbiers', '7874'), ('Saint-Marcellin-en-Forez', '5001'), ('Saint-Martin-la-Plaine', '3815')], 'Haute-Loire': [('Le Puy-en-Velay', '19215'), ('Brioude', '6609'), ('Yssingeaux', '7278'), ('Aurec-sur-Loire', '6142'), ('Beauzac', '2969'), ('Retournac', '2990'), ('Sainte-Sigolène', '5989'), ('Dunières', '2668'), ('Saint-Maurice-de-Lignon', '2634'), ('Sainte-Florine', '3200'), ('Coubon', '3258'), ('Brives-Charensac', '4185'), ('Saint-Germain-Laprade', '3666'), ('Langeac', '3489'), ('Polignac', '2810'), ('Saint-Didier-en-Velay', '3451'), ('Tence', '3074'), ('Espaly-Saint-Marcel', '3497'), ('Saint-Just-Malmont', '4225'), ('Bas-en-Basset', '4377'), ('Monistrol-sur-Loire', '8875'), ('Vals-près-le-Puy', '3420')], 'Loire-Atlantique': [('Derval', '3505'), ('Nantes', '320732'), ('Blain', '9954'), ('La Haie-Fouassière', '4691'), ('Saint-Nazaire', '71394'), ('Châteaubriant', '12031'), ('La Baule-Escoublac', '16255'), ('Ancenis', '7510'), ('Treillières', '9617'), ('Orvault', '27209'), ('Saint-Herblain', '47415'), ('Bouguenais', '19903'), ('Joué-sur-Erdre', '2561'), ('Rezé', '42919'), ('Vertou', '25541'), ('Saint-Sébastien-sur-Loire', '27493'), ('La Montagne', '6290'), ('Clisson', '7435'), ('Indre', '4040'), ('Guérande', '16112'), ('La Chapelle-Launay', '3145'), ('Carquefou', '20365'), ('Crossac', '2983'), ('Le Pallet', '3257'), ('Le Pellerin', '5200'), ('La Turballe', '4734'), ('Saffré', '3952'), ('Saint-Aignan-Grandlieu', '3954'), ('Machecoul', '6334'), ('Couëron', '22309'), ('Ligné', '5302'), ('Prinquiau', '3474'), ('Guenrouët', '3374'), ('Cordemais', '3754'), ('La Chevrolière', '5782'), ('Port-Saint-Père', '2928'), ('Rouans', '3069'), ('Héric', '6288'), ('Pornichet', '11141'), ('Sainte-Luce-sur-Loire', '15319'), ('Missillac', '5398'), ('Grandchamp-des-Fontaines', '6439'), ('Campbon', '3959'), ('Le Cellier', '3974'), ('Saint-Molf', '2716'), ('La Chapelle-des-Marais', '4338'), ('La Plaine-sur-Mer', '4379'), ('Besné', '3183'), ('Saint-Jean-de-Boiseau', '5987'), ('Chauvé', '2930'), ('Vigneux-de-Bretagne', '6175'), ('La Chapelle-sur-Erdre', '19551'), ('Pont-Saint-Martin', '6470'), ('Donges', '8152'), ('Frossay', '3216'), ('Malville', '3531'), ('Arthon-en-Retz', '4213'), ('Saint-Julien-de-Concelles', '7392'), ('Mauves-sur-Loire', '3244'), ('Gorges', '4851'), ('Herbignac', '7046'), ('Pornic', '15859'), ('Saint-Brevin-les-Pins', '14287'), ('Oudon', '3872'), ('Bouvron', '3068'), ('Bouaye', '8008'), ('Bourgneuf-en-Retz', '3667'), ('Erbray', '3043'), ('Fay-de-Bretagne', '3676'), ('Saint-Malo-de-Guersac', '3193'), ('Brains', '2849'), ('La Chapelle-Basse-Mer', '5601'), ('Nozay', '4195'), ('Savenay', '9064'), ('Pontchâteau', '10901'), ('Nort-sur-Erdre', '9082'), ('Sucé-sur-Erdre', '7221'), ('Geneston', '3677'), ('Le Croisic', '4118'), ('Guémené-Penfao', '5233'), ('Vallet', '9460'), ('Legé', '4565'), ('La Chapelle-Heulin', '3357'), ('Batz-sur-Mer', '2824'), ('La Bernerie-en-Retz', '3049'), ('Gétigné', '3719'), ('Château-Thébaud', '3164'), ('Corcoué-sur-Logne', '3022'), ('Haute-Goulaine', '5893'), ('Corsept', '2640'), ('Le Bignon', '3907'), ('Trignac', '7983'), ('La Planche', '2739'), ('Boussay', '2682'), ('Saint-Gildas-des-Bois', '3769'), ('Saint-Mars-du-Désert', '5118'), ('Montbert', '3191'), ('Saint-Père-en-Retz', '4576'), ('Petit-Mars', '3755'), ('Saint-Viaud', '2631'), ('Le Loroux-Bottereau', '8397'), ('Mouzillon', '2865'), ('Maisdon-sur-Sèvre', '3007'), ('Saint-Étienne-de-Montluc', '7515'), ('Divatte-sur-Loire', '6975'), ('Villeneuve-en-Retz', '4962'), ('Vair-sur-Loire', '4764'), ('Machecoul-Saint-Même', '7581'), ('Chaumes-en-Retz', '6895'), ('Loireauxence', '7480'), ("Vallons-de-l'Erdre", '6480'), ('Ancenis-Saint-Géréon', '11037'), ('Sainte-Anne-sur-Brivet', '2933'), ('Saint-Mars-de-Coutais', '2571'), ('Montoir-de-Bretagne', '7180'), ('Saint-André-des-Eaux', '6744'), ('Saint-Lyphard', '4817'), ('Saint-Géréon', '2893'), ('Saint-Herblon', '2631'), ('Saint-Colomban', '3417'), ('Le Landreau', '3083'), ('Sautron', '8478'), ('Varades', '3708'), ('Saint-Philbert-de-Grand-Lieu', '9076'), ('Saint-Nicolas-de-Redon', '3245'), ('Vieillevigne', '4010'), ('Aigrefeuille-sur-Maine', '4014'), ('Basse-Goulaine', '9136'), ('Couffé', '2548'), ('Plessé', '5266'), ('Le Pouliguen', '4015'), ('Mésanger', '4749'), ('Sainte-Pazanne', '6980'), ('Saint-Michel-Chef-Chef', '5233'), ('Thouaré-sur-Loire', '10482'), ('Les Sorinières', '8765'), ('Paimbœuf', '3023'), ('Saint-Joachim', '4087')], 'Loiret': [('Cléry-Saint-André', '3425'), ('Briare', '5213'), ('Châteauneuf-sur-Loire', '8212'), ('Malesherbes', '6089'), ('Puiseaux', '3424'), ('Fleury-les-Aubrais', '21010'), ('Courtenay', '3907'), ('Sandillon', '4114'), ('La Chapelle-Saint-Mesmin', '10180'), ('Corquilleroy', '2815'), ('Dordives', '3280'), ('Le Malesherbois', '8073'), ('Orléans', '116269'), ('Chevilly', '2658'), ('La Ferté-Saint-Aubin', '7404'), ('Montargis', '14976'), ('Pithiviers', '9023'), ('Olivet', '22386'), ('Ingré', '9482'), ('Beaugency', '7339'), ('Gien', '13566'), ('Sully-sur-Loire', '5213'), ('Fay-aux-Loges', '3754'), ('Saran', '16357'), ('Jargeau', '4676'), ('Meung-sur-Loire', '6540'), ('Chécy', '8652'), ('Châtillon-sur-Loire', '3108'), ('Saint-Denis-en-Val', '7589'), ('Ferrières-en-Gâtinais', '3702'), ('Saint-Jean-de-Braye', '21288'), ('Donnery', '2864'), ('Lorris', '2973'), ('Chaingy', '3881'), ('Châlette-sur-Loing', '12688'), ('Traînou', '3448'), ('Ouzouer-sur-Loire', '2641'), ('Pannes', '3688'), ('Villemandeur', '6782'), ('Semoy', '3224'), ('Mardié', '2866'), ('Amilly', '13233'), ('Nogent-sur-Vernisson', '2589'), ('Lailly-en-Val', '3103'), ('Saint-Pryvé-Saint-Mesmin', '6076'), ('Saint-Hilaire-Saint-Mesmin', '3071'), ('Saint-Ay', '3561'), ('Saint-Jean-de-la-Ruelle', '16411'), ('Saint-Cyr-en-Val', '3295'), ("Saint-Denis-de-l'Hôtel", '3021'), ('Ormes', '4147'), ('Saint-Jean-le-Blanc', '9070'), ('Neuville-aux-Bois', '4947')], 'Lot': [('Pradines', '3562'), ('Cahors', '19937'), ('Gourdon', '3981'), ('Figeac', '9802'), ('Gramat', '3469'), ('Saint-Céré', '3414'), ('Souillac', '3201')], 'Lot-et-Garonne': [('Agen', '32602'), ('Bon-Encontre', '6224'), ('Clairac', '2594'), ('Villeneuve-sur-Lot', '21742'), ('Marmande', '17421'), ('Nérac', '6837'), ('Sainte-Livrade-sur-Lot', '6456'), ('Montayral', '2652'), ('Aiguillon', '4421'), ('Casteljaloux', '4518'), ('Miramont-de-Guyenne', '3092'), ('Pujols', '3675'), ('Boé', '5631'), ('Fumel', '4755'), ('Colayrac-Saint-Cirq', '3113'), ('Sainte-Bazeille', '3160'), ('Bias', '2997'), ('Foulayronnes', '5449'), ('Le Passage', '9400'), ('Tonneins', '9130'), ('Pont-du-Casse', '4141'), ('Layrac', '3686')], 'Lozère': [("Saint-Chély-d'Apcher", '4211'), ('Mende', '12318'), ('Langogne', '2887'), ('Marvejols', '4684')], 'Maine-et-Loire': [('Baugé-en-Anjou', '6289'), ('Bagneux', '4329'), ('Chemillé-Melay', '8822'), ('Baugé', '3703'), ('Angers', '155850'), ('Saint-Florent-le-Vieil', '2844'), ('Cholet', '54037'), ('Montreuil-Juigné', '7632'), ('Saumur', '26467'), ('Chalonnes-sur-Loire', '6518'), ('Pellouailles-les-Vignes', '2523'), ('Saint-Lambert-la-Potherie', '2924'), ('Saint-Christophe-du-Bois', '2794'), ('Segré', '6938'), ('Liré', '2505'), ('Gesté', '2756'), ('Écouflant', '4394'), ('Chemillé', '7201'), ('Pouancé', '3286'), ('Briollay', '3084'), ('Champtoceaux', '2526'), ('Maulévrier', '3203'), ('Saint-Macaire-en-Mauges', '7028'), ('Beaupréau', '7449'), ('Les Ponts-de-Cé', '12819'), ('Soucelles', '2581'), ('Seiches-sur-le-Loir', '2912'), ('Avrillé', '14443'), ('Beaucouzé', '5439'), ('Beaufort-en-Vallée', '6682'), ('Brissac-Quincé', '3106'), ('Longué-Jumelles', '6831'), ('Mazé', '5173'), ('Durtal', '3358'), ('Saint-Georges-sur-Loire', '3665'), ('Saint-André-de-la-Marche', '2936'), ('Montreuil-Bellay', '3726'), ('Saint-Germain-sur-Moine', '2970'), ('Corné', '3069'), ("Brain-sur-l'Authion", '3350'), ('Doué-la-Fontaine', '7686'), ('Combrée', '2785'), ('Bouchemaine', '6774'), ('Allonnes', '2975'), ('Andard', '2630'), ('Châteauneuf-sur-Sarthe', '3200'), ("Le Lion-d'Angers", '4965'), ('Le May-sur-Èvre', '3820'), ('Juigné-sur-Loire', '2707'), ('Le Louroux-Béconnais', '3269'), ('Bécon-les-Granits', '2789'), ('Vivy', '2537'), ('Trélazé', '14961'), ('Trémentines', '3088'), ('Saint-Léger-sous-Cholet', '3018'), ('Villedieu-la-Blouère', '2555'), ('La Tessoualle', '3189'), ('Villevêque', '3037'), ('Tiercé', '4432'), ("Saint-Barthélemy-d'Anjou", '9210'), ('Le Plessis-Grammoire', '2552'), ('La Pommeraye', '4002'), ('Mûrs-Erigné', '5660'), ('Montjean-sur-Loire', '3065'), ('Sainte-Gemmes-sur-Loire', '3456'), ("Saint-Sylvain-d'Anjou", '4842'), ('Saint-Pierre-Montlimart', '3315'), ('La Séguinière', '4192'), ('Jallais', '3410'), ('Candé', '2824'), ('Saint-Hilaire-Saint-Florent', '4015'), ("Les Bois d'Anjou", '2601'), ('Beaupréau-en-Mauges', '23419'), ('Chemillé-en-Anjou', '20828'), ('Ingrandes-Le Fresne sur Loire', '2665'), ('Lys-Haut-Layon', '7789'), ('Mauges-sur-Loire', '17924'), ('Montrevault-sur-Èvre', '15668'), ('Sèvremoine', '25162'), ("Orée d'Anjou", '16429'), ('Bellevigne-en-Layon', '5751'), ('Longuenée-en-Anjou', '6306'), ('Val-du-Layon', '3411'), ('Verrières-en-Anjou', '7520'), ('Loire-Authion', '15902'), ('Jarzé Villages', '2752'), ('Beaufort-en-Anjou', '7229'), ('Mazé-Milon', '5925'), ('Erdre-en-Anjou', '5739'), ('Segré-en-Anjou Bleu', '17462'), ('Brissac Loire Aubance', '10851'), ('Les Garennes sur Loire', '4567'), ('Morannes sur Sarthe-Daumeray', '3666'), ('Doué-en-Anjou', '11119'), ("Val d'Erdre-Auxence", '4903'), ("Les Hauts-d'Anjou", '8759'), ("Ombrée d'Anjou", '8970'), ('Noyant-Villages', '5546'), ('Terranjou', '3994'), ("Le Lion-d'Angers", '5023'), ('Baugé-en-Anjou', '11829'), ('Gennes-Val-de-Loire', '8570'), ('Saint-Léger-de-Linières', '3694'), ('Bellevigne-les-Châteaux', '3482'), ('Rives-du-Loir-en-Anjou', '5651'), ('Saint-Lambert-des-Levées', '3392')], 'Manche': [('Tourlaville', '16040'), ('Villedieu-les-Poêles', '3550'), ('Sourdeval', '2716'), ('Saint-Hilaire-du-Harcouët', '3781'), ('Isigny-le-Buat', '3221'), ('Querqueville', '5097'), ('Agon-Coutainville', '2796'), ('Donville-les-Bains', '3109'), ('Saint-Pair-sur-Mer', '4078'), ('Condé-sur-Vire', '3323'), ('Cherbourg-Octeville', '35545'), ('Ducey', '2537'), ('Saint-Lô', '19050'), ('Coutances', '8408'), ('Avranches', '10264'), ('Carentan', '5853'), ('Bréhal', '3455'), ('Valognes', '6822'), ('Pontorson', '3837'), ('Granville', '12513'), ('Agneaux', '4153'), ('Bricquebec', '4014'), ('Saint-James', '2750'), ('Les Pieux', '3220'), ('Équeurdreville-Hainneville', '16392'), ('Octeville', '16948'), ('Cherbourg', '25370'), ('Cherbourg-en-Cotentin', '78549'), ('Grandparigny', '2643'), ('Torigny-les-Villes', '4397'), ('Villedieu-les-Poêles-Rouffigny', '3852'), ('Percy-en-Normandie', '2600'), ('Bourgvallées', '3213'), ('La Haye', '3971'), ('Ducey-Les Chéris', '2799'), ('Bricquebec-en-Cotentin', '5826'), ('Sartilly-Baie-Bocage', '2815'), ('Mortain-Bocage', '2929'), ('Marigny-Le-Lozon', '2687'), ('La Hague', '11173'), ('Condé-sur-Vire', '3382'), ('Condé-sur-Vire', '4044'), ('Gouville-sur-Mer', '3229'), ('Pontorson', '4318'), ('Quettreville-sur-Sienne', '3199'), ('Saint-Hilaire-du-Harcouët', '5850'), ('Sainte-Mère-Église', '3076'), ('Sourdeval', '3156'), ('Saint-James', '4907'), ('Carentan-les-Marais', '10052'), ('Picauville', '3263'), ('Port-Bail-sur-Mer', '2556'), ('Saint-Sauveur-Villages', '3601'), ('Bourgvallées', '3247'), ('Carentan les Marais', '7920')], 'Marne': [('Suippes', '3864'), ('Montmirail', '3571'), ('Fagnières', '4843'), ('Fismes', '5589'), ('Witry-lès-Reims', '4971'), ('Dormans', '2918'), ('Sézanne', '4772'), ('Vitry-le-François', '11376'), ('Ay', '3686'), ('Bezannes', '2978'), ('Bétheny', '7069'), ('Reims', '181194'), ('Épernay', '22433'), ('Sainte-Menehould', '4165'), ('Saint-Brice-Courcelles', '3417'), ('Cormontreuil', '6458'), ('Saint-Memmie', '5474'), ('Tinqueux', '10294'), ('Warmeriville', '2591'), ('Mourmelon-le-Grand', '5150'), ('Aÿ-Champagne', '5309'), ('Blancs-Coteaux', '3332')], 'Haute-Marne': [('Nogent', '3591'), ('Chaumont', '21847'), ('Langres', '7668'), ('Saint-Dizier', '22928'), ('Wassy', '2819'), ('Joinville', '3015')], 'Mayenne': [('Gorron', '2552'), ('Ambrières-les-Vallées', '2639'), ('Villaines-la-Juhel', '2738'), ('Ernée', '5717'), ('Changé', '6209'), ('Azé', '3421'), ('Louverné', '4375'), ("L'Huisserie", '4326'), ('Bonchamp-lès-Laval', '6137'), ('Loiron-Ruillé', '2745'), ('Montsûrs-Saint-Céneré', '2561'), ('Château-Gontier-sur-Mayenne', '16950'), ('Montsûrs', '3162'), ('Évron', '8614'), ('Bazouges', '2775'), ('Renazé', '2529'), ('Laval', '49733'), ('Mayenne', '12823'), ('Château-Gontier', '11805'), ('Évron', '7048'), ('Argentré', '2825'), ('Craon', '4486'), ('Meslay-du-Maine', '2808'), ('Saint-Berthevin', '7353'), ('Cossé-le-Vivien', '3199')], 'Meurthe-et-Moselle': [('Nancy', '105058'), ('Saint-Nicolas-de-Port', '7417'), ('Toul', '15633'), ('Écrouves', '4428'), ('Réhon', '3809'), ('Saulxures-lès-Nancy', '4221'), ('Frouard', '6584'), ('Chaligny', '2826'), ('Lunéville', '17867'), ('Heillecourt', '5403'), ('Trieux', '2605'), ('Pompey', '4870'), ('Briey', '5674'), ('Villerupt', '10003'), ('Laxou', '14366'), ('Seichamps', '5069'), ('Ludres', '6142'), ('Liverdun', '5863'), ('Cosnes-et-Romain', '2741'), ('Jarville-la-Malgrange', '9354'), ('Foug', '2629'), ('Gorcy', '2877'), ('Gondreville', '2712'), ('Lexy', '3816'), ('Pulnoy', '5077'), ('Rosières-aux-Salines', '2870'), ('Blénod-lès-Pont-à-Mousson', '4628'), ('Champigneulles', '6671'), ('Custines', '3082'), ('Longwy', '14774'), ('Baccarat', '4192'), ('Vandœuvre-lès-Nancy', '29942'), ('Dombasle-sur-Meurthe', '9668'), ('Jœuf', '6570'), ('Haucourt-Moulaine', '3394'), ('Auboué', '2617'), ('Bouxières-aux-Dames', '4159'), ('Damelevières', '3155'), ('Varangéville', '3594'), ('Piennes', '2539'), ('Pagny-sur-Moselle', '4105'), ('Saint-Max', '10019'), ('Neuves-Maisons', '6633'), ('Dieulouard', '4772'), ('Hussigny-Godbrange', '3693'), ('Jarny', '8219'), ('Tomblaine', '8856'), ('Longuyon', '5218'), ('Mont-Saint-Martin', '8994'), ('Homécourt', '6267'), ('Essey-lès-Nancy', '8833'), ("Blainville-sur-l'Eau", '3973'), ('Laneuveville-devant-Nancy', '6553'), ('Herserange', '4409'), ('Maxéville', '9818'), ('Villers-lès-Nancy', '14525'), ('Val de Briey', '8069')], 'Meuse': [('Thierville-sur-Meuse', '3165'), ('Ancerville', '2662'), ('Belleville-sur-Meuse', '3040'), ('Stenay', '2507'), ('Étain', '3507'), ('Verdun', '16942'), ('Bar-le-Duc', '14625'), ('Saint-Mihiel', '4039'), ('Commercy', '5386'), ('Ligny-en-Barrois', '4018'), ('Revigny-sur-Ornain', '2802')], 'Morbihan': [('Theix-Noyalo', '8251'), ("Val d'Oust", '2758'), ('Évellys', '3483'), ('Carentoir', '3074'), ('La Gacilly', '3962'), ('Pluméliau-Bieuzy', '4355'), ('Pont-Scorff', '3844'), ('Sérent', '3066'), ('Péaule', '2712'), ('Ploemeur', '17778'), ('Saint-Dolay', '2539'), ('Plumelin', '2770'), ('Baud', '6234'), ('Le Bono', '2533'), ('Theix', '6946'), ('Quéven', '8770'), ('Ploeren', '6635'), ('Sarzeau', '8791'), ('Le Palais', '2544'), ('Quiberon', '4658'), ('Sulniac', '3756'), ('Riantec', '5821'), ('Surzur', '4724'), ('Saint-Nolff', '3847'), ('Plumelec', '2679'), ('Pluneret', '5905'), ('Plouay', '5792'), ('Plumergat', '4178'), ('Arradon', '5392'), ('Le Faouët', '2793'), ('Nivillac', '4677'), ('Hennebont', '16062'), ('Lanester', '23124'), ('Landévant', '3961'), ('Merlevenez', '3238'), ('Inzinzac-Lochrist', '6548'), ('Kervignac', '6786'), ('Cléguer', '3298'), ('Noyal-Pontivy', '3605'), ('Cléguérec', '2828'), ('Lorient', '57246'), ('Locminé', '4505'), ('Plouhinec', '5386'), ('Baden', '4396'), ('Rieux', '2865'), ('Ploërmel', '9791'), ('Pluvigner', '7676'), ('Questembert', '7862'), ('Saint-Jean-Brévelay', '2836'), ('Moréac', '3733'), ("Sainte-Anne-d'Auray", '2770'), ('Plescop', '6145'), ('Saint-Avé', '11895'), ('Séné', '8900'), ('Plougoumelen', '2522'), ('Ploemel', '2988'), ('Pluméliau', '3716'), ('Port-Louis', '2654'), ('Vannes', '53719'), ('Pontivy', '15064'), ('Brech', '6660'), ('Belz', '3760'), ('Auray', '14141'), ('Allaire', '3886'), ('Bignan', '2774'), ('Guidel', '11767'), ('Carentoir', '3288'), ('Caudan', '6920'), ('Gestel', '2637'), ('Elven', '6261'), ('Noyal-Muzillac', '2520'), ('Guer', '6091'), ('Erdeven', '3853'), ('Muzillac', '5066'), ('Locmiquélic', '4039'), ('Mauron', '3145'), ('Languidic', '8064'), ('Grand-Champ', '5552'), ('Camors', '3066'), ('Carnac', '4236'), ('Larmor-Plage', '8319'), ('Monterblanc', '3299'), ('Férel', '3318'), ('Locoal-Mendon', '3464'), ('Gourin', '3779'), ('Crach', '3394')], 'Moselle': [('Hayange', '16005'), ('Bousse', '3205'), ('Faulquemont', '5154'), ('Alsting', '2535'), ('Clouange', '3374'), ('Hagondange', '9287'), ('Audun-le-Tiche', '7048'), ('Uckange', '6926'), ('Saint-Julien-lès-Metz', '3373'), ('Puttelange-aux-Lacs', '3089'), ('Folschviller', '3949'), ('Longeville-lès-Saint-Avold', '3651'), ('Le Ban-Saint-Martin', '4517'), ('Creutzwald', '13042'), ('Cattenom', '2628'), ('Behren-lès-Forbach', '6562'), ('Knutange', '3192'), ('Ars-sur-Moselle', '4704'), ('Maizières-lès-Metz', '11668'), ('Sarralbe', '4485'), ('Rombas', '9861'), ('Yutz', '17143'), ('Ottange', '3099'), ('Moyeuvre-Grande', '7506'), ('Vitry-sur-Orne', '3021'), ('Valmont', '3019'), ('Neufchef', '2590'), ('Manom', '2915'), ('Mondelange', '5631'), ('Fameck', '14442'), ('Talange', '7781'), ('Œting', '2706'), ('Fontoy', '3095'), ('Ham-sous-Varsberg', '2848'), ('Courcelles-Chaussy', '3006'), ('Amnéville', '10416'), ('Nilvange', '4515'), ('Phalsbourg', '4729'), ('Terville', '7134'), ('Rosselange', '2650'), ('Farébersviller', '5493'), ('Serémange-Erzange', '4163'), ('Montois-la-Montagne', '2732'), ('Sainte-Marie-aux-Chênes', '4350'), ('Longeville-lès-Metz', '3955'), ('Marange-Silvange', '6458'), ('Marly', '10108'), ('Woustviller', '2986'), ('Woippy', '14014'), ('Macheren', '2753'), ('Gandrange', '3028'), ('Guénange', '7559'), ('Falck', '2506'), ('Morhange', '3408'), ('Créhange', '3871'), ('Petite-Rosselle', '6298'), ('Morsbach', '2684'), ('Hambach', '2875'), ('Grosbliederstroff', '3296'), ('Moulins-lès-Metz', '5007'), ('Cocheren', '3380'), ('Freyming-Merlebach', '12725'), ('Bertrange', '2801'), ('Saint-Avold', '15415'), ('Florange', '11972'), ('Boulay-Moselle', '5580'), ('Sarrebourg', '12449'), ('Thionville', '40778'), ('Sarreguemines', '20635'), ('Metz', '118489'), ('Dieuze', '2800'), ('Forbach', '21597'), ('Hettange-Grande', '7779'), ('Algrange', '6120'), ('Montigny-lès-Metz', '21879'), ('Bitche', '4951'), ('Spicheren', '3232'), ('Scy-Chazelles', '2675'), ('Stiring-Wendel', '11178'), ('Hombourg-Haut', '6283'), ('Carling', '3415'), ('Bouzonville', '3926'), ("L'Hôpital", '5229'), ('Réhon', '3809'), ('Trieux', '2605'), ('Briey', '5674'), ('Longwy', '14774'), ('Jœuf', '6570'), ('Haucourt-Moulaine', '3394'), ('Auboué', '2617'), ('Piennes', '2539'), ('Longuyon', '5218'), ('Gorcy', '2877'), ('Homécourt', '6267'), ('Herserange', '4409'), ('Marspich', '8497'), ('Merlebach', '7892')], 'Nièvre': [('Garchizy', '3740'), ('Marzy', '3724'), ('Coulanges-lès-Nevers', '3724'), ('La Machine', '3240'), ('Guérigny', '2510'), ('Varennes-Vauzelles', '9201'), ('Nevers', '33005'), ('Clamecy', '3590'), ('Decize', '5233'), ('Imphy', '3202'), ('Cosne-Cours-sur-Loire', '9589'), ('La Charité-sur-Loire', '4742'), ('Fourchambault', '4125')], 'Nord': [('Lille', '236234'), ('Steenvoorde', '4429'), ('Hazebrouck', '21408'), ('Leers', '9378'), ('Roubaix', '98828'), ("Villeneuve-d'Ascq", '61957'), ('Aulnoye-Aymeries', '8840'), ("La Chapelle-d'Armentières", '8577'), ('Dunkerque', '86279'), ('Berlaimont', '3175'), ('Sin-le-Noble', '15428'), ('Arleux', '3177'), ('Maubeuge', '29589'), ('Valenciennes', '43229'), ('Le Quesnoy', '4870'), ('Cambrai', '32176'), ('Tourcoing', '98656'), ('Douai', '39613'), ('Armentières', '24909'), ('Dechy', '5360'), ('Denain', '19877'), ('Avesnes-sur-Helpe', '4195'), ('Croix', '20881'), ('Coudekerque-Branche', '20851'), ('Lambersart', '27400'), ('Bergues', '3600'), ('Marcq-en-Barœul', '38486'), ('Wattrelos', '40898'), ('Loos', '22866'), ('Lys-lez-Lannoy', '13643'), ('Chéreng', '2973'), ('Fourmies', '11563'), ('Gravelines', '11014'), ('Le Cateau-Cambrésis', '7030'), ('Bailleul', '15163'), ('Hornaing', '3562'), ('La Bassée', '6509'), ('Orchies', '8633'), ('Hondschoote', '4060'), ('Bray-Dunes', '4495'), ('Cysoing', '4849'), ('Rœulx', '3807'), ('Iwuy', '3371'), ('Haubourdin', '14725'), ('Caudry', '14121'), ('Ghyvelde', '3176'), ('Bavay', '3264'), ("Condé-sur-l'Escaut", '9515'), ('Anor', '3221'), ('Hautmont', '14517'), ('Téteghem-Coudekerque-Village', '8285'), ('Ghyvelde', '4131'), ('Vieux-Berquin', '2512'), ('Haspres', '2680'), ('Annœullin', '10471'), ('Pecquencourt', '6088'), ('Genech', '2827'), ('Pont-à-Marcq', '2911'), ('Wervicq-Sud', '5353'), ('Wavrin', '7771'), ('Emmerin', '3108'), ('Quesnoy-sur-Deûle', '6931'), ('Anzin', '13329'), ('Waziers', '7402'), ('Ronchin', '19294'), ('Courchelettes', '2842'), ('Lomme', '28006'), ('Jeumont', '10344'), ('Fort-Mardyck', '3426'), ('Monchecourt', '2521'), ('Petite-Forêt', '4887'), ('Aniche', '10085'), ('Saint-Amand-les-Eaux', '15870'), ('Somain', '11934'), ('Comines', '12788'), ('Solesmes', '4290'), ('Seclin', '12463'), ('Escaudœuvres', '3218'), ('Bauvin', '5086'), ('Thumeries', '3884'), ('La Gorgue', '5639'), ('Halluin', '20743'), ('Landrecies', '3482'), ('Bourbourg', '7152'), ('Sainghin-en-Mélantois', '2830'), ('Mouvaux', '13121'), ('Sequedin', '4775'), ('Roost-Warendin', '6037'), ('Lesquin', '9012'), ('Saint-André-lez-Lille', '13151'), ('Hasnon', '3918'), ('Houplin-Ancoisne', '3337'), ('Estaires', '6461'), ('Roncq', '13482'), ('Avesnes-les-Aubert', '3630'), ('Lourches', '3952'), ('Watten', '2599'), ('Phalempin', '4820'), ('Masnières', '2745'), ('Beuvrages', '6809'), ('Wahagnies', '2636'), ('Morbecque', '2524'), ('Nomain', '2549'), ('Templemars', '3465'), ('Cappelle-la-Grande', '7920'), ('Bondues', '9808'), ('Grande-Synthe', '22522'), ('Baisieux', '4777'), ('Hergnies', '4433'), ('Mons-en-Barœul', '21277'), ('La Madeleine', '21856'), ('Douchy-les-Mines', '10293'), ('Marly', '12083'), ('Coutiches', '3077'), ('Auby', '7239'), ('Téteghem', '6926'), ('Wambrechies', '10608'), ('Bousbecque', '4804'), ('Cuincy', '6400'), ('Hem', '18636'), ('Aulnoy-lez-Valenciennes', '7224'), ('Rousies', '4072'), ('Proville', '3124'), ('Neuville-Saint-Rémy', '3782'), ('Auberchicourt', '4551'), ('Wignehies', '2855'), ("Bruay-sur-l'Escaut", '11258'), ('Grand-Fort-Philippe', '5130'), ('Crespin', '4495'), ('Ferrière-la-Grande', '5273'), ('Sains-du-Nord', '2799'), ('Provin', '4403'), ('Lallaing', '6364'), ('Neuville-en-Ferrain', '10264'), ('Masny', '4109'), ('Raimbeaucourt', '4008'), ('Lambres-lez-Douai', '5007'), ('Marpent', '2701'), ('Saint-Saulve', '11276'), ('Trith-Saint-Léger', '6292'), ('Leval', '2516'), ('Hérin', '4127'), ('Wattignies', '15075'), ('Sainghin-en-Weppes', '5481'), ('Haveluy', '3304'), ('Maing', '4066'), ('Onnaing', '8789'), ('Loon-Plage', '6085'), ('Marquette-lez-Lille', '10798'), ('Râches', '2701'), ('Pérenchies', '8542'), ('Abscon', '4309'), ('Flers-en-Escrebieux', '5781'), ('Faches-Thumesnil', '18191'), ('Feignies', '6809'), ('Hallennes-lez-Haubourdin', '4676'), ('Fenain', '5510'), ('Vieux-Condé', '10465'), ('Mérignies', '3199'), ('Linselles', '8275'), ('Lezennes', '3050'), ('Trélon', '2771'), ('Marchiennes', '4584'), ('Raismes', '12324'), ('Santes', '5608'), ('Houplines', '7782'), ('Leffrinckoucke', '4207'), ('Gondecourt', '4031'), ('Montigny-en-Ostrevent', '4793'), ('Famars', '2545'), ('Wormhout', '5672'), ('Flines-lez-Raches', '5598'), ('Louvroil', '6371'), ('Capinghem', '2537'), ('Quiévrechain', '6362'), ('Steenwerck', '3672'), ('Escaudain', '9682'), ('Wasquehal', '20860'), ('Nieppe', '7512'), ('Boussois', '3194'), ('Guesnain', '4689'), ('Beuvry-la-Forêt', '2796'), ('Escautpont', '4208'), ('Fresnes-sur-Escaut', '7499'), ('Saint-Pol-sur-Mer', '20479'), ('Allennes-les-Marais', '3456'), ('Erquinghem-Lys', '5316'), ('Avelin', '2641'), ('Ostricourt', '5454'), ('Hoymille', '3194'), ('Toufflers', '3911'), ('Salomé', '2945'), ('Thiant', '2999'), ('Templeuve-en-Pévèle', '6332'), ('Neuville-sur-Escaut', '2688'), ('Quarouble', '3077'), ('Willems', '3048'), ('Lecelles', '2847'), ('Fretin', '3290'), ('La Sentinelle', '3117'), ('Flers-lez-Lille', '10561')], 'Oise': [('Le Plessis-Belleville', '3833'), ('Coye-la-Forêt', '3976'), ('Auneuil', '2679'), ('Chambly', '10147'), ('Thourotte', '4484'), ('Saint-Maximin', '2847'), ('Fitz-James', '2575'), ('Bresles', '4022'), ('Grandvilliers', '2820'), ('Nogent-sur-Oise', '20780'), ('Chaumont-en-Vexin', '3340'), ('Béthisy-Saint-Pierre', '3108'), ('Choisy-au-Bac', '3288'), ('Crèvecœur-le-Grand', '3503'), ('Sérifontaine', '2739'), ('Mouy', '5237'), ('Gouvieux', '8978'), ('Lamorlaye', '8784'), ('Nanteuil-le-Haudouin', '4201'), ('Noyon', '13190'), ('Senlis', '14760'), ('Chantilly', '10950'), ('Beauvais', '57071'), ('Compiègne', '40615'), ('Crépy-en-Valois', '14788'), ('Creil', '36169'), ('Breteuil', '4221'), ('Saint-Just-en-Chaussée', '6055'), ('Clermont', '10163'), ('Ribécourt-Dreslincourt', '3705'), ('Lacroix-Saint-Ouen', '4891'), ('Rantigny', '2541'), ('Pontpoint', '3242'), ('Neuilly-en-Thelle', '3970'), ('Verberie', '3813'), ('Méru', '14452'), ('Maignelay-Montigny', '2678'), ('Montataire', '13645'), ('Liancourt', '7010'), ('Orry-la-Ville', '3351'), ('Noailles', '2829'), ('Hermes', '2509'), ('Breuil-le-Vert', '3116'), ('Bornel', '3553'), ('La Chapelle-en-Serval', '3140'), ('Estrées-Saint-Denis', '3759'), ('Cires-lès-Mello', '3996'), ('Cauffry', '2621'), ("Saint-Leu-d'Esserent", '4678'), ('Venette', '2884'), ('Précy-sur-Oise', '3220'), ('Pont-Sainte-Maxence', '12602'), ('Laigneville', '4721'), ('Longueil-Annel', '2604'), ('Margny-lès-Compiègne', '8676'), ('Bury', '2961'), ('Breuil-le-Sec', '2667'), ('Agnetz', '3044'), ('Andeville', '3286'), ('Sainte-Geneviève', '3344'), ('Villers-Saint-Paul', '6434'), ('Verneuil-en-Halatte', '4707'), ('Auneuil', '2836'), ('Bornel', '4764')], 'Orne': [('Alençon', '25870'), ('Argentan', '13395'), ('Vimoutiers', '3116'), ('Sées', '4198'), ('Saint-Germain-du-Corbéis', '3780'), ('Saint-Georges-des-Groseillers', '3152'), ('Mortagne-au-Perche', '3775'), ("L'Aigle", '7961'), ('Flers', '14762'), ("Athis-de-l'Orne", '2525'), ('Domfront', '3473'), ('Tinchebray', '2590'), ('La Ferté-Macé', '5810'), ('Tinchebray-Bocage', '4882'), ("Bagnoles de l'Orne Normandie", '2732'), ('Val-au-Perche', '3432'), ('La Ferté-en-Ouche', '3092'), ("Rives d'Andaine", '3051'), ('Athis-Val de Rouvre', '4217'), ('Longny les Villages', '2985'), ('Domfront en Poiraie', '4196'), ('Tourouvre au Perche', '3092'), ('Gouffern en Auge', '3700'), ('La Ferté Macé', '5186')], 'Pas-de-Calais': [('Drocourt', '2926'), ('Oye-Plage', '5583'), ('Bully-les-Mines', '12156'), ('Évin-Malmaison', '4625'), ('Saint-Laurent-Blangy', '6542'), ('Blendecques', '4944'), ('Leforest', '7248'), ('Noyelles-Godault', '5773'), ('Beaurains', '5552'), ('Berck', '13482'), ('Cauchy-à-la-Tour', '2744'), ('Allouagne', '2890'), ('Houdain', '7110'), ('Marck', '10529'), ('Sainte-Catherine', '3562'), ('Coquelles', '2602'), ('Mazingarbe', '8033'), ('Wimereux', '6343'), ('Lillers', '9810'), ('Saint-Pol-sur-Ternoise', '4909'), ('Chocques', '2831'), ('Angres', '4698'), ('Lapugnoy', '3471'), ('Sallaumines', '9594'), ('Montigny-en-Gohelle', '10180'), ('Sangatte', '4845'), ('Laventie', '4979'), ('Libercourt', '8321'), ('Hulluch', '3405'), ('Richebourg', '2646'), ('Calais', '72509'), ('Samer', '4666'), ('Boulogne-sur-Mer', '40251'), ('Arras', '41694'), ('Nœux-les-Mines', '11575'), ('Lens', '31461'), ('Oignies', '9993'), ('Bruay-la-Buissière', '21903'), ('Béthune', '25039'), ('Saint-Omer', '14903'), ('Liévin', '30112'), ('Hénin-Beaumont', '25992'), ('Carvin', '17557'), ('Saint-Martin-au-Laërt', '4077'), ('Longuenesse', '10563'), ('Aire-sur-la-Lys', '9691'), ('Lumbres', '3601'), ('Ardres', '4403'), ('Étaples', '10912'), ('Éleu-dit-Leauwette', '2840'), ('Harnes', '12237'), ('Achicourt', '7957'), ('Éperlecques', '3696'), ('Bapaume', '3748'), ('Vimy', '4358'), ('Barlin', '7544'), ('Courrières', '10554'), ('Auchel', '10272'), ('Le Touquet-Paris-Plage', '4227'), ('Vitry-en-Artois', '4675'), ('Loos-en-Gohelle', '6879'), ('Billy-Berclau', '4885'), ('Guînes', '5618'), ('Avion', '17625'), ('Frévent', '3423'), ('Beuvry', '9219'), ('Saint-Étienne-au-Mont', '5130'), ('Camiers', '2730'), ('Wimille', '3987'), ('Saint-Martin-Boulogne', '11136'), ('Hersin-Coupigny', '6245'), ('Vermelles', '4765'), ('Saint-Nicolas', '4687'), ('Marles-les-Mines', '5542'), ('Méricourt', '11267'), ('Loison-sous-Lens', '5366'), ('Wingles', '8756'), ('Rang-du-Fliers', '4307'), ('Le Portel', '9071'), ('Sains-en-Gohelle', '6060'), ('Sailly-sur-la-Lys', '3910'), ('Noyelles-sous-Lens', '6632'), ('Pont-à-Vendin', '3134'), ('Desvres', '4875'), ('Marquise', '5083'), ('Billy-Montigny', '8177'), ('Cucq', '5059'), ('Fouquières-lès-Lens', '6434'), ('Annezin', '5836'), ('Dourges', '5928'), ('Auxi-le-Château', '2572'), ('Biache-Saint-Vaast', '4305'), ('Brebières', '5088'), ('Anzin-Saint-Aubin', '2751'), ('Divion', '6963'), ('Auchy-les-Mines', '4678'), ('Arques', '9584'), ('Isbergues', '8717'), ('Vendin-le-Vieil', '8479'), ('Annay', '4377'), ('Outreau', '13378'), ('Haisnes', '4398'), ('Audruicq', '5434'), ('Courcelles-lès-Lens', '8104'), ('Douvrin', '5680'), ('Calonne-Ricouart', '5507'), ('Haillicourt', '4837'), ('Dainville', '5681'), ('La Couture', '2677'), ('Gonnehem', '2508'), ('Coulogne', '5430'), ('Grenay', '6799'), ('Burbure', '2833'), ('Aix-Noulette', '3923'), ('Violaines', '3773'), ('Wizernes', '3292'), ('Rinxent', '3038'), ('Verquin', '3501'), ('Saint-Léonard', '3410'), ('Souchez', '2594'), ('Fleurbaix', '2698'), ('Meurchin', '3748'), ('Merlimont', '3401'), ('Neufchâtel-Hardelot', '3745'), ('Labourse', '2893'), ('Lestrem', '4728'), ('Rouvroy', '8945'), ('Équihen-Plage', '2662'), ('Saint-Venant', '3062'), ('Bruay-en-Artois', '22893'), ('Molinghem', '2870'), ('Labuissière', '3647'), ('Saint-Martin-lez-Tatinghem', '5923')], 'Puy-de-Dôme': [('Ennezat', '2520'), ('Aigueperse', '2767'), ('Pont-du-Château', '12182'), ('Ceyrat', '6364'), ('Brassac-les-Mines', '3362'), ('Beaumont', '10664'), ('Courpière', '4017'), ('Veyre-Monton', '3635'), ('Saint-Genès-Champanelle', '3659'), ('Pérignat-lès-Sarliève', '2705'), ('Blanzat', '3840'), ('Volvic', '4623'), ('Chamalières', '17276'), ('Châteaugay', '3105'), ('Aubière', '10904'), ('Le Cendre', '5510'), ('Orcines', '3493'), ('Romagnat', '7791'), ('Saint-Éloy-les-Mines', '3572'), ('Les Martres-de-Veyre', '3984'), ('Vertaizon', '3258'), ('Lezoux', '6256'), ('Vic-le-Comte', '5189'), ('Clermont-Ferrand', '147865'), ('Issoire', '15296'), ('Riom', '19004'), ('Ambert', '6655'), ('Thiers', '11784'), ('Billom', '4777'), ('Maringues', '3125'), ('Royat', '4359'), ('Châtel-Guyon', '6239'), ('Mozac', '3876'), ('La Roche-Blanche', '3483'), ("Cournon-d'Auvergne", '20322'), ('Puy-Guillaume', '2790'), ('Orcet', '2756'), ('Aulnat', '4095'), ('Cébazat', '8805'), ('Gerzat', '10289'), ('Lempdes', '8677'), ('Mur-sur-Allier', '3435')], 'Pyrénées-Atlantiques': [('Pau', '75627'), ('Biarritz', '25787'), ('Bayonne', '51894'), ('Serres-Castet', '4348'), ('Hendaye', '16967'), ('Anglet', '39719'), ('Billère', '12793'), ('Artix', '3428'), ('Idron', '4903'), ('Bizanos', '4563'), ('Lescar', '9665'), ('Pontacq', '2925'), ('Lons', '13707'), ('Nay', '3265'), ('Cambo-les-Bains', '6622'), ('Lahonce', '2506'), ('Hasparren', '7498'), ("Saint-Pierre-d'Irube", '5491'), ('Arcangues', '3195'), ('Morlaàs', '4356'), ('Jurançon', '7102'), ('Oloron-Sainte-Marie', '10594'), ('Saint-Jean-de-Luz', '14196'), ('Ciboure', '6211'), ('Mauléon-Licharre', '2947'), ('Saint-Pée-sur-Nivelle', '7037'), ('Orthez', '10369'), ('Ustaritz', '6918'), ('Sare', '2688'), ('Boucau', '8627'), ('Mourenx', '6122'), ('Salies-de-Béarn', '4568'), ('Urrugne', '10418'), ('Briscous', '2878'), ('Gelos', '3652'), ('Ascain', '4359'), ('Urcuit', '2725'), ('Bassussarry', '3231'), ('Villefranque', '2822'), ('Mouguerre', '5294'), ('Monein', '4435'), ('Sauvagnon', '3411'), ('Gan', '5585'), ('Bidart', '6975'), ('Bordes', '2887')], 'Hautes-Pyrénées': [("Bordères-sur-l'Échez", '5357'), ('Barbazan-Debat', '3498'), ('Lourdes', '13132'), ('Tarbes', '42758'), ('Bagnères-de-Bigorre', '7085'), ('Lannemezan', '5816'), ('Soues', '3066'), ('Argelès-Gazost', '2911'), ('Aureilhan', '7864'), ('Odos', '3290'), ('Vic-en-Bigorre', '4843'), ('Séméac', '5085'), ('Juillan', '4045'), ('Ibos', '2906')], 'Pyrénées-Orientales': [('Alénya', '3645'), ('Pia', '9849'), ('Palau-del-Vidre', '3094'), ('Canohès', '6344'), ('Argelès-sur-Mer', '10260'), ('Le Barcarès', '5771'), ('Ille-sur-Têt', '5513'), ('Rivesaltes', '8888'), ('Saint-Génis-des-Fontaines', '2841'), ('Torreilles', '3839'), ('Saint-André', '3422'), ('Saint-Cyprien', '11040'), ('Saint-Estève', '11719'), ('Pollestres', '5229'), ('Salses-le-Château', '3514'), ('Bages', '4291'), ('Baho', '3431'), ('Bompas', '7426'), ('Baixas', '2612'), ('Villeneuve-de-la-Raho', '3977'), ("Espira-de-l'Agly", '3475'), ('Le Soler', '7752'), ('Toulouges', '7192'), ('Pézilla-la-Rivière', '3812'), ('Millas', '4264'), ('Latour-Bas-Elne', '3158'), ('Ponteilla', '2841'), ('Saleilles', '5564'), ('Saint-Nazaire', '2784'), ('Villelongue-de-la-Salanque', '3248'), ('Saint-Hippolyte', '3198'), ('Céret', '7857'), ('Perpignan', '119344'), ('Banyuls-sur-Mer', '4738'), ('Amélie-les-Bains-Palalda', '3542'), ('Arles-sur-Tech', '2724'), ('Maureillas-las-Illas', '2568'), ('Le Boulou', '5278'), ('Cabestany', '10301'), ('Elne', '9248'), ('Sorède', '3343'), ('Sainte-Marie-la-Mer', '4821'), ("Saint-Féliu-d'Avall", '2874'), ('Thuir', '7924'), ('Claira', '4470'), ('Saint-Laurent-de-la-Salanque', '10071'), ('Port-Vendres', '4016'), ('Prades', '6028'), ('Canet-en-Roussillon', '12284')], 'Bas-Rhin': [('Gambsheim', '5041'), ('Seltz', '3162'), ('Scherwiller', '3217'), ('Châtenois', '4199'), ('Mertzwiller', '3363'), ('Oberhoffen-sur-Moder', '3497'), ('Holtzheim', '3703'), ('Marckolsheim', '4159'), ('Hochfelden', '3997'), ('Oberhausbergen', '5363'), ('Wolfisheim', '4174'), ('Marlenheim', '4250'), ('Plobsheim', '4465'), ('Kilstett', '2515'), ('Truchtersheim', '3257'), ('Sarre-Union', '2796'), ('Bischheim', '17353'), ('Vendenheim', '5971'), ('Weyersheim', '3398'), ('La Wantzenau', '5881'), ('Betschdorf', '4183'), ('Lipsheim', '2664'), ('Mutzig', '6063'), ('Rhinau', '2670'), ('Wasselonne', '5711'), ('Gundershoffen', '3712'), ('Schweighouse-sur-Moder', '4942'), ('Hœnheim', '11304'), ('Bischwiller', '12746'), ('Herrlisheim', '4731'), ('Offendorf', '2518'), ('Reichstett', '4397'), ('Eckbolsheim', '7049'), ('Ostwald', '12985'), ('Hilsenheim', '2649'), ('Lingolsheim', '19797'), ('Lampertheim', '3405'), ('Strasbourg', '290576'), ('Gerstheim', '3449'), ('Geudertheim', '2586'), ('Reichshoffen', '5402'), ('Soufflenheim', '4827'), ('Fegersheim', '5734'), ('Gries', '2874'), ('Huttenheim', '2672'), ('Souffelweyersheim', '7994'), ('Ingwiller', '4048'), ('Erstein', '10661'), ('Marmoutier', '2712'), ('Wissembourg', '7487'), ('Brumath', '10081'), ('Schiltigheim', '33993'), ('Geispolsheim', '7561'), ('Illkirch-Graffenstaden', '27087'), ('Molsheim', '9343'), ('Obernai', '12015'), ('Haguenau', '35448'), ('Sélestat', '19242'), ('Bouxwiller', '3757'), ('Saverne', '11378'), ('Barr', '7197'), ('Bischoffsheim', '3329'), ('Duttlenheim', '2865'), ('Benfeld', '5801'), ('Drusenheim', '5268'), ('La Broque', '2629'), ('Dorlisheim', '2613'), ('Dettwiller', '2610'), ('Mundolsheim', '4749'), ('Eschau', '5424'), ('Hœrdt', '4514'), ('Weitbruch', '2766'), ('Rosheim', '5221'), ('Soultz-sous-Forêts', '3190'), ('Val-de-Moder', '5040'), ('Truchtersheim', '4267'), ('Hochfelden', '4048')], 'Haut-Rhin': [('Bartenheim', '3987'), ('Habsheim', '4999'), ('Bollwiller', '4122'), ('Biesheim', '2530'), ('Hégenheim', '3369'), ('Brunstatt', '6405'), ('Saint-Louis', '22835'), ('Danjoutin', '3620'), ('Offemont', '4219'), ('Morschwiller-le-Bas', '3692'), ('Riedisheim', '12520'), ('Richwiller', '3692'), ('Orbey', '3467'), ('Rixheim', '14204'), ('Volgelsheim', '2680'), ('Rouffach', '4302'), ('Pulversheim', '3017'), ('Munster', '4699'), ('Turckheim', '3831'), ('Zillisheim', '2538'), ('Thann', '7767'), ('Altkirch', '5659'), ('Guebwiller', '11020'), ('Cernay', '11476'), ('Illzach', '14784'), ('Baldersheim', '2667'), ('Mulhouse', '108312'), ('Sainte-Croix-en-Plaine', '3029'), ('Issenheim', '3418'), ('Horbourg-Wihr', '6234'), ('Huningue', '7261'), ('Kembs', '5496'), ('Vieux-Thann', '2815'), ('Village-Neuf', '4421'), ('Masevaux', '3341'), ('Kingersheim', '13263'), ('Lutterbach', '6247'), ('Colmar', '68682'), ('Ribeauvillé', '4665'), ('Pfastatt', '9723'), ('Ensisheim', '7512'), ('Hésingue', '2770'), ('Blotzheim', '4661'), ('Buhl', '3354'), ('Sausheim', '5479'), ('Wintzenheim', '7853'), ('Ingersheim', '4680'), ('Sainte-Marie-aux-Mines', '5054'), ('Soultz-Haut-Rhin', '7048'), ('Wittelsheim', '10362'), ('Sierentz', '3842'), ('Staffelfelden', '4046'), ('Wittenheim', '14776'), ('Kaysersberg Vignoble', '4394'), ('Brunstatt-Didenheim', '8163'), ('Masevaux-Niederbruck', '3720')], 'Rhône': [('Saint-Pierre-la-Palud', '2596'), ('Pollionnay', '2845'), ('Lissieu', '3134'), ("Saint-Jean-d'Ardières", '4859'), ('Mornant', '6144'), ('Saint-Martin-en-Haut', '3853'), ('Communay', '4242'), ('Chasselay', '2791'), ('Quincieux', '3495'), ("Marcy-l'Étoile", '3549'), ('Montagny', '3118'), ("L'Arbresle", '6410'), ('Orliénas', '2541'), ("Collonges-au-Mont-d'Or", '4315'), ('Loire-sur-Rhône', '2618'), ('Limas', '4871'), ("Saint-Cyr-au-Mont-d'Or", '5747'), ('Pierre-Bénite', '10397'), ("Saint-Symphorien-d'Ozon", '5789'), ('Anse', '7538'), ('Fontaines-sur-Saône', '7066'), ('Saint-Laurent-de-Mure', '5458'), ("Champagne-au-Mont-d'Or", '5748'), ('Cours-la-Ville', '3617'), ("Chazay-d'Azergues", '4132'), ('Arnas', '3897'), ('Charbonnières-les-Bains', '5170'), ('Condrieu', '3957'), ('Saint-Fons', '19617'), ('Vaugneray', '5623'), ('Albigny-sur-Saône', '2947'), ('Chaponnay', '4355'), ('Saint-Georges-de-Reneins', '4409'), ('Craponne', '11248'), ('Ternay', '5512'), ('Saint-Genis-Laval', '20914'), ('Saint-Pierre-de-Chandieu', '4579'), ('Thurins', '3053'), ('Messimy', '3402'), ('Chaponost', '8846'), ("Le Bois-d'Oingt", '2554'), ('Fontaines-Saint-Martin', '3033'), ('Irigny', '8750'), ('Cailloux-sur-Fontaines', '2883'), ('Dommartin', '2595'), ('Gleizé', '7496'), ('Vernaison', '4999'), ('Soucieu-en-Jarrest', '4590'), ('Saint-Symphorien-sur-Coise', '3710'), ('Vourles', '3367'), ('Toussieu', '3188'), ('Sérézin-du-Rhône', '2772'), ('La Tour-de-Salvagny', '4117'), ("Saint-Didier-au-Mont-d'Or", '7006'), ("Saint-Germain-au-Mont-d'Or", '3001'), ('Tassin-la-Demi-Lune', '22286'), ('Pusignan', '4191'), ('Saint-Bonnet-de-Mure', '6875'), ('Thizy-les-Bourgs', '6020'), ('Cours', '4390'), ('Chabanière', '4224'), ('Porte des Pierres Dorées', '3907'), ("Val d'Oingt", '4122'), ('Vaugneray', '6080'), ('Beauvallon', '4055'), ('Belleville-en-Beaujolais', '13314'), ('Vindry-sur-Turdine', '5279'), ('Amplepuis', '4919'), ('Lozanne', '2729'), ('Givors', '20285'), ('Francheville', '14906'), ('Oullins', '26553'), ('Dardilly', '8752'), ('Vaulx-en-Velin', '52795'), ('Villefranche-sur-Saône', '36291'), ('Écully', '18948'), ('Taluyers', '2590'), ('Rillieux-la-Pape', '30697'), ('Ampuis', '2749'), ('Pommiers', '2713'), ('Neuville-sur-Saône', '7562'), ('Grézieu-la-Varenne', '6062'), ('Brignais', '12097'), ('Belleville', '8198'), ('Limonest', '3648'), ('Pontcharra-sur-Turdine', '2764'), ('Brindas', '6478'), ('Lentilly', '6480'), ('Tarare', '10490'), ('Saint-Genis-les-Ollières', '5158'), ('Millery', '4318'), ('Colombier-Saugnieu', '2700'), ('Genas', '13181'), ('La Mulatière', '6524')], 'Haute-Saône': [('Vesoul', '14914'), ('Lure', '8046'), ('Gray', '5553'), ('Échenoz-la-Méline', '3249'), ('Arc-lès-Gray', '2533'), ('Luxeuil-les-Bains', '6623'), ('Héricourt', '10646'), ('Ronchamp', '2735'), ('Saint-Loup-sur-Semouse', '3150'), ('Champagney', '3745'), ('Port-sur-Saône', '2954'), ('Fougerolles', '3565'), ('Fougerolles-Saint-Valbert', '3797')], 'Saône-et-Loire': [('Paray-le-Monial', '9214'), ('Chagny', '5528'), ('Chauffailles', '3703'), ('Givry', '3684'), ('Bourbon-Lancy', '4713'), ('Mâcon', '33908'), ('Autun', '12987'), ('Cluny', '4929'), ('Chalon-sur-Saône', '45056'), ('Charolles', '2888'), ('Le Creusot', '21269'), ('Sanvignes-les-Mines', '4316'), ('Gueugnon', '6760'), ('Montceau-les-Mines', '17678'), ('Louhans', '6433'), ('Digoin', '7741'), ('Tournus', '5489'), ('Le Breuil', '3538'), ('Champforgeuil', '2588'), ('Châtenoy-le-Royal', '6148'), ('Saint-Marcel', '6190'), ('Sennecey-le-Grand', '3012'), ('La Chapelle-de-Guinchay', '4171'), ('Blanzy', '6060'), ('Crêches-sur-Saône', '3089'), ('Charnay-lès-Mâcon', '7742'), ('Ouroux-sur-Saône', '3110'), ('Montchanin', '4964'), ('Gergy', '2581'), ('Saint-Vallier', '8576'), ('Saint-Rémy', '6615'), ('Torcy', '2813')], 'Sarthe': [('Saint-Calais', '3244'), ('Changé', '6552'), ('Saint-Saturnin', '2590'), ('Mayet', '3120'), ('Mulsanne', '5225'), ('La Flèche', '14858'), ('Le Mans', '143847'), ('Écommoy', '4744'), ('Coulaines', '7779'), ('Allonnes', '11167'), ('Précigné', '2957'), ('Spay', '2881'), ('La Bazoge', '3646'), ('Teloché', '3056'), ('Le Lude', '3628'), ('Champagné', '3756'), ('Noyen-sur-Sarthe', '2624'), ('Mamers', '5157'), ('Bonnétable', '3791'), ('Arnage', '5503'), ('Saint-Mars-la-Brière', '2698'), ('Moncé-en-Belin', '3703'), ('Vibraye', '2510'), ('La Ferté-Bernard', '8862'), ('Sablé-sur-Sarthe', '12125'), ('Château-du-Loir', '4567'), ('Connerré', '2952'), ('La Milesse', '2637'), ('Montfort-le-Gesnois', '2931'), ('La Suze-sur-Sarthe', '4473'), ('Guécélard', '3092'), ("Savigné-l'Évêque", '4064'), ("Parigné-l'Évêque", '5286'), ('Sargé-lès-le-Mans', '3709'), ('Ruaudin', '3503'), ('Roëzé-sur-Sarthe', '2615'), ('Cérans-Foulletourte', '3369'), ("Yvré-l'Évêque", '4261'), ('Montval-sur-Loir', '6038'), ('Le Lude', '4126'), ('Cherré-Au', '2731'), ('Fresnay-sur-Sarthe', '2889')], 'Savoie': [('Albertville', '19502'), ('Chambéry', '58917'), ('Aix-les-Bains', '30463'), ('Saint-Jean-de-Maurienne', '7621'), ('Bourg Saint Maurice', '7195'), ('Le Bourget-du-Lac', '4966'), ('Aime', '3641'), ('Albens', '3895'), ('Modane', '3075'), ('Bassens', '4785'), ('La Biolle', '2781'), ('Saint-Martin-de-Belleville', '2640'), ('Yenne', '3002'), ('Cognin', '6359'), ('Ugine', '7096'), ('Drumettaz-Clarafond', '2934'), ("Saint-Pierre-d'Albigny", '4097'), ('Barberaz', '5022'), ('La Ravoire', '8530'), ('Barby', '3429'), ('Moûtiers', '3496'), ('Saint-Alban-Leysse', '6285'), ('Gilly-sur-Isère', '3042'), ('Saint-Baldoph', '2759'), ('Aigueblanche', '3213'), ('Mercury', '3299'), ('Jacob-Bellecombette', '4047'), ('Challes-les-Eaux', '5609'), ('Tresserve', '2988'), ('Grésy-sur-Aix', '4595'), ('La Motte-Servolex', '12120'), ('Les Marches', '2682'), ('Les Abrets', '3484'), ('Montmélian', '4102'), ('La Rochette', '3747'), ('Entrelacs', '6315'), ('Les Belleville', '3494'), ('La Plagne Tarentaise', '3766'), ('Aime-la-Plagne', '4444'), ('Porte-de-Savoie', '3796'), ('Saint-Genix-les-Villages', '2905'), ('Grand-Aigueblanche', '3838'), ('Valgelon-La Rochette', '4160'), ('La Léchère', '2635')], 'Haute-Savoie': [('Thorens-Glières', '3223'), ('Sillingy', '5365'), ('Sallanches', '16831'), ('Marnaz', '5642'), ('Saint-Martin-Bellevue', '2732'), ('Metz-Tessy', '3051'), ('Meythet', '8327'), ('Saint-Jorioz', '6008'), ('Valleiry', '4935'), ('Fillinges', '3516'), ('Archamps', '2609'), ('Lugrin', '2504'), ('Cran-Gevrier', '18330'), ('Argonay', '3370'), ('Cluses', '16918'), ('Allinges', '4681'), ('Bonne', '3181'), ('Vétraz-Monthoux', '9003'), ('Chens-sur-Léman', '2792'), ('Douvaine', '6483'), ('Pringy', '4412'), ('Publier', '7341'), ('Sevrier', '4139'), ('Beaumont', '3027'), ('La Balme-de-Sillingy', '5078'), ('Les Houches', '3007'), ('Doussard', '3542'), ('Collonges-sous-Salève', '3948'), ('Sciez', '6348'), ('Viry', '5415'), ('Bons-en-Chablais', '5739'), ('Villaz', '3347'), ('Annecy', '130721'), ('La Roche-sur-Foron', '11149'), ('Pers-Jussy', '3108'), ('Thonon-les-Bains', '35826'), ('Annemasse', '36582'), ('Marignier', '6372'), ('Megève', '3025'), ('Évian-les-Bains', '9108'), ('Épagny', '4118'), ('Étrembières', '2556'), ('Ambilly', '6138'), ('Saint-Julien-en-Genevois', '15873'), ('Taninges', '3394'), ('Passy', '11233'), ('Amancy', '2739'), ('Bonneville', '12509'), ('Morzine', '2717'), ('Magland', '3240'), ('Saint-Pierre-en-Faucigny', '7091'), ('Alby-sur-Chéran', '2576'), ('Poisy', '8233'), ('Saint-Gervais-les-Bains', '5604'), ('Annecy-le-Vieux', '21390'), ('Reignier-Ésery', '8072'), ('Cranves-Sales', '6975'), ('Cruseilles', '4502'), ('Saint-Cergues', '3671'), ('Saint-Jeoire', '3370'), ('Veigy-Foncenex', '3919'), ('Groisy', '3900'), ('Seynod', '21595'), ('Scionzier', '8932'), ('Neuvecelle', '3122'), ('Chavanod', '2683'), ('Faverges', '6943'), ('Ville-la-Grand', '9050'), ('Gaillard', '10071'), ('Thônes', '6598'), ('Thyez', '6330'), ('Rumilly', '15768'), ('Viuz-en-Sallaz', '4454'), ('Epagny Metz-Tessy', '8196'), ('Faverges-Seythenex', '7609'), ('Fillière', '9449'), ('Annecy', '54145'), ('Vallières-sur-Fier', '2562')], 'Paris': [], 'Seine-Maritime': [('Gainneville', '2526'), ('Rouen', '112321'), ('Le Havre', '168290'), ('Dieppe', '28241'), ('Eu', '6716'), ('Le Petit-Quevilly', '22000'), ('Fécamp', '18041'), ('Saint-Pierre-lès-Elbeuf', '8283'), ('Épouville', '2654'), ('Sotteville-lès-Rouen', '29068'), ('Forges-les-Eaux', '3318'), ('Boos', '3923'), ('Le Tréport', '4582'), ('Franqueville-Saint-Pierre', '6105'), ('Gournay-en-Bray', '6041'), ('Barentin', '12398'), ('Sainte-Adresse', '7317'), ('Notre-Dame-de-Bondeville', '6978'), ('Harfleur', '8349'), ('Cany-Barville', '3046'), ('Offranville', '3143'), ('Le Grand-Quevilly', '25963'), ('Neufchâtel-en-Bray', '4678'), ('Criel-sur-Mer', '2630'), ('Maromme', '10663'), ('Buchy', '2840'), ('Bolbec', '11610'), ('Yvetot', '11448'), ('Elbeuf', '16224'), ('Canteleu', '13846'), ('Bois-Guillaume-Bihorel', '21107'), ('Pavilly', '6212'), ("Gonfreville-l'Orcher", '9124'), ('Oissel', '12241'), ('Octeville-sur-Mer', '6018'), ('Darnétal', '9788'), ("Criquetot-l'Esneval", '2562'), ('Déville-lès-Rouen', '10654'), ('Goderville', '2864'), ('Amfreville-la-Mi-Voie', '3306'), ('Bonsecours', '6479'), ('Bihorel', '8264'), ('Saint-Jacques-sur-Darnétal', '3045'), ('Saint-Léger-du-Bourg-Denis', '3599'), ("Saint-Nicolas-d'Aliermont", '3719'), ('Saint-Romain-de-Colbosc', '4259'), ('Yerville', '2514'), ('Saint-Aubin-lès-Elbeuf', '8429'), ('Saint-Valery-en-Caux', '3928'), ('Grand-Couronne', '9739'), ('Lillebonne', '8797'), ('Montivilliers', '15470'), ('Houppeville', '2907'), ('Mont-Saint-Aignan', '19357'), ('Saint-Étienne-du-Rouvray', '28352'), ('Arques-la-Bataille', '2573'), ('Blangy-sur-Bresle', '2907'), ('Caudebec-lès-Elbeuf', '9949'), ('Cléon', '4924'), ('Petit-Couronne', '8693'), ('Montville', '4563'), ('Quincampoix', '3097'), ('Notre-Dame-de-Gravenchon', '8744'), ('Duclair', '4073'), ('Malaunay', '6143'), ('Le Houlme', '4107'), ('Le Mesnil-Esnard', '7886'), ('Isneauville', '3546'), ('Le Trait', '4876'), ('Gruchet-le-Valasse', '3175'), ('Fontaine-la-Mallet', '2633'), ('Rouelles', '3066'), ('Bois-Guillaume', '14592'), ('Petit-Caux', '9628'), ('Port-Jérôme-sur-Seine', '10248'), ('Arelaune-en-Seine', '2522'), ('Rives-en-Seine', '4148'), ('Terres-de-Caux', '4193'), ('Forges-les-Eaux', '3812'), ('Val-de-Scie', '2535'), ('Neuville-lès-Dieppe', '9616')], 'Seine-et-Marne': [('Mouroux', '5654'), ('Trilport', '5045'), ('Lognes', '14353'), ('La Grande-Paroisse', '2833'), ('Chauconin-Neufmontiers', '3554'), ('Souppes-sur-Loing', '5383'), ('Nangis', '8867'), ('Émerainville', '7659'), ('Servon', '3288'), ('La Rochette', '3778'), ('Saint-Fargeau-Ponthierry', '14121'), ('Héricy', '2545'), ('Bois-le-Roi', '5913'), ('Chartrettes', '2543'), ('Champagne-sur-Seine', '6353'), ('Bourron-Marlotte', '2763'), ('Chelles', '55154'), ('Serris', '9494'), ('Ozoir-la-Ferrière', '20532'), ('Pringy', '2954'), ('Combs-la-Ville', '21627'), ('Annet-sur-Marne', '3281'), ('Évry-Grégy-sur-Yerre', '3025'), ('Nemours', '13088'), ('Savigny-le-Temple', '29987'), ('Avon', '13419'), ('Coupvray', '2871'), ('Grisy-Suisnes', '2532'), ('Gretz-Armainvilliers', '8476'), ('Jouarre', '4349'), ('Crécy-la-Chapelle', '4713'), ('Moissy-Cramayel', '17849'), ('Moret-sur-Loing', '4188'), ('Fontenay-Trésigny', '5710'), ('Château-Landon', '2947'), ('Lésigny', '7151'), ('Thorigny-sur-Marne', '10454'), ('Vert-Saint-Denis', '8238'), ('Thomery', '3420'), ('Saint-Pathus', '6110'), ('Moussy-le-Neuf', '3160'), ('Noisiel', '15960'), ('Rubelles', '2831'), ('Esbly', '6404'), ('La Ferté-sous-Jouarre', '9637'), ('Pomponne', '4183'), ('Guérard', '2621'), ('Mitry-Mory', '20648'), ('Roissy-en-Brie', '22851'), ('Le Mée-sur-Seine', '20917'), ('Chessy', '6304'), ('Varennes-sur-Seine', '3601'), ('Lagny-sur-Marne', '21539'), ('Torcy', '22030'), ('Melun', '40844'), ('Fontainebleau', '15696'), ('Provins', '12132'), ('Meaux', '55750'), ('Cesson', '10833'), ('Montereau-Fault-Yonne', '21229'), ('Champs-sur-Marne', '25654'), ('Vaux-le-Pénil', '11101'), ('Donnemarie-Dontilly', '2775'), ('Saint-Mard', '3875'), ('Saint-Soupplets', '3224'), ('Guignes', '4296'), ('Dammarie-les-Lys', '22128'), ('Vernou-la-Celle-sur-Seine', '2643'), ('Brie-Comte-Robert', '18812'), ('Bailly-Romainvilliers', '7416'), ('Dammartin-en-Goële', '10355'), ('Brou-sur-Chantereine', '4809'), ('Coulommiers', '14779'), ('Pommeuse', '2966'), ('Tournan-en-Brie', '8521'), ('Bussy-Saint-Georges', '26597'), ('Lizy-sur-Ourcq', '3533'), ('Nandy', '6235'), ('Claye-Souilly', '12172'), ('Le Châtelet-en-Brie', '4339'), ('Collégien', '3359'), ('Boissise-le-Roi', '3739'), ('Boissy-le-Châtel', '3216'), ('Crégy-lès-Meaux', '5223'), ('Lieusaint', '13853'), ("Verneuil-l'Étang", '3204'), ('Ferrières-en-Brie', '3768'), ('Veneux-les-Sablons', '4696'), ('Faremoutiers', '2944'), ('Saint-Pierre-lès-Nemours', '5394'), ('Montévrain', '13517'), ('Villeparisis', '26580'), ('Chanteloup-en-Brie', '3991'), ('La Ferté-Gaucher', '4933'), ('Magny-le-Hongre', '9107'), ('Vulaines-sur-Seine', '2748'), ('Courtry', '6698'), ('Dampmart', '3417'), ('Chevry-Cossigny', '3927'), ('Vaires-sur-Marne', '13368'), ('Pontault-Combault', '37617'), ('Saint-Germain-sur-Morin', '3770'), ('Chaumes-en-Brie', '3339'), ('Villenoy', '4975'), ('Rozay-en-Brie', '2808'), ('Saint-Mammès', '3391'), ('Quincy-Voisins', '5439'), ('Saint-Thibault-des-Vignes', '6354'), ('Saint-Germain-Laval', '2707'), ('Othis', '6725'), ('Nanteuil-lès-Meaux', '6225'), ('Mormant', '5014'), ('Montry', '3705'), ('Montigny-sur-Loing', '2660'), ('Mareuil-lès-Meaux', '3231'), ('Orvanne', '6795'), ('Moret Loing et Orvanne', '4355'), ('Moret-Loing-et-Orvanne', '12223')], 'Yvelines': [('Le Chesnay-Rocquencourt', '31233'), ('Saint-Germain-en-Laye', '44806'), ("L'Étang-la-Ville", '4453'), ('Maule', '5881'), ('Le Mesnil-le-Roi', '6285'), ('Mareil-Marly', '3677'), ('Mantes-la-Ville', '20818'), ('Juziers', '3843'), ('Louveciennes', '7236'), ('Issou', '3893'), ('Magnanville', '6177'), ('Saint-Rémy-lès-Chevreuse', '7806'), ('Neauphle-le-Château', '3385'), ('Montesson', '14991'), ('Orgeval', '6664'), ('Noisy-le-Roi', '7635'), ('Rocquencourt', '3623'), ('Andrésy', '13302'), ('Le Perray-en-Yvelines', '6597'), ('Rosny-sur-Seine', '6883'), ('Morainvilliers', '3023'), ('Le Port-Marly', '5481'), ('Mézières-sur-Seine', '3683'), ('Poissy', '39187'), ('Versailles', '84808'), ('Magny-les-Hameaux', '9418'), ("Bois-d'Arcy", '15325'), ('Beynes', '7618'), ('Viroflay', '16752'), ('Bailly', '3733'), ('Voisins-le-Bretonneux', '10835'), ('Villiers-Saint-Frédéric', '2906'), ('La Verrière', '6386'), ('Le Mesnil-Saint-Denis', '6777'), ('Maurepas', '17683'), ('Les Essarts-le-Roi', '6668'), ("Saint-Cyr-l'École", '19792'), ('Saint-Germain-en-Laye', '40806'), ('Rambouillet', '27141'), ('Mantes-la-Jolie', '43921'), ('Sartrouville', '52774'), ('Conflans-Sainte-Honorine', '35536'), ('Le Chesnay', '27683'), ('Guyancourt', '29471'), ('Élancourt', '25624'), ('Chatou', '30153'), ('Montigny-le-Bretonneux', '32282'), ('Fontenay-le-Fleury', '13466'), ('Triel-sur-Seine', '12250'), ('Saint-Nom-la-Bretèche', '4920'), ('Épône', '6561'), ('Porcheville', '3174'), ('Achères', '21529'), ("Montfort-l'Amaury", '2935'), ('Jouars-Pontchartrain', '5772'), ('Buc', '6027'), ('Vaux-sur-Seine', '5010'), ('Maurecourt', '4414'), ('Buchelay', '3306'), ('Feucherolles', '2983'), ('Houilles', '32801'), ('Ecquevilly', '4150'), ('Le Pecq', '15716'), ('Meulan-en-Yvelines', '9005'), ('Chambourcy', '5587'), ('Les Mureaux', '33203'), ('Ablis', '3490'), ('Trappes', '32645'), ('La Celle-Saint-Cloud', '20692'), ('Plaisir', '30950'), ('Jouy-en-Josas', '8049'), ('Vélizy-Villacoublay', '23092'), ('Houdan', '3671'), ('Bougival', '8790'), ('Chevreuse', '5610'), ('Chanteloup-les-Vignes', '10426'), ('Bonnières-sur-Seine', '4791'), ('Carrières-sous-Poissy', '16641'), ('Carrières-sur-Seine', '15178'), ('Croissy-sur-Seine', '10031'), ('Villepreux', '11003'), ('Villennes-sur-Seine', '5292'), ('Les Clayes-sous-Bois', '17487'), ('Coignières', '4355'), ('Limay', '17136'), ('Fourqueux', '3944'), ('Freneuse', '4266'), ('Vernouillet', '10104'), ('Gargenville', '7893'), ('Le Vésinet', '15943'), ('Verneuil-sur-Seine', '15980'), ('Marly-le-Roi', '16385'), ('Saint-Arnoult-en-Yvelines', '5818'), ('Aubergenville', '12152')], 'Deux-Sèvres': [('Niort', '59193'), ('Bressuire', '19850'), ('Parthenay', '10089'), ('Échiré', '3441'), ('Thouars', '13886'), ('Airvault', '3270'), ('Melle', '3525'), ("Saint-Maixent-l'École", '7242'), ('Mauléon', '8578'), ('Moncoutant', '3161'), ('Nueil-les-Aubiers', '5536'), ('Aiffres', '5396'), ('Argentonnay', '3183'), ('Mougon-Thorigné', '3417'), ('Aigondigné', '4677'), ("Loretz-d'Argenton", '2620'), ('Melle', '6173'), ('Moncoutant-sur-Sèvre', '5057'), ('Terves', '2617'), ('Chauray', '7083'), ('Celles-sur-Belle', '3907'), ('La Crèche', '5820'), ('Cerizay', '4764'), ('Châtillon-sur-Thouet', '2678'), ('Vouillé', '3345'), ('Frontenay-Rohan-Rohan', '2899'), ('Magné', '2693'), ('Mauzé-sur-le-Mignon', '2891'), ('Thouars', '9225'), ('Thouars', '9302'), ('Thouars', '9475')], 'Somme': [('Doullens', '5890'), ('Corbie', '6292'), ('Montdidier', '6113'), ('Rosières-en-Santerre', '3017'), ('Moreuil', '3986'), ('Abbeville', '22980'), ('Ailly-sur-Somme', '2952'), ('Ailly-sur-Noye', '2833'), ('Albert', '9779'), ('Amiens', '134706'), ('Saleux', '2805'), ('Ham', '4559'), ('Rue', '3111'), ('Friville-Escarbotin', '4491'), ('Flixecourt', '3237'), ('Péronne', '7577'), ('Longueau', '5765'), ('Mers-les-Bains', '2655'), ('Rivery', '3599'), ('Boves', '3281'), ('Villers-Bretonneux', '4592'), ('Camon', '4436'), ('Roye', '5693'), ('Salouël', '3918')], 'Tarn': [('Puygouzon', '3510'), ('Saint-Juéry', '6694'), ('Puygouzon', '3499'), ("Lescure-d'Albigeois", '4574'), ('Rabastens', '5713'), ('Soual', '2622'), ('Coufouleux', '2939'), ('Aiguefonde', '2512'), ('Réalmont', '3480'), ('Albi', '48902'), ('Castres', '42079'), ('Gaillac', '15265'), ('Blaye-les-Mines', '2946'), ('Cagnac-les-Mines', '2578'), ('Lavaur', '10879'), ('Arthès', '2512'), ('Sorèze', '2903'), ('Mazamet', '9996'), ('Carmaux', '9782'), ('Puylaurens', '3160'), ('Aussillon', '5827'), ('Lisle-sur-Tarn', '4676'), ('Pont-de-Larn', '2775'), ('Saint-Sulpice-la-Pointe', '9336'), ('Labruguière', '6506'), ('Graulhet', '12844'), ('Saïx', '3602'), ('Marssac-sur-Tarn', '3395')], 'Tarn-et-Garonne': [('La Ville-Dieu-du-Temple', '3148'), ('Labastide-Saint-Pierre', '3783'), ('Verdun-sur-Garonne', '4808'), ('Saint-Étienne-de-Tulmont', '3901'), ('Valence', '5205'), ('Montauban', '61372'), ('Castelsarrasin', '14114'), ('Moissac', '13378'), ('Lafrançaise', '2835'), ('Grisolles', '4177'), ('Nègrepelisse', '5685'), ('Montbeton', '4241'), ('Beaumont-de-Lomagne', '3773'), ('Caussade', '6808'), ('Montech', '6589'), ('Albias', '3294'), ('Bressols', '3697')], 'Var': [('Rocbaron', '5250'), ('Solliès-Pont', '11762'), ('Seillans', '2669'), ('Le Luc', '11094'), ('Callian', '3341'), ('La Farlède', '9614'), ('Rians', '4278'), ('Le Muy', '9288'), ('Le Castellet', '3873'), ('Trans-en-Provence', '5947'), ('Pignans', '4588'), ('Néoules', '2828'), ('Montauroux', '6548'), ('Saint-Zacharie', '6051'), ('Tourves', '5042'), ('Le Revest-les-Eaux', '3962'), ('Bras', '2629'), ('Carqueiranne', '9518'), ('Pierrefeu-du-Var', '6068'), ('Le Val', '4292'), ("La Cadière-d'Azur", '5574'), ('Pourrières', '5181'), ('Ollioules', '13866'), ('Salernes', '3761'), ('Flassans-sur-Issole', '3564'), ('Le Thoronet', '2552'), ('Saint-Maximin-la-Sainte-Baume', '17095'), ('Flayosc', '4247'), ('Saint-Mandrier-sur-Mer', '6095'), ('Figanières', '2623'), ('Saint-Tropez', '3851'), ('Toulon', '178745'), ('La Croix-Valmer', '3779'), ('Vidauban', '12616'), ('Hyères', '54821'), ('Draguignan', '39433'), ('Tourrettes', '2877'), ('Brignoles', '17361'), ('La Seyne-sur-Mer', '62987'), ('Fréjus', '55750'), ('Vinon-sur-Verdon', '4248'), ('Saint-Raphaël', '36027'), ('La Garde', '25505'), ('La Londe-les-Maures', '10641'), ('La Valette-du-Var', '24087'), ('Six-Fours-les-Plages', '34592'), ('La Motte', '2804'), ('Garéoult', '5303'), ('Bandol', '8403'), ('Roquebrune-sur-Argens', '14937'), ('Cavalaire-sur-Mer', '7586'), ('Cuers', '11962'), ('Nans-les-Pins', '4835'), ('Gonfaron', '4329'), ('Le Pradet', '10277'), ('Les Arcs', '7066'), ('La Crau', '18774'), ('Cogolin', '11311'), ('Barjols', '2990'), ('Saint-Cyr-sur-Mer', '11484'), ('Sanary-sur-Mer', '16889'), ('Puget-sur-Argens', '8062'), ('Carcès', '3380'), ('Fayence', '5735'), ("Les Adrets-de-l'Estérel", '2784'), ('Besse-sur-Issole', '3038'), ('Le Cannet-des-Maures', '4279'), ('Bagnols-en-Forêt', '2862'), ('Le Beausset', '9845'), ('Sainte-Maxime', '14448'), ('Carnoules', '3525'), ('Forcalqueiret', '3101'), ('Lorgues', '9054'), ('Bormes-les-Mimosas', '8162'), ('Régusse', '2642'), ('Le Lavandou', '5985'), ('Grimaud', '4562'), ('Gassin', '2614'), ('Signes', '2927'), ('Solliès-Ville', '2526'), ('Solliès-Toucas', '5753'), ('Le Plan-de-la-Tour', '3048'), ('Puget-Ville', '4357')], 'Vaucluse': [("La Tour-d'Aigues", '4334'), ('Lauris', '3879'), ('Jonquières', '5469'), ('Aubignan', '5778'), ('Entraigues-sur-la-Sorgue', '8595'), ('Velleron', '2976'), ('Caromb', '3399'), ('Le Thor', '8940'), ('Loriol-du-Comtat', '2520'), ('Saint-Saturnin-lès-Apt', '2877'), ('Gargas', '3078'), ('Sainte-Cécile-les-Vignes', '2591'), ('Lapalud', '3830'), ('Sérignan-du-Comtat', '2853'), ('Avignon', '91143'), ('Orange', '28772'), ('Carpentras', '29236'), ('Apt', '11037'), ('Cavaillon', '26236'), ('Le Pontet', '17041'), ('Cadenet', '4201'), ('Althen-des-Paluds', '2851'), ('Bollène', '13439'), ('Bédarrides', '5316'), ('Châteauneuf-de-Gadagne', '3321'), ('Courthézon', '6026'), ('Bédoin', '3093'), ('Caumont-sur-Durance', '4914'), ('Camaret-sur-Aigues', '4542'), ('Mondragon', '3736'), ('Mazan', '6201'), ('Pertuis', '20557'), ('Caderousse', '2651'), ('Cheval-Blanc', '4290'), ('Robion', '4631'), ('Sarrians', '6128'), ('Morières-lès-Avignon', '8786'), ('Monteux', '12935'), ('Saint-Saturnin-lès-Avignon', '4935'), ('Pernes-les-Fontaines', '10170'), ('Valréas', '9403'), ("L'Isle-sur-la-Sorgue", '20042'), ('Vaison-la-Romaine', '5908'), ('Vedène', '11383'), ('Piolenc', '5394'), ('Sorgues', '18893'), ('Malaucène', '2785'), ('Villelaure', '3356')], 'Vendée': [("L'Île-d'Olonne", '2783'), ('Nieul-le-Dolent', '2549'), ('Pouzauges', '5557'), ('Talmont-Saint-Hilaire', '7831'), ('Saint-Gervais', '2697'), ('Sainte-Hermine', '2940'), ('Beauvoir-sur-Mer', '3880'), ('Bretignolles-sur-Mer', '5066'), ('Le Boupère', '3227'), ("Château-d'Olonne", '14411'), ('Chantonnay', '8394'), ('Bournezeau', '3403'), ('Dompierre-sur-Yon', '4344'), ('Aubigny', '3881'), ('La Chaize-le-Vicomte', '3799'), ("L'Herbergement", '3297'), ('Les Clouzeaux', '2831'), ('Coëx', '3285'), ('Commequiers', '3605'), ('Angles', '2878'), ('Les Brouzils', '2816'), ('Essarts en Bocage', '9174'), ('Bellevigny', '6095'), ('Aubigny-Les Clouzeaux', '6880'), ('Montréverd', '3761'), ('Sèvremont', '6423'), ("Rives de l'Yon", '4217'), ('Les Achards', '5271'), ('Montaigu-Vendée', '20424'), ("Les Sables-d'Olonne", '45030'), ('Chanverrie', '5453'), ("L'Île-d'Yeu", '4850'), ('Challans', '21322'), ('Luçon', '9554'), ('La Roche-sur-Yon', '55147'), ("Les Sables-d'Olonne", '14545'), ('Fontenay-le-Comte', '13235'), ('Brem-sur-Mer', '2817'), ('La Guyonnière', '2753'), ('Les Epesses', '2914'), ('La Bruffière', '4028'), ('Aizenay', '9881'), ('La Gaubretière', '3111'), ('Belleville-sur-Vie', '3995'), ('Les Essarts', '5586'), ('La Garnache', '5114'), ('Sallertaine', '3171'), ('La Tranche-sur-Mer', '2904'), ('Jard-sur-Mer', '2665'), ('Les Herbiers', '16250'), ('Boufféré', '3430'), ('Cugand', '3563'), ('Venansault', '4628'), ('Rocheservière', '3400'), ('Olonne-sur-Mer', '15061'), ('Le Poiré-sur-Vie', '8596'), ("Noirmoutier-en-l'Île", '4550'), ('Chavagnes-en-Paillers', '3552'), ('Benet', '4014'), ('La Châtaigneraie', '2586'), ('La Flocellière', '2634'), ('Saint-Christophe-du-Ligneron', '2586'), ('Saint-Gilles-Croix-de-Vie', '7862'), ('Mouchamps', '2922'), ('Montaigu', '5160'), ('La Mothe-Achard', '3108'), ('Mareuil-sur-Lay-Dissais', '2792'), ('La Ferrière', '5315'), ('Saint-Jean-de-Monts', '8696'), ('La Verrie', '3922'), ("L'Aiguillon-la-Presqu'île", '2726'), ('Le Fenouiller', '4854'), ('Saint-Laurent-sur-Sèvre', '3689'), ('Mouilleron-le-Captif', '4971'), ('Saint-Hilaire-de-Loulay', '4577'), ('Saint-Florent-des-Bois', '2833'), ('Saint-Georges-de-Montaigu', '4309'), ('Nesmy', '2912'), ('Mortagne-sur-Sèvre', '6059'), ('Soullans', '4259'), ('Saint-Fulgent', '3814'), ('Treize-Septiers', '3280'), ('Les Lucs-sur-Boulogne', '3522'), ('Saint-Philbert-de-Bouaine', '3535'), ('Saint-Hilaire-de-Riez', '11297')], 'Vienne': [('Montamisé', '3677'), ('Jaunay-Marigny', '7600'), ('Beaumont Saint-Cyr', '2958'), ('Saint-Martin-la-Pallu', '5578'), ('Migné-Auxances', '6154'), ('Smarves', '2892'), ('Dangé-Saint-Romain', '2969'), ('Dissay', '3255'), ('Buxerolles', '10060'), ('Cissé', '2835'), ('Poitiers', '89212'), ('Châtellerault', '31487'), ('Montmorillon', '5940'), ('Vouneuil-sous-Biard', '6060'), ('Lusignan', '2659'), ('Nouaillé-Maupertuis', '2847'), ('Chauvigny', '7062'), ('Iteuil', '2966'), ('Neuville-de-Poitou', '5359'), ('Vendeuvre-du-Poitou', '3214'), ('Civray', '2596'), ('Vouillé', '3683'), ('Naintré', '5941'), ('Loudun', '6712'), ('Chasseneuil-du-Poitou', '4692'), ('Thuré', '2858'), ('Mignaloux-Beauvoir', '5026'), ('Jaunay-Clan', '6278'), ("Nieuil-l'Espoir", '2757'), ('Fontaine-le-Comte', '3934'), ('Saint-Benoît', '7267'), ('Ligugé', '3374'), ('Saint-Georges-lès-Baillargeaux', '4233'), ("Saint-Julien-l'Ars", '2779'), ('Valdivienne', '2749'), ('Vivonne', '4409'), ('Boivre-la-Vallée', '3073'), ('Valence-en-Poitou', '4455'), ('Saint Martin la Pallu', '5205')], 'Haute-Vienne': [('Rilhac-Rancon', '4608'), ('Limoges', '130876'), ('Bessines-sur-Gartempe', '2803'), ('Nexon', '2509'), ('Verneuil-sur-Vienne', '4881'), ('Condat-sur-Vienne', '5128'), ('Saint-Léonard-de-Noblat', '4385'), ('Bellac', '3639'), ('Aixe-sur-Vienne', '5804'), ('Le Palais-sur-Vienne', '6023'), ('Ambazac', '5593'), ('Rochechouart', '3767'), ('Saint-Junien', '11254'), ('Saint-Yrieix-la-Perche', '6718'), ('Feytiat', '6118'), ('Isle', '7847'), ('Boisseuil', '2926'), ("Bosmie-l'Aiguille", '2616'), ('Couzeix', '9518'), ('Saint-Priest-Taurion', '2888'), ('Panazol', '10900'), ('Saint-Just-le-Martel', '2670')], 'Vosges': [('Contrexéville', '3190'), ('Mirecourt', '4949'), ("Raon-l'Étape", '6205'), ('La Broque', '2629'), ('Saint-Dié-des-Vosges', '19576'), ('Épinal', '32256'), ('Saint-Nabord', '3994'), ('Rupt-sur-Moselle', '3529'), ("Le Val-d'Ajol", '3864'), ('Golbey', '8798'), ('Le Thillot', '3376'), ('Saulxures-sur-Moselotte', '2517'), ('Bruyères', '3073'), ('Saint-Étienne-lès-Remiremont', '3847'), ('Charmes', '4690'), ('La Bresse', '4143'), ('Cornimont', '3111'), ('Thaon-les-Vosges', '7553'), ('Chantraine', '3197'), ('Vagney', '3911'), ('Fraize', '2841'), ('Neufchâteau', '6636'), ('Moyenmoutier', '3089'), ('Étival-Clairefontaine', '2610'), ('Anould', '3368'), ('Éloyes', '3122'), ('Vittel', '4832'), ('Gérardmer', '7807'), ('Rambervillers', '5096'), ('Xertigny', '2586'), ('Remiremont', '7691'), ('Granges-Aumontzey', '2613'), ('Thaon-les-Vosges', '8634')], 'Yonne': [('Monéteau', '4125'), ('Migennes', '7258'), ('Joigny', '9557'), ('Tonnerre', '4381'), ('Brienon-sur-Armançon', '3192'), ('Pont-sur-Yonne', '3316'), ('Villeneuve-la-Guyard', '3515'), ('Toucy', '2615'), ('Saint-Florentin', '4240'), ('Villeneuve-sur-Yonne', '5165'), ('Saint-Georges-sur-Baulche', '3225'), ('Paron', '4828'), ('Saint-Clément', '2740'), ('Auxerre', '34451'), ('Appoigny', '3245'), ('Avallon', '6462'), ('Sens', '26688'), ('Charny Orée de Puisaye', '4815'), ('Montholon', '2856')], 'Territoire de Belfort': [('Valdoie', '5269'), ('Offemont', '4219'), ('Belfort', '46443'), ('Beaucourt', '5010'), ('Châtenois-les-Forges', '2667'), ('Delle', '5667'), ('Grandvillars', '3003'), ('Bavilliers', '4659'), ('Giromagny', '2929'), ('Danjoutin', '3620'), ('Essert', '3321')], 'Essonne': [('Bièvres', '4823'), ('Évry', '53641'), ('Itteville', '6522'), ('Palaiseau', '35236'), ('Étampes', '25629'), ('Corbeil-Essonnes', '51234'), ('Gif-sur-Yvette', '21007'), ('Brunoy', '25330'), ('Les Ulis', '25024'), ('Longjumeau', '21269'), ('Sainte-Geneviève-des-Bois', '35830'), ('Grigny', '28201'), ('Draveil', '28602'), ('Montgeron', '23760'), ('Savigny-sur-Orge', '36577'), ('Athis-Mons', '35670'), ('Forges-les-Bains', '3923'), ('Chilly-Mazarin', '19822'), ('Ollainville', '4759'), ('Longpont-sur-Orge', '6375'), ('Briis-sous-Forges', '3342'), ('Lisses', '7333'), ('Ballainvilliers', '4600'), ('Saulx-les-Chartreux', '6416'), ('Breuillet', '8566'), ('Ballancourt-sur-Essonne', '7593'), ('Montlhéry', '7681'), ('Cerny', '3337'), ('Viry-Châtillon', '30865'), ('Méréville', '3059'), ('Saclay', '4256'), ('Wissous', '7133'), ('Vert-le-Petit', '2818'), ('Villiers-sur-Orge', '4600'), ('Linas', '6842'), ('Le Plessis-Pâté', '4140'), ('Saint-Germain-lès-Arpajon', '10983'), ('Ris-Orangis', '29745'), ('Massy', '50644'), ('Saint-Michel-sur-Orge', '20484'), ('Saint-Germain-lès-Corbeil', '7488'), ('Crosne', '9369'), ('Nozay', '4511'), ('Villabé', '5487'), ('Étréchy', '6729'), ('Yerres', '29318'), ('Orsay', '15503'), ('Marcoussis', '8138'), ('Bures-sur-Yvette', '9408'), ('Angerville', '4353'), ('Boutigny-sur-Essonne', '2963'), ('Milly-la-Forêt', '4635'), ('Évry-Courcouronnes', '66851'), ('Le Mérévillois', '3305'), ('Champlan', '2778'), ('Maisse', '2780'), ('Égly', '6322'), ('Mennecy', '15292'), ('Épinay-sous-Sénart', '12266'), ('Le Coudray-Montceaux', '4844'), ('Saintry-sur-Seine', '5797'), ('La Norville', '4331'), ('Igny', '9917'), ('Lardy', '5483'), ('Morangis', '13424'), ('Villemoisson-sur-Orge', '7058'), ('Marolles-en-Hurepoix', '5610'), ('Soisy-sur-Seine', '7213'), ('Saint-Chéron', '5238'), ('Morigny-Champigny', '4346'), ('Dourdan', '10452'), ('Arpajon', '11355'), ('Épinay-sur-Orge', '11184'), ('Leuville-sur-Orge', '4378'), ('Boussy-Saint-Antoine', '7983'), ('Gometz-le-Châtel', '2664'), ('Bondoufle', '10046'), ('Verrières-le-Buisson', '14888'), ('Paray-Vieille-Poste', '7814'), ('Quincy-sous-Sénart', '9426'), ('Limours', '6492'), ('Juvisy-sur-Orge', '17454'), ('Saint-Vrain', '3059'), ('La Ferté-Alais', '3659'), ('Villebon-sur-Yvette', '10466'), ('Fleury-Mérogis', '13641'), ('La Ville-du-Bois', '8010'), ('Brétigny-sur-Orge', '27412'), ('Vigneux-sur-Seine', '31394'), ('Boissy-sous-Saint-Yon', '3828'), ('Saint-Pierre-du-Perray', '11220'), ('Morsang-sur-Orge', '20124'), ('Tigery', '4294'), ('Étiolles', '3144'), ('Bruyères-le-Châtel', '3497'), ('Courcouronnes', '13490'), ('Champcueil', '2867')], 'Hauts-de-Seine': [('Neuilly-sur-Seine', '59599'), ('Issy-les-Moulineaux', '67981'), ('Bagneux', '40936'), ('Levallois-Perret', '66082'), ('Nanterre', '96277'), ('Boulogne-Billancourt', '121583'), ('Colombes', '86534'), ('Saint-Cloud', '30012'), ('Courbevoie', '81558'), ('Asnières-sur-Seine', '87143'), ('Rueil-Malmaison', '78317'), ('Suresnes', '49311'), ('Clamart', '52925'), ('Châtenay-Malabry', '34021'), ('Clichy', '63089'), ('Antony', '62760'), ('Sèvres', '23463'), ('Bois-Colombes', '28841'), ('Châtillon', '36639'), ('Gennevilliers', '48530'), ('Villeneuve-la-Garenne', '24097'), ('Vanves', '27846'), ('Meudon', '45818'), ('Puteaux', '45157'), ('La Garenne-Colombes', '29642'), ('Montrouge', '48352'), ('Garches', '17795'), ('Bourg-la-Reine', '21050'), ('Malakoff', '30950'), ('Le Plessis-Robinson', '30061'), ('Fontenay-aux-Roses', '25531'), ('Sceaux', '20004'), ("Ville-d'Avray", '11225'), ('Chaville', '20771'), ('Vaucresson', '8683')], 'Seine-Saint-Denis': [('Bobigny', '54363'), ('Saint-Denis', '112852'), ('Le Pré-Saint-Gervais', '17351'), ('Gagny', '39661'), ('Livry-Gargan', '45692'), ('Aulnay-sous-Bois', '86969'), ('Montreuil', '111240'), ('Drancy', '72376'), ('Aubervilliers', '88948'), ('Clichy-sous-Bois', '28782'), ('Épinay-sur-Seine', '54611'), ('Noisy-le-Grand', '67871'), ('Le Blanc-Mesnil', '57498'), ('Saint-Ouen-sur-Seine', '50697'), ('Bagnolet', '36060'), ('Pantin', '59846'), ('Neuilly-sur-Marne', '35907'), ('Montfermeil', '27928'), ('Bondy', '54587'), ('Villepinte', '37713'), ('Tremblay-en-France', '36461'), ('Villemomble', '30405'), ('La Courneuve', '45053'), ('Gournay-sur-Marne', '6844'), ('Pierrefitte-sur-Seine', '30861'), ('Les Lilas', '23447'), ('Stains', '38285'), ('Le Raincy', '14569'), ('Le Bourget', '15905'), ('Coubron', '4921'), ('Villetaneuse', '13574'), ('Rosny-sous-Bois', '46024'), ('Vaujours', '7181'), ('Noisy-le-Sec', '44463'), ('Romainville', '30087'), ('Sevran', '51734'), ('Les Pavillons-sous-Bois', '23898'), ('Neuilly-Plaisance', '20878'), ("L'Île-Saint-Denis", '8288'), ('Dugny', '11073')], 'Val-de-Marne': [('Saint-Mandé', '22377'), ('Créteil', '93246'), ('Vitry-sur-Seine', '95510'), ('Villejuif', '55208'), ("L'Haÿ-les-Roses", '32071'), ('Le Kremlin-Bicêtre', '24971'), ('Champigny-sur-Marne', '76990'), ('Saint-Maur-des-Fossés', '74976'), ('Vincennes', '49788'), ('Ivry-sur-Seine', '63748'), ('Fontenay-sous-Bois', '52008'), ('Maisons-Alfort', '56483'), ('Cachan', '30440'), ('Nogent-sur-Marne', '34042'), ('Charenton-le-Pont', '29882'), ('Choisy-le-Roi', '46150'), ('Alfortville', '44805'), ('Orly', '24627'), ('Santeny', '4014'), ('Valenton', '14883'), ('Ormesson-sur-Marne', '10419'), ('Villeneuve-Saint-Georges', '34890'), ('La Queue-en-Brie', '12051'), ('Villiers-sur-Marne', '28895'), ('Chevilly-Larue', '19988'), ('Le Plessis-Trévise', '19829'), ('Fresnes', '28710'), ('Saint-Maurice', '14189'), ('Thiais', '30676'), ('Sucy-en-Brie', '27157'), ('Bry-sur-Marne', '17525'), ('Arcueil', '21788'), ('Rungis', '5657'), ('Limeil-Brévannes', '27833'), ('Joinville-le-Pont', '19652'), ('Périgny', '2688'), ('Gentilly', '18815'), ('Villeneuve-le-Roi', '21679'), ('Bonneuil-sur-Marne', '18062'), ('Boissy-Saint-Léger', '16977'), ('Villecresnes', '11779'), ('Mandres-les-Roses', '4777'), ('Marolles-en-Brie', '4783'), ('Le Perreux-sur-Marne', '33588'), ('Ablon-sur-Seine', '5852'), ('Chennevières-sur-Marne', '18039'), ('Noiseau', '4607')], "Val-d'Oise": [('Jouy-le-Moutier', '16741'), ('Groslay', '8509'), ('Montigny-lès-Cormeilles', '21528'), ('Saint-Martin-du-Tertre', '2697'), ('Le Thillay', '4567'), ('Survilliers', '4213'), ('Viarmes', '5194'), ('Roissy-en-France', '2833'), ('Bernes-sur-Oise', '2733'), ('Soisy-sous-Montmorency', '18033'), ('Cormeilles-en-Parisis', '25406'), ('Louvres', '11037'), ('Méry-sur-Oise', '9865'), ("L'Isle-Adam", '12063'), ('Persan', '13734'), ('Champagne-sur-Oise', '5018'), ("Saint-Ouen-l'Aumône", '24674'), ('Andilly', '2574'), ('Asnières-sur-Oise', '2939'), ('Bouffémont', '6578'), ('Domont', '15504'), ('Bruyères-sur-Oise', '4347'), ('Marly-la-Ville', '5663'), ('Bessancourt', '7878'), ('Vauréal', '16456'), ('Parmain', '5589'), ('Pierrelaye', '9423'), ('Mériel', '5168'), ('Sarcelles', '59196'), ('Luzarches', '4733'), ('Magny-en-Vexin', '5846'), ('Argenteuil', '111038'), ('Cergy', '65911'), ('Pontoise', '32405'), ('Auvers-sur-Oise', '6813'), ('Villiers-le-Bel', '28423'), ('Franconville', '37394'), ('Bezons', '31671'), ('Ermont', '28939'), ('Eaubonne', '25335'), ('Goussainville', '31068'), ('Ézanville', '9656'), ('Beaumont-sur-Oise', '9504'), ('Fosses', '9791'), ('Garges-lès-Gonesse', '43239'), ('Vémars', '2699'), ('Le Plessis-Bouchard', '8514'), ('Arnouville', '14255'), ('Courdimanche', '6709'), ('Montmagny', '14523'), ('Enghien-les-Bains', '11470'), ('Sannois', '26565'), ('Saint-Leu-la-Forêt', '16130'), ('La Frette-sur-Seine', '4701'), ('Saint-Prix', '7036'), ('Deuil-la-Barre', '22149'), ('Gonesse', '26106'), ('Marines', '3446'), ('Presles', '3923'), ('Montsoult', '3633'), ('Osny', '17446'), ('Beauchamp', '8687'), ('Chaumontel', '3241'), ('Écouen', '7125'), ('Puiseux-en-France', '3563'), ('Montlignon', '2937'), ('Saint-Gratien', '20895'), ('Taverny', '26872'), ('Montmorency', '21723'), ('Herblay-sur-Seine', '31314'), ('Margency', '2846'), ('Saint-Brice-sous-Forêt', '15043'), ('Menucourt', '5902'), ('Frépillon', '3342'), ('Éragny', '18296')]}

# Preprocessing du fichier des malades

In [ ]:
# Le fichier "malades.csv" étant trop lourd pour être transmis via Moodle, nous ne l'avons 
# pas inclus dans le fichier zip. Il est cependant toutefois téléchargeable ici :
# https://www.data.gouv.fr/fr/datasets/effectif-de-patients-par-pathologie-sexe-classe-dage-et-territoire-departement-region/

malades = pd.read_csv("malades.csv", sep=";")
malades

,annee,patho_niv1,patho_niv2,patho_niv3,top,cla_age_5,sexe,region,dept,ntop,npop,prev,niveau_prioritaire,libelle_classe_age,libelle_sexe,tri
0,2016,Diabète,Diabète,Diabète,top_FDiabet_ind,75-79,2,76,46,610.0,4230,14.509,"1,2,3",de 75 à 79 ans,femmes,2.0
1,2018,Autres affections de longue durée (dont 31 et 32),Autres affections de longue durée (dont 31 et 32),Autres affections de longue durée (dont 31 et 32),top_ALDAutr_ind,00-04,1,1,971,210.0,10180,2.023,"1,2,3",de 0 à 4 ans,hommes,16.0
2,2015,Diabète,Diabète,Diabète,top_FDiabet_ind,75-79,2,75,23,570.0,3210,17.709,"1,2,3",de 75 à 79 ans,femmes,2.0
3,2016,Diabète,Diabète,Diabète,top_FDiabet_ind,75-79,2,76,82,730.0,4840,15.029,"1,2,3",de 75 à 79 ans,femmes,2.0
4,2018,Autres affections de longue durée (dont 31 et 32),Autres affections de longue durée (dont 31 et 32),Autres affections de longue durée (dont 31 et 32),top_ALDAutr_ind,00-04,1,1,999,210.0,10180,2.023,"1,2,3",de 0 à 4 ans,hommes,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3063709,2020,Maladies psychiatriques,Autres troubles psychiatriques,Autres troubles psychiatriques,top_PsyAutr_ind,70-74,2,93,04,50.0,5890,0.865,"2,3",de 70 à 74 ans,femmes,49.0
3063710,2020,Maladies psychiatriques,Autres troubles psychiatriques,Autres troubles psychiatriques,top_PsyAutr_ind,70-74,2,93,05,40.0,4840,0.723,"2,3",de 70 à 74 ans,femmes,49.0
3063711,2020,Maladies psychiatriques,Autres troubles psychiatriques,Autres troubles psychiatriques,top_PsyAutr_ind,70-74,2,93,06,280.0,37350,0.750,"2,3",de 70 à 74 ans,femmes,49.0
3063712,2020,Maladies psychiatriques,Autres troubles psychiatriques,Autres troubles psychiatriques,top_PsyAutr_ind,70-74,2,93,83,390.0,40660,0.967,"2,3",de 70 à 74 ans,femmes,49.0


In [ ]:
#on se restreint à l'année 2020
malades = malades.loc[malades['annee'] == 2020]

#on se retreint à la catégorie d'age des 60-64 ans : justifié car cette catégorie d'âge est plutôt affectée par les malades (en moyenne plus que les 10-14 ans en tout cas)
malades = malades.loc[malades['cla_age_5'] == "60-64"]

#on enlève certaines colonnes dont nous n'aurons pas besoin
malades = malades.drop(columns=['patho_niv1','patho_niv2','ntop','npop','niveau_prioritaire','libelle_classe_age','libelle_sexe','tri'])

#on renumérote les lignes de 0 à len(malades)-1=26682
malades = malades.reset_index()

#on enleve l'ancien index
malades = malades.drop(columns=["index"])
malades

,annee,patho_niv3,top,cla_age_5,sexe,region,dept,prev
0,2020,Cancer colorectal sous surveillance,top_CanColo_sur,60-64,1,2,999,0.785
1,2020,Cancer colorectal sous surveillance,top_CanColo_sur,60-64,1,3,973,0.718
2,2020,Cancer colorectal sous surveillance,top_CanColo_sur,60-64,1,11,78,0.443
3,2020,Cancer colorectal sous surveillance,top_CanColo_sur,60-64,1,11,91,0.480
4,2020,Cancer colorectal sous surveillance,top_CanColo_sur,60-64,1,11,95,0.455
...,...,...,...,...,...,...,...,...
26678,2020,Autres troubles psychiatriques,top_PsyAutr_ind,60-64,9,84,73,0.522
26679,2020,Autres troubles psychiatriques,top_PsyAutr_ind,60-64,9,84,74,0.536
26680,2020,Autres troubles psychiatriques,top_PsyAutr_ind,60-64,9,93,05,0.826
26681,2020,Autres troubles psychiatriques,top_PsyAutr_ind,60-64,9,94,999,1.065


In [ ]:
liste_lignes_a_drop = []

for i in range(len(malades)):
  dept = malades.loc[malades.index[i],"dept"]
  if len(dept) > 2 or dept == "2A" or dept == "2B": # on retire les lignes où le département n'est pas renseigné, ou les départements d'Outre-mer (971 à 976) ou la Corse
    liste_lignes_a_drop.append(i)

malades.drop(liste_lignes_a_drop, axis=0, inplace=True)
malades

,annee,patho_niv3,top,cla_age_5,sexe,region,dept,prev
2,2020,Cancer colorectal sous surveillance,top_CanColo_sur,60-64,1,11,78,0.443
3,2020,Cancer colorectal sous surveillance,top_CanColo_sur,60-64,1,11,91,0.480
4,2020,Cancer colorectal sous surveillance,top_CanColo_sur,60-64,1,11,95,0.455
5,2020,Cancer colorectal sous surveillance,top_CanColo_sur,60-64,1,24,37,0.436
6,2020,Cancer colorectal sous surveillance,top_CanColo_sur,60-64,1,27,25,0.423
...,...,...,...,...,...,...,...,...
26676,2020,Autres troubles psychiatriques,top_PsyAutr_ind,60-64,9,84,43,0.843
26677,2020,Autres troubles psychiatriques,top_PsyAutr_ind,60-64,9,84,63,1.013
26678,2020,Autres troubles psychiatriques,top_PsyAutr_ind,60-64,9,84,73,0.522
26679,2020,Autres troubles psychiatriques,top_PsyAutr_ind,60-64,9,84,74,0.536


In [ ]:
#l'objectif de cette cellule est d'obtenir la prévalence de chaque maladie dans chaque commune 
#pour ce faire, comme on dispose uniquement de la prévalence par département, on suppose que la prévalence de toutes les communes du département est la même et vaut celle du département


indice = 0
my_dict = {}

for i in range(len(malades)):
  num_dept = malades.loc[malades.index[i],"dept"]
  nom_dept = DEPARTMENTS[num_dept]

  for commune in dico[nom_dept]: # Liste des communes de plus de 2500 habitants dans le département nom_dept
    nom_commune, nb_hab = commune[0], int(commune[1])
    prev = malades.loc[malades.index[i],"prev"]
    if not (np.isnan(prev)):
      nb_malades_commune = int((prev*nb_hab)/100)
      my_dict[indice] = (malades.loc[malades.index[i],"patho_niv3"],malades.loc[malades.index[i],"sexe"],num_dept,nom_commune,nb_hab,prev,nb_malades_commune)
      indice += 1

my_dict

{0: ('Cancer colorectal sous surveillance',
  1,
  '78',
  'Le Chesnay-Rocquencourt',
  31233,
  0.443,
  138),
 1: ('Cancer colorectal sous surveillance',
  1,
  '78',
  'Saint-Germain-en-Laye',
  44806,
  0.443,
  198),
 2: ('Cancer colorectal sous surveillance',
  1,
  '78',
  "L'Étang-la-Ville",
  4453,
  0.443,
  19),
 3: ('Cancer colorectal sous surveillance', 1, '78', 'Maule', 5881, 0.443, 26),
 4: ('Cancer colorectal sous surveillance',
  1,
  '78',
  'Le Mesnil-le-Roi',
  6285,
  0.443,
  27),
 5: ('Cancer colorectal sous surveillance',
  1,
  '78',
  'Mareil-Marly',
  3677,
  0.443,
  16),
 6: ('Cancer colorectal sous surveillance',
  1,
  '78',
  'Mantes-la-Ville',
  20818,
  0.443,
  92),
 7: ('Cancer colorectal sous surveillance',
  1,
  '78',
  'Juziers',
  3843,
  0.443,
  17),
 8: ('Cancer colorectal sous surveillance',
  1,
  '78',
  'Louveciennes',
  7236,
  0.443,
  32),
 9: ('Cancer colorectal sous surveillance', 1, '78', 'Issou', 3893, 0.443, 17),
 10: ('Cancer col

In [ ]:
nouveaux_malades = pd.DataFrame.from_dict(my_dict, orient="index")
nouveaux_malades.rename(columns={0:"patho_niv3", 1:"sexe", 2:"dept", 3:"ville", 4:"nb_hab", 5:"prev", 6:"nb_malades"}, inplace=True)
nouveaux_malades
#on obtient la prévalence pour chaque maladie, pour chaque commune, pour chaque sexe 

,patho_niv3,sexe,dept,ville,nb_hab,prev,nb_malades
0,Cancer colorectal sous surveillance,1,78,Le Chesnay-Rocquencourt,31233,0.443,138
1,Cancer colorectal sous surveillance,1,78,Saint-Germain-en-Laye,44806,0.443,198
2,Cancer colorectal sous surveillance,1,78,L'Étang-la-Ville,4453,0.443,19
3,Cancer colorectal sous surveillance,1,78,Maule,5881,0.443,26
4,Cancer colorectal sous surveillance,1,78,Le Mesnil-le-Roi,6285,0.443,27
...,...,...,...,...,...,...,...
969892,Autres troubles psychiatriques,9,05,Briançon,11339,0.826,93
969893,Autres troubles psychiatriques,9,05,Embrun,6335,0.826,52
969894,Autres troubles psychiatriques,9,05,Chorges,3061,0.826,25
969895,Autres troubles psychiatriques,9,05,Veynes,3216,0.826,26


In [ ]:
nouveaux_malades = nouveaux_malades[nouveaux_malades["patho_niv3"]=="Cancer du poumon actif"] #comme il y avait beaucoup de données, on se restreint à une unique pathologie, ici les cancers du poumon actifs
nouveaux_malades = nouveaux_malades[nouveaux_malades["sexe"] == 1] #on se restreint aux hommes
nouveaux_malades = nouveaux_malades.reset_index()
nouveaux_malades.drop(columns=["index"], inplace=True)
nouveaux_malades

,patho_niv3,sexe,dept,ville,nb_hab,prev,nb_malades
0,Cancer du poumon actif,1,77,Mouroux,5654,0.456,25
1,Cancer du poumon actif,1,77,Trilport,5045,0.456,23
2,Cancer du poumon actif,1,77,Lognes,14353,0.456,65
3,Cancer du poumon actif,1,77,La Grande-Paroisse,2833,0.456,12
4,Cancer du poumon actif,1,77,Chauconin-Neufmontiers,3554,0.456,16
...,...,...,...,...,...,...,...
4480,Cancer du poumon actif,1,06,Auribeau-sur-Siagne,3200,0.503,16
4481,Cancer du poumon actif,1,06,Peymeinade,8192,0.503,41
4482,Cancer du poumon actif,1,06,Gattières,4173,0.503,20
4483,Cancer du poumon actif,1,06,La Roquette-sur-Siagne,5385,0.503,27


In [ ]:
nouveaux_malades.to_csv("nouveaux_malades_poumon.csv", sep=";")
#on le convertit en csv, fichier que l'on utilisera pour créer notre graphe sur Neo4j

# Preprocessing du fichier des établissements

In [ ]:
# Le fichier "etablissements.csv" étant trop lourd pour être transmis via Moodle, nous ne l'avons 
# pas inclus dans le fichier zip. Il est cependant toutefois téléchargeable ici :
# https://www.data.gouv.fr/fr/datasets/finess-extraction-du-fichier-des-etablissements/#/resources
etablissements = pd.read_csv("etablissements.csv", sep=";")

#le csv est divisé en 2 parties, la 1e qui décrit les établissements puis la 2e qui donne plus d'infos sur leur localisation : on ne s'intéressera qu'a la 1e
etablissements_structure = etablissements.loc[:96397]

# On ne garde que la france métropolitaine
etablissements_structure = etablissements.loc[:94000]

#On enlève des colonnes qui ne vont pas nous servir
etablissements_structure = etablissements_structure.drop(columns=['structureet','complrs','compldistrib','telephone','telecopie','categetab','categagretab','siret','dateouv','dateautor','datemaj','numuai'])

etablissements_structure

<ipython-input-42-e3e2c7f4bfc0>:1: DtypeWarning: Columns (1,2,3,6,8,9,10,11,13,14,15,19,21,23,25,27,28,29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  etablissements = pd.read_csv("/content/drive/MyDrive/modal graphes web/etablissements.csv", sep=";")


,nofinesset,nofinessej,rs,rslongue,numvoie,typvoie,voie,compvoie,lieuditbp,commune,departement,libdepartement,ligneachimenement,libcategetab,libcategagretab,codeape,codemft,libmft,codesph,libsph
0,10000024,10780054,CH DE FLEYRIAT,CENTRE HOSPITALIER DE BOURG-EN-BRESSE FLEYRIAT,900.0,RTE,DE PARIS,NaN,NaN,451.0,1,AIN,01440 VIRIAT,Centre Hospitalier (C.H.),Centres Hospitaliers,8610Z,3.0,ARS établissements Publics de santé dotation g...,1.0,Etablissement public de santé
1,10000032,10780062,CH BUGEY SUD,CENTRE HOSPITALIER BUGEY SUD,700.0,AV,DE NARVIK,NaN,BP 139,34.0,1,AIN,01300 BELLEY,Centre Hospitalier (C.H.),Centres Hospitaliers,8610Z,3.0,ARS établissements Publics de santé dotation g...,1.0,Etablissement public de santé
2,10000065,10780096,CH DE TREVOUX - MONTPENSIER,CENTRE HOSPITALIER DE TREVOUX - MONTPENSIER,14.0,R,DE L'HOPITAL,NaN,BP 615,427.0,1,AIN,01606 TREVOUX CEDEX,Centre Hospitalier (C.H.),Centres Hospitaliers,8610Z,3.0,ARS établissements Publics de santé dotation g...,1.0,Etablissement public de santé
3,10000081,10780112,CH DU PAYS DE GEX,CENTRE HOSPITALIER DU PAYS DE GEX,160.0,R,MARC PANISSOD,NaN,BP 437,173.0,1,AIN,01174 GEX CEDEX,"Centre hospitalier, ex Hôpital local",Hôpitaux Locaux,8610Z,3.0,ARS établissements Publics de santé dotation g...,1.0,Etablissement public de santé
4,10000099,10780120,CH DE MEXIMIEUX,CENTRE HOSPITALIER DE MEXIMIEUX,13.0,AV,DU DOCTEUR BOYER,NaN,NaN,244.0,1,AIN,01800 MEXIMIEUX,"Centre hospitalier, ex Hôpital local",Hôpitaux Locaux,8610Z,3.0,ARS établissements Publics de santé dotation g...,1.0,Etablissement public de santé
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93996,950809756,950806927.0,C.A.E : CARRERE,NaN,35.0,R,CARRERE,NaN,NaN,19.0,95,VAL D OISE,95400 ARNOUVILLE,Centre d'Action Educative (C.A.E.),Etab.et Services du Ministère de la Justice po...,NaN,1.0,Etablissement Tarif Libre,NaN,NaN
93997,950809863,750713406.0,CSAPA ANPAA 95,CENTRE DE SOINS ACCOMPAGNEMENT PRÉVENTION EN A...,12.0,BD,MAURICE BERTEAUX,NaN,NaN,18.0,95,VAL D OISE,95100 ARGENTEUIL,Centre soins accompagnement prévention addicto...,Autres Etablissements médico-sociaux,NaN,34.0,ARS / DG dotation globale,NaN,NaN
93998,950810135,750719239.0,SESSAD DE CERGY,SERVICE D EDUCATION ET DE SOINS SPECIALISES A ...,205.0,RES,LES CHENES BRUNS,NaN,NaN,127.0,95,VAL D OISE,95000 CERGY,Service d'Éducation Spéciale et de Soins à Dom...,Services à Domicile ou Ambulatoires pour Handi...,NaN,34.0,ARS / DG dotation globale,NaN,NaN
93999,950810184,950803361.0,C.H.R.S : ETAPE,NaN,113.0,RTE,DE MONTLIGNON,NaN,NaN,203.0,95,VAL D OISE,95600 EAUBONNE,Centre Hébergement & Réinsertion Sociale (C.H....,Etablissements pour Adultes et Familles en Dif...,NaN,30.0,Préfet de région établissements et services so...,NaN,NaN


In [ ]:
#on ne garde que les hôpitaux : on n'a pas besoin des pharmacies, des EPHAD ...
etablissements_structure = etablissements_structure[(etablissements_structure['libcategagretab']=="Centres Hospitaliers") | (etablissements_structure['libcategagretab']=="Hôpitaux Locaux")]

etablissements_structure = etablissements_structure.reset_index()
etablissements_structure = etablissements_structure.drop(columns=['index'])
etablissements_structure

,nofinesset,nofinessej,rs,rslongue,numvoie,typvoie,voie,compvoie,lieuditbp,commune,departement,libdepartement,ligneachimenement,libcategetab,libcategagretab,codeape,codemft,libmft,codesph,libsph
0,10000024,10780054,CH DE FLEYRIAT,CENTRE HOSPITALIER DE BOURG-EN-BRESSE FLEYRIAT,900.0,RTE,DE PARIS,NaN,NaN,451.0,1,AIN,01440 VIRIAT,Centre Hospitalier (C.H.),Centres Hospitaliers,8610Z,3.0,ARS établissements Publics de santé dotation g...,1.0,Etablissement public de santé
1,10000032,10780062,CH BUGEY SUD,CENTRE HOSPITALIER BUGEY SUD,700.0,AV,DE NARVIK,NaN,BP 139,34.0,1,AIN,01300 BELLEY,Centre Hospitalier (C.H.),Centres Hospitaliers,8610Z,3.0,ARS établissements Publics de santé dotation g...,1.0,Etablissement public de santé
2,10000065,10780096,CH DE TREVOUX - MONTPENSIER,CENTRE HOSPITALIER DE TREVOUX - MONTPENSIER,14.0,R,DE L'HOPITAL,NaN,BP 615,427.0,1,AIN,01606 TREVOUX CEDEX,Centre Hospitalier (C.H.),Centres Hospitaliers,8610Z,3.0,ARS établissements Publics de santé dotation g...,1.0,Etablissement public de santé
3,10000081,10780112,CH DU PAYS DE GEX,CENTRE HOSPITALIER DU PAYS DE GEX,160.0,R,MARC PANISSOD,NaN,BP 437,173.0,1,AIN,01174 GEX CEDEX,"Centre hospitalier, ex Hôpital local",Hôpitaux Locaux,8610Z,3.0,ARS établissements Publics de santé dotation g...,1.0,Etablissement public de santé
4,10000099,10780120,CH DE MEXIMIEUX,CENTRE HOSPITALIER DE MEXIMIEUX,13.0,AV,DU DOCTEUR BOYER,NaN,NaN,244.0,1,AIN,01800 MEXIMIEUX,"Centre hospitalier, ex Hôpital local",Hôpitaux Locaux,8610Z,3.0,ARS établissements Publics de santé dotation g...,1.0,Etablissement public de santé
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1468,950045773,950110080.0,CMP CATTP HOPITAL NOVO SITE BEAUMONT,CMP CATTP HOPITAL NOVO SITE BEAUMONT SUR OISE,25.0,R,EDMOND TURCQ,NaN,NaN,52.0,95,VAL D OISE,95260 BEAUMONT SUR OISE,Centre Hospitalier (C.H.),Centres Hospitaliers,NaN,3.0,ARS établissements Publics de santé dotation g...,1.0,Etablissement public de santé
1469,950500017,950110080.0,SSR HOPITAL NOVO SITE AINCOURT,SSR HOPITAL NOVO SITE AINCOURT (EX GHI DU VEXIN),NaN,R,DE LA BUCAILLE,NaN,NaN,8.0,95,VAL D OISE,95510 AINCOURT,Centre Hospitalier (C.H.),Centres Hospitaliers,8610Z,3.0,ARS établissements Publics de santé dotation g...,1.0,Etablissement public de santé
1470,950787135,950110080.0,CENTRE PSYCHOTHERAPIQUE LES OLIVIERS,CENTRE PSYCHOTHERAPIQUE LES OLIVIERS HOPITAL NOVO,NaN,RTE,DE NOISY,NaN,NaN,52.0,95,VAL D OISE,95260 BEAUMONT SUR OISE,Centre Hospitalier (C.H.),Centres Hospitaliers,8610Z,3.0,ARS établissements Publics de santé dotation g...,1.0,Etablissement public de santé
1471,950805879,950110080.0,CMP HOPITAL NOVO SITE BEAUMONT,CMP HOPITAL NOVO SITE BEAUMONT (SECTEUR 8),7.0,R,LEON GODIN,NaN,NaN,52.0,95,VAL D OISE,95260 BEAUMONT SUR OISE,Centre Hospitalier (C.H.),Centres Hospitaliers,NaN,3.0,ARS établissements Publics de santé dotation g...,1.0,Etablissement public de santé


In [ ]:
#on modifie le nom des villes, par exemple en retirant les CEDEX à la fin 

villes = []

for i in range(len(etablissements_structure)):
  nom_ville = etablissements_structure["ligneachimenement"][i][6:]
  nom_ville = nom_ville.replace(" CEDEX", "")
  villes.append(''.join([i for i in nom_ville if not i.isdigit()]))

etablissements_structure["ville"] = villes

etablissements_structure

,nofinesset,nofinessej,rs,rslongue,numvoie,typvoie,voie,compvoie,lieuditbp,commune,...,libdepartement,ligneachimenement,libcategetab,libcategagretab,codeape,codemft,libmft,codesph,libsph,ville
0,10000024,10780054,CH DE FLEYRIAT,CENTRE HOSPITALIER DE BOURG-EN-BRESSE FLEYRIAT,900.0,RTE,DE PARIS,NaN,NaN,451.0,...,AIN,01440 VIRIAT,Centre Hospitalier (C.H.),Centres Hospitaliers,8610Z,3.0,ARS établissements Publics de santé dotation g...,1.0,Etablissement public de santé,VIRIAT
1,10000032,10780062,CH BUGEY SUD,CENTRE HOSPITALIER BUGEY SUD,700.0,AV,DE NARVIK,NaN,BP 139,34.0,...,AIN,01300 BELLEY,Centre Hospitalier (C.H.),Centres Hospitaliers,8610Z,3.0,ARS établissements Publics de santé dotation g...,1.0,Etablissement public de santé,BELLEY
2,10000065,10780096,CH DE TREVOUX - MONTPENSIER,CENTRE HOSPITALIER DE TREVOUX - MONTPENSIER,14.0,R,DE L'HOPITAL,NaN,BP 615,427.0,...,AIN,01606 TREVOUX CEDEX,Centre Hospitalier (C.H.),Centres Hospitaliers,8610Z,3.0,ARS établissements Publics de santé dotation g...,1.0,Etablissement public de santé,TREVOUX
3,10000081,10780112,CH DU PAYS DE GEX,CENTRE HOSPITALIER DU PAYS DE GEX,160.0,R,MARC PANISSOD,NaN,BP 437,173.0,...,AIN,01174 GEX CEDEX,"Centre hospitalier, ex Hôpital local",Hôpitaux Locaux,8610Z,3.0,ARS établissements Publics de santé dotation g...,1.0,Etablissement public de santé,GEX
4,10000099,10780120,CH DE MEXIMIEUX,CENTRE HOSPITALIER DE MEXIMIEUX,13.0,AV,DU DOCTEUR BOYER,NaN,NaN,244.0,...,AIN,01800 MEXIMIEUX,"Centre hospitalier, ex Hôpital local",Hôpitaux Locaux,8610Z,3.0,ARS établissements Publics de santé dotation g...,1.0,Etablissement public de santé,MEXIMIEUX
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1468,950045773,950110080.0,CMP CATTP HOPITAL NOVO SITE BEAUMONT,CMP CATTP HOPITAL NOVO SITE BEAUMONT SUR OISE,25.0,R,EDMOND TURCQ,NaN,NaN,52.0,...,VAL D OISE,95260 BEAUMONT SUR OISE,Centre Hospitalier (C.H.),Centres Hospitaliers,NaN,3.0,ARS établissements Publics de santé dotation g...,1.0,Etablissement public de santé,BEAUMONT SUR OISE
1469,950500017,950110080.0,SSR HOPITAL NOVO SITE AINCOURT,SSR HOPITAL NOVO SITE AINCOURT (EX GHI DU VEXIN),NaN,R,DE LA BUCAILLE,NaN,NaN,8.0,...,VAL D OISE,95510 AINCOURT,Centre Hospitalier (C.H.),Centres Hospitaliers,8610Z,3.0,ARS établissements Publics de santé dotation g...,1.0,Etablissement public de santé,AINCOURT
1470,950787135,950110080.0,CENTRE PSYCHOTHERAPIQUE LES OLIVIERS,CENTRE PSYCHOTHERAPIQUE LES OLIVIERS HOPITAL NOVO,NaN,RTE,DE NOISY,NaN,NaN,52.0,...,VAL D OISE,95260 BEAUMONT SUR OISE,Centre Hospitalier (C.H.),Centres Hospitaliers,8610Z,3.0,ARS établissements Publics de santé dotation g...,1.0,Etablissement public de santé,BEAUMONT SUR OISE
1471,950805879,950110080.0,CMP HOPITAL NOVO SITE BEAUMONT,CMP HOPITAL NOVO SITE BEAUMONT (SECTEUR 8),7.0,R,LEON GODIN,NaN,NaN,52.0,...,VAL D OISE,95260 BEAUMONT SUR OISE,Centre Hospitalier (C.H.),Centres Hospitaliers,NaN,3.0,ARS établissements Publics de santé dotation g...,1.0,Etablissement public de santé,BEAUMONT SUR OISE


In [ ]:
#on elimine les doublons dans la liste des villes des hopitaux 
villes2 = list(set(villes))
print(villes2)

['YZEURE', 'LA ROCHELLE', 'PONTEILS ET BRESIS', 'LONS LE SAUNIER', 'BELLEVILLE', 'VAL COUESNON', 'LE LOROUX BOTTEREAU', 'ST BONNET LE CHATEAU', 'MONTBARD', 'CHALON SUR SAONE', 'CHARTRES', 'VILLEFRANCHE DE ROUERGUE', 'DINAN', 'TONNEINS', 'ST MIHIEL', 'MULHOUSE', 'ST CALAIS', 'BONNETABLE', 'AGEN ', 'CARVIN', 'LE HAVRE', 'FREJUS', 'NEUFCHATEAU', 'LA ROCHETTE', 'AMBERT', 'ST MARCELLIN', 'PITHIVIERS', 'ALENCON', 'TOURCOING', 'ROYAN', 'PLOERMEL', 'CHATILLON SUR INDRE', 'VAL DE BRIEY', 'CAHORS', 'PAIMBOEUF', 'MONTREUIL', 'BOURG ST ANDEOL', 'GRAY', 'LAGNY SUR MARNE', 'ST AUBIN ROUTOT', 'TRANS EN PROVENCE', 'BRIOUDE', 'NANTERRE', 'DOUE EN ANJOU', 'REMIREMONT', 'EAUBONNE', 'EPINAY SUR ORGE', 'AIRE SUR LA LYS', 'MONTFERMEIL', 'AGDE', 'LES SABLES D OLONNE', 'ALTKIRCH', 'LANMEUR', 'MONT ST AIGNAN', 'SARCELLES', 'ST VALERY SUR SOMME', 'VANNES', 'ST LAURENT DU PONT', 'COLMAR', 'THONON LES BAINS', 'MENTON', 'AMILLY', 'NOIRMOUTIER EN L ILE', 'GOURNAY EN BRAY', 'REDON', 'BITCHE', 'ARMENTIERES', 'MARINES

In [ ]:
#on cree un CSV qui va nous aider à créer des noeuds pour les établissements dans notre graphe Neo4j 
etablissements_structure.to_csv("etablissements_structure.csv", sep=";", index=False)

# Calcul des distances par l'API OpenStreetMap


Le but de cette section est de calculer les distances à vol d'oiseau entre les villes des hopitaux et toutes les communes de France (où il y a des malades)

In [ ]:
#on recupère la latitude et la longitude des villes grace à l'API OpenStreetMap

def get_lat_long(address):

    reqUrl = "https://nominatim.openstreetmap.org/search?q={}&format=json&limit=1"

    address = address.lower()
    address = address.replace(" ", "%2B")

    reqUrl = reqUrl.replace("{}", address)

    headersList = {
    "Accept": "*/*",
    "User-Agent": "Thunder Client (https://www.thunderclient.com)" 
    }

    payload = ""
    response = requests.request("GET", reqUrl, data=payload,  headers=headersList).json()


    if response==[]:
        lat, long = -1, -1
    else:
        lat, long = response[0]['lat'], response[0]['lon']
    return lat, long

In [ ]:
#calcule la distance à vol d'oiseau entre deux villes une fois donné leur latitude et longitude

def bird_flight_distance(lat1, lon1, lat2, lon2):
    R = 6371 #rayon de la Terre

    # on convertit en radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # formule de Haversine
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = R * c

    return distance

In [ ]:
# ATTENTION comme cette cellule met du temps à s'exécuter pour la réplication on fournit dans la cellule ci dessous le dictionnaire obtenu
#on recupere la latitude et la longitude de toutes les communes de France (où il y a donc des malades)

lat_long_malades = {}
for dept in dico:
  villes = dico[dept]
  for ville in villes:

    la, lo = get_lat_long(ville[0])
    la = float(la)
    lo = float(lo)
    lat_long_malades[ville[0]] = (lo,la)

In [ ]:
lat_long_malades = {'Beynost': (5.0009242, 45.8400182), 'Saint-Maurice-de-Beynost': (4.97709, 45.8333), 'Villars-les-Dombes': (5.029519, 46.0014072), 'Bourg-en-Bresse': (5.2250324, 46.2051192), 'Châtillon-en-Michaille': (5.7985185, 46.1438962), 'Belley': (5.6861466, 45.760758), 'Bellegarde-sur-Valserine': (5.830749559824268, 46.1073321), 'Oyonnax': (5.673366294950046, 46.2476745), 'Ambérieu-en-Bugey': (5.3595099, 45.9584309), 'Ambronay': (5.3602947, 46.0055071), 'Nantua': (5.6058987, 46.1534255), 'Gex': (6.0588831, 46.3324378), 'Bâgé-la-Ville': (4.942278560486707, 46.3276518), 'Châtillon-sur-Chalaronne': (4.9580413, 46.1191397), 'Arbent': (5.680467, 46.2946684), 'Attignat': (5.16355, 46.289384), 'Chalamont': (5.1710728, 45.9963055), 'Ceyzériat': (5.3242942, 46.1773941), 'Bellignat': (5.6293467, 46.2418811), 'Béligneux': (5.1280072, 45.8677365), 'La Boisse': (5.0328806, 45.8416901), 'Ferney-Voltaire': (6.1076131, 46.2555744), 'Lagnieu': (5.3480192, 45.9034952), 'Cessy': (6.0697412, 46.3168303), 'Montluel': (5.0568249, 45.8498034), 'Balan': (5.0969884, 45.8333789), 'Dagneux': (5.07498, 45.8541), 'Divonne-les-Bains': (6.143049, 46.3570916), 'Vonnas': (4.9925097, 46.2189665), 'Hauteville-Lompnes': (5.600403, 45.9769238), 'Meximieux': (5.1925264, 45.90363), 'Loyettes': (5.2053916, 45.7739423), 'Massieux': (4.8241, 45.9068), 'Villieu-Loyes-Mollon': (5.225316086217485, 45.9289818), 'Péronnas': (5.203589, 46.1798475), 'Culoz': (5.7809063, 45.8476517), 'Jassans-Riottier': (4.7567624, 45.98514), 'Montrevel-en-Bresse': (5.130295, 46.3373228), 'Trévoux': (4.7733198, 45.9410123), "Pont-d'Ain": (5.34554, 46.052), 'Ornex': (6.0931951, 46.2763563), 'Montréal-la-Cluse': (5.5705397, 46.1866471), 'Saint-Maurice-de-Gourdans': (5.1936266, 45.8207025), 'Prévessin-Moëns': (6.0873747, 46.257389), 'Péron': (5.9244804, 46.1896869), 'Saint-Genis-Pouilly': (6.0253162, 46.2440083), 'Miribel': (4.9528525, 45.8235227), 'Neyron': (4.9318223, 45.814595), 'Polliat': (5.1262826, 46.2490546), 'Thoiry': (5.9770947, 46.2337403), 'Saint-Étienne-du-Bois': (5.2939953, 46.2876364), 'Saint-Didier-sur-Chalaronne': (4.8181107, 46.1773643), 'Viriat': (5.2167876, 46.2533181), 'Saint-Denis-lès-Bourg': (5.1906345, 46.202056), 'Feillens': (4.8875665, 46.3360148), 'Montmerle-sur-Saône': (4.7614129, 46.0840019), 'Reyrieux': (4.8220491, 45.9357535), 'Saint-André-de-Corcy': (4.9534729, 45.9281426), 'Replonges': (4.8827131, 46.3092904), 'Val-Revermont': (5.351016903202019, 46.281865249999996), 'Bâgé-Dommartin': (4.975079028663238, 46.32768495), 'Valserhône': (5.799102457185439, 46.12768294999999), "Plateau d'Hauteville": (5.581990102625218, 45.9300815), 'Culoz-Béon': (5.768843651094306, 45.85550365), 'Laon': (3.620686, 49.564665), 'Soissons': (3.327613, 49.3837621), 'Saint-Quentin': (3.2876843, 49.8465253), 'Château-Thierry': (3.4027111, 49.0457362), 'Vervins': (3.9079645, 49.8346074), 'Villers-Cotterêts': (3.0909275, 49.2549479), 'Chauny': (3.2189506, 49.6148945), 'Hirson': (4.0837186, 49.9203419), 'Guise': (3.6262542, 49.8982325), 'Bohain-en-Vermandois': (3.4547638, 49.986708), 'Fère-en-Tardenois': (3.513536, 49.2011635), 'La Fère': (3.3661672, 49.6632869), 'Tergnier': (3.2918511, 49.6558465), 'Crouy': (3.36, 49.4025), 'Athies-sous-Laon': (3.6867911, 49.5722306), 'Saint-Michel': (4.1340444, 49.9186556), 'Charly-sur-Marne': (3.2843296, 48.9754438), 'Beautor': (3.3455556, 49.6516667), 'Fresnoy-le-Grand': (3.4147934, 49.9468967), 'Le Nouvion-en-Thiérache': (3.7858, 50.01805), 'Essômes-sur-Marne': (3.3745637, 49.0301415), 'Villeneuve-Saint-Germain': (3.3594444, 49.38), 'Gauchy': (3.2705556, 49.825), 'Belleu': (3.3355556, 49.3591667), 'Quessy': (3.2914123301477747, 49.671718799999994), 'Anizy-le-Grand': (3.4434471407633414, 49.516053299999996), 'Villeneuve-sur-Aisne': (3.9696109638801182, 49.44020635), 'Saint-Yorre': (3.4614685, 46.0658287), 'Vichy': (3.4203712, 46.1239268), 'Moulins': (3.6847, 49.4217), 'Montluçon': (2.6073963, 46.3400296), 'Désertines': (2.6157229, 46.3556648), 'Dompierre-sur-Besbre': (3.6812752, 46.5216278), "Bourbon-l'Archambault": (3.05804, 46.5857669), 'Bellerive-sur-Allier': (3.4063997, 46.1156244), 'Prémilhat': (2.5370983, 46.3140123), 'Domérat': (2.5368661, 46.359995), 'Lapalisse': (3.6324006, 46.2488636), 'Gannat': (3.1995251, 46.0996068), 'Huriel': (2.477798, 46.3733261), 'Avermes': (3.3102105, 46.5881392), 'Abrest': (3.4450434, 46.0980139), 'Saint-Pourçain-sur-Sioule': (3.2899107, 46.3074423), 'Cusset': (3.4602736, 46.1324239), 'Yzeure': (3.3539775, 46.5676568), 'Saint-Germain-des-Fossés': (3.4320562, 46.2067884), 'Néris-les-Bains': (2.6583028, 46.2898064), 'Commentry': (2.74398, 46.290103), 'Varennes-sur-Allier': (3.4033756, 46.3127469), 'Creuzier-le-Vieux': (3.4382924, 46.1593352), 'Sainte-Tulle': (5.7627076, 43.7858321), 'Volx': (5.8411569, 43.8782761), 'Pierrevert': (5.749258, 43.8118768), 'Forcalquier': (5.780446, 43.9585575), 'Digne-les-Bains': (6.2351431, 44.0918144), 'Barcelonnette': (6.6513554, 44.3862043), 'Manosque': (5.7826657, 43.8338026), 'Sisteron': (5.9443791, 44.1963467), 'Château-Arnoux-Saint-Auban': (6.0080053, 44.0936899), 'Peyruis': (5.9408922, 44.0300301), 'Gréoux-les-Bains': (5.8841886, 43.7581306), 'Valensole': (5.9839867, 43.8379283), 'Les Mées': (0.2321677, 48.3144445), 'Oraison': (5.919898, 43.9166157), 'Villeneuve': (-0.6280356, 45.0835468), 'Laragne-Montéglin': (5.8230176, 44.3148916), 'Gap': (6.0820639, 44.5612032), 'Briançon': (6.6436313, 44.8984037), 'Embrun': (6.4958189, 44.5641764), 'Chorges': (6.2754561, 44.5450837), 'Veynes': (5.8244718, 44.5349009), 'La Bâtie-Neuve': (6.195795, 44.5649889), 'Saint-André-de-la-Roche': (7.2873233, 43.7372922), 'Saint-Vallier-de-Thiey': (6.8478583, 43.6987023), 'Colomars': (7.2183304, 43.7615774), 'Saint-Cézaire-sur-Siagne': (6.7925537, 43.6505213), 'La Colle-sur-Loup': (7.0976053, 43.6857041), 'Saint-Martin-du-Var': (7.1918, 43.82197), 'Le Tignet': (6.8462891, 43.6399782), "L'Escarène": (7.3556964, 43.8342052), 'Tourrette-Levens': (7.27228, 43.7878), 'Contes': (1.9650258, 50.4073056), 'Drap': (7.3201523, 43.7556098), 'Nice': (7.2683912, 43.7009358), 'Cannes': (7.0134418, 43.5515198), 'Antibes': (7.10905, 43.5836), 'Menton': (7.5029213, 43.7753495), 'Vence': (7.1134479, 43.7228003), 'Cagnes-sur-Mer': (7.1513834, 43.6612012), 'Villefranche-sur-Mer': (7.3117225, 43.704061), 'Le Cannet': (7.0191865, 43.5761076), 'Saint-Paul-de-Vence': (7.1216304, 43.697972), 'Beaulieu-sur-Mer': (7.33375, 43.7088), 'Saint-Laurent-du-Var': (7.1906969, 43.6690101), 'Roquebrune-Cap-Martin': (7.4565697, 43.7641047), 'Mougins': (6.9963543, 43.6008898), 'Vallauris': (7.05859, 43.5761001), 'Mandelieu-la-Napoule': (6.9359135, 43.5439153), 'Biot': (7.0966644, 43.6280001), 'La Turbie': (7.4011252, 43.7452982), 'Levens': (7.2401105, 43.8477377), 'Beausoleil': (7.4219677, 43.748281), "Cap-d'Ail": (7.4058222, 43.7223375), 'Pégomas': (6.9306639, 43.5928477), 'Villeneuve-Loubet': (7.1217592, 43.6579947), 'Mouans-Sartoux': (6.9703991, 43.6203852), 'Châteauneuf-Grasse': (6.9758576, 43.6747381), 'Roquefort-les-Pins': (7.0492115, 43.6661456), 'Valbonne': (7.0086255, 43.641141), 'La Trinité': (1.2335758, 48.9893963), 'La Gaude': (7.1503312, 43.7228225), 'Tourrettes-sur-Loup': (7.058912, 43.7158031), 'Sospel': (7.4498235, 43.8785647), 'Saint-Jeannet': (6.1241538, 43.9519438), 'Le Rouret': (7.0059129, 43.6743043), 'Carros': (7.18785, 43.7846), 'Auribeau-sur-Siagne': (6.9105552, 43.6001505), 'Peymeinade': (6.8756858, 43.641991), 'Gattières': (7.1781152, 43.7606843), 'La Roquette-sur-Siagne': (6.9557118, 43.5810901), 'Le Bar-sur-Loup': (6.988256, 43.7012408), 'Saint-Étienne-de-Fontbellon': (4.3780321, 44.5969806), 'Le Teil': (4.6829629, 44.5493789), 'Le Pouzin': (4.7485628, 44.7555185), 'Chomérac': (4.6615361, 44.708414), 'Bourg-Saint-Andéol': (4.6457565, 44.3707629), 'Saint-Péray': (4.8430943, 44.9454417), 'Saint-Jean-de-Muzols': (4.8136624, 45.082847), 'Davézieux': (4.7057799, 45.2567222), 'Le Cheylard': (4.42245, 44.9093), 'Guilherand-Granges': (4.8708275, 44.932545), 'Viviers': (6.4343826, 48.8912366), 'Vals-les-Bains': (4.365739, 44.6553304), 'Charmes-sur-Rhône': (4.83424, 44.8621), 'Les Vans': (4.1332861, 44.405479), 'Villeneuve-de-Berg': (4.4985083, 44.5555132), 'La Voulte-sur-Rhône': (4.7813846, 44.8000756), 'Vernosc-lès-Annonay': (4.7130702, 45.2164175), 'Roiffieux': (4.6549646, 45.2269844), 'Cruas': (4.7625644, 44.6571316), 'Privas': (4.5986733, 44.7352708), 'Aubenas': (4.3902399, 44.6205476), 'Tournon-sur-Rhône': (4.832852, 45.0675156), 'Annonay': (4.6675475, 45.2399639), 'Carignan': (5.1682498, 49.6319856), 'Villers-Semeuse': (4.7455902, 49.7430945), 'Vivier-au-Court': (4.8316667, 49.7355556), 'Vrigne-aux-Bois': (4.857589784763167, 49.73750805), 'Charleville-Mézières': (4.7206939, 49.7735712), 'Sedan': (4.9433409, 49.7033759), 'Rethel': (4.3666467, 49.5091447), 'Vouziers': (4.723819262037698, 49.40228705), 'Givet': (4.8248698, 50.1362582), 'Fumay': (4.7052616, 49.9910213), 'Nouzonville': (4.7445678, 49.814437), 'Bogny-sur-Meuse': (4.7677778, 49.8508333), 'Revin': (4.6374643, 49.9421221), 'Charleville': (3.6664318, 48.813549), 'Mézières': (6.9264097, 46.678377), 'Vrigne aux Bois': (4.857589784763167, 49.73750805), 'La Tour-du-Crieu': (1.6505217, 43.1013375), 'Varilhes': (1.6272972, 43.0424295), 'Foix': (1.6053807, 42.9638998), 'Pamiers': (1.6082177, 43.1147515), 'Saint-Girons': (1.1464455, 42.9841865), 'Lavelanet': (1.8487179, 42.9332121), 'Mirepoix': (0.6719444, 43.7458333), 'Saint-Jean-du-Falga': (1.6254308, 43.0842576), 'Mazères': (-0.2586472, 44.4948895), 'Tarascon-sur-Ariège': (1.6067586, 42.8458016), 'Saverdun': (1.5754613, 43.2336968), 'Les Noës-près-Troyes': (4.0399743, 48.3030024), 'Sainte-Savine': (4.0565359, 48.2931517), 'Bréviandes': (4.0955124, 48.2566295), 'Saint-Julien-les-Villas': (4.0979239, 48.2721492), 'Bar-sur-Aube': (4.7058337, 48.2332047), 'Arcis-sur-Aube': (4.1421213, 48.5362425), 'Bar-sur-Seine': (4.3759103, 48.1126529), 'La Chapelle-Saint-Luc': (4.0413095, 48.3185478), 'Nogent-sur-Seine': (3.4978181, 48.492439), 'Brienne-le-Château': (4.5268522, 48.3941801), 'La Rivière-de-Corps': (4.0279108, 48.2863568), 'Saint-André-les-Vergers': (4.0507314, 48.2784599), 'Romilly-sur-Seine': (3.7264161, 48.5195734), 'Saint-Parres-aux-Tertres': (4.1195915, 48.2979256), 'Rosières-près-Troyes': (4.0643669, 48.2572459), 'Saint-Lyé': (4.0020644, 48.3613368), 'Pont-Sainte-Marie': (4.1014237, 48.3173614), 'Villenauxe-la-Grande': (3.5548725, 48.5903846), 'Aix-Villemaur-Pâlis': (3.7460163322454365, 48.2266775), 'Vinassan': (3.07427, 43.2051), 'Bram': (2.1149613, 43.243622), "Salles-d'Aude": (3.1155056, 43.2390042), 'Quillan': (2.1941008666750825, 42.876584699999995), 'Coursan': (3.0585293, 43.2347407), 'Sigean': (2.9808695, 43.029047), 'Leucate': (3.0270222, 42.910007), "Sallèles-d'Aude": (2.9485821, 43.2567073), 'Port-la-Nouvelle': (3.046461, 43.0203536), 'Fleury': (2.25386, 50.42199), 'Conques-sur-Orbiel': (2.4016599, 43.2696774), 'Gruissan': (3.086422, 43.1050181), 'Villemoustaussou': (2.3653647, 43.2522241), 'Pennautier': (2.3189949, 43.244343), 'Narbonne': (3.0042121, 43.1837661), 'Carcassonne': (2.3491069, 43.2130358), 'Lézignan-Corbières': (2.7574981, 43.2008841), 'Ouveillan': (2.9719822, 43.2887733), 'Trèbes': (2.4412579, 43.2108325), "Cuxac-d'Aude": (2.9993069, 43.2434772), 'Limoux': (2.2176533, 43.0538068), 'Castelnaudary': (1.9533495, 43.3192021), 'Olemps': (2.5541214, 44.3386131), 'Sébazac-Concourès': (2.5992009, 44.4047418), 'Viviez-les-Albres': (-1.0, -1.0), 'Aubin': (-0.4150674, 43.44004), 'Baraqueville': (2.433498, 44.2767537), 'Druelle': (2.5053408, 44.3606219), 'Rodez': (2.5727883, 44.3513062), 'Millau': (3.0777594, 44.1006693), 'Villefranche-de-Rouergue': (2.0375431, 44.3517623), 'Luc-la-Primaube': (2.5346875, 44.3142948), 'Espalion': (2.7616471, 44.5206809), 'Decazeville': (2.2542071, 44.5585148), 'Onet-le-Château': (2.5770621, 44.3747245), 'Bozouls': (2.7211156, 44.4695894), 'Capdenac-Gare': (2.0805165, 44.5741127), 'Saint-Affrique': (2.8866641, 43.9580035), "Sévérac d'Aveyron": (3.043442762730632, 44.3139643), 'Druelle Balsac': (2.441101866953012, 44.378410099999996), 'Saint-Savournin': (5.527214, 43.4063903), "La Roque-d'Anthéron": (5.3122208, 43.7150961), 'Roquefort-la-Bédoule': (5.5916391, 43.2475122), 'Fuveau': (5.5609197, 43.4538997), 'Meyreuil': (5.4952155, 43.4867359), 'Port-de-Bouc': (4.9806998, 43.403098), 'Grans': (5.063379, 43.606758), 'Rognac': (5.2342903, 43.4872341), 'Meyrargues': (5.5284652, 43.6352906), 'Gréasque': (5.5434487, 43.4321624), 'Rognonas': (4.8044205, 43.8997994), 'Le Puy-Sainte-Réparade': (5.4355069, 43.6629062), 'Velaux': (5.2540649, 43.5217717), 'Coudoux': (5.2501215, 43.5584577), 'Septèmes-les-Vallons': (5.3663965, 43.3989529), "Plan-d'Orgon": (4.9982831, 43.8083763), 'Saint-Martin-de-Crau': (4.8111564, 43.6384838), 'Mouriès': (4.871428, 43.6891957), 'Ventabren': (5.2926551, 43.5385249), 'Marseille': (5.3699525, 43.2961743), 'Aix-en-Provence': (5.4474738, 43.5298424), 'Arles': (4.6309653, 43.6776223), 'Bouc-Bel-Air': (5.4127848, 43.4511819), 'Aubagne': (5.5703031, 43.2924385), 'Istres': (4.9884323, 43.5139051), 'Vitrolles': (5.9496043, 44.4347105), 'Saint-Rémy-de-Provence': (4.8317498, 43.7885782), 'La Ciotat': (5.6062495, 43.1758591), 'Salon-de-Provence': (5.0980225, 43.6405237), 'Tarascon': (1.6067586, 42.8458016), 'Miramas': (5.0025809, 43.582733), 'Martigues': (5.0548176, 43.4057279), 'Carry-le-Rouet': (5.1522977, 43.3316442), 'Cassis': (5.5396318, 43.2140359), 'Les Pennes-Mirabeau': (5.3090324, 43.4098822), 'Marignane': (5.2146275, 43.4162729), 'Gémenos': (5.6295178, 43.2961722), 'Port-Saint-Louis-du-Rhône': (4.8026354, 43.3890531), 'Orgon': (5.0394918, 43.790843), 'Graveson': (4.7729441, 43.8516775), 'Allauch': (5.4832687, 43.3351381), 'Éguilles': (5.3543957, 43.5687248), "Berre-l'Étang": (5.1673492, 43.4758249), 'Gardanne': (5.4710661, 43.455613), 'Barbentane': (4.7477259, 43.8986064), 'Auriol': (5.6298611, 43.3688436), 'Alleins': (5.1622362, 43.7043862), 'Cabannes': (4.949561, 43.8610116), 'Châteauneuf-les-Martigues': (5.1646417, 43.3804293), 'La Bouilladisse': (5.5955308, 43.3934248), 'Cabriès': (5.3798882, 43.4411965), 'Charleval': (1.3835702, 49.3720879), 'Eyguières': (5.0297213, 43.695131), 'La Penne-sur-Huveaune': (5.515363, 43.2810535), 'Lançon-Provence': (5.1258628, 43.5913062), 'Fos-sur-Mer': (4.945595, 43.4380714), 'Gignac-la-Nerthe': (5.2329106, 43.3887665), 'Eyragues': (4.8410859, 43.8412123), 'Saint-Andiol': (4.9449219, 43.8358859), 'Pélissanne': (5.1496735, 43.6306683), 'Châteaurenard': (4.85568, 43.8820448), 'Ceyreste': (5.6391975, 43.2183769), 'Lambesc': (5.2620916, 43.6541997), 'Cuges-les-Pins': (5.7014656, 43.2748169), 'Noves': (4.901209, 43.8761887), 'La Destrousse': (5.6058304, 43.3765828), 'Sausset-les-Pins': (5.1094441, 43.332048), 'Ensuès-la-Redonne': (5.2036311, 43.3557165), 'Maillane': (4.7825111, 43.8325251), 'Fontvieille': (4.707674, 43.7264844), 'Plan-de-Cuques': (5.4618046, 43.3460029), 'Peynier': (5.6403316, 43.4471537), 'Carnoux-en-Provence': (5.56555, 43.2583), 'Sénas': (5.078624, 43.7446686), 'Saint-Victoret': (5.2367629, 43.4194836), 'Trets': (5.6851425, 43.4469081), 'Mollégès': (4.9487152, 43.8059139), 'Le Rove': (5.2496375, 43.3690901), 'Peyrolles-en-Provence': (5.5836164, 43.6465639), 'Peypin': (5.5778541, 43.3838133), 'Mallemort': (5.1802855, 43.7314716), 'Rousset': (6.2484788, 44.4776793), 'Saint-Chamas': (5.0341639, 43.5497879), 'Saint-Mitre-les-Remparts': (5.0141117, 43.452893), 'La Fare-les-Oliviers': (5.1922962, 43.5503885), 'Rognes': (5.3469001, 43.6638292), 'Mimet': (5.504383, 43.4139973), 'Jouques': (5.6367503, 43.636907), 'Roquevaire': (5.6046608, 43.3493655), 'Simiane-Collongue': (5.4372321, 43.4283878), 'Saint-Cannat': (5.296775, 43.6207231), 'Venelles': (5.4804956, 43.6003084), 'Bayeux': (-0.7024738, 49.2764624), 'Lisieux': (0.2255168, 49.1460831), 'Falaise': (-0.1999129, 48.8964741), 'Cabourg': (-0.1155085, 49.2934373), 'Deauville': (0.0752778, 49.36), 'Vire': (-0.8872594458318377, 48.8517468), "Pont-l'Évêque": (2.9880799, 49.5643305), 'Honfleur': (0.2338889, 49.4197222), 'Aunay-sur-Odon': (-0.6321995452803886, 49.0154864), 'Hérouville-Saint-Clair': (-0.3353513, 49.2031097), 'Saint-Pierre-sur-Dives': (-0.0333457, 49.0192284), 'Condé-sur-Noireau': (-0.565216636395482, 48.84902185), 'Trouville-sur-Mer': (0.0825, 49.3683333), 'Ouistreham': (-0.258479, 49.2761243), 'Troarn': (-0.1820625, 49.1819395), 'Courseulles-sur-Mer': (-0.4566424, 49.3296058), 'Biéville-Beuville': (-0.3269459, 49.2390116), 'Blainville-sur-Orne': (-0.300348, 49.2283209), 'Villers-sur-Mer': (-0.0052517, 49.3214437), 'Bretteville-sur-Odon': (-0.4176272, 49.1655808), 'Cormelles-le-Royal': (-0.3324069, 49.1546226), 'Mézidon-Canon': (-0.07059065591857375, 49.0792011), 'Lion-sur-Mer': (-0.3160838, 49.3006317), 'Mondeville': (-0.3202231, 49.1771038), 'La Rivière-Saint-Sauveur': (0.2716348, 49.4076106), 'Louvigny': (-0.3937312, 49.1565383), 'Touques': (0.1016715, 49.343477), 'Fontaine-Étoupefour': (-0.453802, 49.1459073), 'Fleury-sur-Orne': (-0.3753212, 49.1478994), 'Colleville-Montgomery': (-0.2970235, 49.27666), 'Argences': (-0.1640626, 49.1270417), 'Douvres-la-Délivrande': (-0.3752486, 49.2970028), 'Carpiquet': (-0.4411659, 49.1834925), 'Villers-Bocage': (2.3141984, 49.9956745), 'Giberville': (-0.2834378, 49.181264), 'Dives-sur-Mer': (-0.1001559, 49.2866417), 'Le Molay-Littry': (-0.8711322, 49.2426447), 'Isigny-sur-Mer': (-1.1008065, 49.3194311), 'Saint-Contest': (-0.4011923, 49.214191), 'Colombelles': (-0.2972472, 49.2041204), 'Saint-Martin-de-Fontenay': (-0.3712366, 49.1171895), 'Moult': (-0.1657186, 49.115483), 'Démouville': (-0.2686813, 49.1781146), "Bretteville-l'Orgueilleuse": (-0.5130521590576549, 49.2137879), 'Verson': (-0.4508333, 49.1555556), 'Hermanville-sur-Mer': (-0.3138889, 49.2844444), 'Luc-sur-Mer': (-0.3555361, 49.3180769), 'Ifs': (-0.3496385, 49.1380679), 'Valdallière': (-0.7241983647769284, 48.8618009), 'Souleuvre en Bocage': (-0.8477914833321669, 48.96157025), 'Condé-en-Normandie': (-0.5483600984601706, 48.88643755), 'Valorbiquet': (0.3366597916787923, 49.0614293), 'Thury-Harcourt-le-Hom': (-0.5164297148958915, 48.986953099999994), "Livarot-Pays-d'Auge": (0.2165978562812309, 49.001345549999996), 'Vire Normandie': (-0.8684694513302919, 48.82015235), 'Saline': (-88.5302584, 37.750075), 'Thue et Mue': (-0.5424107011017105, 49.1962872), 'Moult-Chicheboville': (-0.1891657150797776, 49.112119199999995), 'Saint-Pierre-en-Auge': (0.0364344745481327, 48.987372), "Mézidon Vallée d'Auge": (-0.00045385720584534966, 49.0824073), "Les Monts d'Aunay": (-0.6760951409552416, 48.9907532), 'Noues de Sienne': (-1.0255123680979237, 48.82971105), 'Arpajon-sur-Cère': (2.4570176, 44.9034428), 'Mauriac': (-0.0289804, 44.7450537), 'Ytrac': (2.3633014, 44.9111838), 'Aurillac': (2.4433101, 44.9285441), 'Saint-Flour': (3.5026164, 45.7084425), 'Brie': (3.4833333, 49.6013889), 'Fléac': (-0.5380556, 45.535), "L'Isle-d'Espagnac": (0.2003336, 45.6646231), 'Saint-Yrieix-sur-Charente': (0.1156747, 45.6906297), 'Cognac': (-0.3250175, 45.6931647), 'Angoulême': (0.1576745, 45.6512748), 'Confolens': (0.6728934, 46.0150236), 'Jarnac': (-0.1740904, 45.6791182), 'Barbezieux-Saint-Hilaire': (-0.14194724370412187, 45.481221250000004), 'Champniers': (0.2050544, 45.7145725), 'La Couronne': (0.1013099, 45.608673), 'La Rochefoucauld': (0.39350271291413785, 45.739278), 'Ruffec': (0.1994619, 46.0287057), 'Chasseneuil-sur-Bonnieure': (0.4487238, 45.8239655), 'Châteaubernard': (-0.3151879, 45.6738081), 'Ruelle-sur-Touvre': (0.2186769, 45.6764688), 'Magnac-sur-Touvre': (0.2371275, 45.6658244), 'Châteauneuf-sur-Charente': (-0.0543783, 45.6000693), 'Soyaux': (0.1931511, 45.6412991), 'Roullet-Saint-Estèphe': (0.031769605320544354, 45.577505599999995), 'Gond-Pontouvre': (0.1671129, 45.6677249), 'Montmoreau': (0.15574218295893769, 45.40644745), 'Rouillac': (-2.3667022, 48.3093474), 'La Rochefoucauld-en-Angoumois': (0.3548565852045369, 45.7360333), 'Terres-de-Haute-Charente': (0.5891354482068129, 45.88757235), 'Barbezieux': (-0.1528181, 45.4709161), 'Marennes': (-1.1127949839974698, 45.8224965), 'Saujon': (-0.9244473, 45.6742681), 'Saint-Georges-des-Coteaux': (-0.7111105, 45.763444), 'Marans': (-0.9940538, 46.3101338), 'La Jarrie': (-1.0081735, 46.1290289), "Saint-Pierre-d'Oléron": (-1.3061227, 45.9437695), 'Pons': (-0.5471901, 45.5792284), 'Fouras': (-1.0935837, 45.9883689), 'Montendre': (-0.4095433, 45.285404), 'Soubise': (-1.007683, 45.9262797), 'La Flotte': (-1.325123, 46.1886242), 'Surgères': (-0.7511152, 46.1052715), 'Saint-Georges-de-Didonne': (-0.997949, 45.6024886), "Dolus-d'Oléron": (-1.2619116, 45.9112189), 'Sainte-Marie-de-Ré': (-1.3115373, 46.1491638), 'Meschers-sur-Gironde': (-0.9538537, 45.558105), 'Vaux-sur-Mer': (-1.0623397, 45.645335), 'Nieul-sur-Mer': (-1.16224, 46.2077), 'Bourcefranc-le-Chapus': (-1.1466191, 45.8469846), "L'Houmeau": (-1.18501, 46.1937), 'Saint-Xandre': (-1.1019893, 46.2051404), 'Marsilly': (6.2973783, 49.1000965), 'Saint-Jean-de-Liversay': (-0.8728577, 46.2709561), 'Dompierre-sur-Mer': (-1.0656405, 46.186001), 'Sainte-Soulle': (-1.0157986, 46.188253), 'Angoulins': (-1.1093352, 46.1049651), 'Châtelaillon-Plage': (-1.0903991, 46.0738633), 'Aytré': (-1.1146694, 46.1333325), 'Arvert': (-1.1266705, 45.7409118), 'Royan': (-1.0287636, 45.6245332), 'La Rochelle': (-1.1520434, 46.1591126), 'Périgny': (-0.6053427, 48.9226051), 'Saintes': (-0.6300671, 45.7460663), 'Rochefort': (-0.9687519, 45.9438412), 'Jonzac': (-0.4320984, 45.4454469), 'Échillais': (-0.9524315, 45.8983203), "Saint-Jean-d'Angély": (-0.5195756, 45.9445958), 'Tonnay-Charente': (-0.892337, 45.9444898), 'Médis': (-0.966034, 45.6422545), "Saint-Georges-d'Oléron": (-1.3316424, 45.9790681), "Aigrefeuille-d'Aunis": (-0.9342868, 46.1163928), "Le Château-d'Oléron": (-1.1953301, 45.8866143), 'Saint-Agnant': (-0.96484, 45.86527), 'La Jarne': (-1.072778, 46.128611), 'La Tremblade': (-1.1420694, 45.7687967), 'Lagord': (-1.1511322, 46.186368), 'Gémozac': (-0.6747673, 45.5688319), 'Étaules': (4.9452192, 47.4069368), 'Chaniers': (-0.5578949, 45.718038), 'Breuillet': (-1.0503893, 45.6938702), 'Puilboreau': (-1.11435, 46.1853), 'Saint-Sulpice-de-Royan': (-1.0120694, 45.6727461), 'Saint-Palais-sur-Mer': (-1.0859957, 45.6428769), 'Marennes-Hiers-Brouage': (-1.0679912521809798, 45.8341032), 'Saint-Germain-du-Puy': (2.4823611, 47.0993603), 'La Chapelle-Saint-Ursin': (2.3213011, 47.0627808), 'Saint-Doulchard': (2.3729675, 47.0999731), "La Guerche-sur-l'Aubois": (2.944473, 46.9537428), 'Saint-Florent-sur-Cher': (2.2552381, 46.9921896), 'Méreau': (2.0513, 47.1788), 'Trouy': (2.3565352, 47.0088361), 'Bourges': (2.399125, 47.0811658), 'Vierzon': (2.0691421, 47.2217489), 'Saint-Amand-Montrond': (2.5046503, 46.7227062), 'Aubigny-sur-Nère': (2.4381889, 47.4887418), 'Mehun-sur-Yèvre': (2.2214381, 47.1462117), 'Avord': (2.6511406, 47.0353206), 'Sancoins': (2.9201075, 46.8308788), 'Dun-sur-Auron': (2.5696977, 46.8853506), 'Malemort': (1.5894802604153044, 45.179675599999996), 'Argentat-sur-Dordogne': (1.940425330802527, 45.11658035), 'Brive-la-Gaillarde': (1.5332389, 45.1584982), 'Ussel': (2.3119397, 45.545591), 'Tulle': (1.7706797, 45.2678347), 'Donzenac': (1.5230255, 45.2278169), 'Ussac': (1.5089279, 45.1923638), 'Malemort-sur-Corrèze': (1.5744759370806451, 45.175240200000005), 'Uzerche': (1.5637278, 45.4243315), 'Bort-les-Orgues': (2.4992935, 45.4010305), 'Égletons': (2.045018, 45.4065225), 'Argentat': (1.9238527517102222, 45.0985199), 'Cosnac': (1.58763, 45.131), 'Objat': (1.4097899, 45.2628546), 'Saint-Pantaléon-de-Larche': (1.4456957, 45.1412733), 'Allassac': (1.4755597, 45.257248), 'Grosseto-Prugna': (8.818055732027885, 41.881594050000004), 'Alata': (8.7433833, 41.9768323), 'Bastelicaccia': (8.83624681602052, 41.932943550000005), 'Ajaccio': (8.7376029, 41.9263991), 'Bonifacio': (9.1606179, 41.3878259), 'Sartène': (8.9722646, 41.620958), 'Porto-Vecchio': (9.2794469, 41.5911382), 'Afa': (8.7966629, 41.9838088), 'Propriano': (8.897321033664534, 41.65102485), 'Sarrola-Carcopino': (8.831373535819166, 41.99819295), 'Zonza': (9.1706374, 41.7493663), 'Bastia': (9.4509187, 42.6993979), 'Calvi': (8.7603309, 42.5680204), 'Corte': (9.1511935, 42.3052904), 'Penta-di-Casinca': (9.4595153, 42.4671285), 'Ville-di-Pietrabugno': (9.4321598, 42.7127571), "L'Île-Rousse": (8.9381367, 42.6344797), 'Biguglia': (9.4208224, 42.6269896), 'Borgo': (11.4566004, 46.0533463), 'Furiani': (9.4151285, 42.6584811), 'Ghisonaccia': (9.40536, 42.0149385), 'Ventiseri': (9.3331152, 41.9431721), 'Vescovato': (9.4387142, 42.4936573), 'Lucciana': (9.4175597, 42.5459392), 'San-Martino-di-Lota': (9.4377499, 42.7363431), 'Prunelli-di-Fiumorbo': (9.3246227, 42.0105024), 'Auxonne': (5.3882954, 47.1936509), 'Chenôve': (5.00308, 47.2932), 'Longvic': (5.0574798, 47.2841003), 'Marsannay-la-Côte': (4.9884218, 47.2705441), 'Saint-Apollinaire': (5.0821629, 47.3340704), 'Quetigny': (5.1162694, 47.3125861), 'Chevigny-Saint-Sauveur': (5.1382218, 47.2977106), 'Arc-sur-Tille': (5.1893967, 47.3450572), 'Talant': (5.0047641, 47.3359617), 'Plombières-lès-Dijon': (4.9721576, 47.3396438), 'Fontaine-lès-Dijon': (5.0218591, 47.3416545), 'Genlis': (5.221361, 47.241208), 'Nuits-Saint-Georges': (4.9540989, 47.134979), 'Montbard': (4.3373194, 47.6241674), 'Dijon': (5.0414701, 47.3215806), 'Beaune': (4.8380426, 47.0239064), 'Gevrey-Chambertin': (4.9705666, 47.2261857), 'Is-sur-Tille': (5.1123622, 47.5219136), 'Venarey-les-Laumes': (4.4464783, 47.5421702), 'Semur-en-Auxois': (4.3328705, 47.4902149), 'Châtillon-sur-Seine': (4.5749139, 47.8607467), 'Neuilly-Crimolois': (5.106789, 47.2797526), 'Pordic': (-2.8189686278528208, 48.56955725), 'Quévert': (-2.08416, 48.4628), 'Plouër-sur-Rance': (-2.0030697, 48.5277014), 'Ploubazlanec': (-3.032721, 48.800878), 'Plestin-les-Grèves': (-3.5117711187415352, 48.59726875), 'Plancoët': (-2.2345898, 48.5225082), 'Ploubalay': (-2.1236849993061178, 48.56264455), 'Yffiniac': (-2.6768534, 48.4846721), 'Plémet': (-2.587198219251695, 48.18859125), 'Erquy': (-2.4642309, 48.6286582), 'Saint-Brieuc': (-2.7603283, 48.5141134), 'Guingamp': (-3.1502007, 48.5618483), 'Dinan': (-2.047687, 48.4539775), 'Lannion': (-3.4587994, 48.7322183), 'Léhon': (-2.0558792712367824, 48.43832485), 'Quintin': (-2.9112669, 48.4036054), 'Plaintel': (-2.8210576, 48.4088411), 'Trégastel': (-3.5112796, 48.8267628), 'Pléneuf-Val-André': (-2.5302144762318743, 48.585429149999996), 'Binic': (-2.8248754, 48.6007887), 'Broons': (-2.2620533, 48.3179354), 'Plérin': (-2.7695752, 48.5348817), 'Plœuc-sur-Lié': (-2.7703173992501053, 48.34629875), 'Bégard': (-3.3007868, 48.6298618), 'Merdrignac': (-2.4146917, 48.1900081), 'Plouha': (-2.928296, 48.6755038), 'Plouézec': (-2.9855088, 48.74957), 'Hillion': (-2.6671457, 48.5145266), 'Langueux': (-2.70959, 48.4991), 'Ploufragan': (-2.7933133, 48.4893432), 'Taden': (-2.0165093, 48.4751525), 'Trélivan': (-2.1193713, 48.4321718), 'Ploumagoar': (-3.1315516, 48.544672), 'Grâces': (-3.1846229, 48.5561462), 'Plélo': (-2.9465019, 48.5567179), 'Saint-Quay-Portrieux': (-2.8306322, 48.6521014), 'Perros-Guirec': (-3.4394662, 48.8151133), 'Lamballe': (-2.516502292079859, 48.46244615), 'Rostrenen': (-3.316142, 48.237574), 'Étables-sur-Mer': (-2.8400063011785353, 48.62665135), 'Paimpol': (-3.0484022, 48.7794818), 'Loudéac': (-2.7535644, 48.1762534), 'Pleslin-Trigavou': (-2.076161925465837, 48.52854615), 'Pleudihen-sur-Rance': (-1.9510009, 48.5102781), 'Saint-Cast-le-Guildo': (-2.2502167391995913, 48.6075821), 'Pleumeur-Bodou': (-3.5187179, 48.7732657), 'Lanvallay': (-2.0268152, 48.4555091), 'Plédran': (-2.74218, 48.4434), 'Louannec': (-3.4119987, 48.7939968), 'Ploubezre': (-3.4476934, 48.7039876), 'Trégueux': (-2.7376166, 48.489561), 'Quessoy': (-2.6604289, 48.4208972), 'Pabu': (-3.137756, 48.5880549), 'Plouagat': (-2.988803807818267, 48.531014400000004), 'Trébeurden': (-3.5626633, 48.7694756), 'Le Mené': (-2.5021237041884214, 48.283769050000004), "Plœuc-L'Hermitage": (-2.7850039416742405, 48.3352359), 'Jugon-les-Lacs - Commune nouvelle': (-2.331456614919188, 48.40144075), 'Binic-Étables-sur-Mer': (-2.8392907727959384, 48.616597), 'Beaussais-sur-Mer': (-2.1393342785183203, 48.5626824), 'La Roche-Jaudy': (-3.2426126799398434, 48.730422000000004), 'Châtelaudren-Plouagat': (-2.988803807818267, 48.531014400000004), 'Lamballe-Armor': (-2.4724248506929793, 48.48593005), 'Guéret': (1.8713349, 46.1686865), 'La Souterraine': (1.4861387, 46.2383347), 'Aubusson': (-0.5521714, 48.7842759), 'La Roche-Chalais': (0.006799, 45.152636), 'Prigonrieux': (0.4030054, 44.855306), 'Nontron': (0.6622535, 45.5293074), 'Coulounieix-Chamiers': (0.6922222, 45.1872222), 'Eymet': (0.3984795, 44.6681229), 'Saint-Astier': (0.5227575000000048, 45.13898075), 'Chancelade': (0.6661111, 45.2063889), 'Boulazac': (0.7650347839958426, 45.172486899999996), 'Neuvic': (0.469149, 45.1006366), 'Champcevinel': (0.7272222, 45.2155556), 'Sarlat-la-Canéda': (1.2161321, 44.8879431), 'Périgueux': (0.7184407, 45.1909365), 'Montpon-Ménestérol': (0.15672815508290233, 45.01624955), 'Bergerac': (0.487531, 44.8534568), 'Notre-Dame-de-Sanilhac': (0.7080364376276143, 45.13292945), "Marsac-sur-l'Isle": (0.6619444, 45.1858333), 'Ribérac': (0.3398094, 45.2478033), 'Le Bugue': (0.9273347, 44.9180694), 'Mussidan': (0.3642959, 45.0369509), 'Montignac-Lascaux': (1.1652282, 45.0638704), 'Terrasson-Lavilledieu': (1.3018796, 45.1292139), 'Thiviers': (0.92088, 45.4183646), 'Lamonzie-Saint-Martin': (0.3847222, 44.8433333), 'Lalinde': (0.7387986, 44.8371855), 'La Force': (0.3742764, 44.8704188), 'Trélissac': (0.7835042, 45.1970172), 'Bassillac et Auberoche': (0.8685662334860943, 45.15767985), 'Sanilhac': (4.2497559, 44.5362646), 'Boulazac Isle Manoire': (0.7691334, 45.1801801), 'Brantôme en Périgord': (0.6535460208342219, 45.36202475), 'Charquemont': (6.8237024, 47.2141791), 'Bethoncourt': (6.7984849, 47.5333448), 'Hérimoncourt': (6.8819894, 47.4419332), 'Maîche': (6.800917, 47.2520567), 'Saint-Vit': (5.8100016, 47.1811265), 'Valdahon': (6.3414396, 47.1506866), 'Villers-le-Lac': (6.6690523, 47.0611608), 'Pont-de-Roide-Vermondans': (6.7690738, 47.3864496), 'Seloncourt': (6.8592928, 47.4616568), 'Grand-Charmont': (6.8230306, 47.527578), 'Vieux-Charmont': (6.839312, 47.5237418), 'Voujeaucourt': (6.7705748, 47.4723871), 'Besançon': (6.0243622, 47.2380222), 'Valentigney': (6.8308784, 47.4622925), 'Montbéliard': (6.7977564, 47.5102368), 'Morteau': (6.6047189, 47.0569367), 'Pontarlier': (6.3546345, 46.9040348), 'École-Valentin': (5.98206, 47.272), 'Sochaux': (6.8305344, 47.5138641), 'Étupes': (6.8617959, 47.507832), 'Doubs': (6.235622772820445, 47.06699155), 'Bavans': (6.7297465, 47.4831287), 'Ornans': (6.151746304484529, 47.09472425), 'Thise': (6.0790996, 47.2836567), 'Miserey-Salines': (5.97331, 47.2886014), 'Audincourt': (6.8413684, 47.4819786), 'Baume-les-Dames': (6.3614622, 47.3526707), 'Exincourt': (6.83595, 47.5009), "L'Isle-sur-le-Doubs": (6.5837522, 47.4479052), 'Saône': (6.1168858, 47.2249278), 'Les Fins': (6.6317247, 47.0860106), 'Mandeure': (6.8050839, 47.4519906), 'Les Auxons': (5.9571585, 47.3011685), 'Châteauneuf-sur-Isère': (4.9391394, 45.0148392), 'Livron-sur-Drôme': (4.845507, 44.7726501), 'Peyrins': (5.047774, 45.0922562), 'Saint-Jean-en-Royans': (5.290677, 45.0161452), 'Châteauneuf-du-Rhône': (4.719065, 44.4908219), 'Montmeyran': (4.9772114, 44.8326866), 'Saint-Marcel-lès-Valence': (4.9542484, 44.9711965), 'Malissard': (4.9591496, 44.9006599), 'Donzère': (4.7073439, 44.4442014), 'Chabeuil': (5.0187071, 44.9003613), 'Étoile-sur-Rhône': (4.8934402, 44.8363656), 'Anneyron': (4.8844712, 45.2695877), 'Aouste-sur-Sye': (5.0569017, 44.7154707), 'Beaumont-lès-Valence': (4.9420179, 44.8624984), 'Loriol-sur-Drôme': (4.822279, 44.7518413), 'Portes-lès-Valence': (4.8774936, 44.8753434), 'Montélier': (5.0284008, 44.9383292), "Saint-Donat-sur-l'Herbasse": (4.9822895, 45.1212493), "Pont-de-l'Isère": (4.8714601, 45.004331), 'Mours-Saint-Eusèbe': (5.0510242, 45.070261), 'La Roche-de-Glun': (4.8443331, 45.0133527), 'Valence': (-0.3763353, 39.4697065), 'Die': (139.2394179, 36.5748441), 'Nyons': (5.12838, 44.3555), "Saint-Rambert-d'Albon": (4.8147808, 45.29202), 'Saint-Vallier': (6.3151506, 48.2788193), 'Montélimar': (4.750318, 44.5579391), 'Romans-sur-Isère': (5.0528681, 45.0458886), 'Bourg-de-Péage': (5.0520015, 45.0354375), 'Saint-Paul-Trois-Châteaux': (4.7707325, 44.3475554), 'Bourg-lès-Valence': (4.8953847, 44.9528222), "Tain-l'Hermitage": (4.8370726, 45.0712839), 'Pierrelatte': (4.6976609, 44.3778474), 'Dieulefit': (5.0663637, 44.5236962), 'Alixan': (5.0264482, 44.9750131), 'Crest': (5.0236641, 44.7282675), 'Chatuzange-le-Goubet': (5.0899218, 45.0061626), 'Mercurol-Veaunes': (4.887574406970316, 45.07378975), 'Bernay': (0.5989157, 49.0902278), 'Évreux': (1.1510164, 49.0268903), "Pont-de-l'Arche": (1.1562938, 49.3049098), 'Les Andelys': (1.4215534, 49.2469193), 'Vernon': (1.4886571, 49.0917814), 'Gisors': (1.777826, 49.2789504), 'Verneuil-sur-Avre': (0.9434228814520526, 48.7373168), 'Pacy-sur-Eure': (1.3993509100095425, 49.022157199999995), 'Conches-en-Ouche': (0.9411911, 48.9621726), 'Beaumont-le-Roger': (0.7768341, 49.078161), 'Pont-Audemer': (0.5333167486137402, 49.34758065), 'Le Vaudreuil': (1.204956, 49.2551454), 'Bourgtheroulde-Infreville': (0.8780649692233152, 49.29928105), 'Ivry-la-Bataille': (1.4579416, 48.8852099), 'Breteuil': (2.2939649, 49.6329792), 'Louviers': (1.1646335, 49.2157245), 'Brionne': (0.7181025, 49.1948931), 'Romilly-sur-Andelle': (1.2583031, 49.3314668), 'Beuzeville': (0.3427251, 49.3455531), 'Bourg-Achard': (0.8149501, 49.35255), 'Saint-Sébastien-de-Morsent': (1.0890917, 49.006842), 'Guichainville': (1.1864378, 48.9800436), 'Gaillon': (1.3336126, 49.1605604), 'Saint-Marcel': (1.454725, 49.0984117), 'Gravigny': (1.1624156, 49.0473467), 'Étrépagny': (1.6110749, 49.3065762), 'Le Neubourg': (0.9044712, 49.1496553), 'Val-de-Reuil': (1.2128992, 49.2716839), 'Le Bosc-Roger-en-Roumois': (0.9190382800424554, 49.295798899999994), 'Aubevoye': (1.3331944076081852, 49.17721395), 'Pîtres': (1.2247712, 49.3212629), "Saint-André-de-l'Eure": (1.2735196, 48.9064867), 'Gasny': (1.6031384, 49.090455), 'Ézy-sur-Eure': (1.4198704, 48.8657529), 'Mesnil-en-Ouche': (0.6929441392362816, 48.986905449999995), "Le Thuit de l'Oison": (0.9404133216517714, 49.258027999999996), 'Mesnils-sur-Iton': (1.0632509549219873, 48.86853565), 'Vexin-sur-Epte': (1.5967003814007161, 49.17547615), 'Grand Bourgtheroulde': (0.8689287446752453, 49.3050454), "Le Val d'Hazey": (1.3188976228571019, 49.17136910000001), 'Bosroumois': (0.9188880682137996, 49.291619999999995), 'La Chapelle-Longueville': (1.4269352, 49.1093602), "Verneuil d'Avre et d'Iton": (0.9295044, 48.7386515), 'Chartres': (1.4881434, 48.4438601), 'Dreux': (1.3684254, 48.7358807), 'Châteaudun': (1.3284948, 48.0702298), 'Nogent-le-Rotrou': (0.8211488, 48.321809), 'Épernon': (1.677222, 48.6101745), 'Illiers-Combray': (1.2444186, 48.2994963), 'Bonneval': (1.3853323, 48.181527), 'Voves': (1.632147952379368, 48.269388649999996), 'Cloyes-sur-le-Loir': (1.2240445613419637, 47.99095475), 'Châteauneuf-en-Thymerais': (1.2411312, 48.583196), 'Nogent-le-Roi': (1.5322266, 48.6485761), 'La Loupe': (1.02521, 48.4737), 'Anet': (1.4391896, 48.8552557), 'Saint-Georges-sur-Eure': (1.3552631, 48.4182576), 'Auneau': (1.7714868478474783, 48.4705667), 'Vernouillet': (1.3650922, 48.7166545), 'Maintenon': (1.5773313, 48.5865517), 'Toury': (1.9363143, 48.1962708), 'Champhol': (1.5082448, 48.4685197), 'Mainvilliers': (1.4580047, 48.4489382), 'Brou': (1.1660734, 48.2156349), 'Le Coudray': (1.5011729, 48.4205915), 'Saint-Rémy-sur-Avre': (1.2405119, 48.7628087), 'Saint-Lubin-des-Joncherets': (1.1978005, 48.7673038), 'Luisant': (1.46835, 48.4231), 'Hanches': (1.642421, 48.6006261), 'Lèves': (1.4786967, 48.4782477), 'Gallardon': (1.6896881, 48.5243351), 'Senonches': (1.0322607, 48.5595325), 'Pierres': (-0.7341505, 48.8713065), 'Lucé': (-0.595787, 48.5500968), 'Courville-sur-Eure': (1.2387543, 48.4502626), 'Les Villages Vovéens': (1.5994798162289623, 48.27180505), 'Auneau-Bleury-Saint-Symphorien': (1.7740897343038697, 48.499747850000006), 'Cloyes-les-Trois-Rivières': (1.2342202, 47.9972475), "Vald'Yerre": (1.1224687, 48.0995846), 'Janville-en-Beauce': (1.8468607620260338, 48.1998733), 'Riec-sur-Bélon': (-3.6903285, 47.8428705), 'Ergué-Gabéric': (-4.026792, 47.9965297), 'Pluguffan': (-4.1785594, 47.9805233), 'Kernével': (-3.7778712, 47.9585735), 'Quimper': (-4.1024782, 47.9960325), 'Ploudalmézeau': (-4.6576826, 48.5410278), 'Brest': (-4.4860088, 48.3905283), 'Concarneau': (-3.9223889, 47.8757017), 'Plougasnou': (-3.7922304, 48.6959132), 'Penmarch': (-4.3377655, 47.8126044), 'Morlaix': (-3.8331972, 48.5824932), 'Châteaulin': (-4.0916875, 48.1968026), 'Saint-Yvi': (-3.9371068, 47.9673654), 'Carhaix-Plouguer': (-3.5711477, 48.2764997), 'Plouarzel': (-4.7309245, 48.433792), 'Plouvorn': (-4.0229425, 48.5778741), 'Moëlan-sur-Mer': (-3.6309833, 47.8133996), 'Plouvien': (-4.451225, 48.5305465), 'Pleuven': (-4.0434663, 47.9075933), 'Plouigneau': (-3.697677101963496, 48.571968350000006), 'Crozon': (-4.4894463, 48.2463989), 'Plabennec': (-4.4268474, 48.5020355), 'Plourin-lès-Morlaix': (-3.7879531, 48.5368636), 'Cléder': (-4.1039679, 48.6612675), 'Le Folgoët': (-4.3400067, 48.5635555), 'Plougonvelin': (-4.7192399, 48.3415116), 'Plomelin': (-4.1528995, 47.9352286), "Pont-l'Abbé": (-4.2208461, 47.86592), 'Elliant': (-3.8905448, 47.9951366), 'Bourg-Blanc': (-4.5060451, 48.4991511), 'Saint-Thégonnec Loc-Eguiner': (-3.946256997532075, 48.51119455), 'Milizac-Guipronvel': (-4.56876657572999, 48.47059005), 'Audierne': (-4.545659011908988, 48.02308095), 'Quimperlé': (-3.5505644, 47.8692882), 'Pont-Aven': (-3.7462556, 47.85474), 'Saint-Pol-de-Léon': (-3.9868641, 48.6849601), 'Douarnenez': (-4.3311243, 48.0944459), 'Fouesnant': (-4.0115365, 47.8945402), 'Briec': (-3.999718, 48.1019452), 'Roscoff': (-3.9828983, 48.7255816), 'Lesneven': (-4.3235917, 48.5718493), 'Saint-Renan': (-4.622982, 48.4328451), 'Scaër': (-3.7020023, 48.0268097), 'Bannalec': (-3.6981199, 47.9325142), 'Guipavas': (-4.400979, 48.4346366), 'Landerneau': (-4.2557901, 48.4514803), 'Le Conquet': (-4.7709059, 48.3605295), 'Lannilis': (-4.5215472, 48.569633), 'Taulé': (-3.8995821, 48.603826), 'Rosporden': (-3.8278459, 47.9592684), 'Gouesnou': (-4.465365, 48.451729), 'Guilers': (-4.5583284, 48.4244717), 'Landivisiau': (-4.0699356, 48.5127551), 'Bénodet': (-4.1109828, 47.8746373), 'Saint-Thégonnec': (-3.937426073027055, 48.52303594999999), 'Combrit': (-4.1563101, 47.8873445), 'Guilvinec': (-4.285825639786783, 47.799514599999995), 'Châteauneuf-du-Faou': (-3.8128592, 48.1862985), 'Trégunc': (-3.8514269, 47.854485), 'Gouesnach': (-4.1167181, 47.9119674), 'La Forêt-Fouesnant': (-3.979665, 47.9081865), 'Plougastel-Daoulas': (-4.369679, 48.3742317), 'Loctudy': (-4.1750385, 47.8316473), 'Locmaria-Plouzané': (-4.6444051, 48.3747011), 'Plouescat': (-4.1743159, 48.6572057), 'Clohars-Carnoët': (-3.5826145, 47.7955209), 'Carantec': (-3.9142802, 48.6684418), 'Bohars': (-4.5155977, 48.428823), 'Plogonnec': (-4.1915768, 48.0776491), 'Plouédern': (-4.2459354, 48.4841517), 'Plozévet': (-4.4258889, 47.9859232), 'Plouzané': (-4.6212044, 48.3817165), 'Guiclan': (-3.9624251, 48.5506172), 'Saint-Martin-des-Champs': (-1.3303759, 48.6701144), 'Le Relecq-Kerhuon': (-4.3965573, 48.4075514), 'Rédené': (-3.4613833, 47.8600854), 'Saint-Évarzec': (-4.0217135, 47.9367647), 'Plomeur': (-4.2835502, 47.8411013), 'Milizac': (-4.557663931865585, 48.45694885), 'Pleyber-Christ': (-3.8753049, 48.5047728), 'Landéda': (-4.5720236, 48.5870031), 'Plouguerneau': (-4.506818, 48.6063358), 'Névez': (-3.7921065, 47.8201167), 'Pleyben': (-3.9688719, 48.226109), 'Plonéour-Lanvern': (-4.2826125, 47.9030858), 'Loperhet': (-4.3065629, 48.3754847), 'Melgven': (-3.8349374, 47.9063592), 'Plouhinec': (-4.4877863, 48.0153626), 'Pont-de-Buis-lès-Quimerch': (-4.0859125, 48.2556905), 'Plouénan': (-3.988968, 48.6277816), 'Mellac': (-3.5775882, 47.903995), 'Ploudaniel': (-4.3100487, 48.5369612), 'Plobannalec-Lesconil': (-4.230578491285426, 47.820883949999995), 'Plougonven': (-3.7122667, 48.5212583), 'Bouillargues': (4.4248446, 43.8007432), 'Saint-Laurent-des-Arbres': (4.6991386, 44.0548235), 'Générac': (-0.541972, 45.1717613), 'Bagard': (4.05181, 44.0725), 'Sommières': (4.0893472, 43.7824191), 'Saint-Christol-lez-Alès': (4.07425, 44.0844), 'Aubais': (4.14898, 43.7546), 'Saint-Julien-les-Rosiers': (4.1089897, 44.1739345), 'Saint-Hippolyte-du-Fort': (3.8564364, 43.9643602), 'Caveirac': (4.2627261, 43.825366), "Saint-Laurent-d'Aigouze": (4.1957274, 43.6343255), 'Saint-Hilaire-de-Brethmas': (4.12613, 44.0938), 'Milhaud': (4.3084497, 43.7896802), 'Saint-Martin-de-Valgalgues': (4.0832219, 44.1620752), 'Vauvert': (4.2754772, 43.6924382), 'Boisset-et-Gaujac': (4.0211786, 44.0494413), 'Rousson': (3.2637181, 48.0957914), 'Villeneuve-lès-Avignon': (4.7969224, 43.9636595), 'Vergèze': (4.2204128, 43.7426569), 'Caissargues': (4.3783053, 43.7961462), 'Bellegarde': (0.6244444, 43.4255556), 'Aramon': (4.6810542, 43.8908652), 'Bessèges': (4.1032582, 44.2899584), 'Poulx': (4.42408, 43.9105), 'Rochefort-du-Gard': (4.6903765, 43.9738383), 'Salindres': (4.1594591, 44.167041), 'Bagnols-sur-Cèze': (4.6235831, 44.160481), 'Beauvoisin': (4.3233989, 43.7189318), 'Roquemaure': (4.7788273, 44.0523514), 'Le Grau-du-Roi': (4.1360255, 43.5361048), 'Saint-Ambroix': (4.19978, 44.2600172), 'Redessan': (4.4961043, 43.8318215), 'Jonquières-Saint-Vincent': (4.5559553, 43.8304027), 'Les Salles-du-Gardon': (4.0450517, 44.1969162), 'Saint-Quentin-la-Poterie': (4.4416529, 44.0444747), 'Calvisson': (4.1900505, 43.7853458), 'Nîmes': (4.3600687, 43.8374249), 'Aigues-Vives': (2.5338256, 43.2310412), 'Gallargues-le-Montueux': (4.1743949, 43.7221345), 'Garons': (4.4268453, 43.7694841), 'Alès': (4.0852818, 44.1253665), 'Aigues-Mortes': (4.1912837, 43.5658225), 'Le Vigan': (3.6059407, 43.9912933), 'Beaucaire': (4.6392648, 43.8085436), 'Uzès': (4.4196718, 44.0121279), 'Quissac': (4.0011403, 43.9104511), 'La Grand-Combe': (4.0294228, 44.2102417), 'Anduze': (3.9857082, 44.0545668), "Laudun-l'Ardoise": (4.657231, 44.1056956), 'Saint-Gilles': (3.6756348, 49.2801594), 'Pont-Saint-Esprit': (4.6456631, 44.2538589), 'Aimargues': (4.2095204, 43.683958), 'Uchaud': (4.26861, 43.7572), 'Manduel': (4.4732242, 43.8185561), 'Saint-Privat-des-Vieux': (4.12863, 44.1421), 'Marguerittes': (4.45426, 43.8629), 'Meynes': (4.5614981, 43.8827571), 'Fourques': (4.60931, 43.692905), 'Saint-Geniès-de-Malgoirès': (4.2169644, 43.9461726), 'Bernis': (4.2860742, 43.7656043), 'Rodilhan': (4.434929, 43.8257589), 'Les Angles': (0.0062129, 43.0835762), 'Clarensac': (4.2176541, 43.8266904), 'Montfrin': (4.5911803, 43.8765181), 'Pujaut': (4.7721516, 44.0034948), 'Rieux-Volvestre': (1.20185, 43.258), 'Villefranche-de-Lauragais': (1.7151072, 43.4004663), 'Grenade': (-61.6904045, 12.1360374), 'Montastruc-la-Conseillère': (1.591173, 43.7170378), 'Pibrac': (1.2837373, 43.6189112), 'Baziège': (1.61312, 43.4552), 'Fronton': (1.390285, 43.840295), 'Beauzelle': (1.37302, 43.6689), 'Eaunes': (1.3569079, 43.4247066), 'Merville': (2.6387499, 50.6436584), 'Verfeil': (1.6606008, 43.6570422), 'Frouzins': (1.3228991, 43.516101), 'Villeneuve-Tolosane': (1.3410072, 43.5236403), 'Montberon': (1.4797585, 43.716024), 'Labastidette': (1.247098, 43.4627021), 'Fonsorbes': (1.2314988, 43.5353273), 'Seysses': (1.3115111, 43.4982966), 'Noé': (1.2749497, 43.3545352), 'Miremont': (1.4158823, 43.3692815), 'Roques': (1.3755068, 43.5109039), 'Toulouse': (1.4442469, 43.6044622), 'Fenouillet': (1.3915391, 43.6790033), 'Muret': (1.3262332, 43.4599858), 'Saint-Gaudens': (0.7247218, 43.1077682), 'Auzeville-Tolosane': (1.4829226, 43.5292199), 'Blagnac': (1.37799, 43.6434), 'Colomiers': (1.3282149, 43.6121001), 'Tournefeuille': (1.3466543, 43.5827846), 'Auterive': (1.4775095, 43.3524028), 'Revel': (24.7453688, 59.4372155), 'Montréjeau': (0.5685313, 43.0855843), 'Fontenilles': (1.1911389, 43.5527351), 'Launaguet': (1.4544605, 43.6757142), 'Nailloux': (1.6233628, 43.3558921), 'Plaisance-du-Touch': (1.2978343, 43.5653012), 'Castelmaurou': (1.5325286, 43.6777525), 'Cugnaux': (1.3457537, 43.5369936), 'Aussonne': (1.31799, 43.6828953), 'Ayguesvives': (1.5967912, 43.4381888), 'Balma': (1.5057917, 43.6112217), 'Rieumes': (1.1187328, 43.4123126), 'Cazères': (1.0865093, 43.2056788), 'Caraman': (1.7600805, 43.5292631), 'Carbonne': (1.2250686, 43.2958159), 'Léguevin': (1.2344124, 43.5991512), 'Montesquieu-Volvestre': (1.2301306, 43.2086251), 'Cintegabelle': (1.5313506, 43.3141053), 'Castanet-Tolosan': (1.4980801, 43.517214), 'Montgiscard': (1.5744528, 43.4580362), 'Saint-Lys': (1.1775205, 43.5140434), 'Roquettes': (1.3653644, 43.499511), 'Pins-Justaret': (1.3847977, 43.481576), 'Bruguières': (1.411126, 43.7277106), 'Cornebarrieu': (1.3225828, 43.6479976), 'Buzet-sur-Tarn': (1.6318417, 43.7790638), 'Aucamville': (1.4291655, 43.6687865), 'Quint-Fonsegrives': (1.5308893, 43.5854254), "Castelnau-d'Estrétefonds": (1.3590892, 43.78251), 'Lavernose-Lacasse': (1.2620204, 43.396482), 'Bessières': (1.6076595, 43.8002365), 'Portet-sur-Garonne': (1.4018308, 43.5283006), 'Villemur-sur-Tarn': (1.5039825, 43.8650448), "L'Union": (1.4846885, 43.6501518), 'La Salvetat-Saint-Gilles': (1.2680778, 43.5765783), 'Saint-Jean': (1.5039252, 43.6650391), 'Saint-Geniès-Bellevue': (1.4854271, 43.68255), 'Montrabé': (1.5251637, 43.6425112), 'Pinsaguel': (1.3884681, 43.5099932), 'Mondonville': (1.2879155, 43.6710689), 'Escalquens': (1.5620474, 43.51833), 'Lherm': (1.2223771, 43.4297851), 'Lagardelle-sur-Lèze': (1.388074, 43.4143699), 'Saint-Orens-de-Gameville': (1.5339526, 43.5519847), 'Saint-Jory': (1.3713804, 43.7417552), 'Vernet': (1.4204359935732653, 43.427298), 'Seilh': (1.3534881, 43.694573), 'Saint-Alban': (1.4135768, 43.6923207), 'Lapeyrouse-Fossat': (1.5170896, 43.6927081), 'Longages': (1.2399766, 43.3555742), 'Gratentour': (1.4320099, 43.7217932), 'Pechbonnieu': (1.4670512, 43.7027161), 'Labarthe-sur-Lèze': (1.4005078, 43.4524351), 'Drémil-Lafage': (1.6031831, 43.5977998), 'Ramonville-Saint-Agne': (1.4749101, 43.5460533), 'Venerque': (1.4424794, 43.4333976), 'Bouloc': (1.4051212, 43.7809538), 'Bérat': (1.1759296, 43.3793823), 'Labège': (1.5299721, 43.5292828), 'Lespinasse': (1.3854591, 43.7077986), 'Castelginest': (1.4281537, 43.6929231), 'Fonbeauzard': (1.4321168, 43.6762879), 'Brax': (1.2404373, 43.6168216), 'Gagnac-sur-Garonne': (1.374462, 43.6981454), 'Vic-Fezensac': (0.3041667, 43.7594444), 'Eauze': (0.1007959, 43.8613741), "L'Isle-Jourdain": (1.0816667, 43.6144444), 'Auch': (0.5850507, 43.6463558), 'Condom': (0.3745929, 43.9564563), 'Gimont': (0.8776713, 43.6273203), 'Pavie': (9.137825082826026, 45.036854649999995), 'Lectoure': (0.6219444, 43.9352778), 'Fleurance': (0.6640617, 43.849295), 'Mirande': (0.4041473, 43.5143244), 'Saint-Ciers-sur-Gironde': (-0.6158131, 45.2934579), 'Saint-Savin': (-0.4464038, 45.139407), 'Floirac': (-0.520823, 44.8327), 'Lège-Cap-Ferret': (-1.1472098, 44.7951052), 'Saint-Loubès': (-0.4271005, 44.9167311), 'Saint-Laurent-Médoc': (-0.8216559, 45.1515584), 'Blaye': (-0.6635964, 45.1271231), 'Mérignac': (-0.6469022, 44.842168), 'Langon': (-0.2449274, 44.5530659), 'Pessac': (-0.6308396, 44.805615), 'Bazas': (-0.2126009, 44.4320456), 'Andernos-les-Bains': (-1.0925184, 44.7476897), 'Lesparre-Médoc': (-0.9379402, 45.3081854), 'Audenge': (-1.0198568, 44.6883648), 'Blanquefort': (-0.6300363, 44.9088982), 'Cubzac-les-Ponts': (-0.454723, 44.9711192), 'La Teste-de-Buch': (-1.1491319, 44.6316943), 'Gradignan': (-0.613907, 44.7739701), 'Lormont': (-0.5216683, 44.8791708), "Villenave-d'Ornon": (-0.5595029, 44.7737872), 'Saint-Médard-en-Jalles': (-0.7170764, 44.8958887), 'Ambès': (-0.5291847, 45.0121558), 'Ambarès-et-Lagrave': (-0.4881846, 44.9259011), 'Bassens': (-0.5163222, 44.9015003), 'Belin-Béliet': (-0.7906699, 44.4930554), 'Coutras': (-0.1286261, 45.0409876), 'La Brède': (-0.5279844, 44.6811652), 'Castelnau-de-Médoc': (-0.7990007, 45.0272712), 'Bouliac': (-0.5036626, 44.8140115), 'La Réole': (-0.0350566, 44.5821161), 'Castillon-la-Bataille': (-0.0450873, 44.8533324), 'Arès': (-1.1379928, 44.767644), 'Pauillac': (-0.7484994, 45.1995626), 'Saint-Quentin-de-Baron': (-0.2864432, 44.8226575), 'Sainte-Hélène': (6.6572736, 48.2911559), 'Le Porge': (-1.0927165, 44.8727944), 'Saint-Denis-de-Pile': (-0.2054117, 44.9918062), 'Sainte-Eulalie': (-0.4742854, 44.9061686), 'Pineuilh': (0.2266807, 44.830228), 'Cézac': (-0.42033, 45.09053), 'Le Haillan': (-0.6761661, 44.872105), 'Izon': (-0.363, 44.92014), 'Bordeaux': (-0.5800364, 44.841225), 'Libourne': (-0.2427217, 44.9133742), 'Camblanes-et-Meynac': (-0.4851871, 44.7683222), 'Le Teich': (-1.0216522, 44.6357183), 'Ludon-Médoc': (-0.6037032, 44.9818777), 'Martignas-sur-Jalle': (-0.7761489, 44.8405364), 'Avensan': (-0.7575952, 45.036048), 'Sallebœuf': (-0.396393, 44.8433337), 'Le Barp': (-0.7697752, 44.6063662), 'Arcachon': (-1.1700018, 44.6616428), 'Bègles': (-0.5488542, 44.8074559), 'Le Bouscat': (-0.5992399, 44.8661755), 'Talence': (-0.5879629, 44.8088438), 'Créon': (-0.3482858, 44.7746764), 'Marcheprime': (-0.8534089, 44.6920229), 'Podensac': (-0.3557651, 44.6508185), 'Biganos': (-0.9766226, 44.642014), 'Cadaujac': (-0.5300643, 44.7552279), 'Sainte-Foy-la-Grande': (0.214969, 44.8416496), 'Cestas': (-0.6841225, 44.7411574), 'Bruges': (-0.6111474, 44.8842379), 'Martillac': (-0.5422302, 44.7141704), 'Cadillac-sur-Garonne': (-0.3173533, 44.6360817), 'Lanton': (-1.0360314, 44.7035574), 'Carignan-de-Bordeaux': (-0.474739, 44.812928), 'Listrac-Médoc': (-0.7905774, 45.0736769), 'Tresses': (-0.4635976, 44.8493612), 'Portets': (-0.4238244, 44.6959687), 'Yvrac': (-0.4618132, 44.8807908), 'Hourtin': (-1.0575334, 45.1861565), 'Carbon-Blanc': (-0.50503, 44.8937722), 'Canéjan': (-0.6540587, 44.7626562), "Saint-Médard-d'Eyrans": (-0.5082713, 44.7137024), 'Fargues-Saint-Hilaire': (-0.4440284, 44.8230228), 'Pompignac': (-0.43736, 44.85028), 'Saucats': (-0.5958896, 44.652798), 'Eysines': (-0.6486943, 44.8854667), 'Toulenne': (-0.26324, 44.55729), 'Salles': (-0.8699921, 44.550875), 'Léognan': (-0.5997267, 44.7294474), 'Soulac-sur-Mer': (-1.1228789, 45.513149), 'Galgon': (-0.2723327, 44.9907477), 'Latresne': (-0.4968888, 44.7846424), 'Saint-Caprais-de-Bordeaux': (-0.4363812, 44.7504655), 'Saint-Sulpice-et-Cameyrac': (-0.3909503, 44.9101967), 'Mios': (-0.9349464, 44.6045208), 'Saint-Selve': (-0.4808545, 44.6704442), 'Lacanau': (-1.0784851, 44.9776088), 'Sadirac': (-0.4105625, 44.7806727), 'Gujan-Mestras': (-1.0714014, 44.6360982), 'Artigues-près-Bordeaux': (-0.492353, 44.8599252), 'Saint-André-de-Cubzac': (-0.4440957, 44.9954151), 'Parempuyre': (-0.6052708, 44.9485379), 'Arsac': (-0.6883739, 44.9971866), 'Le Taillan-Médoc': (-0.6693402, 44.9044049), 'Macau': (113.5380454, 22.1899448), "Saint-Seurin-sur-l'Isle": (-0.0009251, 45.0124405), 'Laruscade': (-0.34118, 45.10735), 'La Lande-de-Fronsac': (-0.3814022, 44.9787083), "Saint-Jean-d'Illac": (-0.7815257, 44.8101856), 'Saint-Aubin-de-Médoc': (-0.7246734, 44.9134219), 'Vayres': (-0.3205945, 44.8974025), 'Le Pian-Médoc': (-0.6683986, 44.9549001), 'Montussan': (-0.4286074, 44.8816561), 'Cenon': (-0.5239863, 44.8552882), 'Val de Virvée': (-0.3930986350671708, 45.02103765), 'Margaux-Cantenac': (-0.6702013282453639, 45.03640425), 'Montpellier': (3.8767337, 43.6112422), 'Sussargues': (4.003973, 43.7131593), 'Béziers': (3.2131307, 43.3426562), 'Saint-Gély-du-Fesc': (3.803981, 43.6913873), 'Paulhan': (3.4604451, 43.5418008), 'Saint-Brès': (0.7705556, 43.7797222), 'Gigean': (3.7108476, 43.5008343), 'Saint-Mathieu-de-Tréviers': (3.8644554, 43.7694101), 'Saint-Aunès': (3.9637108, 43.6394182), 'Castries': (3.9868904, 43.6779101), 'Montferrier-sur-Lez': (3.8613428, 43.6665244), 'Sète': (3.6959771, 43.4014434), 'Agde': (3.4771629, 43.3134787), 'Saint-Jean-de-Védas': (3.8264197, 43.5751791), 'Lavérune': (3.8036308, 43.5848366), 'Lattes': (3.9017114, 43.5653993), 'Caux': (3.3635759, 43.509654), 'Cournonterral': (3.7188857, 43.557935), 'Saint-Just': (0.5078478, 45.3348055), 'Prades-le-Lez': (3.862452, 43.6978777), 'Jacou': (3.9107676, 43.6611507), 'Lodève': (3.31892, 43.73106), 'Montarnaud': (3.69844, 43.6495), "Saint-Georges-d'Orques": (3.7788383, 43.6105846), 'Lunel-Viel': (4.0902157, 43.677906), 'Frontignan': (3.7530636, 43.4487615), "Clermont-l'Hérault": (3.4302412, 43.6263176), 'Murviel-lès-Béziers': (3.1436776, 43.4400972), 'Maraussan': (3.15399, 43.3696), 'Vendres': (3.2237054, 43.2686109), 'Juvignac': (3.8101646, 43.6140784), 'Bessan': (3.4263361, 43.3606788), 'Nissan-lez-Enserune': (3.1279229, 43.2900051), 'Puisserguier': (3.0408301, 43.3679188), 'Capestang': (3.0457226, 43.3265792), 'Bédarieux': (3.1576143, 43.6155109), 'Lunel': (4.1348062, 43.6745265), 'Aniane': (3.5916694, 43.6885589), 'Mèze': (3.5988383, 43.4245985), 'Thézan-lès-Béziers': (3.1704075, 43.4217929), 'La Grande-Motte': (4.0860137, 43.5602934), 'Castelnau-le-Lez': (3.9133311, 43.6393171), 'Saint-Drézéry': (3.9775826, 43.7314311), 'Fabrègues': (3.7776283, 43.550677), 'Saint-André-de-Sangonis': (3.5021158, 43.6499836), 'Pézenas': (3.4228323, 43.4611777), 'Montblanc': (3.3672892, 43.3958648), 'Balaruc-le-Vieux': (3.6844938, 43.4603178), 'Vic-la-Gardiole': (3.7961574, 43.4869218), 'Villeneuve-lès-Maguelone': (3.8627227, 43.5328117), 'Marseillan': (0.2138821, 43.3021197), 'Saint-Thibéry': (3.4178651, 43.3977994), 'Ganges': (3.7070719, 43.935142), 'Saint-Clément-de-Rivière': (3.8466598, 43.6843999), 'Sérignan': (3.2798682, 43.2830736), 'Valras-Plage': (3.29192, 43.2496196), 'Mauguio': (4.0098366, 43.6154825), 'Portiragnes': (3.33477, 43.3036), 'Vias': (3.4201599, 43.3107683), 'Palavas-les-Flots': (3.9315347, 43.5278137), 'Balaruc-les-Bains': (3.679545, 43.4444987), 'Teyran': (3.9247682, 43.6829748), 'Clapiers': (3.8868606, 43.6567216), 'Mireval': (3.8018991, 43.5082254), 'Canet': (2.8474926, 43.2292848), 'Cers': (3.3057088, 43.3232846), 'Pignan': (3.7622591, 43.5823652), 'Montagnac': (4.14787, 43.9366), 'Gignac': (3.5512018, 43.6522859), 'Cazouls-lès-Béziers': (3.1013558, 43.3918541), 'Colombiers': (-0.5500038, 45.6444077), 'Boujan-sur-Libron': (3.2483732, 43.3721159), 'Servian': (3.2983696, 43.427043), 'Baillargues': (4.0133114, 43.6609569), 'Cournonsec': (3.6999915, 43.5482372), 'Saint-Martin-de-Londres': (3.7329918, 43.7914507), 'Pérols': (3.9502006, 43.5586003), 'Le Crès': (3.9369165, 43.6500649), 'Villeveyrac': (3.607502, 43.5017177), 'Lespignan': (3.16982, 43.2743), 'Lansargues': (4.0727267, 43.6508831), 'Vendargues': (3.9687448, 43.6568263), 'Mudaison': (4.041147, 43.6493916), 'Lignan-sur-Orb': (3.17294, 43.3828), 'Montbazin': (3.6942986, 43.5158481), 'Magalas': (3.2221621, 43.4724204), 'Poussan': (3.670466, 43.4886565), 'Montady': (3.1254244, 43.3349069), 'Florensac': (3.4631135, 43.3827078), 'Villeneuve-lès-Béziers': (3.2797511, 43.3162817), 'Vailhauquès': (3.7170799, 43.6716001), 'Grabels': (3.7964657, 43.6473888), 'Sauvian': (3.25476, 43.2917), 'Marsillargues': (4.1795225, 43.6658621), 'Rennes': (-1.6800198, 48.1113387), 'Fougères': (-1.1958526, 48.3503362), 'Dinard': (-2.0580178, 48.6320379), 'Montauban-de-Bretagne': (-2.0462319, 48.1998838), 'Saint-Malo': (-2.0260409, 48.649518), 'Saint-Aubin-du-Cormier': (-1.4024515, 48.2577823), 'Plerguer': (-1.8461116, 48.5272074), 'Piré-sur-Seiche': (-1.4299358, 48.0092872), 'Cancale': (-1.8506953, 48.6771721), 'Redon': (-2.0842143, 47.6510682), 'Bréal-sous-Montfort': (-1.867686, 48.0513567), 'Breteil': (-1.8996064, 48.1459113), 'Bédée': (-1.9448081, 48.1799946), 'Étrelles': (-1.1938644, 48.0601694), 'Bains-sur-Oust': (-2.0720346, 47.7043242), 'Saint-Coulomb': (-1.9114049, 48.6755188), 'Crevin': (-1.6629333, 47.9374005), 'Bruz': (-1.7471629, 48.0245814), 'Chantepie': (-1.6171482, 48.088155), 'Pont-Péan': (-1.7023101, 48.0131094), 'Saint-Domineuc': (-1.8760754, 48.3727602), 'Gévezé': (-1.7896366, 48.2204891), 'Saint-Grégoire': (-1.685398, 48.1524129), 'Nouvoitou': (-1.548064, 48.0414284), "L'Hermitage": (-1.8183, 48.1243), 'Saint-Pierre-de-Plesguen': (-1.8843121606715008, 48.45620865), 'Janzé': (-1.495366, 47.9556915), 'Saint-Jouan-des-Guérets': (-1.9730216, 48.5991065), 'Orgères': (0.3418072, 48.7635141), 'La Fresnais': (-1.8437735, 48.5952811), 'Miniac-Morvan': (-1.8995937, 48.5150492), 'La Chapelle-des-Fougeretz': (-1.7341163, 48.178066), 'Louvigné-du-Désert': (-1.1231428, 48.4815182), 'Bourg-des-Comptes': (-1.7388742, 47.9298273), 'Chavagne': (-1.7881749, 48.0539088), 'Guichen': (-1.7970987, 47.9659271), 'Vezin-le-Coquet': (-1.751177, 48.1180947), 'Pacé': (-0.0039061, 48.4454551), 'Corps-Nuds': (-1.5874103, 47.9775099), 'Bourgbarré': (-1.6162609, 47.9958761), 'Pléchâtel': (-1.7496755, 47.8947671), 'Saint-Méloir-des-Ondes': (-1.9051339, 48.6380887), 'Bain-de-Bretagne': (-1.6852229, 47.8414377), 'Cesson-Sévigné': (-1.6038829, 48.1197799), 'Montgermont': (-1.7162062, 48.1557367), 'Montfort-sur-Meu': (-1.955145, 48.1372782), 'Vitré': (-1.214331, 48.1244503), 'Retiers': (-1.3818706, 47.9128732), 'Dol-de-Bretagne': (-1.754257, 48.5498295), 'Combourg': (-1.7500622, 48.4104391), 'Châteaugiron': (-1.5009120306356485, 48.042376399999995), 'Saint-Méen-le-Grand': (-2.1911338, 48.1894709), 'Châteaubourg': (-1.3982411, 48.1108554), 'Tinténiac': (-1.8352875, 48.3293178), 'La Guerche-de-Bretagne': (-1.231454, 47.9426332), 'Iffendic': (-2.0336601, 48.1290768), 'Saint-Jacques-de-la-Lande': (-1.7135029, 48.0806436), 'Pipriac': (-1.9462224, 47.8088832), 'Domloup': (-1.5236949, 48.0626279), 'Sens-de-Bretagne': (-1.5359128, 48.332872), 'Noyal-sur-Vilaine': (-1.5234735, 48.1140789), 'Chartres-de-Bretagne': (-1.7082369, 48.0432406), 'Pleumeleuc': (-1.9191125, 48.1844169), 'Pleurtuit': (-2.05912, 48.5815), 'Romillé': (-1.89245, 48.2153539), 'Betton': (-1.6414898, 48.1817122), 'Melesse': (-1.6962403, 48.2175823), 'Argentré-du-Plessis': (-1.1525093, 48.0574022), 'Vern-sur-Seiche': (-1.6045972, 48.0452053), 'Saint-Erblon': (-1.651345, 48.0181129), 'Plélan-le-Grand': (-2.1003629, 48.0023864), 'Mordelles': (-1.8485204, 48.0733304), "Saint-Aubin-d'Aubigné": (-1.607077, 48.2630217), 'Liffré': (-1.5059748, 48.2145992), 'Maure-de-Bretagne': (-2.011194693475463, 47.895945749999996), 'Saint-Brice-en-Coglès': (-1.3750414419526775, 48.412237700000006), 'Acigné': (-1.5338474, 48.1334094), 'Noyal-Châtillon-sur-Seiche': (-1.6619865, 48.0425725), 'Laillé': (-1.7199066, 47.9784056), 'La Mézière': (-1.7557922, 48.2193772), 'La Bouëxière': (-1.437582, 48.1838953), 'Servon-sur-Vilaine': (-1.4603703, 48.121469), 'Thorigné-Fouillard': (-1.5800238, 48.1535233), "Val-d'Izé": (-1.3018405, 48.1772441), 'Martigné-Ferchaud': (-1.3175553, 47.8312705), 'Guipry': (-1.8537287632953334, 47.827726), 'Guignen': (-1.8613166, 47.9205654), 'Lécousse': (-1.2177018, 48.3683827), 'Goven': (-1.8476016, 48.0054828), 'Le Rheu': (-1.7937675, 48.1006715), 'Guipry-Messac': (-1.8172915901829165, 47.8257106), 'Maen Roch': (-1.3402979215432647, 48.3973839), "Val d'Anast": (-2.0224623314770716, 47.9002239), 'Val-Couesnon': (-1.4566868194322078, 48.44089055), 'Rives-du-Couesnon': (-1.3373151688416574, 48.28779845), "Mesnil-Roc'h": (-1.874167206462296, 48.46380415), 'Piré-Chancé': (-1.4298581886308954, 48.014581), 'Levroux': (1.6392860280931463, 46.9731343), 'Saint-Maur': (1.9183427, 49.6155716), 'Châteauroux': (1.6770956, 46.8203785), 'Issoudun': (1.9889131, 46.9477826), 'Le Blanc': (1.062694, 46.633292), 'La Châtre': (1.9892292, 46.5815678), 'Argenton-sur-Creuse': (1.5191029, 46.5886822), 'Buzançais': (1.4227597, 46.8885028), 'Déols': (1.7010433, 46.8267767), 'Villedieu-sur-Indre': (1.5375901, 46.8474516), 'Ardentes': (1.8320589, 46.7422328), 'Le Poinçonnet': (1.7178194, 46.7641725), 'Chabris': (1.6514258, 47.2527118), 'Langeais': (0.36322372786446244, 47.34762795), 'Tours': (0.6889268, 47.3900474), 'Chinon': (0.2402095, 47.1667182), 'Amboise': (0.983698, 47.4110351), 'Loches': (0.9979549, 47.1281692), 'Azay-le-Rideau': (0.4667128, 47.2614793), 'Bléré': (0.9918234, 47.3279746), 'Joué-lès-Tours': (0.6622353, 47.3511161), 'Descartes': (0.6995098, 46.9741096), 'Sainte-Maure-de-Touraine': (0.6189854, 47.1108281), 'Château-Renault': (0.9084166, 47.5904923), 'Bourgueil': (0.1788873, 47.2827691), 'Vouvray': (0.7998446, 47.4136687), 'Fondettes': (0.5961846, 47.4036631), 'Ballan-Miré': (0.6131325, 47.3413214), 'Montlouis-sur-Loire': (0.8269289, 47.3881023), 'Chambray-lès-Tours': (0.714368, 47.3390872), 'Saint-Cyr-sur-Loire': (0.6658199, 47.4052448), 'La Riche': (0.6572824, 47.3881295), 'Cinq-Mars-la-Pile': (0.4540029, 47.3482173), 'Rochecorbon': (0.7549178, 47.4141177), 'Luynes': (0.5545862, 47.3851411), 'Montbazon': (0.7144705, 47.2872408), "Notre-Dame-d'Oé": (0.7076929, 47.4571183), 'Chanceaux-sur-Choisille': (0.7033519, 47.471275), 'Savonnières': (0.552347, 47.3472325), 'Nazelles-Négron': (0.953293, 47.4319865), 'Saint-Avertin': (0.739923, 47.3577), 'Vernou-sur-Brenne': (0.847688, 47.4197777), 'Sorigny': (0.6939483, 47.241591), 'Azay-sur-Cher': (0.8463048, 47.3482178), 'Esvres': (0.7861713, 47.2845819), 'Athée-sur-Cher': (0.9157178, 47.3208166), 'Artannes-sur-Indre': (0.5996983, 47.2729908), 'Beaumont-en-Véron': (0.1718227, 47.1878662), 'Saint-Pierre-des-Corps': (0.7189761, 47.3898152), 'La Ville-aux-Dames': (0.7683831, 47.3942731), 'La Membrolle-sur-Choisille': (0.6271357, 47.4449038), 'Veigné': (0.7409855, 47.2869884), 'Véretz': (0.8088264, 47.3555532), 'Saint-Branchs': (0.7702016, 47.2269202), 'Monts': (2.0150032, 49.2174941), 'Saint-Martin-le-Beau': (0.9090287, 47.3563119), 'Monnaie': (0.7856396, 47.4996611), "Villette-d'Anthon": (5.1130506, 45.79451), 'La Mure': (5.7871983, 44.9038429), 'Meylan': (5.778361, 45.2097372), 'Roussillon': (4.8124049, 45.3735991), 'Vinay': (5.4021207, 45.2098594), 'Saint-Quentin-Fallavier': (5.1106322, 45.6314982), 'Vizille': (5.7715863, 45.0758358), 'Valencin': (5.01612, 45.6127), 'Tignieu-Jameyzieu': (5.182436929251853, 45.7321789), 'Saint-Georges-de-Commiers': (5.7040122, 45.0382806), 'Saint-Martin-le-Vinoux': (5.7055996, 45.2088706), 'Le Pont-de-Claix': (5.6978136, 45.1236837), 'Sassenage': (5.663687444649805, 45.2066404), 'Villard-de-Lans': (5.5512698, 45.0706211), 'Voreppe': (5.6361825, 45.2928424), 'Vaulx-Milieu': (5.1823217, 45.612328), 'Vif': (5.669033, 45.0554868), 'La Terrasse': (5.9309164, 45.3240258), 'Claix': (5.6729013, 45.1200404), 'Fontaine': (5.6869624, 45.1950183), 'Heyrieux': (5.063674, 45.6290108), 'Chanas': (4.8182954, 45.3182433), 'Beaurepaire': (2.5696774, 49.2916656), 'La Côte-Saint-André': (5.2604941, 45.3937745), 'Grenoble': (5.7357819, 45.1875602), 'Vienne': (16.3725042, 48.2083537), 'Bourgoin-Jallieu': (5.2828123, 45.5872259), 'Échirolles': (5.71927, 45.1437), 'La Tour-du-Pin': (5.4424054, 45.5666202), 'Voiron': (5.5910751, 45.3631386), "Saint-Martin-d'Hères": (5.75448, 45.1836683), "Le Bourg-d'Oisans": (6.0306625, 45.054713), 'Saint-Laurent-du-Pont': (5.7344662, 45.387632), 'Charvieu-Chavagneux': (5.154538192005392, 45.738289699999996), 'Crémieu': (5.2474523, 45.7241948), 'Allevard': (6.0744406, 45.3934726), 'Tullins': (5.4829748, 45.2977714), 'Pontcharra': (6.0204233, 45.4327537), 'Pont-de-Chéruy': (5.1740496, 45.7500102), 'Saint-Marcellin': (5.320347, 45.1539715), 'Crolles': (5.8839432, 45.2845907), 'Le Touvet': (5.9515976, 45.3592324), 'Seyssins': (5.6878134, 45.1593057), 'Fontanil-Cornillon': (5.6654815, 45.2556949), 'Les Avenières': (5.568741061921543, 45.641310399999995), 'Gières': (5.7904194, 45.1798245), 'Le Grand-Lemps': (5.4197156, 45.3979789), 'Saint-Nazaire-les-Eymes': (5.8490243, 45.2566002), 'Saint-Étienne-de-Saint-Geoirs': (5.3440159, 45.3398756), 'Vaulnaveys-le-Haut': (5.8129345, 45.1198654), 'Salaise-sur-Sanne': (4.8189855, 45.3441237), 'Saint-Étienne-de-Crossey': (5.6473505, 45.3782404), 'Saint-Jean-de-Moirans': (5.5818425, 45.3410803), 'Saint-Siméon-de-Bressieux': (5.264369, 45.329026), 'Saint-Clair-de-la-Tour': (5.483523, 45.5723547), 'Saint-André-le-Gaz': (5.5333566, 45.5468932), 'Saint-Clair-du-Rhône': (4.7707377, 45.4387498), "Saint-Georges-d'Espéranche": (5.0808076, 45.5574829), "Saint-Maurice-l'Exil": (4.7749904, 45.3969746), 'Saint-Just-Chaleyssin': (5.00005, 45.5843), 'Pont-Évêque': (4.9147109, 45.5293469), 'Le Péage-de-Roussillon': (4.7961586, 45.3732685), 'Montbonnot-Saint-Martin': (5.80266, 45.2264498), 'Nivolas-Vermelle': (5.3046991, 45.5579059), 'Le Cheylas': (5.9927101, 45.370339), 'Montalieu-Vercieu': (5.4038207, 45.8159548), 'Coublevie': (5.616002, 45.3558251), 'Estrablin': (4.9627643, 45.5163956), 'Diémoz': (5.0940007, 45.5887883), 'Champ-sur-Drac': (5.729987, 45.077144), 'Saint-Égrève': (5.681781, 45.2325079), 'Brié-et-Angonnes': (5.7853698, 45.1128816), 'Le Pont-de-Beauvoisin': (5.6704512, 45.5359674), 'Saint-Ismier': (5.8261521, 45.2481677), 'Chapareillan': (5.9907289, 45.4628848), 'Seyssinet-Pariset': (5.6899617, 45.1790768), 'Dolomieu': (5.5012061, 45.6135424), 'La Buisse': (5.6203141, 45.3333031), 'Le Versoud': (5.8603962, 45.2151366), "L'Isle-d'Abeau": (5.2298398, 45.6230017), 'Saint-Romain-de-Jalionas': (5.2199963, 45.7525436), 'Chasse-sur-Rhône': (4.8114503, 45.5789437), 'Lans-en-Vercors': (5.5885485, 45.1279159), 'Saint-Chef': (5.3650074, 45.6343504), 'Froges': (5.9244034, 45.273157), 'Villard-Bonnot': (5.8882101, 45.2384678), 'La Verpillière': (5.1473108, 45.632836), 'Saint-Jean-de-Bournay': (5.1389879, 45.501892), 'Rives': (5.5047411, 45.3501289), 'Eybens': (5.74812, 45.1524), 'Cessieu': (5.38238, 45.5671), 'Chavanoz': (5.1709524, 45.7768401), 'Bernin': (5.8664744, 45.2693836), 'Aoste': (7.3196649, 45.7370885), 'Apprieu': (5.502492, 45.3992601), 'Domène': (5.8388954, 45.2013065), 'Villefontaine': (5.1580574, 45.614003), 'La Tronche': (5.745715, 45.2090131), 'Morestel': (5.4711032, 45.675676), "Saint-Pierre-d'Allevard": (6.0450811, 45.3739679), "Saint-Martin-d'Uriage": (5.8383075, 45.1507233), 'Corenc': (5.7617836, 45.2196806), 'Ruy-Montceau': (5.346999509653573, 45.5884827), 'Moirans': (5.5647186, 45.3262643), 'Renage': (5.4865278, 45.331967), 'Jarrie': (5.7485905, 45.1053965), 'Varces-Allières-et-Risset': (5.6825158, 45.0885562), 'Crêts en Belledonne': (6.0312270038528055, 45.35715855), 'Autrans-Méaudre en Vercors': (5.547517734754461, 45.17129045), 'Les Avenières Veyrins-Thuellin': (5.553781379454723, 45.640978849999996), 'Les Abrets en Dauphiné': (5.603538537124297, 45.52993845), 'Villages du Lac de Paladru': (5.539572453586494, 45.474320399999996), 'Morez': (6.01616871446755, 46.527073099999996), 'Damparis': (5.4135618, 47.0758071), 'Lons-le-Saunier': (5.5589973, 46.6727037), 'Dole': (5.4925463, 47.0921047), 'Saint-Claude': (5.8643866, 46.3874699), 'Arbois': (5.7727504, 46.9032246), 'Tavaux': (5.40338, 47.0371), 'Les Rousses': (6.0603498, 46.4855507), 'Salins-les-Bains': (5.8786339, 46.9404794), 'Champagnole': (5.9083581, 46.7456049), 'Montmorot': (5.5302024, 46.6769747), 'Poligny': (5.7086756, 46.8348884), 'Hauts de Bienne': (6.0234986, 46.520336), 'Morcenx': (-0.9113736, 44.0323042), 'Saint-Paul-lès-Dax': (-1.0515491, 43.7253162), 'Rion-des-Landes': (-0.9441286797651827, 43.93338835), 'Morcenx-la-Nouvelle': (-0.9135039892050291, 44.03181745), 'Parentis-en-Born': (-1.0691145, 44.3508886), 'Saint-Sever': (-0.574187, 43.757456), 'Pouillon': (3.9491426, 49.3138671), 'Saint-Geours-de-Maremne': (-1.2379292, 43.6879086), "Pontonx-sur-l'Adour": (-0.9251622, 43.7877992), 'Ondres': (6.5941642, 44.114538), 'Labenne': (-1.42811, 43.5977), 'Tartas': (-0.8095934, 43.8323121), 'Mont-de-Marsan': (-0.500972, 43.8911318), 'Dax': (-1.0518771, 43.7084065), 'Capbreton': (-1.4315213, 43.6403659), 'Bénesse-Maremne': (-1.361178, 43.6334222), 'Tarnos': (-1.4611915, 43.5405622), 'Biscarrosse': (-1.1609361, 44.3907065), "Aire-sur-l'Adour": (-0.2647028, 43.7006462), 'Seignosse': (-1.37328, 43.68866), 'Hagetmau': (-0.59393, 43.65506), 'Narrosse': (-1.0077526, 43.6946221), 'Saint-Martin-de-Seignanx': (-1.3870972, 43.5415353), 'Soorts-Hossegor': (-1.4262382, 43.6584045), 'Soustons': (-1.3366453163533616, 43.7536389), 'Sanguinet': (-1.0759377, 44.4834776), 'Mimizan': (-1.2309278, 44.2019796), 'Labouheyre': (-0.9189952, 44.2125451), 'Peyrehorade': (-1.105088, 43.5440941), 'Saint-Vincent-de-Paul': (-0.4692956, 44.9549798), 'Saint-Vincent-de-Tyrosse': (-1.3029669, 43.6604837), 'Tosse': (-1.3337954, 43.6882557), 'Saint-Pierre-du-Mont': (-0.9814739, 49.3893604), 'Veuzain-sur-Loire': (1.1721984, 47.5004881), 'Le Controis-en-Sologne': (1.4281174, 47.4180012), 'Blois': (1.3337639, 47.5876861), 'Vendôme': (1.0662256, 47.7934453), 'Romorantin-Lanthenay': (1.7434925, 47.3592182), 'Salbris': (2.0525543, 47.4264805), 'Chailles': (1.3112812, 47.5344844), 'Mont-près-Chambord': (1.4550818, 47.5611844), 'Montrichard': (1.1828617252709055, 47.3593576), 'Contres': (2.4929328, 46.8666414), 'Villefranche-sur-Cher': (1.7706494, 47.2936361), 'Mer': (-84.6109122, 40.4367866), 'Onzain': (1.167112984465963, 47.51657), 'Montoire-sur-le-Loir': (0.8621777, 47.7527662), 'Selles-sur-Cher': (1.56301, 47.2769), 'Saint-Aignan': (-0.2968396, 44.9486322), 'Saint-Laurent-Nouan': (1.6116899, 47.7169881), 'Noyers-sur-Cher': (1.3811918, 47.2797159), 'Saint-Gervais-la-Forêt': (1.361591, 47.567892), 'Saint-Georges-sur-Cher': (1.1304784, 47.3258694), 'Lamotte-Beuvron': (2.0267353, 47.6052963), 'Cellettes': (0.1478161, 45.8637199), 'Vineuil': (1.6343263, 46.9001285), 'Saint-Ouen': (2.1223737, 50.03683), 'Cour-Cheverny': (1.4588243, 47.512759), 'La Chaussée-Saint-Victor': (1.3587541, 47.6095154), 'Beauce la Romaine': (1.4200123843090457, 47.9359972), 'Montrichard Val de Cher': (1.1843175, 47.3430937), 'Sury-le-Comtal': (4.1829539, 45.5381922), 'Le Coteau': (4.09203, 46.0205), 'Genilac': (4.581123, 45.5321398), 'Boën-sur-Lignon': (4.004279, 45.7456231), 'La Talaudière': (4.4290257, 45.480478), 'Le Chambon-Feugerolles': (4.3247696, 45.3966546), 'Saint-Just-sur-Loire': (4.2629313, 45.5028467), 'Unieux': (4.2620703, 45.3989749), 'Villerest': (4.0369461, 45.9939206), 'Saint-Just-Saint-Rambert': (4.2423759, 45.4994639), 'Saint-Romain-le-Puy': (4.1248446, 45.5549769), 'Saint-Genest-Malifaux': (4.4202655, 45.3401939), 'Saint-Héand': (4.3752871, 45.5296285), "L'Horme": (4.5424593, 45.4873626), 'Riorges': (4.0337193, 46.0437504), 'Roche-la-Molière': (4.3212622, 45.4348987), 'Panissières': (4.34467, 45.7928), 'Pouilly-sous-Charlieu': (4.1092319, 46.1453805), 'Montrond-les-Bains': (4.2299344, 45.6433431), 'Saint-Genest-Lerpt': (4.3311468, 45.4565913), 'Roanne': (4.0729178, 46.0345572), 'Saint-Étienne': (4.3873058, 45.4401467), 'Montbrison': (4.0627318, 45.6072875), 'Saint-Chamond': (4.5098046, 45.4748298), 'Firminy': (4.2858545, 45.3869468), 'Commelle-Vernay': (4.0652848, 45.9930031), 'Lorette': (4.58024, 45.509), 'Feurs': (4.2215124, 45.7440738), 'La Ricamarie': (4.3660765, 45.4037177), 'Mably': (4.0644726, 46.0860078), 'Bourg-Argental': (4.5616181, 45.2973292), 'Balbigny': (4.1833034, 45.8187228), 'Saint-Paul-en-Jarez': (4.5741514, 45.4825111), 'Renaison': (3.9262267, 46.0518286), 'Rive-de-Gier': (4.6187386, 45.5288401), 'Veauche': (4.2892399, 45.5622913), 'Saint-Galmier': (4.3183716, 45.5903147), 'Charlieu': (4.1714892, 46.1588067), 'Savigneux': (4.8481524, 46.000701), 'Pélussin': (4.678089, 45.4179803), 'La Fouillouse': (4.3157199, 45.5009884), "L'Étrat": (4.3755315, 45.4863704), 'Fraisses': (4.2657181, 45.3887948), 'Chavanay': (4.7297216, 45.4140776), 'Chazelles-sur-Lyon': (4.3889953, 45.6375531), 'Andrézieux-Bouthéon': (4.2594811, 45.5250887), 'Bonson': (4.2154175, 45.5225917), 'Villars': (0.7542978, 45.4202981), 'Saint-Priest-en-Jarez': (4.3792689, 45.473408), 'La Grand-Croix': (4.55971, 45.504), 'Saint-Jean-Bonnefonds': (4.4450109, 45.4494687), 'Sorbiers': (4.4507034, 45.4873195), 'Saint-Marcellin-en-Forez': (4.1670804, 45.4973386), 'Saint-Martin-la-Plaine': (4.5926569, 45.546385), 'Le Puy-en-Velay': (3.8855537, 45.0459739), 'Brioude': (3.3853711, 45.293897), 'Yssingeaux': (4.1244363, 45.1435556), 'Aurec-sur-Loire': (4.2008542, 45.3698465), 'Beauzac': (4.0992665, 45.2593451), 'Retournac': (4.0328094, 45.204168), 'Sainte-Sigolène': (4.2360263, 45.2424978), 'Dunières': (4.34461, 45.215), 'Saint-Maurice-de-Lignon': (4.1375076, 45.2230922), 'Sainte-Florine': (3.3179282, 45.4030439), 'Coubon': (3.9177723, 44.9965854), 'Brives-Charensac': (3.92197, 45.0474), 'Saint-Germain-Laprade': (3.9687927, 45.0381325), 'Langeac': (3.4941798, 45.1001248), 'Polignac': (-0.3080556, 45.285), 'Saint-Didier-en-Velay': (4.2753045, 45.301906), 'Tence': (4.289156, 45.1161335), 'Espaly-Saint-Marcel': (3.8657896, 45.0465143), 'Saint-Just-Malmont': (4.3134369, 45.3397467), 'Bas-en-Basset': (4.1116823, 45.3056154), 'Monistrol-sur-Loire': (4.1727546, 45.2926341), 'Vals-près-le-Puy': (3.8757116, 45.0300955), 'Derval': (-1.6702011, 47.6673622), 'Nantes': (-1.5541362, 47.2186371), 'Blain': (-1.7643647, 47.4755536), 'La Haie-Fouassière': (-1.4000716, 47.1546806), 'Saint-Nazaire': (-2.2138905, 47.2733517), 'Châteaubriant': (-1.3757282, 47.7194057), 'La Baule-Escoublac': (-2.3951595, 47.2843726), 'Ancenis': (-1.1816088, 47.3648141), 'Treillières': (-1.6240815, 47.3309103), 'Orvault': (-1.623206, 47.2711217), 'Saint-Herblain': (-1.6346964, 47.2233007), 'Bouguenais': (-1.6233691, 47.179301), 'Joué-sur-Erdre': (-1.421142, 47.4959351), 'Rezé': (-1.5695287, 47.1905456), 'Vertou': (-1.4722242, 47.1685211), 'Saint-Sébastien-sur-Loire': (-1.4992075, 47.2035029), 'La Montagne': (6.59222, 47.9205), 'Clisson': (-1.2817422, 47.0877218), 'Indre': (1.5382051557056249, 46.81210565), 'Guérande': (-2.4298014, 47.3282126), 'La Chapelle-Launay': (-1.9717038, 47.3719848), 'Carquefou': (-1.4921233, 47.2971979), 'Crossac': (-2.1707156, 47.4111944), 'Le Pallet': (-1.3375978, 47.139588), 'Le Pellerin': (-1.7550364, 47.1999241), 'La Turballe': (-2.5080068, 47.3468646), 'Saffré': (-1.5786047, 47.5015277), 'Saint-Aignan-Grandlieu': (-1.6345992, 47.1224436), 'Machecoul': (-1.8480714949844588, 46.991959449999996), 'Couëron': (-1.7271466, 47.2115054), 'Ligné': (0.1100063, 45.9216471), 'Prinquiau': (-2.0107055, 47.3630542), 'Guenrouët': (-1.9503777, 47.5208079), 'Cordemais': (-1.8762761, 47.2909306), 'La Chevrolière': (-1.6117844, 47.0936557), 'Port-Saint-Père': (-1.749713, 47.1327853), 'Rouans': (-1.8598756, 47.1858102), 'Héric': (-1.6523209, 47.4144202), 'Pornichet': (-2.3364238, 47.2613293), 'Sainte-Luce-sur-Loire': (-1.4865339, 47.2493864), 'Missillac': (-2.1590855, 47.4819903), 'Grandchamp-des-Fontaines': (-1.6058096, 47.3651112), 'Campbon': (-1.9693613, 47.4128164), 'Le Cellier': (-1.3465357, 47.3186735), 'Saint-Molf': (-2.4255358, 47.3927175), 'La Chapelle-des-Marais': (-2.2409143, 47.4472597), 'La Plaine-sur-Mer': (-2.1915862, 47.1381083), 'Besné': (-2.0921265, 47.3971609), 'Saint-Jean-de-Boiseau': (-1.7248714, 47.1953115), 'Chauvé': (-1.9860707, 47.1514326), 'Vigneux-de-Bretagne': (-1.7368752, 47.3267919), 'La Chapelle-sur-Erdre': (-1.5518632, 47.2996846), 'Pont-Saint-Martin': (-1.5839895, 47.1231355), 'Donges': (-2.0745708, 47.3185376), 'Frossay': (-1.9341151, 47.2440022), 'Malville': (-1.8635782, 47.3606366), 'Arthon-en-Retz': (-1.942077, 47.1168006), 'Saint-Julien-de-Concelles': (-1.3862271, 47.2520409), 'Mauves-sur-Loire': (-1.393575, 47.2954249), 'Gorges': (2.17745, 50.10992), 'Herbignac': (-2.318167, 47.4472633), 'Pornic': (-2.1040102, 47.1152695), 'Saint-Brevin-les-Pins': (-2.1677298, 47.2473358), 'Oudon': (-1.2861611, 47.3466877), 'Bouvron': (5.8809665, 48.7422218), 'Bouaye': (-1.6917807, 47.1428126), 'Bourgneuf-en-Retz': (-1.924402467420876, 47.03524805), 'Erbray': (-1.3179329, 47.6552164), 'Fay-de-Bretagne': (-1.7898247, 47.4140973), 'Saint-Malo-de-Guersac': (-2.1772785, 47.3516122), 'Brains': (-1.725903, 47.1686607), 'La Chapelle-Basse-Mer': (-1.3573159017634535, 47.27838265), 'Nozay': (4.09529, 48.5085), 'Savenay': (-1.9421244, 47.3592304), 'Pontchâteau': (-2.0880233, 47.4373374), 'Nort-sur-Erdre': (-1.499037, 47.4398626), 'Sucé-sur-Erdre': (-1.5285694, 47.341551), 'Geneston': (-1.5150942, 47.0551962), 'Le Croisic': (-2.5094404, 47.2933971), 'Guémené-Penfao': (-1.8328831, 47.6294113), 'Vallet': (-1.2657431, 47.1610036), 'Legé': (-1.5979446, 46.8867571), 'La Chapelle-Heulin': (-1.3404289, 47.1760091), 'Batz-sur-Mer': (-2.4791931, 47.2771383), 'La Bernerie-en-Retz': (-2.0383793, 47.0814025), 'Gétigné': (-1.2482386, 47.0757663), 'Château-Thébaud': (-1.4213694, 47.1246294), 'Corcoué-sur-Logne': (-1.5772672, 46.965977), 'Haute-Goulaine': (-1.4297979, 47.1990253), 'Corsept': (-2.0603875, 47.2779833), 'Le Bignon': (-1.4907076, 47.0982118), 'Trignac': (-2.1894926, 47.3176944), 'La Planche': (-1.4358522, 47.0165253), 'Boussay': (0.8882705, 46.8411118), 'Saint-Gildas-des-Bois': (-2.0369967, 47.5174681), 'Saint-Mars-du-Désert': (-0.1591084, 48.2937343), 'Montbert': (-1.4877662, 47.0567359), 'Saint-Père-en-Retz': (-2.0444751, 47.2066441), 'Petit-Mars': (-1.4546001, 47.3954408), 'Saint-Viaud': (-2.0191384, 47.2564348), 'Le Loroux-Bottereau': (-1.3472697, 47.2385415), 'Mouzillon': (-1.2839318, 47.1413424), 'Maisdon-sur-Sèvre': (-1.3838618, 47.0972172), 'Saint-Étienne-de-Montluc': (-1.7810546, 47.2739351), 'Divatte-sur-Loire': (-1.3364039244301118, 47.2788103), 'Villeneuve-en-Retz': (-1.8979780094455112, 47.03524805), 'Vair-sur-Loire': (-1.1153162835591488, 47.40185875), 'Machecoul-Saint-Même': (-1.808209766528024, 47.0058802), 'Chaumes-en-Retz': (-1.9183342003558463, 47.1560818), 'Loireauxence': (-1.0239106243320195, 47.4351839), "Vallons-de-l'Erdre": (-1.153122434365907, 47.53490945), 'Ancenis-Saint-Géréon': (-1.195798443873262, 47.3817908), 'Sainte-Anne-sur-Brivet': (-2.005667, 47.4596949), 'Saint-Mars-de-Coutais': (-1.7334222, 47.1131594), 'Montoir-de-Bretagne': (-2.1543171, 47.3285909), 'Saint-André-des-Eaux': (-2.0087518, 48.3711691), 'Saint-Lyphard': (-2.3069996, 47.3989603), 'Saint-Géréon': (-1.206699206461788, 47.370952700000004), 'Saint-Herblon': (-1.0962598, 47.4080247), 'Saint-Colomban': (-1.5833651, 47.0098098), 'Le Landreau': (-1.3062265, 47.2049301), 'Sautron': (-1.6683537, 47.2632611), 'Varades': (-1.0270194380626334, 47.407783949999995), 'Saint-Philbert-de-Grand-Lieu': (-1.6405396, 47.0359151), 'Saint-Nicolas-de-Redon': (-2.0689877, 47.6440928), 'Vieillevigne': (1.6577206, 43.4021153), 'Aigrefeuille-sur-Maine': (-1.3995865, 47.0764444), 'Basse-Goulaine': (-1.4652044, 47.2155058), 'Couffé': (-1.2932184, 47.3909352), 'Plessé': (-1.8864656, 47.5417394), 'Le Pouliguen': (-2.4310663, 47.2714307), 'Mésanger': (-1.2313189, 47.4322236), 'Sainte-Pazanne': (-1.8111609, 47.1031638), 'Saint-Michel-Chef-Chef': (-2.1497904, 47.1816681), 'Thouaré-sur-Loire': (-1.4410916, 47.2668824), 'Les Sorinières': (-1.5302384, 47.148006), 'Paimbœuf': (-2.02641, 47.2871357), 'Saint-Joachim': (-2.2020678, 47.3836823), 'Cléry-Saint-André': (1.7569328, 47.8212402), 'Briare': (2.7413128, 47.6410716), 'Châteauneuf-sur-Loire': (2.2283459, 47.8642836), 'Malesherbes': (2.3972819567297945, 48.2865494), 'Puiseaux': (2.472464, 48.2040248), 'Fleury-les-Aubrais': (1.9208883, 47.9309156), 'Courtenay': (5.3798454, 45.7294097), 'Sandillon': (2.0327513, 47.8455805), 'La Chapelle-Saint-Mesmin': (1.8398174, 47.8869822), 'Corquilleroy': (2.6980927, 48.0451539), 'Dordives': (2.77275, 48.1466), 'Le Malesherbois': (2.3645529772863068, 48.274523849999994), 'Orléans': (1.9086066, 47.9027336), 'Chevilly': (1.8739814, 48.0260305), 'La Ferté-Saint-Aubin': (1.9442053, 47.7169399), 'Montargis': (2.7310072, 47.9978628), 'Pithiviers': (2.2544871, 48.1738051), 'Olivet': (1.9015752, 47.8650984), 'Ingré': (1.8224763, 47.9193001), 'Beaugency': (1.6311944, 47.7778775), 'Gien': (2.6313889, 47.6918575), 'Sully-sur-Loire': (2.3738968, 47.7654098), 'Fay-aux-Loges': (2.1386946, 47.9272265), 'Saran': (84.78074760594323, 25.90835495), 'Jargeau': (2.1242805, 47.8649447), 'Meung-sur-Loire': (1.6968055, 47.8268972), 'Chécy': (2.02135, 47.9019), 'Châtillon-sur-Loire': (2.7530968, 47.5904231), 'Saint-Denis-en-Val': (1.9631806, 47.879147), 'Ferrières-en-Gâtinais': (2.7946987, 48.0927506), 'Saint-Jean-de-Braye': (1.9737186, 47.9200018), 'Donnery': (2.11219, 47.9126), 'Lorris': (2.5157503, 47.8895393), 'Chaingy': (1.7723591, 47.883725), 'Châlette-sur-Loing': (2.738158032225153, 48.021751800000004), 'Traînou': (2.1028645, 47.9704692), 'Ouzouer-sur-Loire': (2.4794838, 47.7664387), 'Pannes': (5.8038889, 48.9329939), 'Villemandeur': (2.70819, 47.9838), 'Semoy': (1.9494742, 47.9337171), 'Mardié': (2.06722, 47.9014), 'Amilly': (1.3930652, 48.4418021), 'Nogent-sur-Vernisson': (2.7421711, 47.8468879), 'Lailly-en-Val': (1.6895989, 47.7709886), 'Saint-Pryvé-Saint-Mesmin': (1.869824, 47.8803102), 'Saint-Hilaire-Saint-Mesmin': (1.8368147, 47.8683824), 'Saint-Ay': (1.7535422, 47.8606378), 'Saint-Jean-de-la-Ruelle': (1.8717431, 47.9098024), 'Saint-Cyr-en-Val': (1.9684671, 47.8320651), "Saint-Denis-de-l'Hôtel": (2.1284595, 47.8723284), 'Ormes': (0.9634334, 49.0533673), 'Saint-Jean-le-Blanc': (-0.6542174, 48.9348406), 'Neuville-aux-Bois': (2.0534134, 48.0695211), 'Pradines': (1.4042655, 44.4794018), 'Cahors': (1.4364999, 44.4495), 'Gourdon': (4.4461818, 46.6414367), 'Figeac': (2.0324321, 44.6092335), 'Gramat': (1.723394, 44.7789948), 'Saint-Céré': (1.8922748, 44.8586149), 'Souillac': (1.46943, 44.8980172), 'Agen': (0.6176112, 44.2015827), 'Bon-Encontre': (0.6683646, 44.1861695), 'Clairac': (0.375571, 44.3601017), 'Villeneuve-sur-Lot': (0.7046455, 44.4056615), 'Marmande': (0.164274, 44.5007939), 'Nérac': (0.3392503, 44.1353979), 'Sainte-Livrade-sur-Lot': (0.59892, 44.4033), 'Montayral': (0.9875538, 44.4802227), 'Aiguillon': (0.3416667, 44.3002778), 'Casteljaloux': (0.0880691, 44.3142971), 'Miramont-de-Guyenne': (0.3593857, 44.6042375), 'Pujols': (-0.0312502, 44.8082356), 'Boé': (0.6294444, 44.1605556), 'Fumel': (0.9684626, 44.4966624), 'Colayrac-Saint-Cirq': (0.5536111, 44.2205556), 'Sainte-Bazeille': (0.0930606, 44.5291668), 'Bias': (-1.2187265, 44.1456048), 'Foulayronnes': (0.64707, 44.23912), 'Le Passage': (5.51253, 45.5326), 'Tonneins': (0.3091667, 44.3905556), 'Pont-du-Casse': (0.6819444, 44.2325), 'Layrac': (0.6608919, 44.1347039), "Saint-Chély-d'Apcher": (3.2757824, 44.8011363), 'Mende': (3.4991057, 44.5180226), 'Langogne': (3.8550146, 44.7266369), 'Marvejols': (3.2901135, 44.5532621), 'Baugé-en-Anjou': (-0.1036842, 47.5612379), 'Bagneux': (3.279444, 49.458056), 'Chemillé-Melay': (-0.732659, 47.2146396), 'Baugé': (-0.1040831, 47.5422732), 'Angers': (-0.5515588, 47.4739884), 'Saint-Florent-le-Vieil': (-1.0108122426865387, 47.34559365), 'Cholet': (-0.8801349, 47.0617293), 'Montreuil-Juigné': (-0.6099197, 47.5291981), 'Saumur': (-0.0785177, 47.2596224), 'Chalonnes-sur-Loire': (-0.76311, 47.3516207), 'Pellouailles-les-Vignes': (-0.4472103122962612, 47.5323942), 'Saint-Lambert-la-Potherie': (-0.6786062, 47.4811787), 'Saint-Christophe-du-Bois': (-0.9448666, 47.029427), 'Segré': (-0.8706239, 47.6846685), 'Liré': (-1.152324643552158, 47.31968245), 'Gesté': (-1.11451, 47.1798), 'Écouflant': (-0.5301394, 47.5315221), 'Chemillé': (-0.7323695720236629, 47.23005335), 'Pouancé': (-1.1729927, 47.7403392), 'Briollay': (-0.5073254, 47.5654856), 'Champtoceaux': (-1.261536557821203, 47.3204002), 'Maulévrier': (-0.7451974, 47.008225), 'Saint-Macaire-en-Mauges': (-0.9928443534994075, 47.1239125), 'Beaupréau': (-0.985013628460941, 47.2122889), 'Les Ponts-de-Cé': (-0.5270662, 47.4286962), 'Soucelles': (-0.4307951690356452, 47.5844476), 'Seiches-sur-le-Loir': (-0.3619433, 47.5765866), 'Avrillé': (-0.5956539, 47.5045657), 'Beaucouzé': (-0.636681, 47.4763), 'Beaufort-en-Vallée': (-0.21245573106941965, 47.43125595), 'Brissac-Quincé': (-0.44251974223851, 47.341066749999996), 'Longué-Jumelles': (-0.1034531, 47.4346214), 'Mazé': (-0.28937490430425306, 47.4576317), 'Durtal': (-0.2453787, 47.6705186), 'Saint-Georges-sur-Loire': (-0.7610073, 47.4082853), 'Saint-André-de-la-Marche': (-1.0084765458671576, 47.0954294), 'Montreuil-Bellay': (-0.1554622, 47.1308239), 'Saint-Germain-sur-Moine': (-1.1209236476392501, 47.12151355), 'Corné': (-0.3514559, 47.471056), "Brain-sur-l'Authion": (-0.42272057610645347, 47.46433825), 'Doué-la-Fontaine': (-0.27589532041237075, 47.19224005), 'Combrée': (-1.0236212019348816, 47.707505100000006), 'Bouchemaine': (-0.6085356, 47.4234644), 'Allonnes': (1.6561556, 48.3304943), 'Andard': (-0.3856130713687841, 47.4829969), 'Châteauneuf-sur-Sarthe': (-0.4976700182029784, 47.68193015), "Le Lion-d'Angers": (-0.7370899308201252, 47.628958850000004), 'Le May-sur-Èvre': (-0.8917359, 47.1377641), 'Juigné-sur-Loire': (-0.48343257232794157, 47.398584650000004), 'Le Louroux-Béconnais': (-0.8855198, 47.5215888), 'Bécon-les-Granits': (-0.8012205, 47.5034433), 'Vivy': (-0.0558806, 47.326892), 'Trélazé': (-0.4652452, 47.4460674), 'Trémentines': (-0.7885143, 47.1219319), 'Saint-Léger-sous-Cholet': (-0.9141649, 47.0949813), 'Villedieu-la-Blouère': (-1.0593262184422927, 47.159235699999996), 'La Tessoualle': (-0.850644, 47.0050398), 'Villevêque': (-0.4273829996816577, 47.5391914), 'Tiercé': (-0.4673732, 47.6152682), "Saint-Barthélemy-d'Anjou": (-0.4932941, 47.4683081), 'Le Plessis-Grammoire': (-0.4308085, 47.5004506), 'La Pommeraye': (-0.4245081, 48.909216), 'Mûrs-Erigné': (-0.5361153, 47.3992042), 'Montjean-sur-Loire': (-0.8611689, 47.3877012), 'Sainte-Gemmes-sur-Loire': (-0.5587967, 47.4243064), "Saint-Sylvain-d'Anjou": (-0.47799013176375205, 47.5129785), 'Saint-Pierre-Montlimart': (-1.0257527, 47.2706494), 'La Séguinière': (-0.9399268, 47.0636081), 'Jallais': (-0.8640796851326958, 47.20089315), 'Candé': (-1.0414584, 47.560347), 'Saint-Hilaire-Saint-Florent': (-0.1231025, 47.2712287), "Les Bois d'Anjou": (-0.1891696, 47.4866054), 'Beaupréau-en-Mauges': (-0.9183549426012452, 47.20279205), 'Chemillé-en-Anjou': (-0.6725711312155349, 47.22107895), 'Ingrandes-Le Fresne sur Loire': (-0.9295464, 47.4037231), 'Lys-Haut-Layon': (-0.423740369520669, 47.150343449999994), 'Mauges-sur-Loire': (-0.9463142468193277, 47.3434076), 'Montrevault-sur-Èvre': (-1.0500983062603204, 47.2494064), 'Sèvremoine': (-1.0821288289702786, 47.08775155), "Orée d'Anjou": (-1.2103047021300555, 47.30424605), 'Bellevigne-en-Layon': (-0.5354509319541243, 47.272419850000006), 'Longuenée-en-Anjou': (-0.670853, 47.5498843), 'Val-du-Layon': (-0.6568825985212126, 47.3135124), 'Verrières-en-Anjou': (-0.4675412, 47.5179398), 'Loire-Authion': (-0.3825314294033625, 47.469642), 'Jarzé Villages': (-0.2490142582178278, 47.56063235), 'Beaufort-en-Anjou': (-0.2162487, 47.4399992), 'Mazé-Milon': (-0.2724533, 47.4554296), 'Erdre-en-Anjou': (-0.8346007, 47.6009896), 'Segré-en-Anjou Bleu': (-0.8715571, 47.6811163), 'Brissac Loire Aubance': (-0.44288581283807194, 47.33942025), 'Les Garennes sur Loire': (-0.4719574443962181, 47.38502545), 'Morannes sur Sarthe-Daumeray': (-0.40199577740731685, 47.718064049999995), 'Doué-en-Anjou': (-0.2938656, 47.1896337), "Val d'Erdre-Auxence": (-0.8855532, 47.5202712), "Les Hauts-d'Anjou": (-0.5434044652604251, 47.697020699999996), "Ombrée d'Anjou": (-1.1665791, 47.7451895), 'Noyant-Villages': (0.10717869531132722, 47.51117495), 'Terranjou': (-0.4355920572666552, 47.2690011), 'Gennes-Val-de-Loire': (-0.28138612404349483, 47.335043299999995), 'Saint-Léger-de-Linières': (-0.7058718, 47.4624511), 'Bellevigne-les-Châteaux': (-0.04832051289504743, 47.1781487), 'Rives-du-Loir-en-Anjou': (-0.4509653049530439, 47.557136400000005), 'Saint-Lambert-des-Levées': (-0.07016477206342236, 47.287763749999996), 'Tourlaville': (-1.5706728933964897, 49.6401033), 'Villedieu-les-Poêles': (-1.2220882, 48.8381226), 'Sourdeval': (-0.8935215571763233, 48.7147351), 'Saint-Hilaire-du-Harcouët': (-1.085461778865405, 48.5714837), 'Isigny-le-Buat': (-1.1716128, 48.6200556), 'Querqueville': (-1.7056207, 49.6574288), 'Agon-Coutainville': (-1.58178, 49.0413), 'Donville-les-Bains': (-1.5813204, 48.8488845), 'Saint-Pair-sur-Mer': (-1.5665949, 48.8139133), 'Condé-sur-Vire': (-1.0250661118570192, 49.059865099999996), 'Cherbourg-Octeville': (-1.6285770868661507, 49.6343543), 'Ducey': (-1.2799223688562862, 48.61287995), 'Saint-Lô': (-1.0906637, 49.1157004), 'Coutances': (-1.4454857, 49.047792), 'Avranches': (-1.3634379895766875, 48.688189), 'Carentan': (-1.2463444, 49.30329), 'Bréhal': (-1.5118727, 48.898168), 'Valognes': (-1.4708174, 49.5089547), 'Pontorson': (-1.4981210717832734, 48.57362635), 'Granville': (-1.5959213, 48.8376284), 'Agneaux': (-1.114372, 49.1165234), 'Bricquebec': (-1.6321305, 49.4717312), 'Saint-James': (-1.32375, 48.5232), 'Les Pieux': (-1.8083081, 49.513753), 'Équeurdreville-Hainneville': (-1.6744434619124282, 49.64390935), 'Octeville': (-1.6448063, 49.6254784), 'Cherbourg': (-1.6249565, 49.6425343), 'Cherbourg-en-Cotentin': (-1.6249565, 49.6425343), 'Grandparigny': (-1.0863680427734792, 48.59449195), 'Torigny-les-Villes': (-0.9416840064138204, 48.99631835), 'Villedieu-les-Poêles-Rouffigny': (-1.2230159708654684, 48.824754999999996), 'Percy-en-Normandie': (-1.2019534053166545, 48.917350400000004), 'Bourgvallées': (-1.1078329478479656, 49.0365768), 'La Haye': (4.3113461, 52.0799838), 'Ducey-Les Chéris': (-1.2708983702513963, 48.61997005), 'Bricquebec-en-Cotentin': (-1.6658135848271107, 49.46208075), 'Sartilly-Baie-Bocage': (-1.443765150078303, 48.744845), 'Mortain-Bocage': (-0.924256777713777, 48.6076865), 'Marigny-Le-Lozon': (-1.264809944187318, 49.12576905), 'La Hague': (-1.7991053325251822, 49.65483205), 'Gouville-sur-Mer': (-1.577379293627522, 49.097905749999995), 'Quettreville-sur-Sienne': (-1.4772562033980272, 48.95960175), 'Sainte-Mère-Église': (-1.3268247811335525, 49.411770399999995), 'Carentan-les-Marais': (-1.2046684948980775, 49.3060982), 'Picauville': (-1.3919822168758198, 49.37973705), 'Port-Bail-sur-Mer': (-1.6756393629759971, 49.34012355), 'Saint-Sauveur-Villages': (-1.3877433571366429, 49.1341435), 'Carentan les Marais': (-1.2046684948980775, 49.3060982), 'Suippes': (4.531783, 49.1308493), 'Montmirail': (3.5381991, 48.8708848), 'Fagnières': (4.3169497, 48.9644633), 'Fismes': (3.6802766, 49.3072444), 'Witry-lès-Reims': (4.115644, 49.2902975), 'Dormans': (3.6383732, 49.0745659), 'Sézanne': (3.7223381, 48.7240897), 'Vitry-le-François': (4.5851622, 48.7262225), 'Ay': (3.9970886150997327, 49.0621668), 'Bezannes': (3.9864282, 49.2195271), 'Bétheny': (4.0535127, 49.2845383), 'Reims': (4.031926, 49.2577886), 'Épernay': (3.9528655, 49.042561), 'Sainte-Menehould': (4.8967637, 49.0904192), 'Saint-Brice-Courcelles': (3.9866881, 49.2619631), 'Cormontreuil': (4.0534118, 49.2234466), 'Saint-Memmie': (4.3847288, 48.9512156), 'Tinqueux': (3.98601, 49.2479), 'Warmeriville': (4.2202414, 49.3499703), 'Mourmelon-le-Grand': (4.3660294, 49.1373278), 'Aÿ-Champagne': (4.0150810116635745, 49.055833500000006), 'Blancs-Coteaux': (3.985709032042527, 48.922717750000004), 'Nogent': (5.3469428, 48.029544), 'Chaumont': (5.1395849, 48.1111324), 'Langres': (5.333947, 47.8639612), 'Saint-Dizier': (4.9473513, 48.6371125), 'Wassy': (4.9489836, 48.4996383), 'Joinville': (5.1377537, 48.4433253), 'Gorron': (-0.8129799, 48.4123473), 'Ambrières-les-Vallées': (-0.6297627, 48.4033446), 'Villaines-la-Juhel': (-0.2780934, 48.3451696), 'Ernée': (-0.9351069, 48.2967935), 'Changé': (0.2839276, 47.9875095), 'Azé': (4.7607483, 46.431937), 'Louverné': (-0.7174, 48.1214342), "L'Huisserie": (-0.7681717, 48.0229521), 'Bonchamp-lès-Laval': (-0.6992069, 48.0746999), 'Loiron-Ruillé': (-0.9386869953271073, 48.0620375), 'Montsûrs-Saint-Céneré': (-0.580837758744267, 48.11355745), 'Château-Gontier-sur-Mayenne': (-0.703179069901837, 47.820917550000004), 'Montsûrs': (-0.5522215177316291, 48.137962900000005), 'Évron': (-0.37536173463129474, 48.1460174), 'Bazouges': (-0.7186433, 47.8352789), 'Renazé': (-1.0557842, 47.793333), 'Laval': (-0.7734022, 48.0706687), 'Mayenne': (-0.6491273812007092, 48.1507819), 'Château-Gontier': (-0.7066665, 47.8263058), 'Argentré': (-0.642135, 48.0835596), 'Craon': (-0.9518383, 47.84831), 'Meslay-du-Maine': (-0.5537919, 47.9500183), 'Saint-Berthevin': (-0.8290936, 48.0690907), 'Cossé-le-Vivien': (-0.9117797, 47.9444647), 'Nancy': (6.1834097, 48.6937223), 'Saint-Nicolas-de-Port': (6.3022451, 48.6310196), 'Toul': (5.8941759, 48.6762624), 'Écrouves': (5.8399328, 48.6794767), 'Réhon': (5.7535172, 49.5014297), 'Saulxures-lès-Nancy': (6.2455185, 48.6876255), 'Frouard': (6.1293789, 48.7601755), 'Chaligny': (6.0803932, 48.6256516), 'Lunéville': (6.4919563, 48.5916164), 'Heillecourt': (6.1947541, 48.6530512), 'Trieux': (5.9299434, 49.3243194), 'Pompey': (6.1229922, 48.7658301), 'Briey': (5.966079338759432, 49.2583938), 'Villerupt': (5.9326783, 49.468468), 'Laxou': (6.1522232, 48.6848709), 'Seichamps': (6.2611518, 48.7145001), 'Ludres': (6.1609279, 48.6217058), 'Liverdun': (6.0630033, 48.7494954), 'Cosnes-et-Romain': (5.7125127, 49.5187211), 'Jarville-la-Malgrange': (6.2023462, 48.6735375), 'Foug': (5.7866205, 48.6846355), 'Gorcy': (5.6819176, 49.5368217), 'Gondreville': (2.9549603, 49.2162403), 'Lexy': (5.7302118, 49.502997), 'Pulnoy': (6.2567749, 48.7004346), 'Rosières-aux-Salines': (6.3325266, 48.5944191), 'Blénod-lès-Pont-à-Mousson': (6.0530227, 48.8811732), 'Champigneulles': (6.1651167, 48.7342216), 'Custines': (6.1422309, 48.791174), 'Longwy': (5.7679082, 49.5145564), 'Baccarat': (6.7381417, 48.4487414), 'Vandœuvre-lès-Nancy': (6.1724618, 48.6599676), 'Dombasle-sur-Meurthe': (6.3493799, 48.6231282), 'Jœuf': (6.0092074, 49.2302031), 'Haucourt-Moulaine': (5.8063556, 49.4894749), 'Auboué': (5.9750234, 49.2128736), 'Bouxières-aux-Dames': (6.1638113, 48.7511972), 'Damelevières': (6.3871266, 48.557112), 'Varangéville': (6.3130958, 48.6340871), 'Piennes': (5.7846233, 49.3085543), 'Pagny-sur-Moselle': (6.0197497, 48.9837551), 'Saint-Max': (6.2074377, 48.7005548), 'Neuves-Maisons': (6.1035434, 48.6155888), 'Dieulouard': (6.0683191, 48.8416054), 'Hussigny-Godbrange': (5.8680329, 49.4936689), 'Jarny': (5.8821708, 49.1569586), 'Tomblaine': (6.2106707, 48.6864409), 'Longuyon': (5.600732, 49.4461066), 'Mont-Saint-Martin': (3.6389239, 49.282103), 'Homécourt': (5.9926232, 49.2249744), 'Essey-lès-Nancy': (6.2220764, 48.7053172), "Blainville-sur-l'Eau": (6.4078881, 48.5557744), 'Laneuveville-devant-Nancy': (6.2314017, 48.656251), 'Herserange': (5.79533, 49.5191), 'Maxéville': (6.1641314, 48.7119046), 'Villers-lès-Nancy': (6.1536378, 48.6731344), 'Val de Briey': (5.9571236995611905, 49.266237849999996), 'Thierville-sur-Meuse': (5.348031, 49.1706321), 'Ancerville': (6.3878009, 49.0306024), 'Belleville-sur-Meuse': (5.370347, 49.1814053), 'Stenay': (5.1883537, 49.4916388), 'Étain': (5.6363489, 49.21335), 'Verdun': (5.3867277, 49.1589278), 'Bar-le-Duc': (5.1623805, 48.7712673), 'Saint-Mihiel': (5.5423046, 48.891207), 'Commercy': (5.5914753, 48.763203), 'Ligny-en-Barrois': (5.3253723, 48.685936), 'Revigny-sur-Ornain': (4.9870514, 48.8284503), 'Theix-Noyalo': (-2.637546715043245, 47.630494549999995), "Val d'Oust": (-2.423324555586654, 47.8693004), 'Évellys': (-2.8593386770704394, 47.97141775), 'Carentoir': (-2.1362362244234725, 47.81775695), 'La Gacilly': (-2.1519418829005823, 47.772628850000004), 'Pluméliau-Bieuzy': (-2.9905791817846676, 47.97265305), 'Pont-Scorff': (-3.4013848, 47.8347721), 'Sérent': (-2.5053307, 47.8225319), 'Péaule': (-2.3570901, 47.5811276), 'Ploemeur': (-3.4270534, 47.7354591), 'Saint-Dolay': (-2.1550384, 47.5452971), 'Plumelin': (-2.8869263, 47.8620653), 'Baud': (-3.0199362, 47.8758768), 'Le Bono': (-2.947793, 47.6402896), 'Theix': (-2.6474422423022155, 47.63506645), 'Quéven': (-3.4161464, 47.788792), 'Ploeren': (-2.8690655, 47.6554463), 'Sarzeau': (-2.7641997, 47.5287694), 'Le Palais': (-3.1548668, 47.3470611), 'Quiberon': (-3.12152, 47.4874), 'Sulniac': (-2.5685294, 47.6766517), 'Riantec': (-3.3088391, 47.7119271), 'Surzur': (-2.6304421, 47.5790425), 'Saint-Nolff': (-2.6523793, 47.7032918), 'Plumelec': (-2.6396508, 47.8378469), 'Pluneret': (-2.9571398, 47.6756752), 'Plouay': (-3.3470319507258073, 47.9181255), 'Plumergat': (-2.9163053, 47.7422676), 'Arradon': (-2.8229152, 47.62798), 'Le Faouët': (-3.0727913, 48.6849235), 'Nivillac': (-2.2829889, 47.5330834), 'Hennebont': (-3.2751174, 47.8058026), 'Lanester': (-3.3444489, 47.7639726), 'Landévant': (-3.1194205, 47.7636187), 'Merlevenez': (-3.2351341, 47.7359916), 'Inzinzac-Lochrist': (-3.2662889, 47.8434609), 'Kervignac': (-3.2388668, 47.7635947), 'Cléguer': (-3.3826297, 47.8548766), 'Noyal-Pontivy': (-2.882738, 48.0676349), 'Cléguérec': (-3.0700203, 48.1244286), 'Lorient': (-3.3660907, 47.7477336), 'Locminé': (-2.8362402, 47.8874274), 'Baden': (8.1390286, 48.1928371), 'Rieux': (1.58039, 49.93468), 'Ploërmel': (-2.39808, 47.9323786), 'Pluvigner': (-3.0095755, 47.7770909), 'Questembert': (-2.4527138, 47.6604074), 'Saint-Jean-Brévelay': (-2.7229551, 47.8451004), 'Moréac': (-2.8203462, 47.9199666), "Sainte-Anne-d'Auray": (-2.95522, 47.70339), 'Plescop': (-2.8062224, 47.6980235), 'Saint-Avé': (-2.7358875, 47.6903849), 'Séné': (-2.7373441, 47.6199584), 'Plougoumelen': (-2.9175888, 47.6525123), 'Ploemel': (-3.0717048, 47.6508427), 'Pluméliau': (-2.9893010175124104, 47.96848), 'Port-Louis': (57.5028044, -20.1624522), 'Vannes': (-2.7599079, 47.6586772), 'Pontivy': (-2.9675702, 48.0647729), 'Brech': (-2.9969193, 47.721585), 'Belz': (-3.1687227, 47.6759681), 'Auray': (-2.9833718, 47.6664485), 'Allaire': (-2.1625362, 47.6373804), 'Bignan': (-2.7718416, 47.8799364), 'Guidel': (-3.4914955, 47.7894226), 'Caudan': (-3.3404445, 47.81081), 'Gestel': (-3.4421869, 47.8036641), 'Elven': (-2.5905844, 47.7325736), 'Noyal-Muzillac': (-2.45712, 47.59148), 'Guer': (-2.1221276, 47.9036655), 'Erdeven': (-3.1557523, 47.6417056), 'Muzillac': (-2.4842784, 47.5553035), 'Locmiquélic': (-3.3424834, 47.7240686), 'Mauron': (-2.2850738, 48.0829709), 'Languidic': (-3.1583936, 47.8340055), 'Grand-Champ': (-2.8452562, 47.7584409), 'Camors': (-3.0000626, 47.847484), 'Carnac': (-3.0794428, 47.5836629), 'Larmor-Plage': (-3.376302, 47.7147553), 'Monterblanc': (-2.6798827, 47.7438832), 'Férel': (-2.3437351, 47.4824862), 'Locoal-Mendon': (-3.1050963, 47.711724), 'Gourin': (-3.6042254, 48.1385797), 'Crach': (-3.0011326, 47.6166941), 'Hayange': (6.0636733, 49.3304353), 'Bousse': (6.1983902, 49.2762832), 'Faulquemont': (6.5958205, 49.0487769), 'Alsting': (7.0003939, 49.1793035), 'Clouange': (6.0942709, 49.2606328), 'Hagondange': (6.1678213, 49.2540313), 'Audun-le-Tiche': (5.9572377, 49.4728242), 'Uckange': (6.1588684, 49.3007649), 'Saint-Julien-lès-Metz': (6.2033148, 49.1347537), 'Puttelange-aux-Lacs': (6.9316068, 49.0530406), 'Folschviller': (6.6862279, 49.0704468), 'Longeville-lès-Saint-Avold': (6.6459628, 49.1187388), 'Le Ban-Saint-Martin': (6.1511577, 49.122521), 'Creutzwald': (6.6963095, 49.2052869), 'Cattenom': (6.244669, 49.4074405), 'Behren-lès-Forbach': (6.935203, 49.1715929), 'Knutange': (6.0390472, 49.3407224), 'Ars-sur-Moselle': (6.0763114, 49.0769676), 'Maizières-lès-Metz': (6.1600345, 49.210252), 'Sarralbe': (7.0299169, 48.9998765), 'Rombas': (6.0938369, 49.2515048), 'Yutz': (6.1919276, 49.3615058), 'Ottange': (6.0183193, 49.4425009), 'Moyeuvre-Grande': (6.0453982, 49.2514437), 'Vitry-sur-Orne': (6.1062202, 49.2686949), 'Valmont': (0.5124798, 49.7444276), 'Neufchef': (6.0244822, 49.3165277), 'Manom': (6.1883115, 49.3714653), 'Mondelange': (6.168332, 49.2619543), 'Fameck': (6.1041352, 49.3043497), 'Talange': (6.1736941, 49.2368955), 'Œting': (6.9161261, 49.1745826), 'Fontoy': (5.995021, 49.3570812), 'Ham-sous-Varsberg': (6.6467482, 49.1792357), 'Courcelles-Chaussy': (6.4054894, 49.1092707), 'Amnéville': (6.1420739, 49.2603613), 'Nilvange': (6.0507501, 49.341511), 'Phalsbourg': (7.2586207, 48.767098), 'Terville': (6.1343914, 49.3449986), 'Rosselange': (6.0705587, 49.2574126), 'Farébersviller': (6.8752992, 49.1189722), 'Serémange-Erzange': (6.0912274, 49.321249), 'Montois-la-Montagne': (6.0231576, 49.2178917), 'Sainte-Marie-aux-Chênes': (5.9987938, 49.1921063), 'Longeville-lès-Metz': (6.1361244, 49.1152448), 'Marange-Silvange': (6.1164164, 49.2160771), 'Marly': (3.5435909, 50.3500747), 'Woustviller': (7.003809, 49.0758475), 'Woippy': (6.1519224, 49.1515848), 'Macheren': (6.7747461, 49.1002426), 'Gandrange': (6.1250201, 49.2705976), 'Guénange': (6.1960648, 49.2978357), 'Falck': (6.6398114, 49.222549), 'Morhange': (6.6458064, 48.9250518), 'Créhange': (6.5796867, 49.0609761), 'Petite-Rosselle': (6.8488569, 49.2074936), 'Morsbach': (6.865618, 49.1672234), 'Hambach': (7.0372308, 49.0613593), 'Grosbliederstroff': (7.0254677, 49.1564879), 'Moulins-lès-Metz': (6.1075035, 49.1068515), 'Cocheren': (6.8572032, 49.1440632), 'Freyming-Merlebach': (6.8184829, 49.1469437), 'Bertrange': (6.193137, 49.3133457), 'Saint-Avold': (6.7071754, 49.1043397), 'Florange': (6.122827, 49.3278069), 'Boulay-Moselle': (6.4935123, 49.1842243), 'Sarrebourg': (7.0557352, 48.734239), 'Thionville': (6.1675872, 49.3579272), 'Sarreguemines': (7.0708865, 49.1094836), 'Metz': (6.1763552, 49.1196964), 'Dieuze': (6.7179948, 48.8104847), 'Forbach': (6.874619, 49.1835376), 'Hettange-Grande': (6.1516319, 49.407298), 'Algrange': (6.0464343, 49.3570892), 'Montigny-lès-Metz': (6.153041, 49.0999832), 'Bitche': (7.4254577, 49.0508729), 'Spicheren': (6.96079222998615, 49.1928381), 'Scy-Chazelles': (6.1141574, 49.1144083), 'Stiring-Wendel': (6.9276846, 49.2011432), 'Hombourg-Haut': (6.7745606, 49.1241267), 'Carling': (6.7181281, 49.1682805), 'Bouzonville': (6.535247, 49.2901634), "L'Hôpital": (6.7334061, 49.1609381), 'Marspich': (6.0809355, 49.337554), 'Merlebach': (6.8166048, 49.1496124), 'Garchizy': (3.0947519, 47.0467368), 'Marzy': (3.0931689, 46.9804476), 'Coulanges-lès-Nevers': (3.1880756, 47.0054628), 'La Machine': (3.4658381, 46.8899094), 'Guérigny': (3.1937606, 47.0866061), 'Varennes-Vauzelles': (3.1456935, 47.013196), 'Nevers': (3.1577203, 46.9876601), 'Clamecy': (3.3619444, 49.4286111), 'Decize': (3.4616097, 46.8284259), 'Imphy': (3.2627526, 46.9281117), 'Cosne-Cours-sur-Loire': (2.9273937, 47.4115613), 'La Charité-sur-Loire': (3.0185784, 47.1793255), 'Fourchambault': (3.0827327, 47.017788), 'Lille': (3.0635282, 50.6365654), 'Steenvoorde': (2.5816407, 50.8101261), 'Hazebrouck': (2.5360331, 50.7226113), 'Leers': (3.2399516, 50.6792095), 'Roubaix': (3.1741734, 50.6915893), "Villeneuve-d'Ascq": (3.1314002, 50.6193174), 'Aulnoye-Aymeries': (3.8336383, 50.2002502), "La Chapelle-d'Armentières": (2.8965721, 50.6727867), 'Dunkerque': (2.3772525, 51.0347708), 'Berlaimont': (3.8123133, 50.2026418), 'Sin-le-Noble': (3.1117295, 50.3644331), 'Arleux': (3.1052613, 50.2816945), 'Maubeuge': (3.9674127, 50.2796417), 'Valenciennes': (3.5234846, 50.3579317), 'Le Quesnoy': (3.6364504, 50.2473417), 'Cambrai': (3.2346145, 50.1757546), 'Tourcoing': (3.1605714, 50.7235038), 'Douai': (3.0804641, 50.3675677), 'Armentières': (2.882169, 50.6867281), 'Dechy': (3.1299151, 50.354883), 'Denain': (3.3943729, 50.3270926), 'Avesnes-sur-Helpe': (3.9321056, 50.1218835), 'Croix': (3.1564365, 50.6802124), 'Coudekerque-Branche': (2.3894321, 51.020878), 'Lambersart': (3.0223644, 50.6477924), 'Bergues': (2.4325247, 50.9683886), 'Marcq-en-Barœul': (3.1043032, 50.6767018), 'Wattrelos': (3.2153907, 50.7016015), 'Loos': (3.0162523, 50.6092082), 'Lys-lez-Lannoy': (3.21621, 50.6713), 'Chéreng': (3.2022404, 50.6111045), 'Fourmies': (4.0458492, 50.0171741), 'Gravelines': (2.1273118, 50.9870697), 'Le Cateau-Cambrésis': (3.5417479, 50.1058353), 'Bailleul': (2.7349286, 50.7396668), 'Hornaing': (3.33736, 50.3694), 'La Bassée': (2.8046597, 50.5316305), 'Orchies': (3.2425313, 50.4745024), 'Hondschoote': (2.5862639, 50.9808564), 'Bray-Dunes': (2.5245134, 51.071002), 'Cysoing': (3.215489, 50.5677767), 'Rœulx': (3.3344358, 50.3075282), 'Iwuy': (3.3216106, 50.2329376), 'Haubourdin': (2.9864421, 50.6093554), 'Caudry': (3.4105032, 50.1243363), 'Ghyvelde': (2.501430297653145, 51.0446525), 'Bavay': (3.7949291, 50.2978864), "Condé-sur-l'Escaut": (3.5928883, 50.4483415), 'Anor': (4.1009549, 49.991385), 'Hautmont': (3.9200611, 50.2513704), 'Téteghem-Coudekerque-Village': (2.4530818448868095, 51.009179450000005), 'Vieux-Berquin': (2.6441759, 50.6948494), 'Haspres': (3.415738, 50.260129), 'Annœullin': (2.92648, 50.5279), 'Pecquencourt': (3.2150839, 50.3775793), 'Genech': (3.2141332, 50.5305709), 'Pont-à-Marcq': (3.1158401, 50.5223657), 'Wervicq-Sud': (3.0497929, 50.7728871), 'Wavrin': (2.932545, 50.570559), 'Emmerin': (3.0057584, 50.5908589), 'Quesnoy-sur-Deûle': (3.0008466, 50.7121678), 'Anzin': (3.5020684, 50.3703966), 'Waziers': (3.11252, 50.3838), 'Ronchin': (3.0778827, 50.6056749), 'Courchelettes': (3.060771, 50.3425863), 'Lomme': (2.9870927, 50.6457022), 'Jeumont': (4.0996569, 50.2971012), 'Fort-Mardyck': (2.3067298, 51.0292833), 'Monchecourt': (3.2095541, 50.3072925), 'Petite-Forêt': (3.47833, 50.3749), 'Aniche': (3.250989, 50.3309169), 'Saint-Amand-les-Eaux': (3.4281142, 50.4491519), 'Somain': (3.2810972, 50.3589237), 'Comines': (3.0066319, 50.7653265), 'Solesmes': (3.4975923, 50.1857534), 'Seclin': (3.0252819, 50.5452102), 'Escaudœuvres': (3.2666564, 50.1945683), 'Bauvin': (2.89353, 50.5191), 'Thumeries': (3.06638, 50.4725), 'La Gorgue': (2.7134006, 50.6372599), 'Halluin': (3.1249234, 50.7826693), 'Landrecies': (3.6913023, 50.1250142), 'Bourbourg': (2.1973702, 50.9466619), 'Sainghin-en-Mélantois': (3.1613643, 50.5893806), 'Mouvaux': (3.13689, 50.7029), 'Sequedin': (2.9821639, 50.6253159), 'Roost-Warendin': (3.10168, 50.4144), 'Lesquin': (3.10997, 50.5888), 'Saint-André-lez-Lille': (3.04468, 50.6608), 'Hasnon': (3.3866775, 50.4164903), 'Houplin-Ancoisne': (3.0011995, 50.5621058), 'Estaires': (2.7226506, 50.6440257), 'Roncq': (3.1209016, 50.7531232), 'Avesnes-les-Aubert': (3.3777081, 50.1976627), 'Lourches': (3.3532859, 50.3149962), 'Watten': (2.212066, 50.8324023), 'Phalempin': (3.0180377, 50.5172168), 'Masnières': (3.207531, 50.1147329), 'Beuvrages': (3.5071608, 50.3856245), 'Wahagnies': (3.0322175, 50.48797), 'Morbecque': (2.5155576, 50.6918534), 'Nomain': (3.2480344, 50.4994441), 'Templemars': (3.0528537, 50.5762334), 'Cappelle-la-Grande': (2.3677943, 50.9977218), 'Bondues': (3.0921638, 50.7034622), 'Grande-Synthe': (2.3029972, 51.0134813), 'Baisieux': (3.2410393, 50.6148786), 'Hergnies': (3.5232218, 50.4740852), 'Mons-en-Barœul': (3.1093516, 50.6428547), 'La Madeleine': (3.0733338, 50.654401), 'Douchy-les-Mines': (3.3909158, 50.2988884), 'Coutiches': (3.2015337, 50.4524298), 'Auby': (3.06175, 50.4136), 'Téteghem': (2.4574795178578706, 51.01573705), 'Wambrechies': (3.0521806, 50.6861944), 'Bousbecque': (3.076757, 50.7720856), 'Cuincy': (3.0452058, 50.3821492), 'Hem': (3.1839429, 50.6540895), 'Aulnoy-lez-Valenciennes': (3.5314621, 50.330848), 'Rousies': (4.0052619, 50.2726393), 'Proville': (3.2079258, 50.1580558), 'Neuville-Saint-Rémy': (3.21763, 50.1879), 'Auberchicourt': (3.2329285, 50.3319796), 'Wignehies': (4.0040972, 50.013991), "Bruay-sur-l'Escaut": (3.5379105, 50.397277), 'Grand-Fort-Philippe': (2.1045965, 51.0014456), 'Crespin': (3.6613971, 50.4177989), 'Ferrière-la-Grande': (3.9942662, 50.2560051), 'Sains-du-Nord': (4.0112313, 50.0953683), 'Provin': (2.9133145, 50.5151406), 'Lallaing': (3.16915, 50.3865), 'Neuville-en-Ferrain': (3.1525135, 50.7502701), 'Masny': (3.2035349, 50.3498274), 'Raimbeaucourt': (3.10637, 50.4362), 'Lambres-lez-Douai': (3.0572513, 50.3564041), 'Marpent': (4.0809622, 50.2930252), 'Saint-Saulve': (3.5547336, 50.3710165), 'Trith-Saint-Léger': (3.4845838, 50.3245906), 'Leval': (3.8306751, 50.1789453), 'Hérin': (3.4529374, 50.3576305), 'Wattignies': (3.0511655, 50.5878779), 'Sainghin-en-Weppes': (2.8927387, 50.5634089), 'Haveluy': (3.4037295, 50.3525832), 'Maing': (3.4883887, 50.3079322), 'Onnaing': (3.6011231, 50.3870769), 'Loon-Plage': (2.2192457, 50.9946103), 'Marquette-lez-Lille': (3.0647175, 50.6760968), 'Râches': (3.136211, 50.4149446), 'Pérenchies': (2.9744656, 50.6679447), 'Abscon': (3.29753, 50.3297), 'Flers-en-Escrebieux': (3.0614209, 50.3978149), 'Faches-Thumesnil': (3.0697877, 50.6026031), 'Feignies': (3.914409, 50.2954202), 'Hallennes-lez-Haubourdin': (2.9632568, 50.6140245), 'Fenain': (3.29915, 50.367), 'Vieux-Condé': (3.5675784, 50.4724228), 'Mérignies': (3.1103335, 50.506087), 'Linselles': (3.0794271, 50.737818), 'Lezennes': (3.1165338, 50.6146341), 'Trélon': (4.1035648, 50.061164), 'Marchiennes': (3.2825837, 50.4075026), 'Raismes': (3.4823888, 50.3916313), 'Santes': (2.955471, 50.5977229), 'Houplines': (2.9104189, 50.6904589), 'Leffrinckoucke': (2.438674, 51.0509395), 'Gondecourt': (2.9830581, 50.5432418), 'Montigny-en-Ostrevent': (3.1858602, 50.3646715), 'Famars': (3.5188176, 50.3151251), 'Wormhout': (2.468733, 50.8827628), 'Flines-lez-Raches': (3.1833569, 50.4269337), 'Louvroil': (3.9623154, 50.2644717), 'Capinghem': (2.9629646, 50.6447131), 'Quiévrechain': (3.6700494, 50.3956463), 'Steenwerck': (2.7763161, 50.7005793), 'Escaudain': (3.3438289, 50.3333247), 'Wasquehal': (3.1312447, 50.6691577), 'Nieppe': (2.8403065, 50.703612), 'Boussois': (4.0391856, 50.2906747), 'Guesnain': (3.15059, 50.3493), 'Beuvry-la-Forêt': (3.2864498, 50.4593377), 'Escautpont': (3.55175, 50.4241), 'Fresnes-sur-Escaut': (3.5801267, 50.4321092), 'Saint-Pol-sur-Mer': (2.3422205, 51.0273731), 'Allennes-les-Marais': (2.95043, 50.5404), 'Erquinghem-Lys': (2.8476686, 50.6768306), 'Avelin': (3.0812217, 50.5378519), 'Ostricourt': (3.0314735, 50.4538801), 'Hoymille': (2.4485047, 50.9730181), 'Toufflers': (3.2315753, 50.6619357), 'Salomé': (2.83819, 50.5365), 'Thiant': (3.4494452, 50.3045875), 'Templeuve-en-Pévèle': (3.1740541, 50.5326276), 'Neuville-sur-Escaut': (3.3523537, 50.2993698), 'Quarouble': (3.6244948, 50.3863821), 'Willems': (3.2383226, 50.6306376), 'Lecelles': (3.395199, 50.4714838), 'Fretin': (3.1350524, 50.5595962), 'La Sentinelle': (3.476136, 50.3496627), 'Flers-lez-Lille': (-1.0, -1.0), 'Le Plessis-Belleville': (2.7572516, 49.0972552), 'Coye-la-Forêt': (2.4671146, 49.1425141), 'Auneuil': (1.9951528702984023, 49.3706181), 'Chambly': (2.2444986, 49.1662774), 'Thourotte': (2.8832881, 49.4790123), 'Saint-Maximin': (2.4505702, 49.2203384), 'Fitz-James': (2.4283593, 49.3918395), 'Bresles': (2.2525151, 49.4091281), 'Grandvilliers': (1.9428278, 49.6647765), 'Nogent-sur-Oise': (2.4676727, 49.2744711), 'Chaumont-en-Vexin': (1.8821405, 49.2625682), 'Béthisy-Saint-Pierre': (2.8032, 49.3054), 'Choisy-au-Bac': (2.8774424, 49.439618), 'Crèvecœur-le-Grand': (2.08184, 49.611), 'Sérifontaine': (1.7686195, 49.3554483), 'Mouy': (2.3205165, 49.3196544), 'Gouvieux': (2.41521, 49.1902), 'Lamorlaye': (2.42438, 49.1598), 'Nanteuil-le-Haudouin': (2.8089282, 49.141185), 'Noyon': (3.0000288, 49.5805561), 'Senlis': (2.1523287, 50.5331612), 'Chantilly': (2.4636686, 49.1931736), 'Beauvais': (2.0823355, 49.4300997), 'Compiègne': (2.8263171, 49.4179497), 'Crépy-en-Valois': (2.8900507, 49.2358509), 'Creil': (2.4733771, 49.2607914), 'Saint-Just-en-Chaussée': (2.4337604, 49.5052334), 'Clermont': (2.4141907, 49.3780921), 'Ribécourt-Dreslincourt': (2.9231228, 49.5107071), 'Lacroix-Saint-Ouen': (2.7895045, 49.356047), 'Rantigny': (2.440098, 49.3311881), 'Pontpoint': (2.65396, 49.3023), 'Neuilly-en-Thelle': (2.2859429, 49.2242307), 'Verberie': (2.7288582, 49.3094836), 'Méru': (2.1355087, 49.2361978), 'Maignelay-Montigny': (2.51345, 49.552), 'Montataire': (2.4342569, 49.2561311), 'Liancourt': (2.46591, 49.3312), 'Orry-la-Ville': (2.5137195, 49.1312314), 'Noailles': (2.2000013, 49.3269), 'Hermes': (2.2461658, 49.3571636), 'Breuil-le-Vert': (2.4370282, 49.3623895), 'Bornel': (0.6706425, 44.6197337), 'La Chapelle-en-Serval': (2.52775, 49.122), 'Estrées-Saint-Denis': (2.6440507, 49.4266493), 'Cires-lès-Mello': (2.3559541, 49.2714584), 'Cauffry': (2.449229, 49.316926), "Saint-Leu-d'Esserent": (2.4215332, 49.2174377), 'Venette': (2.8007412, 49.4162216), 'Précy-sur-Oise': (2.3722215, 49.2066545), 'Pont-Sainte-Maxence': (2.6045838, 49.3012483), 'Laigneville': (2.4447527, 49.2956967), 'Longueil-Annel': (2.863719, 49.469411), 'Margny-lès-Compiègne': (2.8169708, 49.4246132), 'Bury': (2.3439338, 49.3132772), 'Breuil-le-Sec': (2.4513755, 49.3758211), 'Agnetz': (2.3845656, 49.3829563), 'Andeville': (2.16852, 49.2597), 'Sainte-Geneviève': (4.0722222, 49.7197222), 'Villers-Saint-Paul': (2.491905, 49.2883865), 'Verneuil-en-Halatte': (2.5188916, 49.273664), 'Alençon': (0.0911374, 48.4312059), 'Argentan': (-0.0223386, 48.74289), 'Vimoutiers': (0.1970867, 48.9297086), 'Sées': (0.1734544, 48.603378), 'Saint-Germain-du-Corbéis': (0.0636128, 48.4221718), 'Saint-Georges-des-Groseillers': (-0.572685, 48.7702155), 'Mortagne-au-Perche': (0.5501095, 48.5205106), "L'Aigle": (0.6278039, 48.7651713), 'Flers': (2.25255, 50.32053), "Athis-de-l'Orne": (-0.5007676, 48.8102537), 'Domfront': (2.5553031, 49.6032078), 'Tinchebray': (-0.7329038, 48.7636321), 'La Ferté-Macé': (-0.38203748624326295, 48.580906), 'Tinchebray-Bocage': (-0.7588936850362377, 48.7283279), "Bagnoles de l'Orne Normandie": (-0.421248254317265, 48.5697763), 'Val-au-Perche': (0.7766779992984025, 48.288410400000004), 'La Ferté-en-Ouche': (0.5539372666267839, 48.84362045), "Rives d'Andaine": (-0.46728879827175207, 48.5352175), 'Athis-Val de Rouvre': (-0.47272689250000083, 48.793238900000006), 'Longny les Villages': (0.7589869818264303, 48.54184755), 'Domfront en Poiraie': (-0.6225916540840775, 48.5864842), 'Tourouvre au Perche': (0.614361525227036, 48.606194), 'Gouffern en Auge': (0.14389870797163967, 48.785094650000005), 'La Ferté Macé': (-0.38203748624326295, 48.580906), 'Drocourt': (2.9246358, 50.3922158), 'Oye-Plage': (2.04198, 50.9809), 'Bully-les-Mines': (2.71989, 50.4464), 'Évin-Malmaison': (3.0325007, 50.4372552), 'Saint-Laurent-Blangy': (2.8016229, 50.2993922), 'Blendecques': (2.2817948, 50.7170841), 'Leforest': (3.0588, 50.4388), 'Noyelles-Godault': (2.996459, 50.419769), 'Beaurains': (2.7886987, 50.2632628), 'Berck': (1.5711617, 50.4052576), 'Cauchy-à-la-Tour': (2.4521125, 50.5031042), 'Allouagne': (2.50706, 50.5325), 'Houdain': (2.5378571, 50.4524327), 'Marck': (1.95199, 50.949), 'Sainte-Catherine': (2.763456, 50.304864), 'Coquelles': (1.8000938, 50.9363698), 'Mazingarbe': (2.7185236, 50.4693066), 'Wimereux': (1.6119368, 50.7696528), 'Lillers': (2.47604, 50.5606), 'Saint-Pol-sur-Ternoise': (2.3361053, 50.3812113), 'Chocques': (2.5696057, 50.5391865), 'Angres': (2.7561083, 50.4091556), 'Lapugnoy': (2.5342, 50.5204), 'Sallaumines': (2.8591238, 50.4205435), 'Montigny-en-Gohelle': (2.92988, 50.4269), 'Sangatte': (1.753713, 50.9458037), 'Laventie': (2.7719156, 50.6283001), 'Libercourt': (3.0105187, 50.4802102), 'Hulluch': (2.8134207, 50.4868473), 'Richebourg': (2.7399295, 50.5792526), 'Calais': (1.8538446, 50.9524769), 'Samer': (1.7469188, 50.6392717), 'Boulogne-sur-Mer': (1.6118771, 50.7259985), 'Arras': (2.7772211, 50.291048), 'Nœux-les-Mines': (2.6620888, 50.4808898), 'Lens': (2.8319805, 50.4291723), 'Oignies': (2.99387, 50.4643), 'Bruay-la-Buissière': (2.5461922, 50.4821963), 'Béthune': (2.6402097, 50.5292608), 'Saint-Omer': (2.2534183, 50.7515771), 'Liévin': (2.7738, 50.4245), 'Hénin-Beaumont': (2.9469339, 50.4191721), 'Carvin': (2.9577519, 50.4934145), 'Saint-Martin-au-Laërt': (2.2257042817928827, 50.756194550000004), 'Longuenesse': (2.24833, 50.7371), 'Aire-sur-la-Lys': (2.4000602, 50.6395937), 'Lumbres': (2.12179, 50.7051363), 'Ardres': (1.9786068, 50.8536324), 'Étaples': (1.6386252, 50.5139552), 'Éleu-dit-Leauwette': (2.81099, 50.4181), 'Harnes': (2.9068246, 50.44741), 'Achicourt': (2.7578569, 50.2731465), 'Éperlecques': (2.1525694, 50.8061561), 'Bapaume': (2.8497506, 50.1038538), 'Vimy': (2.8107935, 50.3732652), 'Barlin': (13.3888599, 52.5170365), 'Courrières': (2.94621, 50.4521), 'Auchel': (2.4682059, 50.5037492), 'Le Touquet-Paris-Plage': (1.5909325, 50.5211202), 'Vitry-en-Artois': (2.981135, 50.3260011), 'Loos-en-Gohelle': (2.792593, 50.4576111), 'Billy-Berclau': (2.8674884, 50.5189176), 'Guînes': (1.8704408, 50.8690859), 'Avion': (2.8343891, 50.4126128), 'Frévent': (2.2929785, 50.2782128), 'Beuvry': (2.6864794, 50.5337191), 'Saint-Étienne-au-Mont': (1.6260428, 50.6816857), 'Camiers': (1.61441, 50.5638), 'Wimille': (1.6303277, 50.7641091), 'Saint-Martin-Boulogne': (1.633867, 50.7258932), 'Hersin-Coupigny': (2.6497198, 50.4492663), 'Vermelles': (2.746528, 50.4884634), 'Saint-Nicolas': (2.7767966, 50.3022047), 'Marles-les-Mines': (2.5060412, 50.5019493), 'Méricourt': (2.8664217, 50.4015803), 'Loison-sous-Lens': (2.8623894, 50.4426254), 'Wingles': (2.85813, 50.4953), 'Rang-du-Fliers': (1.6439351, 50.4168975), 'Le Portel': (1.5736655, 50.7074811), 'Sains-en-Gohelle': (2.6833883, 50.4473282), 'Sailly-sur-la-Lys': (2.7701687, 50.6579904), 'Noyelles-sous-Lens': (2.8760012, 50.4307524), 'Pont-à-Vendin': (2.88627, 50.4738), 'Desvres': (1.8348266, 50.6678737), 'Marquise': (1.699554, 50.8125819), 'Billy-Montigny': (2.9116301, 50.4179484), 'Cucq': (1.6246227, 50.4865763), 'Fouquières-lès-Lens': (2.913055, 50.4297974), 'Annezin': (2.6193, 50.5415), 'Dourges': (2.9844016, 50.4340484), 'Auxi-le-Château': (2.1177614, 50.2314908), 'Biache-Saint-Vaast': (2.9440409, 50.3116723), 'Brebières': (3.0224298, 50.3372093), 'Anzin-Saint-Aubin': (2.7466712, 50.3133381), 'Divion': (2.50722, 50.4694), 'Auchy-les-Mines': (2.7853708, 50.5120767), 'Arques': (2.3062068, 50.7396639), 'Isbergues': (2.4527186, 50.6238724), 'Vendin-le-Vieil': (2.8671553, 50.4756482), 'Annay': (2.8855, 50.4628), 'Outreau': (1.5845684, 50.7036916), 'Haisnes': (2.8009381, 50.5081773), 'Audruicq': (2.0752362, 50.8786489), 'Courcelles-lès-Lens': (3.018367, 50.418154), 'Douvrin': (2.8318133, 50.5101937), 'Calonne-Ricouart': (2.481762, 50.4869689), 'Haillicourt': (2.57793, 50.4751), 'Dainville': (2.72248, 50.2807141), 'La Couture': (2.7145125, 50.5811855), 'Gonnehem': (2.5738425, 50.5622867), 'Coulogne': (1.8840904, 50.9257432), 'Grenay': (2.7396733, 50.4482588), 'Burbure': (2.4655642, 50.5370155), 'Aix-Noulette': (2.7148579, 50.4251621), 'Violaines': (2.7884031, 50.5414315), 'Wizernes': (2.2283996, 50.709962), 'Rinxent': (1.73997, 50.8064), 'Verquin': (2.6411556, 50.5011555), 'Saint-Léonard': (1.6270864, 50.6888137), 'Souchez': (2.7410877, 50.391009), 'Fleurbaix': (2.8321763, 50.6537034), 'Meurchin': (2.8900936, 50.49539), 'Merlimont': (1.614809, 50.4553061), 'Neufchâtel-Hardelot': (1.6306156, 50.6188491), 'Labourse': (2.679865, 50.4972978), 'Lestrem': (2.6857534, 50.6216212), 'Rouvroy': (2.9036734, 50.393427), 'Équihen-Plage': (1.5716648, 50.679393), 'Saint-Venant': (2.5492669, 50.6237649), 'Bruay-en-Artois': (2.5420986117066224, 50.48362575), 'Molinghem': (2.443245, 50.6099541), 'Labuissière': (4.194796, 50.3141612), 'Saint-Martin-lez-Tatinghem': (2.2150835399743327, 50.7495707), 'Ennezat': (3.2208875, 45.8976947), 'Aigueperse': (3.20189, 46.0219319), 'Pont-du-Château': (3.2492577, 45.7971315), 'Ceyrat': (3.0668817, 45.7328967), 'Brassac-les-Mines': (3.3294831, 45.4130887), 'Beaumont': (-94.1018461, 30.0860459), 'Courpière': (3.5325463, 45.7559617), 'Veyre-Monton': (3.1648468, 45.6732958), 'Saint-Genès-Champanelle': (3.0202426, 45.7193702), 'Pérignat-lès-Sarliève': (3.1405545, 45.736057), 'Blanzat': (3.0771704, 45.8283549), 'Volvic': (3.0375701, 45.872542), 'Chamalières': (3.0681174, 45.776018), 'Châteaugay': (3.0850695, 45.8502246), 'Aubière': (3.1144195, 45.7512575), 'Le Cendre': (3.1856364, 45.7205658), 'Orcines': (3.0113839, 45.7853457), 'Romagnat': (3.098485, 45.7280876), 'Saint-Éloy-les-Mines': (2.8345577, 46.1611691), 'Les Martres-de-Veyre': (3.1903821, 45.6843764), 'Vertaizon': (3.2881194, 45.7703602), 'Lezoux': (3.3805242, 45.8281092), 'Vic-le-Comte': (3.2433311, 45.6433495), 'Clermont-Ferrand': (3.0819427, 45.7774551), 'Issoire': (3.2502213, 45.5432973), 'Riom': (3.1136667, 45.893068), 'Ambert': (3.742639, 45.5504338), 'Thiers': (3.5489707, 45.855795), 'Billom': (3.3390575, 45.7220503), 'Maringues': (3.328959, 45.9210457), 'Royat': (3.0477919, 45.7643631), 'Châtel-Guyon': (3.06111, 45.9205), 'Mozac': (3.0953843, 45.8940829), 'La Roche-Blanche': (3.1276022, 45.7085345), "Cournon-d'Auvergne": (3.1962804, 45.7392306), 'Puy-Guillaume': (3.4755085, 45.9598042), 'Orcet': (3.17459, 45.7067), 'Aulnat': (3.16383, 45.7949413), 'Cébazat': (3.1000933, 45.8305758), 'Gerzat': (3.1435185, 45.8250404), 'Lempdes': (3.1979033, 45.769032), 'Mur-sur-Allier': (3.2431779566477275, 45.76034145), 'Pau': (-0.3685668, 43.2957547), 'Biarritz': (-1.552726590666314, 43.471143749999996), 'Bayonne': (-1.4736657, 43.4945144), 'Serres-Castet': (-0.3541989, 43.3861879), 'Hendaye': (-1.7616499, 43.3641518), 'Anglet': (-1.5149935, 43.4813927), 'Billère': (-0.3905556, 43.3030044), 'Artix': (-0.5722401, 43.397429), 'Idron': (-0.3117531, 43.2910849), 'Bizanos': (-0.3531403, 43.2887961), 'Lescar': (-0.4357266, 43.3333773), 'Pontacq': (-0.1144186, 43.1856262), 'Lons': (5.5589973, 46.6727037), 'Nay': (-105.0000001, 22.0000001), 'Cambo-les-Bains': (-1.3994687, 43.3609139), 'Lahonce': (-1.3902495, 43.4831925), 'Hasparren': (-1.3049538, 43.38388), "Saint-Pierre-d'Irube": (-1.4474527, 43.4731208), 'Arcangues': (-1.5216228, 43.4367292), 'Morlaàs': (-0.2649452, 43.3454394), 'Jurançon': (-0.3896425, 43.2871825), 'Oloron-Sainte-Marie': (-0.6115586, 43.1922408), 'Saint-Jean-de-Luz': (-1.6635695, 43.3870763), 'Ciboure': (-1.6676238, 43.3844003), 'Mauléon-Licharre': (-0.8885873, 43.2233373), 'Saint-Pée-sur-Nivelle': (-1.5504787, 43.3557594), 'Orthez': (-0.7738363, 43.4874963), 'Ustaritz': (-1.4562207, 43.39876), 'Sare': (-1.5793011, 43.3129609), 'Boucau': (-1.4857538, 43.5236127), 'Mourenx': (-0.6294321, 43.3708943), 'Salies-de-Béarn': (-0.9234025, 43.4703515), 'Urrugne': (-1.7003017, 43.3624715), 'Briscous': (-1.3339998, 43.4602151), 'Gelos': (-0.3708013, 43.2849213), 'Ascain': (-1.620839, 43.3449393), 'Urcuit': (-1.3367618, 43.4856054), 'Bassussarry': (-1.5149178, 43.4464338), 'Villefranque': (-1.4522561, 43.4367357), 'Mouguerre': (-1.4149834, 43.4694806), 'Monein': (-0.5786974, 43.3211151), 'Sauvagnon': (-0.3858932, 43.404401), 'Gan': (116.0, 28.0), 'Bidart': (-1.5898909, 43.4391702), 'Bordes': (-0.2817162, 43.234542), "Bordères-sur-l'Échez": (0.0500621, 43.2583864), 'Barbazan-Debat': (0.1176276, 43.1964736), 'Lourdes': (-0.0464975, 43.0940904), 'Tarbes': (0.0781021, 43.232858), 'Bagnères-de-Bigorre': (0.1530914, 43.0658212), 'Lannemezan': (0.384241, 43.1252298), 'Soues': (2.05322, 49.95632), 'Argelès-Gazost': (-0.0997681, 43.00511), 'Aureilhan': (0.0965226, 43.2426749), 'Odos': (0.0581, 43.1973846), 'Vic-en-Bigorre': (0.0548805, 43.3860159), 'Séméac': (0.1046315, 43.2284141), 'Juillan': (0.024292, 43.2011), 'Ibos': (0.0045982, 43.2325985), 'Alénya': (2.9806725, 42.6390997), 'Pia': (2.9191579, 42.7452716), 'Palau-del-Vidre': (2.9600905, 42.5717989), 'Canohès': (2.8371319, 42.6533713), 'Argelès-sur-Mer': (3.0253613, 42.5476734), 'Le Barcarès': (3.0335243, 42.7946794), 'Ille-sur-Têt': (2.6224853, 42.6709665), 'Rivesaltes': (2.8709277, 42.7683568), 'Saint-Génis-des-Fontaines': (2.9207275, 42.5428648), 'Torreilles': (2.992913, 42.7548029), 'Saint-André': (0.8514079, 43.273316), 'Saint-Cyprien': (1.0461817, 44.8693315), 'Saint-Estève': (2.8456715, 42.7100287), 'Pollestres': (2.8709063, 42.6420519), 'Salses-le-Château': (2.9181798, 42.8333802), 'Bages': (2.9924502, 43.1236647), 'Baho': (2.82168, 42.7042), 'Bompas': (1.6139895, 42.8661676), 'Baixas': (2.8091531, 42.7498075), 'Villeneuve-de-la-Raho': (2.9182491, 42.6359815), "Espira-de-l'Agly": (2.8325432, 42.7771379), 'Le Soler': (2.8007536, 42.6824044), 'Toulouges': (2.8319821, 42.6700342), 'Pézilla-la-Rivière': (2.7693489, 42.6940024), 'Millas': (2.6975072, 42.692017), 'Latour-Bas-Elne': (3.0024845, 42.6076675), 'Ponteilla': (2.8139027, 42.6272392), 'Saleilles': (2.94911, 42.6555), 'Villelongue-de-la-Salanque': (2.9834029, 42.7264387), 'Saint-Hippolyte': (-0.1218554, 44.8698075), 'Céret': (2.7488069, 42.485804), 'Perpignan': (2.8953121, 42.6985304), 'Banyuls-sur-Mer': (3.1275893, 42.4794084), 'Amélie-les-Bains-Palalda': (2.6653321, 42.4741898), 'Arles-sur-Tech': (2.6343451, 42.4562149), 'Maureillas-las-Illas': (2.8054056, 42.48966), 'Le Boulou': (0.5863309, 45.4051111), 'Cabestany': (2.9403263, 42.6799514), 'Elne': (2.9705605, 42.6015016), 'Sorède': (2.9564544, 42.5304019), 'Sainte-Marie-la-Mer': (3.01762, 42.7251), "Saint-Féliu-d'Avall": (2.743105, 42.6802608), 'Thuir': (2.7557657, 42.6291282), 'Claira': (2.9556064, 42.7601553), 'Saint-Laurent-de-la-Salanque': (2.9891635, 42.7732114), 'Port-Vendres': (3.1056167, 42.5204146), 'Prades': (4.3160861, 44.6382), 'Canet-en-Roussillon': (3.0105105, 42.704359), 'Gambsheim': (7.8864256, 48.6919519), 'Seltz': (8.1067978, 48.8920496), 'Scherwiller': (7.4195389, 48.286161), 'Châtenois': (7.4013652, 48.272548), 'Mertzwiller': (7.6823034, 48.8659947), 'Oberhoffen-sur-Moder': (7.8671359, 48.7826029), 'Holtzheim': (7.6431803, 48.5595788), 'Marckolsheim': (7.5444161, 48.165441), 'Hochfelden': (7.562195025290636, 48.755472), 'Oberhausbergen': (7.6857854, 48.6056872), 'Wolfisheim': (7.6672188, 48.5875517), 'Marlenheim': (7.4900618, 48.6217367), 'Plobsheim': (7.7267224, 48.4707922), 'Kilstett': (7.8573782, 48.6749481), 'Truchtersheim': (7.618187103761335, 48.65842085), 'Sarre-Union': (7.0899329, 48.9404465), 'Bischheim': (7.7523716, 48.6148396), 'Vendenheim': (7.7109609, 48.6656016), 'Weyersheim': (7.802508, 48.7182493), 'La Wantzenau': (7.8277685, 48.6583055), 'Betschdorf': (7.9065678, 48.8974813), 'Lipsheim': (7.6644772, 48.490003), 'Mutzig': (7.4555778, 48.5395474), 'Rhinau': (7.7054878, 48.3180965), 'Wasselonne': (7.4472685, 48.6373266), 'Gundershoffen': (7.6538032, 48.9052729), 'Schweighouse-sur-Moder': (7.7358163, 48.821213), 'Hœnheim': (7.7554925, 48.6212867), 'Bischwiller': (7.8597466, 48.7682281), 'Herrlisheim': (7.9061317, 48.7288855), 'Offendorf': (7.9123631, 48.7118236), 'Reichstett': (7.7537839, 48.6483814), 'Eckbolsheim': (7.6883102, 48.5806873), 'Ostwald': (7.7102193, 48.5425109), 'Hilsenheim': (7.5632388, 48.2877709), 'Lingolsheim': (7.6817943, 48.5559512), 'Lampertheim': (7.7003567, 48.6503574), 'Strasbourg': (7.7507127, 48.584614), 'Gerstheim': (7.7018376, 48.3803691), 'Geudertheim': (7.7561385, 48.7232088), 'Reichshoffen': (7.667539, 48.9335648), 'Soufflenheim': (7.9636887, 48.8306043), 'Fegersheim': (7.6798031, 48.4897313), 'Gries': (7.8138887, 48.7530555), 'Huttenheim': (7.580719, 48.3589814), 'Souffelweyersheim': (7.7416302, 48.6355292), 'Ingwiller': (7.4798581, 48.8718112), 'Erstein': (7.6632641, 48.4227385), 'Marmoutier': (7.381985, 48.6904394), 'Wissembourg': (7.9445109, 49.0365935), 'Brumath': (7.708107, 48.7309406), 'Schiltigheim': (7.7484488, 48.6047148), 'Geispolsheim': (7.6437386, 48.5146952), 'Illkirch-Graffenstaden': (7.7116218, 48.5298611), 'Molsheim': (7.4924679, 48.5419351), 'Obernai': (7.4817123, 48.4623358), 'Haguenau': (7.7885978, 48.8172236), 'Sélestat': (7.454217, 48.2594396), 'Bouxwiller': (7.4800291, 48.8254997), 'Saverne': (7.3625953, 48.7419909), 'Barr': (7.4492808, 48.4088704), 'Bischoffsheim': (7.4901898, 48.4876657), 'Duttlenheim': (7.5655858, 48.5242179), 'Benfeld': (7.5952375, 48.3707281), 'Drusenheim': (7.951689, 48.7631141), 'La Broque': (7.211743, 48.4684871), 'Dorlisheim': (7.4862698, 48.5246813), 'Dettwiller': (7.4675594, 48.7535135), 'Mundolsheim': (7.7150066, 48.6416216), 'Eschau': (7.7153003, 48.4856711), 'Hœrdt': (7.7837447, 48.698517), 'Weitbruch': (7.779797, 48.7556693), 'Rosheim': (7.4698832, 48.496421), 'Soultz-sous-Forêts': (7.8842704, 48.9374463), 'Val-de-Moder': (7.600393348506518, 48.839586600000004), 'Bartenheim': (7.4766365, 47.6354901), 'Habsheim': (7.4164754, 47.7286566), 'Bollwiller': (7.2623733, 47.8600613), 'Biesheim': (7.5448609, 48.0408514), 'Hégenheim': (7.5261691, 47.5624713), 'Brunstatt': (7.329411448629806, 47.716560650000005), 'Saint-Louis': (-15.2954502, 16.1876131), 'Danjoutin': (6.8628326, 47.618521), 'Offemont': (6.87575, 47.6609928), 'Morschwiller-le-Bas': (7.2706716, 47.7365524), 'Riedisheim': (7.3682858, 47.7492477), 'Richwiller': (7.281152, 47.7798816), 'Orbey': (7.1622187, 48.1272548), 'Rixheim': (7.3968786, 47.7441969), 'Volgelsheim': (7.5471221, 48.0164772), 'Rouffach': (7.2994054, 47.9569146), 'Pulversheim': (7.3029778, 47.8359062), 'Munster': (-8.570897276292667, 52.307621600000004), 'Turckheim': (7.278011, 48.0878217), 'Zillisheim': (7.2938793, 47.696435), 'Thann': (7.1020957, 47.8101008), 'Altkirch': (7.2401542, 47.6238204), 'Guebwiller': (7.2113271, 47.9094629), 'Cernay': (0.325, 49.0211111), 'Illzach': (7.347339, 47.7815313), 'Baldersheim': (7.378094, 47.8007333), 'Mulhouse': (7.3389937, 47.7467233), 'Sainte-Croix-en-Plaine': (7.3859443, 48.0079341), 'Issenheim': (7.253469, 47.9024133), 'Horbourg-Wihr': (7.4002423, 48.0805815), 'Huningue': (7.5794613, 47.5941514), 'Kembs': (7.5048054, 47.6895561), 'Vieux-Thann': (7.1217043, 47.8070778), 'Village-Neuf': (7.5696755, 47.6054816), 'Masevaux': (7.0088624485807856, 47.78173105), 'Kingersheim': (7.3386856, 47.7923002), 'Lutterbach': (7.2813576, 47.7599822), 'Colmar': (7.3579641, 48.0777517), 'Ribeauvillé': (7.3248871, 48.1924602), 'Pfastatt': (7.3025075, 47.7697441), 'Ensisheim': (7.3529099, 47.8665246), 'Hésingue': (7.5186109, 47.5751587), 'Blotzheim': (7.4966546, 47.6014104), 'Buhl': (7.0853, 48.7253), 'Sausheim': (7.3720907, 47.787994), 'Wintzenheim': (7.2925545, 48.0724515), 'Ingersheim': (7.3022832, 48.0968771), 'Sainte-Marie-aux-Mines': (7.1851184, 48.2476662), 'Soultz-Haut-Rhin': (7.2288038, 47.8870216), 'Wittelsheim': (7.2402432, 47.8091086), 'Sierentz': (7.4533853, 47.6536197), 'Staffelfelden': (7.2552171, 47.8270012), 'Wittenheim': (7.3373681, 47.8080799), 'Kaysersberg Vignoble': (7.247852991000449, 48.154958449999995), 'Brunstatt-Didenheim': (7.318939398098226, 47.716560650000005), 'Masevaux-Niederbruck': (7.008867680624187, 47.78172725), 'Saint-Pierre-la-Palud': (4.6129642, 45.7887025), 'Pollionnay': (4.6607806, 45.7659276), 'Lissieu': (4.742215, 45.8652176), "Saint-Jean-d'Ardières": (4.7378381, 46.1251695), 'Mornant': (4.670592, 45.6193985), 'Saint-Martin-en-Haut': (4.5594655, 45.6602362), 'Communay': (4.8324705, 45.6043302), 'Chasselay': (5.33692, 45.2569), 'Quincieux': (4.77378, 45.9143), "Marcy-l'Étoile": (4.7071338, 45.7816572), 'Montagny': (4.2343363, 46.032283), "L'Arbresle": (4.6165809, 45.8362368), 'Orliénas': (4.7188841, 45.6595515), "Collonges-au-Mont-d'Or": (4.8411566, 45.8229657), 'Loire-sur-Rhône': (4.8063326, 45.5589736), 'Limas': (4.7087667, 45.9751893), "Saint-Cyr-au-Mont-d'Or": (4.8175958, 45.8156548), 'Pierre-Bénite': (4.8241323, 45.7028321), "Saint-Symphorien-d'Ozon": (4.855816, 45.6329361), 'Anse': (4.7170072, 45.9364662), 'Fontaines-sur-Saône': (4.8488338, 45.8325696), 'Saint-Laurent-de-Mure': (5.0461188, 45.6872816), "Champagne-au-Mont-d'Or": (4.789458, 45.7954913), 'Cours-la-Ville': (4.3217677, 46.0991652), "Chazay-d'Azergues": (4.71116, 45.8759), 'Arnas': (4.7071167, 46.0235247), 'Charbonnières-les-Bains': (4.7459, 45.7787), 'Condrieu': (4.7676199, 45.4624294), 'Saint-Fons': (4.84935, 45.7001), 'Vaugneray': (4.651686407892688, 45.7316723), 'Albigny-sur-Saône': (4.8323149, 45.8706791), 'Chaponnay': (4.9345198, 45.6274322), 'Saint-Georges-de-Reneins': (4.723272, 46.0629911), 'Craponne': (4.7231057, 45.743476), 'Ternay': (0.7774903, 47.7304215), 'Saint-Genis-Laval': (4.7928827, 45.6950021), 'Saint-Pierre-de-Chandieu': (5.0100345, 45.6446435), 'Thurins': (4.6395008, 45.6815791), 'Messimy': (4.67418, 45.6985), 'Chaponost': (4.7469158, 45.7107025), "Le Bois-d'Oingt": (4.5856762, 45.9211413), 'Fontaines-Saint-Martin': (4.8547594, 45.8442606), 'Irigny': (4.8216902, 45.6733141), 'Cailloux-sur-Fontaines': (4.8737656, 45.8523803), 'Dommartin': (2.3895684, 49.8006199), 'Gleizé': (4.6971455, 45.9888277), 'Vernaison': (4.80986, 45.6467), 'Soucieu-en-Jarrest': (4.7036797, 45.6780477), 'Saint-Symphorien-sur-Coise': (4.4550523, 45.6313062), 'Vourles': (4.7753246, 45.6603571), 'Toussieu': (4.9849791, 45.6568916), 'Sérézin-du-Rhône': (4.8247139, 45.6283376), 'La Tour-de-Salvagny': (4.7163893, 45.8129372), "Saint-Didier-au-Mont-d'Or": (4.7979353, 45.8115147), "Saint-Germain-au-Mont-d'Or": (4.80259, 45.8828), 'Tassin-la-Demi-Lune': (4.7657747, 45.7598386), 'Pusignan': (5.06785, 45.7561), 'Saint-Bonnet-de-Mure': (5.0224963, 45.6945331), 'Thizy-les-Bourgs': (4.310490309298064, 46.05038665), 'Cours': (0.5897222, 44.3230556), 'Chabanière': (4.620813457112174, 45.5853824), 'Porte des Pierres Dorées': (4.648763193683868, 45.96278955), "Val d'Oingt": (4.568752532051882, 45.93867675), 'Beauvallon': (4.9031348, 44.8571562), 'Belleville-en-Beaujolais': (4.7486553, 46.1089382), 'Vindry-sur-Turdine': (4.496296828085184, 45.88870745), 'Amplepuis': (4.3313792, 45.9730698), 'Lozanne': (4.683398, 45.8563065), 'Givors': (4.7696529, 45.5844238), 'Francheville': (4.8817511, 47.4542392), 'Oullins': (4.8039331, 45.7153254), 'Dardilly': (4.7537701, 45.8064992), 'Vaulx-en-Velin': (4.9206153, 45.7784255), 'Villefranche-sur-Saône': (4.726611, 45.9864749), 'Écully': (4.7785644, 45.7754498), 'Taluyers': (4.7191738, 45.6406556), 'Rillieux-la-Pape': (4.8994366, 45.823514), 'Ampuis': (4.811206, 45.4888765), 'Pommiers': (3.2720429, 49.3935702), 'Neuville-sur-Saône': (4.8395051, 45.8765557), 'Grézieu-la-Varenne': (4.6910373, 45.7479949), 'Brignais': (4.7540703, 45.6739755), 'Belleville': (6.0982683, 48.8176714), 'Limonest': (4.7708336, 45.8356702), 'Pontcharra-sur-Turdine': (4.4897842, 45.8748314), 'Brindas': (4.6940681, 45.7206375), 'Lentilly': (4.6633554, 45.8179161), 'Tarare': (4.4371374, 45.8942555), 'Saint-Genis-les-Ollières': (4.72654, 45.7601), 'Millery': (4.3163267, 47.5154205), 'Colombier-Saugnieu': (5.1119241, 45.7113787), 'Genas': (4.9965605, 45.7330585), 'La Mulatière': (4.8096049, 45.7292173), 'Vesoul': (6.1544694, 47.6197395), 'Lure': (6.4946806, 47.6863376), 'Gray': (5.5919867, 47.4455642), 'Échenoz-la-Méline': (6.1379125, 47.6008519), 'Arc-lès-Gray': (5.5818459, 47.4575385), 'Luxeuil-les-Bains': (6.3808865, 47.817177), 'Héricourt': (2.2536, 50.34478), 'Ronchamp': (6.632869, 47.7002959), 'Saint-Loup-sur-Semouse': (6.2731365, 47.8849389), 'Champagney': (5.9065734, 47.2562434), 'Port-sur-Saône': (6.0454479, 47.6876801), 'Fougerolles': (6.416006243289244, 47.902459), 'Fougerolles-Saint-Valbert': (6.422725472378433, 47.89260605), 'Paray-le-Monial': (4.1177959, 46.450559), 'Chagny': (4.7523461, 46.910578), 'Chauffailles': (4.338189, 46.2077321), 'Givry': (4.7432564, 46.7831191), 'Bourbon-Lancy': (3.7737674, 46.6193905), 'Mâcon': (4.8322266, 46.3036683), 'Autun': (4.2989478, 46.9510313), 'Cluny': (4.6575708, 46.4339138), 'Chalon-sur-Saône': (4.8529605, 46.7888997), 'Charolles': (4.2743162, 46.435311), 'Le Creusot': (4.4285961, 46.8054064), 'Sanvignes-les-Mines': (4.2922042, 46.6654678), 'Gueugnon': (4.0633825, 46.6005232), 'Montceau-les-Mines': (4.3631681, 46.6740455), 'Louhans': (5.2232175, 46.6291425), 'Digoin': (3.9786291, 46.4810422), 'Tournus': (4.9122425, 46.5617373), 'Le Breuil': (3.6460985, 48.9747452), 'Champforgeuil': (4.8350294, 46.8203731), 'Châtenoy-le-Royal': (4.8145788, 46.794589), 'Sennecey-le-Grand': (4.869504, 46.6409587), 'La Chapelle-de-Guinchay': (4.7555202, 46.2110224), 'Blanzy': (4.3899508, 46.7037419), 'Crêches-sur-Saône': (4.7867962, 46.246363), 'Charnay-lès-Mâcon': (4.784542, 46.3098212), 'Ouroux-sur-Saône': (4.9536545, 46.721557), 'Montchanin': (4.4699684, 46.747427), 'Gergy': (4.9470751, 46.8766663), 'Saint-Rémy': (4.2996321, 47.6367211), 'Torcy': (2.02302, 50.4830626), 'Saint-Calais': (0.7434396, 47.9201366), 'Saint-Saturnin': (3.9019931, 48.613152), 'Mayet': (0.2752165, 47.7592389), 'Mulsanne': (0.2460607, 47.9069356), 'La Flèche': (-0.0746149, 47.6968408), 'Le Mans': (0.1967849, 48.0073849), 'Écommoy': (0.2733338, 47.8286006), 'Coulaines': (0.2068691, 48.0306873), 'Précigné': (-0.3246558, 47.7664332), 'Spay': (0.1510542, 47.9241442), 'La Bazoge': (-1.0367071894865312, 48.6422686), 'Teloché': (0.2698822, 47.887679), 'Le Lude': (0.15514092058276263, 47.6572143), 'Champagné': (0.33179, 48.0168), 'Noyen-sur-Sarthe': (-0.0992579, 47.8723118), 'Mamers': (0.3696376, 48.3505932), 'Bonnétable': (0.4269856, 48.1768212), 'Arnage': (0.1889245, 47.9269544), 'Saint-Mars-la-Brière': (0.3718076, 48.0301287), 'Moncé-en-Belin': (0.2038619, 47.8941045), 'Vibraye': (0.7341899, 48.0604122), 'La Ferté-Bernard': (0.6554511, 48.1860576), 'Sablé-sur-Sarthe': (-0.3352048, 47.8400712), 'Château-du-Loir': (0.4310473863354066, 47.6995205), 'Connerré': (0.4933923, 48.0604578), 'La Milesse': (0.1361917, 48.0647399), 'Montfort-le-Gesnois': (0.403157, 48.049078), 'La Suze-sur-Sarthe': (0.0275915, 47.8895538), 'Guécélard': (0.1294751, 47.8771768), "Savigné-l'Évêque": (0.2962126, 48.0773296), "Parigné-l'Évêque": (0.376251, 47.9324), 'Sargé-lès-le-Mans': (0.2402009, 48.0339235), 'Ruaudin': (0.266523, 47.9437), 'Roëzé-sur-Sarthe': (0.0683017, 47.8933701), 'Cérans-Foulletourte': (0.0735833, 47.8259328), "Yvré-l'Évêque": (0.270926, 48.0137909), 'Montval-sur-Loir': (0.44213227296009794, 47.69545275), 'Cherré-Au': (0.6612376981585295, 48.17433375), 'Fresnay-sur-Sarthe': (0.01920527167183339, 48.2837686), 'Albertville': (6.3925417, 45.6754622), 'Chambéry': (5.9203636, 45.5662672), 'Aix-les-Bains': (5.9146362, 45.6886292), 'Saint-Jean-de-Maurienne': (6.3453329, 45.2775258), 'Bourg Saint Maurice': (6.7694313, 45.6185284), 'Le Bourget-du-Lac': (5.8595715, 45.6470041), 'Aime': (6.6229794183746336, 45.5555869), 'Albens': (5.935358924332197, 45.78627875), 'Modane': (6.6728169, 45.2015369), 'La Biolle': (5.9275429, 45.7567025), 'Saint-Martin-de-Belleville': (6.5000761369605815, 45.3503871), 'Yenne': (5.7587461, 45.7036956), 'Cognin': (5.8933927, 45.5587047), 'Ugine': (6.4205158, 45.7490363), 'Drumettaz-Clarafond': (5.9285677, 45.6582224), "Saint-Pierre-d'Albigny": (6.1548267, 45.569016), 'Barberaz': (5.9421274, 45.5582393), 'La Ravoire': (5.9643319, 45.5586324), 'Barby': (4.3129272, 49.5217295), 'Moûtiers': (6.5340061, 45.4849883), 'Saint-Alban-Leysse': (5.9557957, 45.5812325), 'Gilly-sur-Isère': (6.3490978, 45.659271), 'Saint-Baldoph': (5.952071, 45.5358369), 'Aigueblanche': (6.512083614338553, 45.52055675), 'Mercury': (6.3360129, 45.6753463), 'Jacob-Bellecombette': (5.916979, 45.551703), 'Challes-les-Eaux': (5.9831637, 45.5482519), 'Tresserve': (5.8955448, 45.6731168), 'Grésy-sur-Aix': (5.9317779, 45.7213843), 'La Motte-Servolex': (5.8767935, 45.5984113), 'Les Marches': (6.001357354363555, 45.48631485), 'Les Abrets': (5.597710960334607, 45.54341925), 'Montmélian': (6.0587974, 45.5009946), 'La Rochette': (2.662899, 48.5045038), 'Entrelacs': (-74.0035168, 46.1126751), 'Les Belleville': (6.503499, 45.3830517), 'La Plagne Tarentaise': (6.694657308933131, 45.568546749999996), 'Aime-la-Plagne': (6.612139747257789, 45.56609185), 'Porte-de-Savoie': (6.0154506, 45.5008746), 'Saint-Genix-les-Villages': (5.6590017, 45.6058079), 'Grand-Aigueblanche': (6.497122, 45.4990371), 'Valgelon-La Rochette': (6.1165424, 45.4565137), 'La Léchère': (6.450393967516041, 45.52627615), 'Thorens-Glières': (6.306224482398606, 45.9873594), 'Sillingy': (6.0359723, 45.9471123), 'Sallanches': (6.6302551, 45.9361246), 'Marnaz': (6.5264868, 46.0589925), 'Saint-Martin-Bellevue': (6.139013985944965, 45.973566000000005), 'Metz-Tessy': (6.106940139890374, 45.9404188), 'Meythet': (6.095924491965651, 45.919176750000005), 'Saint-Jorioz': (6.1642972, 45.8333544), 'Valleiry': (5.967928, 46.1079478), 'Fillinges': (6.3424293, 46.1586371), 'Archamps': (6.1332458, 46.1354538), 'Lugrin': (6.6640618, 46.402458), 'Cran-Gevrier': (6.106538, 45.9079102), 'Argonay': (6.1428018, 45.9440884), 'Cluses': (6.5804218, 46.0603171), 'Allinges': (6.4649753, 46.3367823), 'Bonne': (7.10066, 50.735851), 'Vétraz-Monthoux': (6.2578532, 46.1741035), 'Chens-sur-Léman': (6.2674591, 46.3268916), 'Douvaine': (6.3024961, 46.3059806), 'Pringy': (4.5154884, 48.7798117), 'Publier': (6.5436956, 46.3870276), 'Sevrier': (6.1404951, 45.8641144), 'La Balme-de-Sillingy': (6.0409321, 45.9604149), 'Les Houches': (6.799138, 45.891508), 'Doussard': (6.2200226, 45.776103), 'Collonges-sous-Salève': (6.1449533, 46.1409508), 'Sciez': (6.3752502, 46.331359), 'Viry': (4.3343506, 46.472464), 'Bons-en-Chablais': (6.3702817, 46.264918), 'Villaz': (6.1915748, 45.951157), 'Annecy': (6.1288847, 45.8992348), 'La Roche-sur-Foron': (6.3121658, 46.0672189), 'Pers-Jussy': (6.2676644, 46.1066771), 'Thonon-les-Bains': (6.4779448, 46.3731303), 'Annemasse': (6.2341093, 46.1934005), 'Marignier': (6.4992902, 46.0903808), 'Megève': (6.617932, 45.8567089), 'Évian-les-Bains': (6.5897931, 46.400979), 'Épagny': (5.0601263, 47.447525), 'Étrembières': (6.2261757, 46.1786453), 'Ambilly': (6.2224568, 46.1960316), 'Saint-Julien-en-Genevois': (6.0812467, 46.1444789), 'Taninges': (6.5913801, 46.1075116), 'Passy': (4.5343419, 46.5412837), 'Amancy': (6.33, 46.075), 'Bonneville': (2.258570041718889, 50.0776486), 'Morzine': (6.7091057, 46.1799561), 'Magland': (6.6207249, 46.0203509), 'Saint-Pierre-en-Faucigny': (6.3680865, 46.0589885), 'Alby-sur-Chéran': (6.020555, 45.8190536), 'Poisy': (6.0626528, 45.9220363), 'Saint-Gervais-les-Bains': (6.7113338, 45.8928549), 'Annecy-le-Vieux': (6.1420652, 45.9191358), 'Reignier-Ésery': (6.27021838880828, 46.1367059), 'Cranves-Sales': (6.2919875, 46.1876567), 'Cruseilles': (6.1069469, 46.0338359), 'Saint-Cergues': (6.3213839, 46.2357361), 'Saint-Jeoire': (6.4582302, 46.1386318), 'Veigy-Foncenex': (6.2567358, 46.2695471), 'Groisy': (6.171294, 46.0164774), 'Seynod': (6.0992785, 45.8880398), 'Scionzier': (6.5498475, 46.0572538), 'Neuvecelle': (6.6122858, 46.3955991), 'Chavanod': (6.0525071, 45.8842342), 'Faverges': (6.291683001877855, 45.7525572), 'Ville-la-Grand': (6.2495954, 46.2025309), 'Gaillard': (6.211169, 46.1838437), 'Thônes': (6.3241844, 45.8819884), 'Thyez': (6.5398722, 46.0828653), 'Rumilly': (2.01477, 50.57651), 'Viuz-en-Sallaz': (6.4104203, 46.147716), 'Epagny Metz-Tessy': (6.093513616324771, 45.942964200000006), 'Faverges-Seythenex': (6.303820595655985, 45.734328000000005), 'Fillière': (6.274954022443695, 46.0057264), 'Vallières-sur-Fier': (5.9352533, 45.9007169), 'Gainneville': (0.256677, 49.5174), 'Rouen': (1.0939658, 49.4404591), 'Le Havre': (0.1079732, 49.4938975), 'Dieppe': (1.0791444, 49.9246182), 'Eu': (1.4175744, 50.0491699), 'Le Petit-Quevilly': (1.06208, 49.4238), 'Fécamp': (0.374891, 49.7577729), 'Saint-Pierre-lès-Elbeuf': (1.0476368, 49.2745217), 'Épouville': (0.2260209, 49.5635663), 'Sotteville-lès-Rouen': (1.089746, 49.4151549), 'Forges-les-Eaux': (1.5351516211598595, 49.61296845), 'Boos': (1.2007981, 49.3877403), 'Le Tréport': (1.3827656, 50.0591099), 'Franqueville-Saint-Pierre': (1.1825, 49.4027), 'Gournay-en-Bray': (1.7241534, 49.4824794), 'Barentin': (0.9522586, 49.5459699), 'Sainte-Adresse': (0.0805655, 49.5052482), 'Notre-Dame-de-Bondeville': (1.0487027, 49.4883509), 'Harfleur': (0.200141, 49.5075158), 'Cany-Barville': (0.6358291, 49.7868965), 'Offranville': (1.0502415, 49.8703506), 'Le Grand-Quevilly': (1.0507926, 49.4068986), 'Neufchâtel-en-Bray': (1.4421413, 49.7350035), 'Criel-sur-Mer': (1.3139444, 50.0160928), 'Maromme': (1.0308208, 49.476737), 'Buchy': (1.3583539, 49.5843382), 'Bolbec': (0.4742448, 49.5739091), 'Yvetot': (0.7538866, 49.6179384), 'Elbeuf': (1.0108998, 49.2871086), 'Canteleu': (1.0352069, 49.4507987), 'Bois-Guillaume-Bihorel': (1.140749199694191, 49.4759231), 'Pavilly': (0.9534847, 49.5677557), "Gonfreville-l'Orcher": (0.2338372, 49.5039262), 'Oissel': (1.0957098, 49.3424375), 'Octeville-sur-Mer': (0.1161519, 49.5553201), 'Darnétal': (1.1519145, 49.4449083), "Criquetot-l'Esneval": (0.2661419, 49.6449576), 'Déville-lès-Rouen': (1.0495889, 49.4695338), 'Goderville': (0.3661017, 49.6456303), 'Amfreville-la-Mi-Voie': (1.12356, 49.4068), 'Bonsecours': (1.1239654, 49.4222278), 'Bihorel': (1.1159756, 49.4543405), 'Saint-Jacques-sur-Darnétal': (1.2055682, 49.4408138), 'Saint-Léger-du-Bourg-Denis': (1.16414, 49.4288), "Saint-Nicolas-d'Aliermont": (1.2140639, 49.878403), 'Saint-Romain-de-Colbosc': (0.3579164, 49.5313601), 'Yerville': (0.8974236, 49.6688705), 'Saint-Aubin-lès-Elbeuf': (1.0197, 49.3027), 'Saint-Valery-en-Caux': (0.7106831, 49.8597004), 'Grand-Couronne': (1.0043489, 49.3534994), 'Lillebonne': (0.534411, 49.5188282), 'Montivilliers': (0.192418, 49.5456435), 'Houppeville': (1.0813157, 49.5097935), 'Mont-Saint-Aignan': (1.0813, 49.4679), 'Saint-Étienne-du-Rouvray': (1.09163, 49.3831), 'Arques-la-Bataille': (1.126523, 49.8806133), 'Blangy-sur-Bresle': (1.629493, 49.9326076), 'Caudebec-lès-Elbeuf': (1.02611, 49.2814), 'Cléon': (1.0328344, 49.3141297), 'Petit-Couronne': (1.0333, 49.3818), 'Montville': (1.0750949, 49.5479845), 'Quincampoix': (1.17464, 49.521), 'Notre-Dame-de-Gravenchon': (0.5680054, 49.4898201), 'Duclair': (0.8754968, 49.482563), 'Malaunay': (1.0403092, 49.5280751), 'Le Houlme': (1.03741, 49.5096), 'Le Mesnil-Esnard': (1.1391732, 49.4119899), 'Isneauville': (1.1416384, 49.4988394), 'Le Trait': (0.8087513, 49.4819151), 'Gruchet-le-Valasse': (0.4866308, 49.5569416), 'Fontaine-la-Mallet': (0.1460207, 49.5363508), 'Rouelles': (5.085199825850805, 47.80395905), 'Bois-Guillaume': (1.1199476, 49.4704819), 'Petit-Caux': (1.2592728536242959, 49.9613613), 'Port-Jérôme-sur-Seine': (0.6005385938946798, 49.505435), 'Arelaune-en-Seine': (0.7518237306842186, 49.464944599999995), 'Rives-en-Seine': (0.7611546217382126, 49.52662945), 'Terres-de-Caux': (0.6080501864125021, 49.65182935), 'Val-de-Scie': (1.1523195353569395, 49.71483755), 'Neuville-lès-Dieppe': (1.1038797, 49.926907), 'Mouroux': (3.0375419, 48.8213771), 'Trilport': (2.9471072, 48.9585984), 'Lognes': (2.632722, 48.8384157), 'La Grande-Paroisse': (2.9009239, 48.3860553), 'Chauconin-Neufmontiers': (2.8380097, 48.9734656), 'Souppes-sur-Loing': (2.7426674, 48.1868057), 'Nangis': (3.0134672, 48.5549685), 'Émerainville': (2.6212919, 48.8115813), 'Servon': (-1.419401, 48.6006852), 'Saint-Fargeau-Ponthierry': (2.5273471, 48.5361215), 'Héricy': (2.7643819, 48.4492392), 'Bois-le-Roi': (1.34378, 48.86276), 'Chartrettes': (2.7005289, 48.4881157), 'Champagne-sur-Seine': (2.8028617, 48.3948577), 'Bourron-Marlotte': (2.7056355, 48.3408614), 'Chelles': (3.0344372, 49.3549458), 'Serris': (2.7855615, 48.8567), 'Ozoir-la-Ferrière': (2.6719421, 48.7620904), 'Combs-la-Ville': (2.564877, 48.6646994), 'Annet-sur-Marne': (2.7189327, 48.9263555), 'Évry-Grégy-sur-Yerre': (2.6362499, 48.6560723), 'Nemours': (2.6953079, 48.268026), 'Savigny-le-Temple': (2.5798953, 48.5958681), 'Avon': (2.7207225, 48.4048867), 'Coupvray': (2.7970238, 48.8843923), 'Grisy-Suisnes': (2.6654299, 48.6851555), 'Gretz-Armainvilliers': (2.7359603, 48.7410407), 'Jouarre': (3.129898, 48.9265013), 'Crécy-la-Chapelle': (2.9086092, 48.8578447), 'Moissy-Cramayel': (2.5937073, 48.6272298), 'Moret-sur-Loing': (2.813519159377565, 48.3633576), 'Fontenay-Trésigny': (2.8722456, 48.7100369), 'Château-Landon': (2.7006781, 48.1501591), 'Lésigny': (2.6146976, 48.7441196), 'Thorigny-sur-Marne': (2.7115737, 48.8841856), 'Vert-Saint-Denis': (2.6252499, 48.5671024), 'Thomery': (2.784838, 48.4090286), 'Saint-Pathus': (2.79415, 49.0745), 'Moussy-le-Neuf': (2.6014953, 49.0647805), 'Noisiel': (2.6200032, 48.8457347), 'Rubelles': (2.682406, 48.5590252), 'Esbly': (2.8130733, 48.9009645), 'La Ferté-sous-Jouarre': (3.1294221, 48.9500674), 'Pomponne': (2.6913606, 48.8816121), 'Guérard': (2.9580234, 48.8239829), 'Mitry-Mory': (2.6183971, 48.9839674), 'Roissy-en-Brie': (2.6545195, 48.7903787), 'Le Mée-sur-Seine': (2.6335912, 48.5381859), 'Chessy': (2.7658363, 48.8798792), 'Varennes-sur-Seine': (2.9301414, 48.3766097), 'Lagny-sur-Marne': (2.7056056, 48.8773018), 'Melun': (2.6608169, 48.539927), 'Fontainebleau': (2.7015872, 48.4049375), 'Provins': (3.2988089, 48.5602959), 'Meaux': (2.8773541, 48.9582708), 'Cesson': (2.6021145, 48.5624381), 'Montereau-Fault-Yonne': (2.9557006, 48.3849647), 'Champs-sur-Marne': (2.6027219, 48.8526892), 'Vaux-le-Pénil': (2.6831642, 48.5255542), 'Donnemarie-Dontilly': (3.1288116, 48.4793547), 'Saint-Mard': (3.5828, 49.3863), 'Saint-Soupplets': (2.8062379, 49.038354), 'Guignes': (2.7954418, 48.6354878), 'Dammarie-les-Lys': (2.6347018, 48.5150876), 'Vernou-la-Celle-sur-Seine': (2.8470169, 48.3889172), 'Brie-Comte-Robert': (2.6166737, 48.6904522), 'Bailly-Romainvilliers': (2.8212923, 48.8468159), 'Dammartin-en-Goële': (2.6835682, 49.0547758), 'Brou-sur-Chantereine': (2.6289001, 48.8856842), 'Coulommiers': (3.0843818, 48.8121301), 'Pommeuse': (3.0153334, 48.8163232), 'Tournan-en-Brie': (2.7645642, 48.7403305), 'Bussy-Saint-Georges': (2.6973858, 48.8430257), 'Lizy-sur-Ourcq': (3.0218144, 49.0273072), 'Nandy': (2.564017, 48.5813635), 'Claye-Souilly': (2.6883703, 48.9460871), 'Le Châtelet-en-Brie': (2.792873, 48.5044282), 'Collégien': (2.673176, 48.8362114), 'Boissise-le-Roi': (2.5737655, 48.5280356), 'Boissy-le-Châtel': (3.1363451, 48.8208816), 'Crégy-lès-Meaux': (2.87437, 48.9779), 'Lieusaint': (-1.4782994, 49.4753601), "Verneuil-l'Étang": (2.8358459, 48.6451703), 'Ferrières-en-Brie': (2.7045126, 48.8213887), 'Veneux-les-Sablons': (2.79523361364013, 48.38106345), 'Faremoutiers': (2.9982641, 48.8004306), 'Saint-Pierre-lès-Nemours': (2.6870183, 48.2637127), 'Montévrain': (2.748344, 48.8741286), 'Villeparisis': (2.6176193, 48.939344), 'Chanteloup-en-Brie': (2.7395512, 48.8555777), 'La Ferté-Gaucher': (3.305711, 48.7812807), 'Magny-le-Hongre': (2.8135257, 48.8631156), 'Vulaines-sur-Seine': (2.7637818, 48.430803), 'Courtry': (2.60248, 48.9158), 'Dampmart': (2.7355279, 48.8852184), 'Chevry-Cossigny': (2.6617094, 48.7244459), 'Vaires-sur-Marne': (2.6398292, 48.8732841), 'Pontault-Combault': (2.6068064, 48.8009132), 'Saint-Germain-sur-Morin': (2.8514287, 48.8829414), 'Chaumes-en-Brie': (2.842398, 48.6647753), 'Villenoy': (2.8596553, 48.9412182), 'Rozay-en-Brie': (2.96346, 48.6848), 'Saint-Mammès': (2.807564, 48.3887016), 'Quincy-Voisins': (2.87886, 48.8997), 'Saint-Thibault-des-Vignes': (2.6877273, 48.8683837), 'Saint-Germain-Laval': (2.9903343, 48.4007243), 'Othis': (2.6705562, 49.0779972), 'Nanteuil-lès-Meaux': (2.89893, 48.9306), 'Mormant': (2.8904431, 48.6066366), 'Montry': (2.8282263, 48.8875007), 'Montigny-sur-Loing': (2.757804, 48.3405032), 'Mareuil-lès-Meaux': (2.8634773, 48.9267289), 'Orvanne': (2.8159449, 48.3730289), 'Moret Loing et Orvanne': (2.8159449, 48.3730289), 'Moret-Loing-et-Orvanne': (2.8159449, 48.3730289), 'Le Chesnay-Rocquencourt': (2.1218066, 48.8298674), 'Saint-Germain-en-Laye': (2.0917742569617808, 48.93560465), "L'Étang-la-Ville": (2.0726292, 48.8707694), 'Maule': (-71.48868, -35.5972284), 'Le Mesnil-le-Roi': (2.128842, 48.9396822), 'Mareil-Marly': (2.0764516, 48.8823232), 'Mantes-la-Ville': (1.7109449, 48.9748938), 'Juziers': (1.8456775, 48.9930057), 'Louveciennes': (2.1130641, 48.8616969), 'Issou': (1.7924733, 48.988093), 'Magnanville': (1.6847431, 48.9681039), 'Saint-Rémy-lès-Chevreuse': (2.071109, 48.7054888), 'Neauphle-le-Château': (1.9038201, 48.8111045), 'Montesson': (2.1492681, 48.9076421), 'Orgeval': (3.70325, 49.508), 'Noisy-le-Roi': (2.0645422, 48.8439572), 'Rocquencourt': (2.4172616, 49.6502698), 'Andrésy': (2.0610573, 48.9821445), 'Le Perray-en-Yvelines': (1.8538535, 48.6924665), 'Rosny-sur-Seine': (1.6280317, 49.001733), 'Morainvilliers': (1.9416886, 48.9274749), 'Le Port-Marly': (2.109145, 48.8809876), 'Mézières-sur-Seine': (1.7961756, 48.9608736), 'Poissy': (2.0428293, 48.927439), 'Versailles': (2.1266886, 48.8035403), 'Magny-les-Hameaux': (2.0839562, 48.724051), "Bois-d'Arcy": (3.7165618, 47.5475818), 'Beynes': (1.8735151, 48.8558793), 'Viroflay': (2.1730571, 48.7995951), 'Bailly': (2.9706092, 49.4962015), 'Voisins-le-Bretonneux': (2.0510182, 48.7586644), 'Villiers-Saint-Frédéric': (1.8914159, 48.8196898), 'La Verrière': (1.9554697, 48.7555258), 'Le Mesnil-Saint-Denis': (1.9606249, 48.7399787), 'Maurepas': (2.8478956, 49.9905624), 'Les Essarts-le-Roi': (1.8946206, 48.717601), "Saint-Cyr-l'École": (2.0672138, 48.799708), 'Rambouillet': (1.819207, 48.6452851), 'Mantes-la-Jolie': (1.7140683, 48.9891971), 'Sartrouville': (2.1585037, 48.9409016), 'Conflans-Sainte-Honorine': (2.0949768, 48.9938791), 'Le Chesnay': (2.1257808, 48.8264842), 'Guyancourt': (2.0706084, 48.7709414), 'Élancourt': (1.949352, 48.7681182), 'Chatou': (2.1573695, 48.8897044), 'Montigny-le-Bretonneux': (2.0381228, 48.7698937), 'Fontenay-le-Fleury': (2.0461549, 48.8106578), 'Triel-sur-Seine': (2.0030818, 48.980925), 'Saint-Nom-la-Bretèche': (2.0205241, 48.8641009), 'Épône': (1.8126696, 48.9573716), 'Porcheville': (1.7766633, 48.9717033), 'Achères': (2.4491821, 47.2774561), "Montfort-l'Amaury": (1.8070765, 48.779142), 'Jouars-Pontchartrain': (1.9000212, 48.8032681), 'Buc': (2.3402667, 43.0376272), 'Vaux-sur-Seine': (1.9636465, 49.0061998), 'Maurecourt': (2.0625234, 48.9975159), 'Buchelay': (1.6716534, 48.9793554), 'Feucherolles': (1.9723953, 48.8725969), 'Houilles': (2.1868998, 48.9229416), 'Ecquevilly': (1.9226814, 48.9510622), 'Le Pecq': (2.1026594, 48.8976716), 'Meulan-en-Yvelines': (1.9100835, 49.0049712), 'Chambourcy': (2.0396454, 48.9058999), 'Les Mureaux': (1.909928, 48.9931578), 'Ablis': (1.8368758, 48.517153), 'Trappes': (1.9988356, 48.7760957), 'La Celle-Saint-Cloud': (2.136145, 48.8478753), 'Plaisir': (1.9476363, 48.817399), 'Jouy-en-Josas': (2.1684529, 48.76464), 'Vélizy-Villacoublay': (2.1900052, 48.7809496), 'Houdan': (1.6019247, 48.7912001), 'Bougival': (2.1377696, 48.8642975), 'Chevreuse': (2.0408005, 48.7079689), 'Chanteloup-les-Vignes': (2.0302362, 48.9784414), 'Bonnières-sur-Seine': (1.5809775, 49.037481), 'Carrières-sous-Poissy': (2.0341747, 48.9437694), 'Carrières-sur-Seine': (2.1779083, 48.9134112), 'Croissy-sur-Seine': (2.1327881, 48.8791554), 'Villepreux': (2.007049, 48.8323013), 'Villennes-sur-Seine': (1.9987648, 48.9404491), 'Les Clayes-sous-Bois': (1.9871825, 48.8214478), 'Coignières': (1.9201632, 48.7511787), 'Limay': (1.7353686, 48.9921015), 'Fourqueux': (2.056790092971905, 48.88620905), 'Freneuse': (1.0785477, 49.3122251), 'Gargenville': (1.8104637, 48.9901469), 'Le Vésinet': (2.1347291, 48.894254), 'Verneuil-sur-Seine': (1.9756367, 48.9792052), 'Marly-le-Roi': (2.0965935, 48.8681762), 'Saint-Arnoult-en-Yvelines': (1.9350788, 48.572604), 'Aubergenville': (1.8554319, 48.9591392), 'Niort': (-0.4649403, 46.3241132), 'Bressuire': (-0.4929017, 46.8425572), 'Parthenay': (-0.2472687, 46.6481266), 'Échiré': (-0.4154901, 46.3919107), 'Thouars': (-0.2057893552874009, 46.983202750000004), 'Airvault': (-0.1368880662471918, 46.821993750000004), 'Melle': (7.32049, 44.5620476), "Saint-Maixent-l'École": (-0.2019344, 46.4096855), 'Mauléon': (0.5674923, 42.9594856), 'Moncoutant': (-0.5865093211079355, 46.7314449), 'Nueil-les-Aubiers': (-0.5920969, 46.9435791), 'Aiffres': (-0.4175237, 46.2866383), 'Argentonnay': (-0.41706425504327926, 46.9720249), 'Mougon-Thorigné': (-0.26893290795341784, 46.2887193), 'Aigondigné': (-0.2686333428079816, 46.2846033), "Loretz-d'Argenton": (-0.2570333700604056, 47.05786705), 'Moncoutant-sur-Sèvre': (-0.5748935211569981, 46.709708449999994), 'Terves': (-0.5089931, 46.8046169), 'Chauray': (-0.3745955, 46.3606879), 'Celles-sur-Belle': (-0.21020600942692336, 46.257713949999996), 'La Crèche': (-0.2970798, 46.3624398), 'Cerizay': (-0.6672118, 46.8207452), 'Châtillon-sur-Thouet': (-0.2362671, 46.6598371), 'Vouillé': (-0.3677431, 46.314256), 'Frontenay-Rohan-Rohan': (-0.5404118, 46.2544923), 'Magné': (-0.5431161, 46.3148541), 'Mauzé-sur-le-Mignon': (-0.6709409, 46.1957534), 'Doullens': (2.3410532, 50.1572085), 'Corbie': (2.5110728, 49.9082911), 'Montdidier': (2.570196, 49.6460204), 'Rosières-en-Santerre': (2.7020278, 49.8130117), 'Moreuil': (2.486822, 49.7705466), 'Abbeville': (1.8337029, 50.1060835), 'Ailly-sur-Somme': (2.1980997, 49.928085), 'Ailly-sur-Noye': (2.3645669, 49.7580993), 'Albert': (2.6509223, 50.0018022), 'Amiens': (2.2956951, 49.8941708), 'Saleux': (2.2389702, 49.8605488), 'Ham': (3.0733031, 49.746322), 'Rue': (1.6673051, 50.272874), 'Friville-Escarbotin': (1.5452219, 50.0869946), 'Flixecourt': (2.082582, 50.0117096), 'Péronne': (2.9321899, 49.9289578), 'Longueau': (2.3574279, 49.8702523), 'Mers-les-Bains': (1.3889703, 50.0656325), 'Rivery': (2.3216925, 49.9005828), 'Boves': (2.3932912, 49.8463524), 'Villers-Bretonneux': (2.518383, 49.8680878), 'Camon': (2.3446546, 49.8874246), 'Roye': (2.7930755, 49.6986186), 'Salouël': (2.24173, 49.8709), 'Puygouzon': (2.169829029490022, 43.88783805), 'Saint-Juéry': (2.2096659, 43.9490655), "Lescure-d'Albigeois": (2.1691356, 43.9538382), 'Rabastens': (1.7240388, 43.8212439), 'Soual': (2.1148775, 43.5545042), 'Coufouleux': (1.7306871, 43.8175853), 'Aiguefonde': (2.3168355, 43.4937763), 'Réalmont': (2.1889958, 43.7760912), 'Albi': (2.147899, 43.9277552), 'Castres': (3.2383333, 49.8044444), 'Gaillac': (1.8967943, 43.9016766), 'Blaye-les-Mines': (2.1323326, 44.0301194), 'Cagnac-les-Mines': (2.1383353, 43.9875724), 'Lavaur': (1.0230515, 44.6129697), 'Arthès': (2.2079468, 43.9540167), 'Sorèze': (2.0678397, 43.4539616), 'Mazamet': (2.3764363, 43.4902317), 'Carmaux': (2.1583033, 44.0496296), 'Puylaurens': (2.0134036, 43.5726147), 'Aussillon': (2.3553618, 43.4879811), 'Lisle-sur-Tarn': (1.8100746, 43.8526551), 'Pont-de-Larn': (2.4058399107415935, 43.5302274), 'Saint-Sulpice-la-Pointe': (1.686477, 43.7737894), 'Labruguière': (2.2634222, 43.5402951), 'Graulhet': (1.99723, 43.7578), 'Saïx': (2.1832358, 43.581767), 'Marssac-sur-Tarn': (2.0293467, 43.9176823), 'La Ville-Dieu-du-Temple': (1.2187309, 44.036304), 'Labastide-Saint-Pierre': (1.3672004, 43.918889), 'Verdun-sur-Garonne': (1.2363889, 43.8538889), 'Saint-Étienne-de-Tulmont': (1.4627778, 44.0488889), 'Montauban': (1.3549991, 44.0175835), 'Castelsarrasin': (1.1072875, 44.0395151), 'Moissac': (1.0846682, 44.1050886), 'Lafrançaise': (1.2413889, 44.1286111), 'Grisolles': (3.35857, 49.1403), 'Nègrepelisse': (1.5225, 44.0761111), 'Montbeton': (1.2858333, 44.0119444), 'Beaumont-de-Lomagne': (0.9875923, 43.8817933), 'Caussade': (1.5365375, 44.1608142), 'Montech': (1.2305556, 43.9577778), 'Albias': (1.4502778, 44.0908333), 'Bressols': (1.3391102, 43.9682789), 'Rocbaron': (6.0910097, 43.3040144), 'Solliès-Pont': (6.0420201, 43.190571), 'Seillans': (6.6445779, 43.6364106), 'Le Luc': (6.3128995, 43.3946278), 'Callian': (0.2765967, 43.6288155), 'La Farlède': (6.04317, 43.1597), 'Rians': (2.6131277, 47.1855657), 'Le Muy': (6.566111, 43.4713932), 'Le Castellet': (5.9789367, 43.9383814), 'Trans-en-Provence': (6.4867321, 43.5039337), 'Pignans': (6.229509, 43.3023198), 'Néoules': (6.0134378, 43.3109287), 'Montauroux': (6.765164, 43.617944), 'Saint-Zacharie': (5.707967, 43.3839412), 'Tourves': (5.9243299, 43.407701), 'Le Revest-les-Eaux': (5.9253838, 43.1768514), 'Bras': (5.9534995, 43.4720265), 'Carqueiranne': (6.0766933, 43.0910087), 'Pierrefeu-du-Var': (6.14307, 43.2272615), 'Le Val': (6.0733753, 43.4390685), "La Cadière-d'Azur": (5.7549753, 43.1960022), 'Pourrières': (5.734276, 43.5035923), 'Ollioules': (5.8483538, 43.1397205), 'Salernes': (6.2325706, 43.5640073), 'Flassans-sur-Issole': (6.2138119, 43.367837), 'Le Thoronet': (6.3023262, 43.4522605), 'Saint-Maximin-la-Sainte-Baume': (5.862294, 43.4522053), 'Flayosc': (6.3980138, 43.5340489), 'Saint-Mandrier-sur-Mer': (5.92658, 43.0761), 'Figanières': (6.4975914, 43.5704687), 'Saint-Tropez': (6.6405225, 43.2727191), 'Toulon': (5.9304919, 43.1257311), 'La Croix-Valmer': (6.5676698, 43.208027), 'Vidauban': (6.4314327, 43.4273183), 'Hyères': (6.1301614, 43.1202573), 'Draguignan': (6.4627333, 43.5374662), 'Tourrettes': (6.7015297, 43.6236134), 'Brignoles': (6.0616452, 43.4064401), 'La Seyne-sur-Mer': (5.8788948, 43.1007714), 'Fréjus': (6.7360182, 43.4330308), 'Vinon-sur-Verdon': (5.8127808, 43.7245231), 'Saint-Raphaël': (1.0747165, 45.3043079), 'La Garde': (6.0449399, 45.0691999), 'La Londe-les-Maures': (6.2355346, 43.1381313), 'La Valette-du-Var': (5.9834654, 43.1378245), 'Six-Fours-les-Plages': (5.8393953, 43.0935105), 'La Motte': (-2.733408, 48.2349806), 'Garéoult': (6.0457357, 43.3293378), 'Bandol': (5.7523647, 43.1357717), 'Roquebrune-sur-Argens': (6.6363623, 43.4433565), 'Cavalaire-sur-Mer': (6.531917, 43.1722952), 'Cuers': (6.072772, 43.2372099), 'Nans-les-Pins': (5.7840241, 43.3732163), 'Gonfaron': (6.2897851, 43.3199051), 'Le Pradet': (6.0223774, 43.1037608), 'Les Arcs': (6.4786269, 43.4629097), 'La Crau': (6.0737053, 43.1497034), 'Cogolin': (6.53659, 43.25), 'Barjols': (6.0056803, 43.558133), 'Saint-Cyr-sur-Mer': (5.711516, 43.1810087), 'Sanary-sur-Mer': (5.8008839, 43.1177238), 'Puget-sur-Argens': (6.6844356, 43.4561681), 'Carcès': (6.1826133, 43.4760038), 'Fayence': (6.6949071, 43.6238666), "Les Adrets-de-l'Estérel": (6.814462, 43.5258374), 'Besse-sur-Issole': (6.1770266, 43.3494306), 'Le Cannet-des-Maures': (6.341972, 43.3902105), 'Bagnols-en-Forêt': (6.6981788, 43.5376002), 'Le Beausset': (5.8007195, 43.1988273), 'Sainte-Maxime': (6.637714, 43.30871), 'Carnoules': (6.1898612, 43.3024863), 'Forcalqueiret': (6.0824736, 43.3377105), 'Lorgues': (6.3600329, 43.4929987), 'Bormes-les-Mimosas': (6.3419285, 43.1506968), 'Régusse': (6.1327063, 43.6550445), 'Le Lavandou': (6.3701286, 43.1383795), 'Grimaud': (6.5222979, 43.2737698), 'Gassin': (6.585476, 43.2291533), 'Signes': (5.8636703, 43.2908597), 'Solliès-Ville': (6.0378603, 43.182302), 'Solliès-Toucas': (6.0251908, 43.2058411), 'Le Plan-de-la-Tour': (6.54537, 43.3384), 'Puget-Ville': (6.138408, 43.2848554), "La Tour-d'Aigues": (5.5477306, 43.724804), 'Lauris': (5.3126795, 43.7466047), 'Jonquières': (2.730011, 49.3899096), 'Aubignan': (5.0242503, 44.0991698), 'Entraigues-sur-la-Sorgue': (4.9272616, 44.0023187), 'Velleron': (5.0292193, 43.9562757), 'Caromb': (5.1078494, 44.1111263), 'Le Thor': (4.9945784, 43.928823), 'Loriol-du-Comtat': (5.0003202, 44.076276), 'Saint-Saturnin-lès-Apt': (5.3847356, 43.9441346), 'Gargas': (1.4679519, 43.755877), 'Sainte-Cécile-les-Vignes': (4.8868861, 44.2464525), 'Lapalud': (4.6865913, 44.3061604), 'Sérignan-du-Comtat': (4.8445684, 44.1893038), 'Avignon': (4.8059012, 43.9492493), 'Orange': (-117.8704931, 33.7500378), 'Carpentras': (5.0487681, 44.0554043), 'Apt': (5.3970648, 43.8757936), 'Cavaillon': (5.0383522, 43.8356808), 'Le Pontet': (6.2282666, 45.4934683), 'Cadenet': (5.3737893, 43.7346782), 'Althen-des-Paluds': (4.95753, 44.0051), 'Bollène': (4.7484092, 44.2803882), 'Bédarrides': (4.8982969, 44.0407425), 'Châteauneuf-de-Gadagne': (4.9442687, 43.927603), 'Courthézon': (4.8835125, 44.0867088), 'Bédoin': (5.1806559, 44.1240383), 'Caumont-sur-Durance': (4.9457232, 43.893915), 'Camaret-sur-Aigues': (4.8736859, 44.1650029), 'Mondragon': (4.7135032, 44.2385626), 'Mazan': (5.1281078, 44.0565965), 'Pertuis': (5.5032671, 43.6951468), 'Caderousse': (4.7571511, 44.1025641), 'Cheval-Blanc': (5.0635597, 43.800822), 'Robion': (5.112215, 43.8441157), 'Sarrians': (4.9717219, 44.0831417), 'Morières-lès-Avignon': (4.9042769, 43.940965), 'Monteux': (4.9968733, 44.0349281), 'Saint-Saturnin-lès-Avignon': (4.9294712, 43.9567795), 'Pernes-les-Fontaines': (5.059255, 43.9984627), 'Valréas': (4.9906089, 44.3851455), "L'Isle-sur-la-Sorgue": (5.0515385, 43.9185354), 'Vaison-la-Romaine': (5.075737, 44.2407641), 'Vedène': (4.9035659, 43.9787328), 'Piolenc': (4.7628292, 44.178621), 'Sorgues': (4.8725016, 44.0066403), 'Malaucène': (5.1321258, 44.1732744), 'Villelaure': (5.433956, 43.7110499), "L'Île-d'Olonne": (-1.782259, 46.5614321), 'Nieul-le-Dolent': (-1.5077617, 46.5745648), 'Pouzauges': (-0.8368869, 46.7829569), 'Talmont-Saint-Hilaire': (-1.618977, 46.4666884), 'Saint-Gervais': (5.4820229, 45.2022356), 'Sainte-Hermine': (-1.0606752, 46.5552047), 'Beauvoir-sur-Mer': (-2.0413971, 46.9169461), 'Bretignolles-sur-Mer': (-1.8538889, 46.6286111), 'Le Boupère': (-0.9252778, 46.7955556), "Château-d'Olonne": (-1.7361111, 46.505), 'Chantonnay': (-1.0505509, 46.6868121), 'Bournezeau': (-1.1706004, 46.6359566), 'Dompierre-sur-Yon': (-1.3904471, 46.7369838), 'Aubigny': (2.4833605, 49.8999821), 'La Chaize-le-Vicomte': (-1.2968122, 46.6722073), "L'Herbergement": (-1.3758333, 46.91), 'Les Clouzeaux': (-1.5093051, 46.6304999), 'Coëx': (-1.7594444, 46.6986111), 'Commequiers': (-1.8397386, 46.7612887), 'Angles': (-1.4027911, 46.40716), 'Les Brouzils': (-1.3213179, 46.8869394), 'Essarts en Bocage': (-1.2128522788918927, 46.7860407), 'Bellevigny': (-1.4039332745583355, 46.78746285), 'Aubigny-Les Clouzeaux': (-1.4842768695956505, 46.61229555), 'Montréverd': (-1.4424379079711862, 46.91043725), 'Sèvremont': (-0.8562486147797005, 46.840470100000005), "Rives de l'Yon": (-1.3368104355740524, 46.589736349999995), 'Les Achards': (-1.6603019907165417, 46.59235305), 'Montaigu-Vendée': (-1.292745559745998, 46.974115), "Les Sables-d'Olonne": (-1.7924575344192666, 46.5002031), 'Chanverrie': (-0.9868360771394622, 46.94669245), "L'Île-d'Yeu": (-2.34917690050466, 46.71018105), 'Challans': (-1.8773787, 46.847943), 'Luçon': (-1.1645595, 46.4555953), 'La Roche-sur-Yon': (-1.4269698, 46.6705431), 'Fontenay-le-Comte': (-0.8064003, 46.4660771), 'Brem-sur-Mer': (-1.8297222, 46.6055556), 'La Guyonnière': (-1.2486111, 46.9663889), 'Les Epesses': (-0.8996996, 46.8834978), 'La Bruffière': (-1.1963889, 47.0138889), 'Aizenay': (-1.6084351, 46.7391797), 'La Gaubretière': (-1.0647926, 46.9427746), 'Belleville-sur-Vie': (-1.4300397, 46.7842036), 'Les Essarts': (0.9921207721113022, 48.880996999999994), 'La Garnache': (-1.8297437, 46.891487), 'Sallertaine': (-1.9557544, 46.8609485), 'La Tranche-sur-Mer': (-1.4380541, 46.3440097), 'Jard-sur-Mer': (-1.5758538, 46.4173018), 'Les Herbiers': (-1.0150302, 46.8714322), 'Boufféré': (-1.3391297, 46.9624742), 'Cugand': (-1.2550427, 47.0636617), 'Venansault': (-1.5113138, 46.6858331), 'Rocheservière': (-1.50975, 46.9385391), 'Olonne-sur-Mer': (-1.7687019, 46.5358685), 'Le Poiré-sur-Vie': (-1.5094862, 46.7682395), "Noirmoutier-en-l'Île": (-2.2441449, 47.000089), 'Chavagnes-en-Paillers': (-1.2507462, 46.8931258), 'Benet': (-0.5961111, 46.3683333), 'La Châtaigneraie': (-0.7391667, 46.65), 'La Flocellière': (-0.8623841, 46.830193), 'Saint-Christophe-du-Ligneron': (-1.7667509, 46.8254111), 'Saint-Gilles-Croix-de-Vie': (-1.9254545, 46.692985), 'Mouchamps': (-1.0635913, 46.7786856), 'Montaigu': (3.83264, 49.535), 'La Mothe-Achard': (-1.6591947, 46.6180017), 'Mareuil-sur-Lay-Dissais': (-1.2200228, 46.5350994), 'La Ferrière': (6.085802096183492, 45.280845), 'Saint-Jean-de-Monts': (-2.0588889, 46.7936111), 'La Verrie': (-0.9938889, 46.9619444), "L'Aiguillon-la-Presqu'île": (-1.309758641451055, 46.31680605), 'Le Fenouiller': (-1.8991667, 46.7202778), 'Saint-Laurent-sur-Sèvre': (-0.8888005, 46.9597454), 'Mouilleron-le-Captif': (-1.4590747, 46.7203336), 'Saint-Hilaire-de-Loulay': (-1.3297222, 47.0036111), 'Saint-Florent-des-Bois': (-1.3147222, 46.5944444), 'Saint-Georges-de-Montaigu': (-1.2966952, 46.9477099), 'Nesmy': (-1.4025, 46.5922222), 'Mortagne-sur-Sèvre': (-0.9525, 46.9925), 'Soullans': (-1.8986111, 46.7966667), 'Saint-Fulgent': (-1.1766667, 46.8538889), 'Treize-Septiers': (-1.2303654, 46.9850566), 'Les Lucs-sur-Boulogne': (-1.4948903, 46.8440826), 'Saint-Philbert-de-Bouaine': (-1.5194263, 46.9860317), 'Saint-Hilaire-de-Riez': (-1.9441667, 46.7219444), 'Montamisé': (0.421787, 46.6208411), 'Jaunay-Marigny': (0.3768296, 46.7455607), 'Beaumont Saint-Cyr': (0.4482795740759365, 46.7253903), 'Saint-Martin-la-Pallu': (0.32615331523852775, 46.72958425), 'Migné-Auxances': (0.3113812, 46.6267464), 'Smarves': (0.35802, 46.5121), 'Dangé-Saint-Romain': (0.6076714, 46.9360856), 'Dissay': (0.4280238, 46.6985186), 'Buxerolles': (4.9321318, 47.8072225), 'Cissé': (0.2279887, 46.6462525), 'Poitiers': (0.340196, 46.5802596), 'Châtellerault': (0.545812, 46.8180047), 'Montmorillon': (0.8682027, 46.4249573), 'Vouneuil-sous-Biard': (0.2729321, 46.573287), 'Lusignan': (0.1223765, 46.4351549), 'Nouaillé-Maupertuis': (0.4131096, 46.5085993), 'Chauvigny': (0.6439464, 46.5695048), 'Iteuil': (0.3113773, 46.4887916), 'Neuville-de-Poitou': (0.2450626, 46.6852065), 'Vendeuvre-du-Poitou': (0.3259791842556715, 46.72136895), 'Civray': (0.2950117, 46.1508396), 'Naintré': (0.4823734, 46.7638945), 'Loudun': (0.081826, 47.0102263), 'Chasseneuil-du-Poitou': (0.3748389, 46.6505634), 'Thuré': (0.4573359, 46.8318215), 'Mignaloux-Beauvoir': (0.4148949, 46.5443255), 'Jaunay-Clan': (0.36249102317505283, 46.6945306), "Nieuil-l'Espoir": (0.4562577, 46.4847979), 'Fontaine-le-Comte': (0.2663806, 46.5341041), 'Saint-Benoît': (2.0639828, 43.0169012), 'Ligugé': (0.3325911, 46.5201627), 'Saint-Georges-lès-Baillargeaux': (0.401259, 46.6687442), "Saint-Julien-l'Ars": (0.506826, 46.5583041), 'Valdivienne': (0.6364036, 46.5094492), 'Vivonne': (0.2584318, 46.4262128), 'Boivre-la-Vallée': (0.09027120640148145, 46.53914), 'Valence-en-Poitou': (0.21558079577487071, 46.316931), 'Saint Martin la Pallu': (0.32615331523852775, 46.72958425), 'Rilhac-Rancon': (1.3159469, 45.8979417), 'Limoges': (1.2644847, 45.8354243), 'Bessines-sur-Gartempe': (1.3685926, 46.1090993), 'Nexon': (1.1867578, 45.6781476), 'Verneuil-sur-Vienne': (1.1273441, 45.8492352), 'Condat-sur-Vienne': (1.231575, 45.7937549), 'Saint-Léonard-de-Noblat': (1.4924586, 45.8377349), 'Bellac': (1.0502545, 46.1209036), 'Aixe-sur-Vienne': (1.1362125, 45.7967581), 'Le Palais-sur-Vienne': (1.3241667, 45.865), 'Ambazac': (1.4019278, 45.958226), 'Rochechouart': (0.8202782, 45.8218406), 'Saint-Junien': (0.9011111, 45.8872222), 'Saint-Yrieix-la-Perche': (1.2033138, 45.5127494), 'Feytiat': (1.331823, 45.8097115), 'Isle': (1.2238259, 45.8055591), 'Boisseuil': (1.3307721, 45.7699722), "Bosmie-l'Aiguille": (1.2027499, 45.7647001), 'Couzeix': (1.2379652, 45.8789214), 'Saint-Priest-Taurion': (1.397452, 45.8863817), 'Panazol': (1.3059422, 45.8369971), 'Saint-Just-le-Martel': (1.3858298, 45.862378), 'Contrexéville': (5.8946281, 48.180496), 'Mirecourt': (6.1312974, 48.3005302), "Raon-l'Étape": (6.8437227, 48.406984), 'Saint-Dié-des-Vosges': (6.9477708, 48.2873153), 'Épinal': (6.4503643, 48.1747684), 'Saint-Nabord': (6.5780769, 48.0474263), 'Rupt-sur-Moselle': (6.6597589, 47.9269076), "Le Val-d'Ajol": (6.4836408, 47.9256946), 'Golbey': (6.4364631, 48.1955731), 'Le Thillot': (6.7633816, 47.8804149), 'Saulxures-sur-Moselotte': (6.7697167, 47.9499536), 'Bruyères': (6.7204365, 48.2115397), 'Saint-Étienne-lès-Remiremont': (6.6110083, 48.0203315), 'Charmes': (3.3786111, 49.6516667), 'La Bresse': (6.8766915, 48.0066282), 'Cornimont': (6.8397595, 47.9639269), 'Thaon-les-Vosges': (6.412361233645147, 48.249484499999994), 'Chantraine': (6.4358835, 48.1680353), 'Vagney': (6.7172863, 48.0091314), 'Fraize': (7.0022069, 48.1863539), 'Neufchâteau': (5.6948571, 48.3553014), 'Moyenmoutier': (6.9128471, 48.3794555), 'Étival-Clairefontaine': (6.8652182, 48.3640674), 'Anould': (6.9469287, 48.1851024), 'Éloyes': (6.6112215, 48.0926885), 'Vittel': (5.9418851, 48.2020717), 'Gérardmer': (6.8776965, 48.0705772), 'Rambervillers': (6.6361176, 48.3466306), 'Xertigny': (6.4008806, 48.0453308), 'Remiremont': (6.5912615, 48.0158951), 'Granges-Aumontzey': (6.804521906966393, 48.1342375), 'Monéteau': (3.5794275, 47.8483265), 'Migennes': (3.5225691, 47.9647312), 'Joigny': (3.3995767, 47.9812486), 'Tonnerre': (3.9733288, 47.8565193), 'Brienon-sur-Armançon': (3.6159046, 47.9933948), 'Pont-sur-Yonne': (3.2045813, 48.2852895), 'Villeneuve-la-Guyard': (3.0665404, 48.3398099), 'Toucy': (3.2939186, 47.7338629), 'Saint-Florentin': (3.7280106, 48.00172), 'Villeneuve-sur-Yonne': (3.2942751, 48.0820102), 'Saint-Georges-sur-Baulche': (3.5312159, 47.8048527), 'Paron': (3.2470551, 48.1785718), 'Saint-Clément': (4.0716276, 49.7765278), 'Auxerre': (3.570579, 47.7961287), 'Appoigny': (3.523723, 47.8781778), 'Avallon': (3.9070658, 47.4885365), 'Sens': (3.282606, 48.1978559), 'Charny Orée de Puisaye': (3.1059231592187717, 47.8785383), 'Montholon': (3.2796271280274683, 47.9014175), 'Valdoie': (6.8413823, 47.6651582), 'Belfort': (6.8628942, 47.6379599), 'Beaucourt': (6.9256448, 47.4851387), 'Châtenois-les-Forges': (6.8497211, 47.5581966), 'Delle': (6.9977521, 47.5075682), 'Grandvillars': (6.976952, 47.5379779), 'Bavilliers': (6.8347586, 47.6220703), 'Giromagny': (6.8251787, 47.7418553), 'Essert': (3.7753011099922347, 47.6424655), 'Bièvres': (3.7119444, 49.4941667), 'Évry': (3.25604, 48.26406), 'Itteville': (2.3418427, 48.5145073), 'Palaiseau': (2.2453842, 48.7144587), 'Étampes': (2.1614464, 48.4344621), 'Corbeil-Essonnes': (2.4818087, 48.6137734), 'Gif-sur-Yvette': (2.134529, 48.7018823), 'Brunoy': (2.5044613, 48.6979312), 'Les Ulis': (2.1701573, 48.681708), 'Longjumeau': (2.2953297, 48.6943793), 'Sainte-Geneviève-des-Bois': (2.3259129, 48.6407872), 'Grigny': (2.065831, 50.3856239), 'Draveil': (2.4083576, 48.6851514), 'Montgeron': (2.4605107, 48.7038873), 'Savigny-sur-Orge': (2.3458417, 48.6794572), 'Athis-Mons': (2.3890941, 48.7079028), 'Forges-les-Bains': (2.0996273, 48.6290401), 'Chilly-Mazarin': (2.3123582, 48.7026161), 'Ollainville': (5.8062324, 48.2677463), 'Longpont-sur-Orge': (2.2919546, 48.64262), 'Briis-sous-Forges': (2.1243349, 48.6246916), 'Lisses': (2.4237886, 48.5960009), 'Ballainvilliers': (2.2944783, 48.6752123), 'Saulx-les-Chartreux': (2.2664925, 48.6919585), 'Ballancourt-sur-Essonne': (2.3858373, 48.5262129), 'Montlhéry': (2.2710952, 48.6396904), 'Cerny': (2.3296911, 48.4762984), 'Viry-Châtillon': (2.3750351, 48.6708729), 'Méréville': (6.1526, 48.5922), 'Saclay': (2.172576, 48.7305162), 'Wissous': (2.3268669, 48.7316381), 'Vert-le-Petit': (2.3663092, 48.5512575), 'Villiers-sur-Orge': (2.3010786, 48.6585041), 'Linas': (2.2701898, 48.6306286), 'Le Plessis-Pâté': (2.3256213, 48.6116007), 'Saint-Germain-lès-Arpajon': (2.2557107, 48.5948294), 'Ris-Orangis': (2.4151151, 48.6538754), 'Massy': (1.399492, 49.6902106), 'Saint-Michel-sur-Orge': (2.3027478, 48.6302735), 'Saint-Germain-lès-Corbeil': (2.4913639, 48.6261053), 'Crosne': (2.4645591, 48.72046), 'Villabé': (2.4544951, 48.5885996), 'Étréchy': (3.9446163, 48.885541), 'Yerres': (2.4879753, 48.716811), 'Orsay': (2.187457, 48.699184), 'Marcoussis': (2.230811, 48.6427131), 'Bures-sur-Yvette': (2.1632719, 48.6972179), 'Angerville': (-0.0328008, 49.2430262), 'Boutigny-sur-Essonne': (2.3805733, 48.437427), 'Milly-la-Forêt': (2.4702697, 48.4028842), 'Évry-Courcouronnes': (2.4381816, 48.629966), 'Le Mérévillois': (2.0819830696746484, 48.31992755), 'Champlan': (2.2758626, 48.7073405), 'Maisse': (2.3755851, 48.3942835), 'Égly': (2.2216686, 48.5774044), 'Mennecy': (2.4347044, 48.5666645), 'Épinay-sous-Sénart': (2.5162, 48.6877), 'Le Coudray-Montceaux': (2.4855792, 48.5653177), 'Saintry-sur-Seine': (2.4963163, 48.5926056), 'La Norville': (2.2583893, 48.5856753), 'Igny': (5.76504, 47.4782), 'Lardy': (2.2654419, 48.5204818), 'Morangis': (3.9198107, 48.9778812), 'Villemoisson-sur-Orge': (2.3322424, 48.6612042), 'Marolles-en-Hurepoix': (2.2986957, 48.5623029), 'Soisy-sur-Seine': (2.4490991, 48.6502928), 'Saint-Chéron': (4.5600716, 48.6362731), 'Morigny-Champigny': (2.1814129, 48.4461337), 'Dourdan': (2.0153689, 48.5288442), 'Arpajon': (2.4570176, 44.9034428), 'Épinay-sur-Orge': (2.324446, 48.6725622), 'Leuville-sur-Orge': (2.2674029, 48.6155573), 'Boussy-Saint-Antoine': (2.5299905, 48.6887478), 'Gometz-le-Châtel': (2.1386067, 48.6784419), 'Bondoufle': (2.3805119, 48.6137157), 'Verrières-le-Buisson': (2.2653844, 48.7467819), 'Paray-Vieille-Poste': (2.3637838, 48.7126093), 'Quincy-sous-Sénart': (2.5405195, 48.6712928), 'Limours': (2.075826, 48.6455882), 'Juvisy-sur-Orge': (2.373809, 48.6904063), 'Saint-Vrain': (4.80309, 48.69421), 'La Ferté-Alais': (2.3476234, 48.4824089), 'Villebon-sur-Yvette': (2.2276489, 48.7000046), 'Fleury-Mérogis': (2.3609313, 48.629965), 'La Ville-du-Bois': (2.2687891, 48.6610752), 'Brétigny-sur-Orge': (2.3075288, 48.6099887), 'Vigneux-sur-Seine': (2.4172117, 48.6998552), 'Boissy-sous-Saint-Yon': (2.2124998, 48.5486852), 'Saint-Pierre-du-Perray': (2.505381, 48.6137867), 'Morsang-sur-Orge': (2.3527728, 48.6641415), 'Tigery': (2.5064593, 48.6396178), 'Étiolles': (2.4775273, 48.6380667), 'Bruyères-le-Châtel': (2.1909499, 48.5922609), 'Courcouronnes': (2.411893948759139, 48.622557), 'Champcueil': (2.4500815, 48.5149123), 'Neuilly-sur-Seine': (2.2695658, 48.884683), 'Issy-les-Moulineaux': (2.273457, 48.8250508), 'Levallois-Perret': (2.2881683, 48.892956), 'Nanterre': (2.2071267, 48.8924273), 'Boulogne-Billancourt': (2.240206, 48.8356649), 'Colombes': (2.2543577, 48.922788), 'Saint-Cloud': (2.219344, 48.8437412), 'Courbevoie': (2.2561602, 48.8953328), 'Asnières-sur-Seine': (2.2890454, 48.9105948), 'Rueil-Malmaison': (2.1802832, 48.87778), 'Suresnes': (2.2255727, 48.8709053), 'Clamart': (2.2630292, 48.800368), 'Châtenay-Malabry': (2.2775316, 48.7670404), 'Clichy': (2.30551, 48.9026), 'Antony': (2.2959423, 48.753554), 'Sèvres': (2.2127083, 48.8247407), 'Bois-Colombes': (2.2674892, 48.9148269), 'Châtillon': (5.7302769, 46.6591772), 'Gennevilliers': (2.2940122, 48.9254221), 'Villeneuve-la-Garenne': (2.3324727, 48.9350488), 'Vanves': (2.289259, 48.8212037), 'Meudon': (2.2385432, 48.8126688), 'Puteaux': (2.2368863, 48.8841522), 'La Garenne-Colombes': (2.2465748, 48.9069349), 'Montrouge': (2.3194375, 48.8188544), 'Garches': (2.1853699, 48.8437486), 'Bourg-la-Reine': (2.316085, 48.7795178), 'Malakoff': (2.3019814, 48.8211559), 'Le Plessis-Robinson': (2.2619632, 48.7811049), 'Fontenay-aux-Roses': (2.2870373, 48.7911052), 'Sceaux': (4.021844858050334, 47.514728500000004), "Ville-d'Avray": (2.1894241, 48.8279362), 'Chaville': (2.1883487, 48.8086915), 'Vaucresson': (2.1575981, 48.8392676), 'Bobigny': (2.4452231, 48.906387), 'Saint-Denis': (2.3580232, 48.935773), 'Le Pré-Saint-Gervais': (2.4040111, 48.8830565), 'Gagny': (2.5361783, 48.8853708), 'Livry-Gargan': (2.5298854, 48.917335), 'Aulnay-sous-Bois': (2.499789, 48.934231), 'Montreuil': (1.7631125, 50.4638918), 'Drancy': (2.4455201, 48.9229803), 'Aubervilliers': (2.3821895, 48.9146078), 'Clichy-sous-Bois': (2.5460722, 48.910939), 'Épinay-sur-Seine': (2.3145039, 48.9525181), 'Noisy-le-Grand': (2.5529612, 48.8478586), 'Le Blanc-Mesnil': (2.4631476, 48.9385489), 'Saint-Ouen-sur-Seine': (2.334267, 48.911729), 'Bagnolet': (2.4173658, 48.8688199), 'Pantin': (2.4019804, 48.8965023), 'Neuilly-sur-Marne': (2.5290045, 48.8565557), 'Montfermeil': (2.5660399, 48.8992679), 'Bondy': (2.48291, 48.9031), 'Villepinte': (2.0855714, 43.2814917), 'Tremblay-en-France': (2.5589558, 48.9802035), 'Villemomble': (2.5076181, 48.881886), 'La Courneuve': (2.3896123, 48.9267236), 'Gournay-sur-Marne': (2.5769908, 48.8597463), 'Pierrefitte-sur-Seine': (2.3601022, 48.9637589), 'Les Lilas': (2.415796, 48.8795658), 'Stains': (2.3825154, 48.9565669), 'Le Raincy': (2.5155626, 48.8986666), 'Le Bourget': (2.4258354, 48.9351403), 'Coubron': (2.57922, 48.9166), 'Villetaneuse': (2.3439698, 48.9611722), 'Rosny-sous-Bois': (2.4875193, 48.8716626), 'Vaujours': (2.5691921, 48.9302563), 'Noisy-le-Sec': (2.4516055, 48.8897751), 'Romainville': (2.4358326, 48.8852958), 'Sevran': (2.5298164, 48.937634), 'Les Pavillons-sous-Bois': (2.5034999, 48.9082988), 'Neuilly-Plaisance': (2.5060144, 48.8608712), "L'Île-Saint-Denis": (2.3397438, 48.9355885), 'Dugny': (2.414733, 48.9538155), 'Saint-Mandé': (2.4159739, 48.8432434), 'Créteil': (2.4530731, 48.7771486), 'Vitry-sur-Seine': (2.39164, 48.7876), 'Villejuif': (2.3633048, 48.7921098), "L'Haÿ-les-Roses": (2.3373021, 48.7790514), 'Le Kremlin-Bicêtre': (2.3570028, 48.8125893), 'Champigny-sur-Marne': (2.5107384, 48.8137759), 'Saint-Maur-des-Fossés': (2.4853015, 48.8033057), 'Vincennes': (2.4396714, 48.8474508), 'Ivry-sur-Seine': (2.3872525, 48.8122302), 'Fontenay-sous-Bois': (2.4749347, 48.8490721), 'Maisons-Alfort': (2.4309703, 48.8012045), 'Cachan': (2.3340758, 48.7945413), 'Nogent-sur-Marne': (2.4916592, 48.8387639), 'Charenton-le-Pont': (2.4159508, 48.8198479), 'Choisy-le-Roi': (2.4093664, 48.7630238), 'Alfortville': (2.4197113, 48.8051624), 'Orly': (2.402391, 48.7431683), 'Santeny': (2.5788488, 48.7348571), 'Valenton': (2.4699741, 48.7443189), 'Ormesson-sur-Marne': (2.5483998, 48.785641), 'Villeneuve-Saint-Georges': (2.44782, 48.7305272), 'La Queue-en-Brie': (2.5749323, 48.7903338), 'Villiers-sur-Marne': (2.5406696, 48.8262055), 'Chevilly-Larue': (2.3469255, 48.7714153), 'Le Plessis-Trévise': (2.5732616, 48.8091528), 'Fresnes': (3.3705556, 49.5544444), 'Saint-Maurice': (7.3363439, 48.3275249), 'Thiais': (2.391024, 48.7643664), 'Sucy-en-Brie': (2.5221284, 48.7711323), 'Bry-sur-Marne': (2.5193322, 48.8352872), 'Arcueil': (2.3366502, 48.8064996), 'Rungis': (2.3472665, 48.7484048), 'Limeil-Brévannes': (2.4809129, 48.7489857), 'Joinville-le-Pont': (2.4668077, 48.8183717), 'Gentilly': (2.3483549, 48.8153656), 'Villeneuve-le-Roi': (2.4077003, 48.7350308), 'Bonneuil-sur-Marne': (2.4869009, 48.7737116), 'Boissy-Saint-Léger': (2.5124717, 48.7479193), 'Villecresnes': (2.5327265, 48.7208822), 'Mandres-les-Roses': (2.5471633, 48.7031725), 'Marolles-en-Brie': (3.162953, 48.7761644), 'Le Perreux-sur-Marne': (2.5077111, 48.840627), 'Ablon-sur-Seine': (2.421509, 48.7247582), 'Chennevières-sur-Marne': (2.530442, 48.7965001), 'Noiseau': (2.5480492, 48.7749788), 'Jouy-le-Moutier': (2.03194, 49.0116), 'Groslay': (2.3458615, 48.9867112), 'Montigny-lès-Cormeilles': (2.1989225, 48.9959113), 'Saint-Martin-du-Tertre': (3.257737, 48.2121679), 'Le Thillay': (2.46926, 49.0053), 'Survilliers': (2.5434395, 49.1025201), 'Viarmes': (2.3693685, 49.125863), 'Roissy-en-France': (2.5176985, 49.0032865), 'Bernes-sur-Oise': (2.2985653, 49.1620172), 'Soisy-sous-Montmorency': (2.3002885, 48.9875044), 'Cormeilles-en-Parisis': (2.1998877, 48.9759637), 'Louvres': (2.5096751, 49.0449139), 'Méry-sur-Oise': (2.1836324, 49.0634561), "L'Isle-Adam": (2.2186324, 49.1125294), 'Persan': (2.2765575, 49.1512109), 'Champagne-sur-Oise': (2.2312397, 49.1339431), "Saint-Ouen-l'Aumône": (2.1059841, 49.044765), 'Andilly': (5.881351, 48.7662318), 'Asnières-sur-Oise': (2.3541273, 49.1339089), 'Bouffémont': (2.2991001, 49.0431342), 'Domont': (2.3278097, 49.0285536), 'Bruyères-sur-Oise': (2.3301301, 49.1573165), 'Marly-la-Ville': (2.5034402, 49.0823913), 'Bessancourt': (2.2136286, 49.0372777), 'Vauréal': (2.0205773, 49.0300763), 'Parmain': (2.2090453, 49.1142087), 'Pierrelaye': (2.1557195, 49.0231851), 'Mériel': (2.2048038, 49.0790561), 'Sarcelles': (2.3796245, 48.9960813), 'Luzarches': (2.4223849, 49.1124996), 'Magny-en-Vexin': (1.7864359, 49.1554403), 'Argenteuil': (2.2481797, 48.9479069), 'Cergy': (2.0388736, 49.0527528), 'Pontoise': (2.1008067, 49.0508845), 'Auvers-sur-Oise': (2.1718687, 49.0701875), 'Villiers-le-Bel': (2.3880847, 49.0074999), 'Franconville': (6.4527267, 48.5000906), 'Bezons': (2.2105491, 48.9250016), 'Ermont': (2.2585679, 48.9917676), 'Eaubonne': (2.2794326, 48.9906696), 'Goussainville': (2.4733628, 49.0323168), 'Ézanville': (2.361889, 49.031707), 'Beaumont-sur-Oise': (2.2849324, 49.1419322), 'Fosses': (2.5161998, 49.0986649), 'Garges-lès-Gonesse': (2.399024, 48.9703841), 'Vémars': (2.5659272, 49.0672657), 'Le Plessis-Bouchard': (2.2330815, 49.0016863), 'Arnouville': (2.4202824, 48.9892126), 'Courdimanche': (1.999555, 49.0354894), 'Montmagny': (2.343561, 48.9728571), 'Enghien-les-Bains': (2.3068482, 48.9699732), 'Sannois': (2.2568296, 48.9717267), 'Saint-Leu-la-Forêt': (2.2480799, 49.0212001), 'La Frette-sur-Seine': (2.178345, 48.9740461), 'Saint-Prix': (4.0730519, 46.9553018), 'Deuil-la-Barre': (2.3275141, 48.9756423), 'Gonesse': (2.4500537, 48.9863555), 'Marines': (1.9831777, 49.1451519), 'Presles': (-0.781373, 48.8769135), 'Montsoult': (2.312001, 49.0704484), 'Osny': (2.0614598, 49.0660487), 'Beauchamp': (2.1993602, 49.0115876), 'Chaumontel': (2.4303, 49.1209286), 'Écouen': (2.3799378, 49.0196554), 'Puiseux-en-France': (2.5009544, 49.0597512), 'Montlignon': (2.2862669, 49.008969), 'Saint-Gratien': (2.4081341, 49.9656998), 'Taverny': (2.2234971, 49.0265888), 'Montmorency': (2.3197111, 48.9891693), 'Herblay-sur-Seine': (2.166162, 48.9898588), 'Margency': (2.28847, 49.002), 'Saint-Brice-sous-Forêt': (2.3568948, 48.9973824), 'Menucourt': (1.9815885, 49.027615), 'Frépillon': (2.2032431, 49.0512302), 'Éragny': (2.0910817, 49.0192178)}

In [ ]:
# ATTENTION comme cette cellule met du temps à s'exécuter pour la réplication on fournit dans la cellule ci dessous le dictionnaire obtenu
#on fait la meme chose pour les établissements
lat_long_etablissements = {}
for ville in villes2:
 
  la, lo = get_lat_long(ville)
  la = float(la)
  lo = float(lo)
  lat_long_etablissements[ville] = (lo,la)

In [ ]:
lat_long_etablissements = {'BORT LES ORGUES': (2.4992935, 45.4010305), 'DIEULEFIT': (5.0663637, 44.5236962), 'TAVERNY': (2.2234971, 49.0265888), 'PONTEILS ET BRESIS': (3.97586, 44.4033), 'PERONNE': (2.9321899, 49.9289578), 'NERIS LES BAINS': (2.6583028, 46.2898064), 'ST HERBLAIN': (-1.6346964, 47.2233007), 'LA MEMBROLLE SUR CHOISILLE': (0.6271357, 47.4449038), 'LUNEVILLE': (6.4919563, 48.5916164), 'ST DIE DES VOSGES': (6.9477708, 48.2873153), 'ROYE': (2.7930755, 49.6986186), 'LE PALAIS': (-3.1548668, 47.3470611), 'ST LEONARD DE NOBLAT': (1.4924586, 45.8377349), 'ARNOUVILLE': (2.4202824, 48.9892126), 'FONTAINEBLEAU': (2.7015872, 48.4049375), 'DENAIN': (3.3943729, 50.3270926), 'DECAZEVILLE': (2.2542071, 44.5585148), 'CROZON': (-4.4894463, 48.2463989), 'BOSCAMNANT': (-0.0682303, 45.1982588), 'ST ETIENNE DE TINEE': (6.9249179, 44.2564013), 'PONT L EVEQUE': (2.9880799, 49.5643305), 'DOULLENS': (2.3410532, 50.1572085), 'BAR SUR SEINE': (4.3759103, 48.1126529), 'QUINGEY': (5.8831349, 47.1038613), 'CREST': (5.0236641, 44.7282675), 'ST BONNET LE CHATEAU': (4.06555, 45.4235), 'NIVILLAC': (-2.2829889, 47.5330834), 'MEULAN EN YVELINES': (1.9100835, 49.0049712), 'CARPENTRAS': (5.0487681, 44.0554043), 'MARINES': (1.9831777, 49.1451519), 'MOUTHE': (6.1932371, 46.7115978), 'ST AULAYE PUYMANGOU': (0.11173209745349706, 45.18153735), 'RIBEAUVILLE': (3.5866666, 50.0436112), 'MURAT': (2.9084037, 46.4008462), 'ROANNE': (4.0729178, 46.0345572), 'ENVAL': (3.0503, 45.9), 'GUERET': (1.8713349, 46.1686865), 'PARENTIS EN BORN': (-1.0691145, 44.3508886), 'TONNEINS': (0.3091667, 44.3905556), 'FREJUS': (6.7360182, 43.4330308), 'CHARTRES': (1.4881434, 48.4438601), 'SIERENTZ': (7.4533853, 47.6536197), 'DAX': (-1.0518771, 43.7084065), 'TROYES': (4.0746257, 48.2971626), 'THOISSEY': (4.8015014, 46.1721833), 'LAON': (3.620686, 49.564665), 'ASTUGUE': (0.0702519, 43.0927344), 'MORTAGNE AU PERCHE': (0.5501095, 48.5205106), 'FAYE L ABBESSE': (-0.3525362, 46.8303723), 'CANCALE': (-1.8506953, 48.6771721), 'ST MARTIN DES CHAMPS': (-3.8439407, 48.574994), 'PUTEAUX': (2.2368863, 48.8841522), 'BEAUVAIS': (2.0823355, 49.4300997), 'FALAISE': (-0.1999129, 48.8964741), 'CHATEAU CHINON': (3.9330028, 47.0643977), 'SELESTAT': (7.454217, 48.2594396), 'PORT LA NOUVELLE': (3.046461, 43.0203536), 'CASTELLANE': (6.5131814, 43.8462177), 'NEUFCHATEAU': (5.6948571, 48.3553014), 'LE COUDRAY': (1.5011729, 48.4205915), 'WISSEMBOURG': (7.9445109, 49.0365935), 'CORNIL': (1.6915746, 45.2114467), 'BEAUREPAIRE': (2.5696774, 49.2916656), 'MONTDIDIER': (2.570196, 49.6460204), 'ST VALLIER': (6.3151506, 48.2788193), 'TULLINS': (5.4829748, 45.2977714), 'PLATEAU D HAUTEVILLE': (5.581990102625218, 45.9300815), 'TARASCON SUR ARIEGE': (1.6067586, 42.8458016), 'AVALLON': (3.9070658, 47.4885365), 'FOUGERES': (-1.1958526, 48.3503362), 'VALENCE D AGEN': (0.8902778, 44.1111111), 'VITRY LE FRANCOIS': (4.5851622, 48.7262225), 'SENS': (3.282606, 48.1978559), 'BLOIS': (1.3337639, 47.5876861), 'SECLIN': (3.0252819, 50.5452102), 'GISORS': (1.777826, 49.2789504), 'NERAC': (0.3392503, 44.1353979), 'RUMILLY': (2.01477, 50.57651), 'ST AUBIN ROUTOT': (0.3253756, 49.5240901), 'MONTREUIL': (1.7631125, 50.4638918), 'MULHOUSE': (7.3389937, 47.7467233), 'LE BAILLEUL': (-0.1613837, 47.7679046), 'TRAMAYES': (4.6004306, 46.3077301), 'EU': (1.4175744, 50.0491699), 'BEDARIEUX': (3.1576143, 43.6155109), 'BRIE COMTE ROBERT': (2.6166737, 48.6904522), 'GRAULHET': (1.99723, 43.7578), 'ST ANDRE DE CUBZAC': (-0.4440957, 44.9954151), 'DREUX': (1.3684254, 48.7358807), 'LA PORTE DU DER': (4.811324406741573, 48.477413), 'LANDERNEAU': (-4.2557901, 48.4514803), 'PARAY LE MONIAL': (4.1177959, 46.450559), 'LAMARCHE': (5.7822, 48.0688), 'BRIEY': (5.966079338759432, 49.2583938), 'VILLENEUVE DE BERG': (4.4985083, 44.5555132), 'LANNION': (-3.4587994, 48.7322183), 'MUNSTER': (-8.570897276292667, 52.307621600000004), 'PRIVAS': (4.5986733, 44.7352708), 'LUNEL': (4.1348062, 43.6745265), 'VALOGNES': (-1.4708174, 49.5089547), 'GUEBWILLER': (7.2113271, 47.9094629), 'MONTFORT SUR MEU': (-1.955145, 48.1372782), 'BRIANCON': (6.6436313, 44.8984037), 'PONTAULT COMBAULT': (2.6068064, 48.8009132), 'NOUZONVILLE': (4.7445678, 49.814437), 'FORBACH': (6.874619, 49.1835376), 'MARCONNE': (2.0450843, 50.3704702), 'DOMME': (1.2140018, 44.8022673), 'LE FOLGOET': (-4.3400067, 48.5635555), 'AJACCIO': (8.7376029, 41.9263991), 'ROUBAIX': (3.1741734, 50.6915893), 'ST CLAUDE': (5.8643866, 46.3874699), 'MONTFERMEIL': (2.5660399, 48.8992679), 'ST MAUR': (1.9183427, 49.6155716), 'MENDE': (3.4991057, 44.5180226), 'VIMOUTIERS': (0.1970867, 48.9297086), 'ERSTEIN': (7.6632641, 48.4227385), 'CHELLES': (3.0344372, 49.3549458), 'SEES': (0.1734544, 48.603378), 'PITHIVIERS': (2.2544871, 48.1738051), 'PALAISEAU': (2.2453842, 48.7144587), 'CHATILLON SUR SEINE': (4.5749139, 47.8607467), 'CARENTOIR': (-2.1362362244234725, 47.81775695), 'CONDE SUR L ESCAUT': (3.5928883, 50.4483415), 'CHEMILLE EN ANJOU': (-0.6725711312155349, 47.22107895), 'AUTUN': (4.2989478, 46.9510313), 'CLAMECY': (3.3619444, 49.4286111), 'VERRIERES': (4.9074122, 49.065134), 'SAVENAY': (-1.9421244, 47.3592304), 'DOMFRONT EN POIRAIE': (-0.6225916540840775, 48.5864842), 'BAPAUME': (2.8497506, 50.1038538), 'NEUVILLE AUX BOIS': (2.0534134, 48.0695211), 'APT': (5.3970648, 43.8757936), 'PAIMBOEUF': (-2.02641, 47.2871357), 'LA CHATAIGNERAIE': (-0.7391667, 46.65), 'FISMES': (3.6802766, 49.3072444), 'YSSINGEAUX': (4.1244363, 45.1435556), 'ARRAS': (2.7772211, 50.291048), 'HAUBOURDIN': (2.9864421, 50.6093554), 'ISSOIRE': (3.2502213, 45.5432973), 'ST CYR L ECOLE': (2.0672138, 48.799708), 'HAM': (3.0733031, 49.746322), 'GUERANDE': (-2.4298014, 47.3282126), 'JONZAC': (-0.4320984, 45.4454469), 'LOCHES': (0.9979549, 47.1281692), 'ABRESCHVILLER': (7.0979423, 48.6370241), 'BEAUMONT SUR OISE': (2.2849324, 49.1419322), 'ST RAPHAEL': (1.0747165, 45.3043079), 'LAMASTRE': (4.5796372, 44.9854888), 'CAHORS ': (1.4364999, 44.4495), 'MONTBELIARD': (6.7977564, 47.5102368), 'MORTEAU': (6.6047189, 47.0569367), 'BELLEY': (5.6861466, 45.760758), 'VIERZON': (2.0691421, 47.2217489), 'AVION': (2.8343891, 50.4126128), 'LABOUHEYRE': (-0.9189952, 44.2125451), 'SARREGUEMINES': (7.0708865, 49.1094836), 'VERDUN': (5.3867277, 49.1589278), 'VAUCOULEURS': (5.6656143, 48.6017525), 'MAULEON SOULE': (-0.8913386053069048, 43.2209834), 'MARSEILLE ': (5.3699525, 43.2961743), 'STE MARIE AUX MINES': (7.1851184, 48.2476662), 'VALENCAY': (1.5666078, 47.161511), 'VITTEL': (5.9418851, 48.2020717), 'ANTIBES': (7.10905, 43.5836), 'ST LAURENT DE CHAMOUSSET': (4.4645889, 45.7386043), 'COGOLIN': (6.53659, 43.25), 'NEUILLY SUR MARNE': (2.5290045, 48.8565557), 'PENNE D AGENAIS': (0.8205394, 44.3891155), 'FAINS VEEL': (5.1238423, 48.7913628), 'LA BASSEE': (2.8046597, 50.5316305), 'LE FAOUET': (-3.0727913, 48.6849235), 'MONTMORENCY': (2.3197111, 48.9891693), 'BERGERAC': (0.487531, 44.8534568), 'ELBEUF': (1.0108998, 49.2871086), 'ST CHAMOND': (4.5098046, 45.4748298), 'PLOERMEL': (-2.39808, 47.9323786), 'LA FERTE BERNARD': (0.6554511, 48.1860576), 'RAMBOUILLET': (1.819207, 48.6452851), 'PONT DE VEYLE': (4.88836, 46.2635), 'ST GENIEZ D OLT ET D AUBRA': (-1.0, -1.0), 'ST LO': (-1.0906637, 49.1157004), 'ST JEAN DE MAURIENNE': (6.3453329, 45.2775258), 'BRUYERES': (6.7204365, 48.2115397), 'MITRY MORY': (2.6183971, 48.9839674), 'BAYONNE': (-1.4736657, 43.4945144), 'CANNES LA BOCCA': (6.9865537, 43.5486667), 'AMILLY': (1.3930652, 48.4418021), 'BASTIA': (9.4509187, 42.6993979), 'VERNEUIL SUR AVRE': (0.9434228814520526, 48.7373168), 'CHAUDES AIGUES': (3.0044296, 44.8534478), 'CHATEAU GONTIER SUR MAYENN': (-1.0, -1.0), 'ST SULPICE LA POINTE': (1.686477, 43.7737894), 'BEAUCAIRE': (4.6392648, 43.8085436), 'ARGENTEUIL': (2.2481797, 48.9479069), 'JOIGNY': (3.3995767, 47.9812486), 'VANNES': (-2.7599079, 47.6586772), 'PERPIGNAN ': (2.8953121, 42.6985304), 'DIGNE LES BAINS': (6.2351431, 44.0918144), 'VOUZIERS': (4.723819262037698, 49.40228705), 'BEAUPREAU EN MAUGES': (-0.9183549426012452, 47.20279205), 'CORBIE': (2.5110728, 49.9082911), 'FOURMIES': (4.0458492, 50.0171741), 'CHAMBONAS': (4.1290677, 44.416701), 'LESNEVEN': (-4.3235917, 48.5718493), 'YERRES': (2.4879753, 48.716811), 'CREPY EN VALOIS': (2.8900507, 49.2358509), 'VILLEDIEU LES POELES ROUFF': (-1.0, -1.0), 'NIORT': (-0.4649403, 46.3241132), 'AMBOISE': (0.983698, 47.4110351), 'VENDOME': (1.0662256, 47.7934453), 'ORNANS': (6.151746304484529, 47.09472425), 'LUXEUIL LES BAINS': (6.3808865, 47.817177), 'PAMIERS': (1.6082177, 43.1147515), 'BOURG ST MAURICE': (6.7694313, 45.6185284), 'CANDE': (111.6928724, 29.034552), 'AURAY': (-2.9833718, 47.6664485), 'ST MEEN LE GRAND': (-2.1911338, 48.1894709), 'CASTILLON LA BATAILLE': (-0.0450873, 44.8533324), 'NARBONNE': (3.0042121, 43.1837661), 'CALAIS': (1.8538446, 50.9524769), 'MOLSHEIM': (7.4924679, 48.5419351), 'VILLENEUVE SUR YONNE': (3.2942751, 48.0820102), 'GIMONT': (0.8776713, 43.6273203), 'AGDE': (3.4771629, 43.3134787), 'MONTELIMAR': (4.750318, 44.5579391), 'NONTRON': (0.6622535, 45.5293074), 'AVESNELLES': (3.9473593, 50.1182023), 'ERR': (2.0328526, 42.4406621), 'TERRANJOU': (-0.4355920572666552, 47.2690011), 'ROSHEIM': (7.4698832, 48.496421), 'ST BRIEUC ': (-2.7603283, 48.5141134), 'MAMERS': (0.3696376, 48.3505932), 'ST MIHIEL': (5.5423046, 48.891207), 'ST JEAN D ANGELY': (-0.5195756, 45.9445958), 'VARENNES VAUZELLES': (3.1456935, 47.013196), 'GAUCHIN VERLOINGT': (2.3124098, 50.3953916), 'VILLENEUVE SUR LOT': (0.7046455, 44.4056615), 'SERRIERES': (4.6811536, 46.3123259), 'VAISON LA ROMAINE': (5.075737, 44.2407641), 'ST JULIEN EN GENEVOIS CEDE': (-1.0, -1.0), 'PARIS ': (2.3483915, 48.8534951), 'BILLOM': (3.3390575, 45.7220503), 'MONTMIRAIL': (3.5381991, 48.8708848), 'BRETAGNE DE MARSAN': (-0.4593708, 43.8473121), 'LA GARDE': (6.0449399, 45.0691999), 'COLMAR': (7.3579641, 48.0777517), 'BAR SUR AUBE': (4.7058337, 48.2332047), 'LE NEUBOURG': (0.9044712, 49.1496553), 'FIGEAC': (2.0324321, 44.6092335), 'ST FELICIEN': (4.62778, 45.0867), 'VAL DE BRIEY': (5.9571236995611905, 49.266237849999996), 'MURET': (1.3262332, 43.4599858), 'BERCK': (1.5711617, 50.4052576), 'AVESNES SUR HELPE': (3.9321056, 50.1218835), 'MOYENMOUTIER': (6.9128471, 48.3794555), 'HYERES': (6.1301614, 43.1202573), 'VERVINS': (3.9079645, 49.8346074), 'CHINON': (0.2402095, 47.1667182), 'LANGRES': (5.333947, 47.8639612), 'TOURNON SUR RHONE': (4.832852, 45.0675156), 'PORNIC': (-2.1040102, 47.1152695), 'PLOEMEUR': (-3.4270534, 47.7354591), 'PERIGUEUX': (0.7184407, 45.1909365), 'CREIL ': (2.4733771, 49.2607914), 'EAUBONNE': (2.2794326, 48.9906696), 'GRANDRIS': (4.4761219, 46.0386194), 'AIX LES BAINS': (5.9146362, 45.6886292), 'MAURIAC': (-0.0289804, 44.7450537), 'NEUVILLE SUR SAONE': (4.8395051, 45.8765557), 'CHATEAU RENAULT': (0.9084166, 47.5904923), 'MOUTIER ROZEILLE': (2.1986651, 45.9175892), 'ANNONAY': (4.6675475, 45.2399639), 'ROMILLY SUR SEINE': (3.7264161, 48.5195734), 'FLERS': (2.25255, 50.32053), 'CAMBRAI': (3.2346145, 50.1757546), 'BOULOGNE SUR MER': (1.6118771, 50.7259985), 'TARBES ': (0.0781021, 43.232858), 'PONT STE MARIE': (4.1014237, 48.3173614), 'ST MARTIN DE RE': (-1.3681861, 46.2016893), 'TOULON': (5.9304919, 43.1257311), 'BONNETABLE': (0.4269856, 48.1768212), 'RIEZ': (6.0930048, 43.8176907), 'NEMOURS': (2.6953079, 48.268026), 'LA SEYNE SUR MER': (5.8788948, 43.1007714), 'BAZAS': (-0.2126009, 44.4320456), 'MEAUX': (2.8773541, 48.9582708), 'USSEL': (2.3119397, 45.545591), 'ST SAIRE': (1.4914456, 49.6937431), 'MELLE': (7.32049, 44.5620476), 'CHALLANS': (-1.8773787, 46.847943), 'TOULON SUR ARROUX': (4.1387169, 46.693513), 'CHARLEVILLE MEZIERES': (4.7206939, 49.7735712), 'ENSISHEIM': (7.3529099, 47.8665246), 'BELLAC': (1.0502545, 46.1209036), 'VERSAILLES': (2.1266886, 48.8035403), 'MEZE': (-0.4724581, 49.0785317), 'CLERMONT L HERAULT': (3.4302412, 43.6263176), 'CREVECOEUR LE GRAND': (2.08184, 49.611), 'HOUDAN': (1.6019247, 48.7912001), 'BOULOGNE BILLANCOURT': (2.240206, 48.8356649), 'PONTARLIER': (6.3546345, 46.9040348), 'CAMPAGNE LES HESDIN': (1.8774929, 50.397913), 'BAR LE DUC': (5.1623805, 48.7712673), 'RIVES SUR FURE': (5.4996953, 45.354196), 'CHAGNY': (4.7523461, 46.910578), 'BUZANCAIS': (1.4227597, 46.8885028), 'BAGNERES DE LUCHON': (0.5912279, 42.7910617), 'LISIEUX': (0.2255168, 49.1460831), 'PONTIVY': (-2.9675702, 48.0647729), 'ESPALION': (2.7616471, 44.5206809), 'GOURNAY EN BRAY': (1.7241534, 49.4824794), 'ST OMER': (2.2534183, 50.7515771), 'STE MAURE DE TOURAINE': (0.6189854, 47.1108281), 'CASTRES': (3.2383333, 49.8044444), 'GAP': (6.0820639, 44.5612032), 'ROQUEBILLIERE': (7.307673, 44.0127173), 'PORTO VECCHIO': (9.2794469, 41.5911382), 'LANGOGNE': (3.8550146, 44.7266369), 'RENAZE': (-1.0557842, 47.793333), 'DUNKERQUE': (2.3772525, 51.0347708), 'HENIN BEAUMONT': (2.9469339, 50.4191721), 'LIGNY EN BARROIS': (5.3253723, 48.685936), 'ST GOBAIN': (3.3759, 49.5967), 'LE PONT DE BEAUVOISIN': (5.6704512, 45.5359674), 'VILLAINES LA JUHEL': (-0.2780934, 48.3451696), 'CONFOLENS': (0.6728934, 46.0150236), 'STE FOY LA GRANDE': (0.214969, 44.8416496), 'DOUAI': (3.0804641, 50.3675677), 'QUINTIN': (-2.9112669, 48.4036054), 'ST AMAND MONTROND': (2.5046503, 46.7227062), 'SAINTES': (-0.6300671, 45.7460663), 'HAGUENAU': (7.7885978, 48.8172236), 'FECAMP': (0.374891, 49.7577729), 'PUGET THENIERS': (6.8951839, 43.9555155), 'GLEIZE': (4.6971455, 45.9888277), 'BAVILLIERS': (6.8347586, 47.6220703), 'MONTAIGU': (3.83264, 49.535), 'ST NAZAIRE': (-2.2138905, 47.2733517), 'EPAGNY METZ TESSY': (6.093513616324771, 45.942964200000006), 'ORTHEZ': (-0.7738363, 43.4874963), 'BELFORT': (6.8628942, 47.6379599), 'ANTONNE ET TRIGONANT': (0.8382445773021518, 45.2276171), 'ST CALAIS': (0.7434396, 47.9201366), 'L ISLE SUR LA SORGUE': (5.0515385, 43.9185354), 'LA LOUPE': (1.02521, 48.4737), 'MOISSAC': (1.0846682, 44.1050886), 'ALBIGNY SUR SAONE': (4.8323149, 45.8706791), 'LE CHESNAY': (2.1257808, 48.8264842), 'ST QUENTIN': (3.2876843, 49.8465253), 'NOGARO': (-0.0327778, 43.7591667), 'LES MUREAUX': (1.909928, 48.9931578), 'PARTHENAY': (-0.2472687, 46.6481266), 'OLEMPS': (2.5541214, 44.3386131), 'PONT ST ESPRIT': (4.6456631, 44.2538589), 'FLEURANCE': (0.6640617, 43.849295), 'PFASTATT': (7.3025075, 47.7697441), 'BOURBON LANCY': (3.7737674, 46.6193905), 'RIANTEC': (-3.3088391, 47.7119271), 'CHAMPAGNOLE': (5.9083581, 46.7456049), 'AUBUSSON': (-0.5521714, 48.7842759), 'ANCENIS': (-1.1816088, 47.3648141), 'SOSPEL': (7.4498235, 43.8785647), 'RODEZ ': (2.5727883, 44.3513062), 'LECTOURE': (0.6219444, 43.9352778), 'BISCHWILLER': (7.8597466, 48.7682281), 'DIEPPE': (1.0791444, 49.9246182), 'LA TESTE': (-1.1429337, 44.6369726), 'CLICHY': (2.30551, 48.9026), 'JOSSIGNY': (2.7531163, 48.8386762), 'LAURIS': (5.3126795, 43.7466047), 'LONS LE SAUNIER': (5.5589973, 46.6727037), 'CASTELNAUDARY': (1.9533495, 43.3192021), 'ST DENIS': (2.3580232, 48.935773), 'LE CHAMBON FEUGEROLLES CED': (-1.0, -1.0), 'CAMBO LES BAINS': (-1.3994687, 43.3609139), 'LYS HAUT LAYON': (-0.423740369520669, 47.150343449999994), 'VERTOU': (-1.4722242, 47.1685211), 'ALBERT': (2.6509223, 50.0018022), 'ALENCON': (0.0911374, 48.4312059), 'DINARD': (-2.0580178, 48.6320379), 'LA FERE': (3.3661672, 49.6632869), 'MORESTEL': (5.4711032, 45.675676), 'GASSIN': (6.585476, 43.2291533), 'ALFORTVILLE': (2.4197113, 48.8051624), 'COURCHEVEL': (6.6349892, 45.4140984), 'MEXIMIEUX': (5.1925264, 45.90363), 'POLIGNY': (5.7086756, 46.8348884), 'NEUFCHATEL EN BRAY': (1.4421413, 49.7350035), 'SAUMUR': (-0.0785177, 47.2596224), 'CRICQUEBOEUF': (0.1466667, 49.4030556), 'CERGY': (2.0388736, 49.0527528), 'LE CROISIC': (-2.5094404, 47.2933971), 'GUEMENE SUR SCORFF': (-3.2027629, 48.0658593), 'ST VINCENT DE TYROSSE': (-1.3029669, 43.6604837), 'SARREBOURG': (7.0557352, 48.734239), 'ST LIZIER': (1.1354926, 43.0025191), 'CRAON': (-0.9518383, 47.84831), 'LANMEUR': (-3.7165916, 48.6472122), 'RUFFEC': (0.1994619, 46.0287057), 'GEX': (6.0588831, 46.3324378), 'LA TOUR': (6.431111, 46.133611), 'SOMAIN': (3.2810972, 50.3589237), 'TULLE': (1.7706797, 45.2678347), 'LAVAL': (-0.7734022, 48.0706687), 'BAGNERES DE BIGORRE': (0.1530914, 43.0658212), 'CHAMONIX MONT BLANC': (6.8727506, 45.9246705), 'BOURG ST ANDEOL': (4.6457565, 44.3707629), 'CORCOUE SUR LOGNE': (-1.5772672, 46.965977), 'CHAUNY': (3.2189506, 49.6148945), 'GUEUGNON': (4.0633825, 46.6005232), 'ERNEE': (-0.9351069, 48.2967935), 'QUIMPER': (-4.1024782, 47.9960325), 'EVAUX LES BAINS': (2.4870429, 46.176149), 'MONTLUCON': (2.6073963, 46.3400296), 'LAMALOU LES BAINS': (3.08514, 43.5962), 'CHALONS EN CHAMPAGNE': (4.3628851, 48.9566218), 'LA GUICHE': (4.4506059, 46.5445899), 'MONTVAL SUR LOIR': (0.44213227296009794, 47.69545275), 'ST VALERY EN CAUX': (0.7106831, 49.8597004), 'MONTAUBAN': (1.3549991, 44.0175835), 'YZEURE': (3.3539775, 46.5676568), 'LARAGNE MONTEGLIN': (5.8230176, 44.3148916), 'L HERM': (1.6858757, 42.9724292), 'LAGNY SUR MARNE': (2.7056056, 48.8773018), 'COMMERCY': (5.5914753, 48.763203), 'MULHOUSE ': (7.3389937, 47.7467233), 'MONTBARD': (4.3373194, 47.6241674), 'ST VALERY SUR SOMME': (1.6279147, 50.1887006), 'CHATEAUBRIANT': (-1.3757282, 47.7194057), 'AURILLAC': (2.4433101, 44.9285441), 'MAUREPAS': (2.8478956, 49.9905624), 'ST JEAN DE LUZ': (-1.6635695, 43.3870763), 'BOURBONNE LES BAINS': (5.7512344, 47.9538267), 'CHALON SUR SAONE': (4.8529605, 46.7888997), 'LANGON': (-0.2449274, 44.5530659), 'BRIGNOLES': (6.0616452, 43.4064401), 'LA ROCHETTE': (2.662899, 48.5045038), 'ST GERMAIN EN LAYE': (2.0917742569617808, 48.93560465), 'ROCHEFORT': (-0.9687519, 45.9438412), 'ST AMAND LES EAUX': (3.4281142, 50.4491519), 'MOULINS': (3.6847, 49.4217), 'LA ROCHE SUR FORON': (6.3121658, 46.0672189), 'DECHY': (3.1299151, 50.354883), 'DECIZE': (3.4616097, 46.8284259), 'ALBERTVILLE': (6.3925417, 45.6754622), 'VICHY': (3.4203712, 46.1239268), 'BESANCON': (6.0243622, 47.2380222), 'AX LES THERMES': (1.8379765, 42.7193597), 'ARBOIS': (5.7727504, 46.9032246), 'BEAUGENCY': (1.6311944, 47.7778775), 'NOYAL PONTIVY': (-2.882738, 48.0676349), 'ST AVOLD': (6.7071754, 49.1043397), 'SALLANCHES': (6.6302551, 45.9361246), 'HENNEBONT': (-3.2751174, 47.8058026), 'AVRANCHES': (-1.3634379895766875, 48.688189), 'SULLY SUR LOIRE': (2.3738968, 47.7654098), 'MONT DE MARSAN': (-0.500972, 43.8911318), 'SAULT': (5.4087581, 44.0918084), 'LA GUERCHE DE BRETAGNE': (-1.231454, 47.9426332), 'LILLEBONNE': (0.534411, 49.5188282), 'LONGUENESSE': (2.24833, 50.7371), 'TREGUIER': (-3.2319779, 48.784201), 'BEZIERS': (3.2131307, 43.3426562), 'ST YRIEIX LA PERCHE': (1.2033138, 45.5127494), 'VILLENEUVE ST GEORGES CEDE': (-1.0, -1.0), 'QUIMPERLE': (-3.5505644, 47.8692882), 'LA ROCHEFOUCAULD EN ANGOUM': (-1.0, -1.0), 'TARASCON': (1.6067586, 42.8458016), 'LEZIGNAN CORBIERES': (2.7574981, 43.2008841), 'EXCIDEUIL': (1.0490102, 45.336641), 'CAVAILLON': (5.0383522, 43.8356808), 'PERTUIS': (5.5032671, 43.6951468), 'TONNERRE': (3.9733288, 47.8565193), 'AUXONNE': (5.3882954, 47.1936509), 'GHISONACCIA': (9.40536, 42.0149385), 'NEGREPELISSE': (1.5225, 44.0761111), 'ORGELET': (5.6109831, 46.5205835), 'LA ROCHE SUR YON ': (-1.4269698, 46.6705431), 'CONFLANS STE HONORINE': (2.0949768, 48.9938791), 'THANN': (7.1020957, 47.8101008), 'GRAY': (5.5919867, 47.4455642), 'ST GIRONS': (1.1464455, 42.9841865), 'MONTFORT EN CHALOSSE': (-0.8396241, 43.7107059), 'LOOS': (3.0162523, 50.6092082), 'ST GIRONS D AIGUEVIVES': (-0.5416993, 45.1401281), 'CONDRIEU': (4.7676199, 45.4624294), 'GRASSE': (6.9239103, 43.6589011), 'EPINAY SUR ORGE': (2.324446, 48.6725622), 'JOINVILLE': (5.1377537, 48.4433253), 'LA ROCHELLE': (-1.1520434, 46.1591126), 'TENDE': (7.5933916, 44.0889416), 'BAILLEUL': (2.7349286, 50.7396668), 'EPINAL': (6.4503643, 48.1747684), 'COURBEVOIE': (2.2561602, 48.8953328), 'ABBEVILLE': (1.8337029, 50.1060835), 'ISSOUDUN': (1.9889131, 46.9477826), 'TOURCOING': (3.1605714, 50.7235038), 'TREVOU TREGUIGNEC': (-3.3584522, 48.8120844), 'DEUIL LA BARRE': (2.3275141, 48.9756423), 'ARPAJON': (2.4570176, 44.9034428), 'AINCOURT': (1.771976, 49.0719037), 'VIC FEZENSAC': (0.3041667, 43.7594444), 'MABLY': (4.0644726, 46.0860078), 'PONT STE MAXENCE': (2.6045838, 49.3012483), 'BRIOUDE': (3.3853711, 45.293897), 'ST GILLES CROIX DE VIE': (-1.9254545, 46.692985), 'EPERNAY': (3.9528655, 49.042561), 'THIERS': (3.5489707, 45.855795), 'LANGEAC': (3.4941798, 45.1001248), 'VIGNEUX SUR SEINE': (2.4172117, 48.6998552), 'ST PIERRE DU MONT': (-0.9814739, 49.3893604), 'COGNAC': (-0.3250175, 45.6931647), 'ARLES': (4.6309653, 43.6776223), 'BURES SUR YVETTE': (2.1632719, 48.6972179), 'LAVAUR': (1.0230515, 44.6129697), 'AUCH': (0.5850507, 43.6463558), 'BOULAY': (6.4935123, 49.1842243), 'OBERNAI': (7.4817123, 48.4623358), 'VALS LES BAINS': (4.365739, 44.6553304), 'LENS': (2.8319805, 50.4291723), 'LURE': (6.4946806, 47.6863376), 'PARIS': (2.3483915, 48.8534951), 'FRAIZE': (7.0022069, 48.1863539), 'CHERBOURG OCTEVILLE': (-1.6285770868661507, 49.6343543), 'EVRY COURCOURONNES': (2.4381816, 48.629966), 'GOURDON': (4.4461818, 46.6414367), 'ST LAURENT DU PONT': (5.7344662, 45.387632), 'RIOM': (3.1136667, 45.893068), 'VESOUL': (6.1544694, 47.6197395), 'MORTAIN BOCAGE': (-0.924256777713777, 48.6076865), 'MODANE': (6.6728169, 45.2015369), 'MONTEREAU': (2.5717138, 47.8564484), 'TREVENANS': (6.8623067, 47.5712429), 'LE MONT DORE': (-0.1992419, 46.7319309), 'GUINGAMP': (-3.1502007, 48.5618483), 'AUXERRE': (3.570579, 47.7961287), 'ST ANGEL': (2.9338786, 45.9957342), 'HONFLEUR': (0.2338889, 49.4197222), 'ANGOULEME': (0.1576745, 45.6512748), 'EVREUX': (1.1510164, 49.0268903), 'ST GEOIRE EN VALDAINE': (5.6355662, 45.4563548), 'ST JAMES': (-90.7989525, 30.0096684), 'CLERMONT': (2.4141907, 49.3780921), 'LOUHANS': (5.2232175, 46.6291425), 'BOLBEC': (0.4742448, 49.5739091), 'VIC EN BIGORRE': (0.0548805, 43.3860159), 'PORT DE BOUC': (4.9806998, 43.403098), 'BARBEZIEUX ST HILAIRE': (-0.14194724370412187, 45.481221250000004), 'PROVINS': (3.2988089, 48.5602959), 'VIENNE': (16.3725042, 48.2083537), 'ST RENAN': (-4.622982, 48.4328451), 'PLOUGONVEN': (-3.7122667, 48.5212583), 'MONTRICHARD': (1.1828617252709055, 47.3593576), 'BAYEUX': (-0.7024738, 49.2764624), 'CHATEAU ARNOUX ST AUBAN': (6.0080053, 44.0936899), 'BRUAY SUR L ESCAUT': (3.5379105, 50.397277), 'LE HAVRE': (0.1079732, 49.4938975), 'LORIENT': (-3.3660907, 47.7477336), 'LODEVE': (3.31892, 43.73106), 'LE THILLOT': (6.7633816, 47.8804149), 'HEROUVILLE ST CLAIR': (-0.3353513, 49.2031097), 'ST DIZIER': (4.9473513, 48.6371125), 'ST HILAIRE DU HARCOUET': (-1.085461778865405, 48.5714837), 'POMPEY': (6.1229922, 48.7658301), 'DOUARNENEZ': (-4.3311243, 48.0944459), 'CHATILLON SUR INDRE': (1.1745103, 46.9865819), 'BONIFACIO': (9.1606179, 41.3878259), 'BULLY LES MINES': (2.71989, 50.4464), 'THONON LES BAINS': (6.4779448, 46.3731303), 'NOYON': (3.0000288, 49.5805561), 'RIBERAC': (0.3398094, 45.2478033), 'PERPIGNAN': (2.8953121, 42.6985304), 'MELUN': (2.6608169, 48.539927), 'MENTON': (7.5029213, 43.7753495), 'ST PIERRE D OLERON': (-1.3061227, 45.9437695), 'CARHAIX PLOUGUER': (-3.5711477, 48.2764997), 'LA CIOTAT': (5.6062495, 43.1758591), 'MAULEON': (0.5674923, 42.9594856), 'ST GAUDENS': (0.7247218, 43.1077682), 'BULLION': (1.9945126, 48.6221663), 'WASSY': (4.9489836, 48.4996383), 'CANNES': (7.0134418, 43.5515198), 'ORANGE': (-117.8704931, 33.7500378), 'ROUBAIX ': (3.1741734, 50.6915893), 'SEGRE EN ANJOU BLEU': (-0.8715571, 47.6811163), 'BUCHELAY': (1.6716534, 48.9793554), 'COSNE COURS SUR LOIRE CEDE': (-1.0, -1.0), 'MAZAMET': (2.3764363, 43.4902317), 'JARNY': (5.8821708, 49.1569586), 'MARMANDE': (0.164274, 44.5007939), 'TREVOUX': (4.7733198, 45.9410123), 'GUISE': (3.6262542, 49.8982325), 'BEAUFORT EN ANJOU': (-0.2162487, 47.4399992), 'DOURDAN': (2.0153689, 48.5288442), 'MAGNY EN VEXIN': (1.7864359, 49.1554403), 'CHALONNES SUR LOIRE': (-0.76311, 47.3516207), 'SEZANNE': (3.7223381, 48.7240897), 'LE VESINET': (2.1347291, 48.894254), 'ST MAIXENT L ECOLE': (-0.2019344, 46.4096855), 'TRONGET': (3.0677484, 46.42222), 'LIBOURNE': (-0.2427217, 44.9133742), 'CARVIN': (2.9577519, 50.4934145), 'CALVI': (8.7603309, 42.5680204), 'JUVISY SUR ORGE': (2.373809, 48.6904063), 'VALREAS': (4.9906089, 44.3851455), 'VILLENEUVE ST GEORGES': (2.44782, 48.7305272), 'LEVROUX': (1.6392860280931463, 46.9731343), 'JOYEUSE': (4.2414292, 44.4801272), 'CHATEAUROUX': (1.6770956, 46.8203785), 'OLORON STE MARIE': (-0.6115586, 43.1922408), 'CRAPONNE SUR ARZON': (3.8459302, 45.3317039), 'SEDAN': (4.9433409, 49.7033759), 'TOURNUS': (4.9122425, 46.5617373), 'ROMANS SUR ISERE': (5.0528681, 45.0458886), 'DRAGUIGNAN': (6.4627333, 43.5374662), 'BELLEVILLE': (6.0982683, 48.8176714), 'ST MAURICE': (7.3363439, 48.3275249), 'PAIMPOL': (-3.0484022, 48.7794818), 'CAHORS': (1.4364999, 44.4495), 'NIDERVILLER': (7.1075055, 48.7121812), 'MOURENX': (-0.6294321, 43.3708943), 'LE PUY EN VELAY': (3.8855537, 45.0459739), 'SEURRE': (5.1487861, 46.9995787), 'CIREY SUR VEZOUZE': (6.9500348, 48.5811011), 'SEMUR EN AUXOIS': (4.3328705, 47.4902149), 'CHAUMONT': (5.1395849, 48.1111324), 'BLAYE': (-0.6635964, 45.1271231), 'MATTAINCOURT': (6.1339, 48.2799), 'FORCALQUIER': (5.780446, 43.9585575), 'TARBES': (0.0781021, 43.232858), 'VITTEAUX': (4.5417701, 47.3972983), 'TARARE': (4.4371374, 45.8942555), 'PLAISIR': (1.9476363, 48.817399), 'GAILLAC': (1.8967943, 43.9016766), 'GRAMAT': (1.723394, 44.7789948), 'CARCASSONNE ': (2.3491069, 43.2130358), 'CLUNY': (4.6575708, 46.4339138), 'BREIL SUR ROYA': (7.5139664, 43.9375321), 'STE MENEHOULD': (4.8967637, 49.0904192), 'SEVRES': (2.2127083, 48.8247407), 'AIX EN PROVENCE ': (5.4474738, 43.5298424), 'LA ROCHELLE ': (-1.1520434, 46.1591126), 'PONTOISE': (2.1008067, 49.0508845), 'ST ASTIER': (0.5227575000000048, 45.13898075), 'BEAUNE LA ROLANDE': (2.42804, 48.06949), 'VALENCIENNES': (3.5234846, 50.3579317), 'CLISSON': (-1.2817422, 47.0877218), 'GIVORS': (4.7696529, 45.5844238), 'SARCELLES': (2.3796245, 48.9960813), 'ST MALO': (-2.0260409, 48.649518), 'ST JEAN DE VERGES': (1.6114072, 43.0123688), 'CHAROLLES': (4.2743162, 46.435311), 'MAUBEUGE': (3.9674127, 50.2796417), 'NOGENT SUR SEINE': (3.4978181, 48.492439), 'ST JUST LA PENDUE': (4.240052, 45.8915699), 'DOUE EN ANJOU': (-0.2938656, 47.1896337), 'VIRE': (-0.8872594458318377, 48.8517468), 'ST ROMAIN DE COLBOSC': (0.3579164, 49.5313601), 'LA TOUR DU PIN': (5.4424054, 45.5666202), 'FIRMINY': (4.2858545, 45.3869468), 'FELLERIES': (4.0320248, 50.1477534), 'LONGUE JUMELLES': (-0.1034531, 47.4346214), 'WASQUEHAL': (3.1312447, 50.6691577), 'AVIGNON ': (4.8059012, 43.9492493), 'DOLE': (5.4925463, 47.0921047), 'STE FOY LES LYON': (4.7934767, 45.7362255), 'BITCHE': (7.4254577, 49.0508729), 'VILLEFRANCHE DE ROUERGUE': (2.0375431, 44.3517623), 'MIRANDE': (0.4041473, 43.5143244), 'PONT A MOUSSON': (6.0547691, 48.9025946), 'LORMES': (3.8182762, 47.2890627), 'DAINVILLE': (2.72248, 50.2807141), 'BOEN SUR LIGNON': (4.004279, 45.7456231), 'CHAUMONT EN VEXIN': (1.8821405, 49.2625682), 'SOISSONS': (3.327613, 49.3837621), 'FUMAY': (4.7052616, 49.9910213), 'ST MARTIN DU TERTRE': (3.257737, 48.2121679), 'LA SOUTERRAINE': (1.4861387, 46.2383347), 'AULNAY SOUS BOIS': (2.499789, 48.934231), 'NANTERRE': (2.2071267, 48.8924273), 'BARENTIN': (0.9522586, 49.5459699), 'BUSSY ST MARTIN': (2.6921301, 48.8485248), 'PEZENAS': (3.4228323, 43.4611777), 'HIRSON': (4.0837186, 49.9203419), 'MANOSQUE': (5.7826657, 43.8338026), 'LE NOUVION EN THIERACHE': (3.7858, 50.01805), 'SOULTZ HAUT RHIN': (7.2288038, 47.8870216), 'CHATEL SUR MOSELLE': (6.3932974, 48.3131315), 'MONTBRISON': (4.0627318, 45.6072875), 'VAL COUESNON': (-1.4566868194322078, 48.44089055), 'VITRE': (-1.214331, 48.1244503), 'LE CATEAU CAMBRESIS': (3.5417479, 50.1058353), 'ST PIERRE DE BOEUF': (4.7497626, 45.3829499), 'LES ANDELYS': (1.4215534, 49.2469193), 'LOMBEZ': (0.9111013, 43.4757035), 'CORTE': (9.1511935, 42.3052904), 'DARNETAL': (1.1519145, 49.4449083), 'OYONNAX': (5.673366294950046, 46.2476745), 'L ILE D YEU': (-2.34917690050466, 46.71018105), 'LE MANS': (0.1967849, 48.0073849), 'CASTELSARRASIN': (1.1072875, 44.0395151), 'ARS SUR MOSELLE': (6.0763114, 49.0769676), 'EVRON': (-0.37536173463129474, 48.1460174), 'LE BLANC': (1.062694, 46.633292), 'HAUTMONT': (3.9200611, 50.2513704), 'BELLEME': (0.5629418, 48.3753612), 'GIEN': (2.6313889, 47.6918575), 'CHAMBRAY LES TOURS': (0.714368, 47.3390872), 'BEAUMONT DE LOMAGNE': (0.9875923, 43.8817933), 'AIRE SUR LA LYS': (2.4000602, 50.6395937), 'COMPIEGNE': (2.8263171, 49.4179497), 'LUYNES': (0.5545862, 47.3851411), 'ST MARTIN D URIAGE': (5.8383075, 45.1507233), 'MAGNAC LAVAL': (1.1668215, 46.2150181), 'ST SEVER': (-0.574187, 43.757456), 'DOL DE BRETAGNE': (-1.754257, 48.5498295), 'ST PALAIS': (-1.0345115, 43.3273438), 'MONT ST AIGNAN': (1.0813, 49.4679), 'LE PERREUX SUR MARNE': (2.5077111, 48.840627), 'OMBREE D ANJOU': (-1.1665791, 47.7451895), 'LOUVIERS': (1.1646335, 49.2157245), 'NOIRMOUTIER EN L ILE': (-2.2441449, 47.000089), 'ST PONS DE THOMIERES': (2.7598776, 43.4891622), 'PONT AUDEMER': (0.5333167486137402, 49.34758065), 'CONDOM': (0.3745929, 43.9564563), 'VALENCE ': (-0.3763353, 39.4697065), 'GRANDE SYNTHE': (2.3029972, 51.0134813), 'NEUILLY SUR SEINE': (2.2695658, 48.884683), 'BRUAY LA BUISSIERE': (2.5461922, 50.4821963), 'ST POURCAIN SUR SIOULE': (3.2899107, 46.3074423), 'CONCARNEAU': (-3.9223889, 47.8757017), 'BOURBON L ARCHAMBAULT': (3.05804, 46.5857669), 'LA CLAYETTE': (4.3092992, 46.2905337), 'LES SABLES D OLONNE': (-1.7924575344192666, 46.5002031), 'LA MURE D ISERE': (5.7836919567382505, 44.912125), 'ST JUNIEN': (0.9011111, 45.8872222), 'ST CLOUD': (-94.1642004, 45.5616075), 'ST CERE': (1.8922748, 44.8586149), 'GONESSE': (2.4500537, 48.9863555), 'CONDAT': (1.6634008, 44.9898406), 'SALINS LES BAINS': (5.8786339, 46.9404794), 'MARTIGUES': (5.0548176, 43.4057279), 'PAU': (-0.3685668, 43.2957547), 'CARENTAN LES MARAIS': (-1.2046684948980775, 49.3060982), 'SAVERNE': (7.3625953, 48.7419909), 'ST MARTIN BOULOGNE': (1.633867, 50.7258932), 'MAUVEZIN': (0.9293955, 43.3849761), 'NUITS ST GEORGES': (4.9540989, 47.134979), 'COULOMMIERS': (3.0843818, 48.8121301), 'ST BENOIT LA FORET': (0.3217875, 47.2210296), 'SEIGNOSSE': (-1.37328, 43.68866), 'DUNKERQUE ': (2.3772525, 51.0347708), 'PRADES': (4.3160861, 44.6382), 'LARGENTIERE': (4.2932518, 44.5418516), 'MOUTIERS TARENTAISE': (6.529988687824937, 45.485712649999996), 'MORLAIX': (-3.8331972, 48.5824932), 'CHOISY LE ROI': (2.4093664, 48.7630238), 'SALLES LA SOURCE': (2.5144663, 44.4366137), 'BAGNOLS SUR CEZE': (4.6235831, 44.160481), 'VALLON PONT D ARC': (4.39289, 44.4048), 'MOREZ': (6.01616871446755, 46.527073099999996), 'MACHECOUL ST MEME': (-1.808209766528024, 47.0058802), 'MACON': (-83.6324022, 32.8406946), 'GOLBEY': (6.4364631, 48.1955731), 'DOMONT': (2.3278097, 49.0285536), 'AIGUILLES': (6.8686622, 44.781105), 'LE QUESNOY': (3.6364504, 50.2473417), 'FONTENAY SOUS BOIS': (2.4749347, 48.8490721), 'AMBERT': (3.742639, 45.5504338), 'LILLE': (3.0635282, 50.6365654), 'UZES': (4.4196718, 44.0121279), 'WATTRELOS': (3.2153907, 50.7016015), 'ST PIERRE D ALBIGNY': (6.1548267, 45.569016), 'CHATEAUDUN': (1.3284948, 48.0702298), 'ALBI ': (2.147899, 43.9277552), 'GORDES': (5.2002043, 43.9110641), 'ST AIGNAN': (4.8468284, 49.6525533), 'GRANVILLE': (-1.5959213, 48.8376284), 'SARLAT LA CANEDA': (1.2161321, 44.8879431), 'SARTENE': (8.9722646, 41.620958), 'NOGENT LE ROTROU': (0.8211488, 48.321809), 'MONTIVILLIERS': (0.192418, 49.5456435), 'NYONS': (5.12838, 44.3555), 'CHATEAU THIERRY': (3.4027111, 49.0457362), 'RIVE DE GIER': (4.6187386, 45.5288401), 'REDON': (-2.0842143, 47.6510682), 'GRAND FOUGERAY': (-1.7323074, 47.7234217), 'TOUL': (5.8941759, 48.6762624), 'RANG DU FLIERS': (1.6439351, 50.4168975), 'BOURGOIN JALLIEU': (5.2828123, 45.5872259), 'ALTKIRCH': (7.2401542, 47.6238204), 'LE LUDE': (0.15514092058276263, 47.6572143), 'LANNEMEZAN': (0.384241, 43.1252298), 'BETHUNE': (2.6402097, 50.5292608), 'FEURS': (4.2215124, 45.7440738), 'LONGJUMEAU': (2.2953297, 48.6943793), 'CASTELJALOUX': (0.0880691, 44.3142971), 'BUIS LES BARONNIES': (5.2747917, 44.2767853), 'BEAUJEU': (4.5966558, 46.148552), 'CONTAMINE SUR ARVE': (6.3395716, 46.1274947), 'AUBENAS': (4.3902399, 44.6205476), 'AUBAGNE': (5.5703031, 43.2924385), 'FLORAC TROIS RIVIERES': (3.5940267, 44.3238185), 'MAEN ROCH': (-1.3402979215432647, 48.3973839), 'ROYAN': (-1.0287636, 45.6245332), 'MONTCEAU LES MINES': (4.3631681, 46.6740455), 'CORBEIL ESSONNES': (2.4818087, 48.6137734), 'ALES': (8.815632, 39.768245), 'MONTMEDY': (5.3672502, 49.5194191), 'CERNAY': (0.325, 49.0211111), 'JOUARS PONTCHARTRAIN': (1.9000212, 48.8032681), 'FONTENAY LE COMTE': (-0.8064003, 46.4660771), 'ST POL DE LEON': (-3.9868641, 48.6849601), 'LES MONTS D AUNAY': (-0.6760951409552416, 48.9907532), 'ORSAY': (2.187457, 48.699184), 'RETHEL': (4.3666467, 49.5091447), 'MIMIZAN': (-1.2309278, 44.2019796), 'ZUYDCOOTE': (2.4907115, 51.0639093), 'LOURDES': (-0.0464975, 43.0940904), 'NEVERS': (3.1577203, 46.9876601), 'LA FERTE MACE': (-0.38203748624326295, 48.580906), 'LAVELANET': (1.8487179, 42.9332121), 'BOURGES': (2.399125, 47.0811658), 'BOIS D ARCY': (3.7165618, 47.5475818), 'JANZE': (-1.495366, 47.9556915), 'AIRE SUR L ADOUR': (-0.2647028, 43.7006462), 'LE LUC': (6.3128995, 43.3946278), 'THIZY LES BOURGS': (4.310490309298064, 46.05038665), 'VAUX SUR MER': (-1.0623397, 45.645335), 'MAYENNE': (-0.6491273812007092, 48.1507819), 'BRUMATH': (7.708107, 48.7309406), 'LE LOROUX BOTTEREAU': (-1.3472697, 47.2385415), 'MILLAU': (3.0777594, 44.1006693), 'JEUMONT': (4.0996569, 50.2971012), 'MARENNES HIERS BROUAGE': (-1.0679912521809798, 45.8341032), 'SILLE LE GUILLAUME': (-0.1275755, 48.1841304), 'DIEUZE': (6.7179948, 48.8104847), 'VIRIAT': (5.2167876, 46.2533181), 'BERNAY': (0.5989157, 49.0902278), 'ST LOUIS': (-90.1910154, 38.6280278), 'COMBOURG': (-1.7500622, 48.4104391), 'BISCARROSSE': (-1.1609361, 44.3907065), 'BAUGE EN ANJOU': (-0.1036842, 47.5612379), 'L AIGLE': (0.6278039, 48.7651713), 'TRAPPES': (1.9988356, 48.7760957), 'HAZEBROUCK': (2.5360331, 50.7226113), 'QUILLAN': (2.1941008666750825, 42.876584699999995), 'FUMEL': (0.9684626, 44.4966624), 'LES FOSSES': (-0.3703803, 46.1643449), 'ETAMPES': (2.1614464, 48.4344621), 'LE VIGAN': (3.6059407, 43.9912933), 'ARGENTAN': (-0.0223386, 48.74289), 'BRIVE LA GAILLARDE': (1.5332389, 45.1584982), 'CHATEAU GONTIER': (-0.7066665, 47.8263058), 'ETAPLES': (1.6386252, 50.5139552), 'LE CHEYLARD': (4.42245, 44.9093), 'BOURGANEUF': (1.7567677, 45.9545107), 'CHAMBON': (-0.8497848, 46.1257314), 'FRONTIGNAN': (3.7530636, 43.4487615), 'NOZAY': (4.09529, 48.5085), 'AGEN ': (0.6176112, 44.2015827), 'COUTANCES': (-1.4454857, 49.047792), 'SENLIS': (2.1523287, 50.5331612), 'CHARLIEU': (4.1714892, 46.1588067), 'BAUME LES DAMES': (6.3614622, 47.3526707), 'ARNAY LE DUC': (4.4858635, 47.1309349), 'LE DORAT': (1.0822603, 46.2144367), 'THOUARS': (-0.2057893552874009, 46.983202750000004), 'BOURG EN BRESSE': (5.2250324, 46.2051192), 'CHAMBERY': (5.9203636, 45.5662672), 'BARCELONNETTE': (6.6513554, 44.3862043), 'EMBRUN': (6.4958189, 44.5641764), 'DINAN': (-2.047687, 48.4539775), 'SOTTEVILLE LES ROUEN': (1.089746, 49.4151549), 'LOUDEAC': (-2.7535644, 48.1762534), 'ST POL SUR TERNOISE': (2.3361053, 50.3812113), 'POISSY': (2.0428293, 48.927439), 'CHOLET': (-0.8801349, 47.0617293), 'ISPOURE': (-1.2337588, 43.1704612), 'SALON DE PROVENCE': (5.0980225, 43.6405237), 'EGLY': (2.2216686, 48.5774044), 'LAMBALLE': (-2.516502292079859, 48.46244615), 'TRANS EN PROVENCE': (6.4867321, 43.5039337), 'SETE': (3.6959771, 43.4014434), 'RUEIL MALMAISON': (2.1802832, 48.87778), 'IS SUR TILLE': (5.1123622, 47.5219136), 'LANDIVISIAU': (-4.0699356, 48.5127551), 'SEYNE LES ALPES': (6.3563226, 44.3521974), 'CRETEIL': (2.4530731, 48.7771486), 'PLEMET': (-2.587198219251695, 48.18859125), 'ALLAUCH': (5.4832687, 43.3351381), 'CHATEAUNEUF SUR CHARENTE': (-0.0543783, 45.6000693), 'MANTES LA JOLIE': (1.7140683, 48.9891971), 'ST SYMPHORIEN SUR COISE': (4.4550523, 45.6313062), 'SAULIEU': (4.230692, 47.2775177), 'LE LAVANDOU': (6.3701286, 43.1383795), 'BEAUNE': (4.8380426, 47.0239064), 'BRESSUIRE': (-0.4929017, 46.8425572), 'LA REOLE': (-0.0350566, 44.5821161), 'LUCON': (120.73967457219265, 15.5919971), 'ST NICOLAS DE PORT': (6.3022451, 48.6310196), 'DIE': (139.2394179, 36.5748441), 'MARVEJOLS': (3.2901135, 44.5532621), 'LA CHARITE SUR LOIRE': (3.0185784, 47.1793255), 'SISTERON': (5.9443791, 44.1963467), 'ANNECY': (6.1288847, 45.8992348), 'ARMENTIERES': (2.882169, 50.6867281), 'REVEL': (24.7453688, 59.4372155), 'ST FLOUR': (3.5026164, 45.7084425), 'ST AFFRIQUE': (2.8866641, 43.9580035), 'CAUDRY': (3.4105032, 50.1243363), 'MARIGNANE': (5.2146275, 43.4162729), 'ST MARCELLIN': (5.320347, 45.1539715), 'LIMOUX': (2.2176533, 43.0538068), 'ST BRIEUC': (-2.7603283, 48.5141134), 'PONT DE VAUX': (4.937635, 46.4316899), 'PAYS DE BELVES': (1.0068584, 44.7751847), 'ROMORANTIN LANTHENAY': (1.7434925, 47.3592182), 'VERNON': (1.4886571, 49.0917814), 'REMIREMONT': (6.5912615, 48.0158951), 'GERARDMER': (6.8776965, 48.0705772), 'ST SAULVE': (3.5547336, 50.3710165), 'VERFEIL': (1.6606008, 43.6570422), 'YVETOT': (0.7538866, 49.6179384), 'LA CHATRE': (1.9892292, 46.5815678), 'SANCERRE': (2.8390333, 47.3307251), 'BOURG ACHARD': (0.8149501, 49.35255), 'LONGWY': (5.7679082, 49.5145564), 'ST CHELY D APCHER': (3.2757824, 44.8011363), 'PELUSSIN': (4.678089, 45.4179803), 'FRUGES': (2.1349286, 50.5150387), 'LE MANS ': (0.1967849, 48.0073849)}

In [ ]:
#on calcule ainsi les distances à vol d'oiseau entre les établissements et toutes les communes
#remarque : calculer les distance en voiture aurait été possible techniquement également via l'API OpenStreetMap mais vu le nombre de distances à calculer, cela aurait pris des semaines vu les limitations de l'API qui n'autorise qu'un nombre maximal de requetes par seconde

dico_distances = {}

for ville_h in lat_long_etablissements:

  lo1, la1 = lat_long_etablissements[ville_h]
  for ville_m in lat_long_malades:
    lo2, la2 = lat_long_malades[ville_m]
    dico_distances[(ville_h, ville_m)] = bird_flight_distance(la1,lo1,la2,lo2)

# Création du fichier contenant les distances utiles

In [ ]:
# Nous nous servons du dictionnaire créé dans la partie précédente pour
# stocker les distances entre toutes nos villes
edges_data = []
for couple in dico_distances:
  ville_h, ville_m = couple[0], couple[1]
  edges_data.append({"Malade":ville_m, "Etablissement":ville_h, "Distance":dico_distances[(ville_h, ville_m)]})

In [ ]:
# Enregistrement des distances sous un fichier "edges.csv"
with open('edges_data.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = ["Malade", "Etablissement", "Distance"])
    writer.writeheader()
    writer.writerows(edges_data)

# Suppression des grandes distances

In [ ]:
edges = pd.read_csv("edges_data.csv")

In [ ]:
# Nous ne conservons que les distances inférieures à 50km
edges = edges[edges["Distance"]<50]

In [ ]:
# Export dans le fichier "small_edges_data.csv"
edges.to_csv("small_edges_data.csv", sep=";")

# Création du graphe

In [ ]:
# Nous combinons les données sur les malades, sur les établissements, et sur les distances
# pour créer un graphe où les malades sont reliés par une arête
# aux établissements à moins de 50km de chez eux
data = []

for i in range(len(edges)):
  ville_m, ville_h, distance = edges.loc[edges.index[i],"Malade"], edges.loc[edges.index[i],"Etablissement"], edges.loc[edges.index[i],"Distance"]
  sous_malades = np.array(nouveaux_malades[nouveaux_malades.ville == ville_m].index)
  sous_etablissements = np.array(etablissements_structure[etablissements_structure.ville == ville_h].nofinesset)
  for sous_malade in sous_malades:
    for sous_etablissement in sous_etablissements:
      data.append({"id_malade":sous_malade, "id_etablissement":sous_etablissement, "dist":distance})

In [ ]:
# Export dans le fichier "edges_poumon_50km.csv"
with open("edges_poumon_50km.csv", 'w') as csvfile:
  writer = csv.DictWriter(csvfile, fieldnames=["id_malade", "id_etablissement", "dist"], delimiter=";")
  writer.writeheader()
  for row in data:
      writer.writerow(row)